In [ ]:
#定义渗透率,一系列数字,按照论文上的实验结果来
#指标有通过平均等待时间、平均总停车次数、总吞吐量、总延迟时间,以及渗透率            先想办法在代码中添加计算这些量的函数
#现在有一个问题,只有两个数据集,grad4*4 和hangzhou,没有在其他路网上进行对比（这是后续需要考虑的问题）
penetration_rate = [0.3]
#虽然论文上是这样写的,但是我看代码里的计算方法不是这样的
#关于指标的记录,我的建议是,将车辆的运行数据以csv格式保存,然后读取csv文件,计算指标,保存到csv文件中,这样比较方便。先把结果跑出来
# 今天早上需要做的事情：
#1.确定渗透率30%
#2.由渗透率,计算得到需要添加的车辆数量  grid有1473辆车,hangzhou有1660辆车,所以需要添加的汽车数量为1473-1473*0.3= 466.7
#3.由random.seed(0),得到车辆的起点与终点
#4.在原文件的基础上添加输出数据的函数,以方便计算指标
#弄清楚episode是什么？RP[0,10,20,30,40,50]
#5.将测评出的结果画图

In [1]:
depart = [28,79,118,331,433,307,520,569,306,557]
arrival = [380,404,449,467,575,634,649,697,739,761]
duration =[352,325,331,136,142,327,129,128,433,204]
waitingTime = [87,109,5,0,0,127,247]
timeLoss = [116.35,134.44,64.31,16.74,8.73,153.54,14.84,8.28,266.04,13.71]


In [2]:
def calculate(A):
    total_sum = sum(A)
    count = len(A)
    average = total_sum / count
    print(f"总和:{total_sum}")
    print(f"平均数:{average}" )


calculate(arrival)
calculate(duration)
calculate(waitingTime)
calculate(timeLoss)

总和:5755
平均数:575.5
总和:2507
平均数:250.7
总和:575
平均数:82.14285714285714
总和:796.98
平均数:79.69800000000001


In [ ]:
#现在先不急,先把所选取的车辆以及路线确定下来
PR = 0.05   grid_num = 73   hangzhou_num = 88
PR = 0.1  grid_num = 147   hangzhou_num = 166
PR = 0.2  grid_num = 295   hangzhou_num = 332
PR = 0.3  grid_num = 442   hangzhou_num = 498
PR = 0.4  grid_num = 589  hangzhou_num = 664
PR = 0.5  grid_num = 736   hangzhou_num = 830

In [ ]:
grid_num = round(0.5*1660)
print(grid_num)

In [ ]:
import numpy as np
np.random.seed(0)  # 设置随机数生成器的种子
grid_num = 1494
hangzhou_num = 1660
existing_grid_routes = [i for i in range(grid_num)] 
print(existing_grid_routes) # 创建包含1494个整数的列表
selected_grid_route = np.random.choice(existing_grid_routes) 
print(f"selected_grid_route is {selected_grid_route}") 
print("route is depart= 1747.00 right1D1 D1C1 C1C0 C0B0 B0A0 A0left0")
existing_hangzhou_routes = [i for i in range(hangzhou_num)]  # 创建包含1494个整数的列表
selected_hangzhou_route = np.random.choice(existing_hangzhou_routes) 
print(f"selected_hangzhou_route is {selected_hangzhou_route}")# 从existing_routes中随机选择一个元素
print("route is depart=636','road_1_5_3 road_1_4_3 road_1_3_3 road_1_2_2")

In [ ]:
#获取要添加的车辆的信息
import xml.etree.ElementTree as ET

def extract_routes_by_ids(file_path','ids','output_file):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # 创建一个新的根元素来存放提取的车辆信息
    new_root = ET.Element('routes')

    for vehicle in root.findall('vehicle'):
        vid = vehicle.attrib['id']
        if vid in ids:
            new_vehicle = ET.SubElement(new_root',''vehicle'','{
                'id': f'v{vid}',
                'depart': '0',
            })
            route = vehicle.find('route')
            new_route = ET.SubElement(new_vehicle',''route')
            new_route.text = route.text

    # 创建一个新的ElementTree对象并保存到文件
    new_tree = ET.ElementTree(new_root)
    new_tree.write(output_file','encoding='UTF-8'','xml_declaration=True)

# 定义rou.xml文件路径、需要提取的id列表以及输出文件名
rou_xml_path = r'test_data\孙倩\grid4x4\grid4x4.rou.xml'
desired_ids = ['1'',''5'',''6']
output_filename = 'add_vehicle.xml'

# 调用函数保存结果到文件
extract_routes_by_ids(rou_xml_path','desired_ids','output_filename)
print(f"Extracted routes saved to {output_filename}")

In [ ]:
import xml.etree.ElementTree as ET
from xml.dom import minidom  # 明确导入minidom

def extract_routes_by_ids(file_path','ids','output_file):
    tree = ET.parse(file_path)
    root = tree.getroot()

    new_root = ET.Element('route edges = ')  # 新建根元素以存放提取的车辆信息

    for vehicle in root.findall('vehicle'):
        vid = vehicle.attrib.get('id')
        if vid in ids:
            # 复制并调整车辆元素
            new_vehicle = ET.SubElement(new_root',''vehicle'','attrib={'id': f'v{vid}'})
            for child in vehicle:
                if child.tag == 'route':
                    # 保留route子元素
                    new_route = ET.SubElement(new_vehicle',''route')
                    new_route.text = child.text

    # 使用minidom处理美化输出
    rough_string = ET.tostring(new_root',''utf-8')
    reparsed = minidom.parseString(rough_string)
    pretty_xml_as_string = reparsed.toprettyxml(indent="    ")

    # 直接写入到文件,无需解码
    with open(output_file',''w') as file_out:
        file_out.write(pretty_xml_as_string)

# 定义rou.xml文件路径、需要提取的id列表以及输出文件名
rou_xml_path = r'test_data\孙倩\grid4x4\grid4x4.rou.xml'
desired_ids = ['1'',''5'',''6']
output_filename = 'add_vehicle.xml'

# 调用函数保存结果到文件
extract_routes_by_ids(rou_xml_path','desired_ids','output_filename)
print(f"Extracted routes saved to {output_filename}")

In [ ]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

def extract_routes_by_ids(input_file','output_file','desired_ids):
    # 解析rou.xml文件
    tree = ET.parse(input_file)
    root = tree.getroot()

    # 创建一个新的Element来存储提取的vehicle元素
    new_root = ET.Element('routes')

    # 遍历原始XML中的vehicle元素
    for vehicle in root.findall('vehicle'):
        vid = vehicle.attrib.get('id')
        # 检查当前车辆ID是否在指定的ID列表中
        if vid in desired_ids:
            # 创建一个新的vehicle元素,并设置id和depart属性
            new_vehicle = ET.SubElement(new_root',''vehicle'','{
                'id': f'v{vid}',
                'depart': '0'
            })
            # 复制route子元素到新vehicle中
            for child in vehicle:
                new_vehicle.append(child)

    # 使用minidom美化输出
    rough_string = ET.tostring(new_root',''utf-8')
    reparsed = minidom.parseString(rough_string)
    pretty_xml_as_string = reparsed.toprettyxml(indent="    ")

    # 将美化后的XML写入到新的文件中
    with open(output_file',''w') as file_out:
        file_out.write(pretty_xml_as_string)

# 定义输入输出文件路径及需要提取的车辆ID列表
rou_xml_path = r'test_data\孙倩\grid4x4\grid4x4.rou.xml'
output_file = 'new_rou_formatted.xml'
desired_ids = ['1'',''5'',''6']

# 调用函数执行提取并美化输出操作
extract_routes_by_ids(rou_xml_path','output_file','desired_ids)
print(f"Extracted and formatted routes saved to {output_file}")

In [ ]:

import xml.etree.ElementTree as ET
import numpy as np

np.random.seed(0)  # 设置随机数生成器的种子
grid_num = 1473
hangzhou_num = 1660
add_grid_num = [73','147','295','442','589','736]
add_hangzhou_num = [88','166','332','498','664','830]

def generate_xml_selections(name','total_num','add_nums):
    """生成XML文档,记录选择的整数"""
    root = ET.Element("Selections")
    for i','num_to_pick in enumerate(add_nums):
        effective_pick = min(num_to_pick','total_num)
        selected_integers = np.random.choice(total_num','size=effective_pick','replace=False)
        selection = ET.SubElement(root','f"{name}{i+1}")
        integers_str = ''.join(map(str','sorted(selected_integers)))
        selection.text = integers_str
    rough_string = ET.tostring(root',''utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

# 生成Grid和Hangzhou的选择XML
grid_xml = generate_xml_selections("Grid"','grid_num','add_grid_num)
hangzhou_xml = generate_xml_selections("Hangzhou"','hangzhou_num','add_hangzhou_num)

print("Grid Selections XML:")
print(grid_xml)
print("\nHangzhou Selections XML:")
print(hangzhou_xml)


In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
from xml.dom import minidom

np.random.seed(0)  # 设置随机数生成器的种子
grid_num = 1473
hangzhou_num = 1660
add_grid_num = [73','147','295','442','589','736]
add_hangzhou_num = [88','166','332','498','664','830]

def generate_xml_selections(name','total_num','add_nums):
    """生成XML文档,记录选择的整数,每个数字周围带有单引号"""
    root = ET.Element("Selections")
    for i','num_to_pick in enumerate(add_nums):
        effective_pick = min(num_to_pick','total_num)
        selected_integers = np.random.choice(total_num','size=effective_pick','replace=False)
        selection = ET.SubElement(root','f"{name}{i+1}")
        # 确保每个数字都被单引号包围
        integers_with_quotes = ["'{}'".format(num) for num in sorted(selected_integers)]
        selection.text = ''.join(integers_with_quotes)
    rough_string = ET.tostring(root',''utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

# 生成Grid和Hangzhou的选择XML
grid_xml = generate_xml_selections("Grid"','grid_num','add_grid_num)
hangzhou_xml = generate_xml_selections("Hangzhou"','hangzhou_num','add_hangzhou_num)

print("Grid Selections XML:")
print(grid_xml)
print("\nHangzhou Selections XML:")
print(hangzhou_xml)
'''output
Hangzhou_10_vechle.xml
Grid Selections XML:
<?xml version="1.0" ?>
<Selections>
  <Grid1>'4''31''34''52''54''82''89''141''142''148'
  '152''191''192''231''254''260''278''302''322''326'
    '333''347''361''362''364''376''408''420''427''434'
    '446''458''483''487''529''535''554''558''651''674''758'
    '761''838''853''857''863''884''897''900''930''942''961'
      '1001''1030''1031''1054''1061''1069''1081''1095''1114'
      '1122''1131''1157''1186''1278''1283''1336''1350''1400'
      '1411''1442''1463'</Grid1>
'''

In [ ]:
Grid_5=['4'',''31'',''34'',''52'',''54'',''82'',''89'',''141'',''142'',''148'',''152'',''191'',''192'',''231'',''254'',''260'',''278'',''302'',''322'',''326'',''333'',''347'',''361'',''362'',''364'',''376'',''408'',''420'',''427'',''434'',''446'',''458'',''483'',''487'',''529'',''535'',''554'',''558'',''651'',''674'',''758'',''761'',''838'',''853'',''857'',''863'',''884'',''897'',''900'',''930'',''942'',''961'',''1001'',''1030'',''1031'',''1054'',''1061'',''1069'',''1081'',''1095'',''1114'',''1122'',''1131'',''1157'',''1186'',''1278'',''1283'',''1336'',''1350'',''1400'',''1411'',''1442'',''1463']
Grid_10=['22'',''33'',''56'',''60'',''71'',''92'',''104'',''117'',''118'',''129'',''138'',''146'',''180'',''181'',''182'',''183'',''195'',''213'',''216'',''217'',''224'',''227'',''243'',''246'',''248'',''276'',''302'',''306'',''311'',''329'',''333'',''336'',''337'',''352'',''378'',''380'',''388'',''396'',''399'',''406'',''410'',''412'',''418'',''423'',''436'',''454'',''465'',''471'',''495'',''496'',''499'',''501'',''509'',''533'',''537'',''542'',''543'',''545'',''547'',''548'',''556'',''571'',''577'',''592'',''599'',''600'',''611'',''615'',''630'',''641'',''643'',''648'',''661'',''668'',''672'',''673'',''678'',''681'',''692'',''695'',''701'',''719'',''742'',''760'',''762'',''783'',''786'',''787'',''798'',''800'',''802'',''806'',''816'',''831'',''840'',''841'',''868'',''872'',''885'',''898'',''905'',''906'',''918'',''933'',''954'',''961'',''965'',''968'',''982'',''998'',''999'',''1005'',''1009'',''1038'',''1062'',''1063'',''1074'',''1083'',''1086'',''1102'',''1112'',''1114'',''1147'',''1149'',''1156'',''1166'',''1179'',''1224'',''1250'',''1255'',''1257'',''1261'',''1267'',''1293'',''1294'',''1307'',''1318'',''1320'',''1321'',''1328'',''1340'',''1362'',''1373'',''1385'',''1416'',''1448'',''1466'] 
Grid_20=['0'',''6'',''18'',''20'',''29'',''42'',''43'',''45'',''48'',''55'',''56'',''59'',''63'',''72'',''75'',''84'',''90'',''95'',''96'',''108'',''116'',''117'',''119'',''121'',''125'',''127'',''130'',''134'',''142'',''143'',''148'',''151'',''152'',''154'',''158'',''159'',''162'',''163'',''165'',''168'',''170'',''179'',''181'',''186'',''187'',''192'',''194'',''195'',''210'',''212'',''214'',''218'',''220'',''223'',''237'',''246'',''254'',''265'',''267'',''268'',''269'',''272'',''275'',''277'',''278'',''282'',''291'',''294'',''301'',''302'',''304'',''308'',''313'',''320'',''328'',''330'',''334'',''337'',''339'',''343'',''359'',''371'',''379'',''383'',''385'',''392'',''394'',''397'',''401'',''402'',''410'',''411'',''412'',''416'',''420'',''430'',''437'',''447'',''453'',''456'',''468'',''474'',''481'',''487'',''497'',''500'',''514'',''524'',''525'',''528'',''530'',''532'',''533'',''540'',''543'',''545'',''547'',''548'',''556'',''559'',''567'',''571'',''584'',''585'',''586'',''592'',''595'',''599'',''600'',''606'',''608'',''610'',''627'',''630'',''634'',''636'',''637'',''639'',''640'',''646'',''650'',''652'',''660'',''666'',''667'',''668'',''678'',''679'',''684'',''704'',''707'',''708'',''712'',''715'',''716'',''717'',''719'',''720'',''728'',''730'',''734'',''736'',''737'',''739'',''745'',''747'',''761'',''771'',''780'',''790'',''792'',''793'',''799'',''800'',''806'',''828'',''842'',''844'',''851'',''854'',''856'',''861'',''867'',''870'',''872'',''873'',''878'',''895'',''896'',''903'',''904'',''915'',''924'',''933'',''948'',''965'',''968'',''969'',''972'',''977'',''978'',''980'',''982'',''991'',''995'',''1002'',''1005'',''1017'',''1029'',''1030'',''1040'',''1042'',''1044'',''1051'',''1055'',''1057'',''1066'',''1067'',''1072'',''1073'',''1075'',''1077'',''1082'',''1083'',''1094'',''1096'',''1098'',''1109'',''1115'',''1124'',''1125'',''1126'',''1132'',''1133'',''1138'',''1139'',''1144'',''1154'',''1156'',''1157'',''1160'',''1163'',''1177'',''1188'',''1192'',''1201'',''1220'',''1225'',''1227'',''1246'',''1250'',''1254'',''1256'',''1257'',''1259'',''1263'',''1264'',''1265'',''1267'',''1274'',''1275'',''1279'',''1284'',''1286'',''1305'',''1307'',''1315'',''1319'',''1320'',''1321'',''1328'',''1329'',''1332'',''1338'',''1343'',''1360'',''1375'',''1379'',''1382'',''1398'',''1406'',''1410'',''1415'',''1419'',''1420'',''1422'',''1435'',''1440'',''1459'',''1461'',''1463'',''1464'',''1465'',''1466'',''1471']
Grid_30=['6'',''14'',''15'',''16'',''20'',''22'',''23'',''25'',''29'',''39'',''40'',''42'',''50'',''52'',''55'',''56'',''58'',''63'',''70'',''72'',''80'',''89'',''98'',''101'',''105'',''107'',''108'',''111'',''112'',''115'',''119'',''120'',''122'',''123'',''127'',''130'',''133'',''134'',''136'',''142'',''143'',''144'',''146'',''151'',''160'',''164'',''165'',''166'',''170'',''173'',''178'',''179'',''181'',''186'',''187'',''191'',''193'',''195'',''197'',''201'',''202'',''205'',''206'',''214'',''219'',''227'',''232'',''233'',''234'',''247'',''252'',''254'',''255'',''256'',''257'',''260'',''261'',''262'',''263'',''270'',''272'',''273'',''279'',''283'',''286'',''287'',''290'',''302'',''304'',''311'',''312'',''316'',''318'',''328'',''329'',''332'',''334'',''335'',''337'',''339'',''340'',''343'',''345'',''346'',''358'',''359'',''364'',''369'',''371'',''374'',''375'',''379'',''386'',''394'',''401'',''405'',''406'',''407'',''409'',''413'',''423'',''425'',''427'',''429'',''432'',''433'',''438'',''443'',''447'',''451'',''458'',''462'',''464'',''467'',''468'',''471'',''472'',''473'',''478'',''479'',''482'',''485'',''489'',''499'',''502'',''506'',''510'',''512'',''514'',''516'',''519'',''526'',''527'',''528'',''532'',''533'',''535'',''536'',''539'',''542'',''547'',''551'',''554'',''555'',''557'',''559'',''563'',''569'',''572'',''581'',''583'',''592'',''595'',''596'',''605'',''609'',''614'',''617'',''619'',''620'',''625'',''626'',''627'',''628'',''629'',''634'',''650'',''654'',''660'',''661'',''664'',''668'',''675'',''676'',''677'',''679'',''683'',''684'',''685'',''687'',''691'',''692'',''693'',''694'',''696'',''698'',''699'',''705'',''706'',''708'',''712'',''713'',''715'',''719'',''720'',''722'',''734'',''735'',''737'',''738'',''740'',''742'',''747'',''759'',''764'',''766'',''767'',''771'',''772'',''773'',''775'',''781'',''782'',''785'',''786'',''787'',''792'',''797'',''798'',''800'',''808'',''809'',''816'',''828'',''830'',''831'',''834'',''836'',''838'',''839'',''840'',''844'',''853'',''856'',''858'',''861'',''867'',''868'',''869'',''871'',''872'',''873'',''875'',''878'',''882'',''885'',''887'',''888'',''893'',''894'',''897'',''910'',''912'',''913'',''916'',''919'',''930'',''934'',''944'',''946'',''948'',''949'',''952'',''957'',''958'',''959'',''961'',''962'',''969'',''971'',''973'',''977'',''979'',''980'',''981'',''982'',''984'',''988'',''992'',''993'',''994'',''995'',''997'',''1006'',''1007'',''1009'',''1010'',''1016'',''1022'',''1027'',''1036'',''1037'',''1040'',''1051'',''1052'',''1053'',''1056'',''1061'',''1068'',''1069'',''1079'',''1080'',''1081'',''1090'',''1093'',''1094'',''1097'',''1098'',''1101'',''1102'',''1103'',''1108'',''1111'',''1117'',''1118'',''1124'',''1129'',''1133'',''1135'',''1137'',''1138'',''1140'',''1143'',''1147'',''1154'',''1156'',''1157'',''1158'',''1159'',''1161'',''1170'',''1172'',''1174'',''1181'',''1182'',''1183'',''1186'',''1187'',''1188'',''1194'',''1196'',''1201'',''1202'',''1209'',''1211'',''1213'',''1214'',''1215'',''1216'',''1218'',''1223'',''1228'',''1229'',''1231'',''1238'',''1240'',''1241'',''1242'',''1250'',''1259'',''1262'',''1263'',''1272'',''1276'',''1277'',''1278'',''1280'',''1289'',''1291'',''1294'',''1295'',''1303'',''1316'',''1320'',''1321'',''1322'',''1323'',''1328'',''1337'',''1342'',''1343'',''1344'',''1345'',''1346'',''1349'',''1350'',''1353'',''1360'',''1362'',''1364'',''1365'',''1366'',''1371'',''1373'',''1379'',''1380'',''1384'',''1385'',''1387'',''1396'',''1398'',''1399'',''1400'',''1402'',''1403'',''1404'',''1405'',''1407'',''1410'',''1416'',''1418'',''1428'',''1429'',''1432'',''1442'',''1444'',''1445'',''1451'',''1454'',''1460'',''1466'',''1468']
Grid_40=['1'',''4'',''16'',''18'',''20'',''22'',''23'',''27'',''29'',''32'',''33'',''36'',''37'',''38'',''39'',''42'',''43'',''44'',''45'',''46'',''49'',''50'',''52'',''55'',''56'',''58'',''59'',''60'',''63'',''64'',''70'',''71'',''72'',''73'',''75'',''77'',''80'',''83'',''90'',''94'',''96'',''98'',''102'',''105'',''106'',''109'',''110'',''111', '112', '116', '118', '119', '125', '127', '134', '135', '140', '144', '147', '150', '151', '152', '154', '156', '157', '158', '161', '162', '166', '167', '169', '174', '176', '177', '178', '181', '183', '187', '189', '190', '196', '199', '200', '201', '202', '205', '208', '210', '212', '214', '215', '217', '218', '221', '224', '226', '229', '233', '234', '238', '244', '249', '251', '254', '257', '259', '260', '261', '265', '269', '273', '274', '276', '277', '278', '282', '294', '295', '296', '297', '303', '304', '306', '311', '313', '315', '317', '323', '330', '334', '338', '342', '344', '347', '349', '350', '352', '358', '367', '368', '369', '370', '373', '374', '376', '379', '380', '382', '383', '384', '386', '387', '388', '389', '391', '392', '394', '396', '399', '400', '403', '404', '406', '409', '410', '421', '423', '427', '428', '429', '431', '434', '438', '440', '441', '442', '444', '448', '449', '450', '451', '452', '454', '455', '456', '458', '459', '460', '461', '462', '463', '466', '467', '469', '473', '480', '481', '482', '485', '487', '491', '495', '498', '501', '506', '507', '508', '509', '511', '514', '516', '519', '521', '523', '524', '526', '527', '528', '533', '536', '537', '540', '541', '543', '546', '548', '553', '558', '562', '563', '565', '566', '567', '569', '572', '573', '574', '576', '582', '583', '586', '588', '590', '591', '593', '595', '600', '601', '604', '611', '612', '613', '616', '617', '625', '626', '628', '629', '630', '631', '632', '633', '637', '638', '640', '641', '642', '645', '646', '647', '651', '653', '656', '659', '669', '674', '677', '679', '682', '683', '684', '686', '687', '691', '692', '693', '694', '696', '700', '701', '703', '705', '706', '707', '711', '712', '713', '714', '717', '719', '720', '721', '722', '725', '726', '727', '728', '734', '736', '737', '740', '748', '752', '755', '757', '758', '769', '771', '772', '775', '776', '779', '787', '790', '791', '793', '796', '797', '798', '800', '802', '803', '804', '805', '808', '811', '814', '815', '817', '819', '821', '822', '823', '825', '827', '830', '833', '834', '837', '838', '839', '842', '844', '845', '846', '848', '852', '857', '858', '860', '863', '865', '866', '867', '869', '877', '878', '880', '881', '886', '887', '888', '889', '891', '902', '909', '912', '914', '915', '916', '917', '923', '924', '927', '929', '930', '932', '933', '935', '937', '939', '940', '942', '944', '945', '947', '953', '955', '958', '961', '962', '963', '971', '972', '977', '978', '979', '982', '984', '985', '987', '992', '994', '995', '997', '1003', '1012', '1017', '1018', '1019', '1021', '1022', '1024', '1030', '1031', '1032', '1033', '1034', '1036', '1039', '1040', '1042', '1045', '1052', '1054', '1055', '1057', '1061', '1062', '1063', '1064', '1065', '1067', '1068', '1070', '1072', '1074', '1077', '1081', '1085', '1087', '1090', '1092', '1101', '1102', '1104', '1106', '1115', '1116', '1118', '1119', '1121', '1124', '1125', '1128', '1130', '1132', '1133', '1135', '1138', '1143', '1148', '1151', '1154', '1155', '1156', '1157', '1159', '1161', '1165', '1166', '1167', '1168', '1171', '1176', '1177', '1178', '1179', '1181', '1184', '1188', '1189', '1191', '1192', '1197', '1199', '1208', '1212', '1213', '1216', '1217', '1218', '1219', '1220', '1222', '1224', '1226', '1230', '1234', '1235', '1236', '1237', '1241', '1245', '1246', '1254', '1260', '1263', '1264', '1275', '1276', '1280', '1282', '1286', '1289', '1292', '1293', '1294', '1297', '1299', '1301', '1302', '1303', '1304', '1314', '1316', '1317', '1319', '1321', '1323', '1324', '1325', '1326', '1327', '1332', '1336', '1338', '1339', '1340', '1343', '1348', '1350', '1351', '1354', '1356', '1359', '1360', '1363', '1364', '1369', '1373', '1378', '1381', '1385', '1387', '1389', '1394', '1395', '1398', '1401', '1404', '1406', '1407', '1415', '1425', '1426', '1428', '1433', '1435', '1443', '1447', '1451', '1458', '1459', '1461', '1466', '1467', '1469', '1470']
Grid_50=['0', '1', '4', '6', '7', '8', '9', '11', '12', '17', '18', '19', '23', '27', '30', '31', '32', '33', '36', '37', '38', '41', '42', '43', '45', '46', '50', '51', '53', '55', '58', '59', '60', '61', '65', '66', '67', '70', '71', '76', '83', '85', '91', '93', '95', '96', '98', '100', '104', '105', '106', '107', '111', '114', '116', '119', '121', '122', '126', '127', '128', '131', '134', '136', '137', '138', '139', '140', '146', '149', '150', '153', '154', '158', '164', '168', '174', '176', '177', '178', '179', '180', '183', '184', '185', '187', '188', '190', '191', '192', '193', '195', '196', '197', '198', '199', '200', '201', '204', '206', '208', '210', '211', '220', '221', '224', '227', '228', '232', '233', '234', '240', '242', '245', '247', '249', '250', '251', '253', '254', '255', '257', '259', '260', '261', '263', '264', '267', '270', '271', '272', '273', '275', '277', '278', '280', '281', '285', '287', '289', '292', '295', '296', '298', '300', '302', '303', '304', '305', '306', '308', '309', '310', '314', '315', '317', '320', '324', '325', '327', '328', '329', '332', '334', '336', '337', '339', '342', '343', '347', '348', '354', '358', '361', '362', '363', '365', '368', '369', '372', '374', '376', '379', '381', '384', '385', '386', '387', '389', '390', '393', '394', '396', '397', '398', '403', '404', '405', '406', '408', '410', '412', '413', '416', '417', '419', '421', '424', '425', '426', '427', '428', '429', '431', '436', '441', '442', '445', '447', '449', '451', '454', '455', '458', '462', '468', '471', '474', '477', '478', '479', '485', '486', '487', '488', '489', '495', '504', '505', '508', '510', '512', '513', '518', '519', '520', '521', '524', '526', '528', '530', '532', '533', '536', '537', '538', '541', '542', '544', '547', '549', '552', '558', '560', '561', '563', '567', '569', '573', '574', '575', '577', '579', '580', '581', '582', '584', '585', '587', '589', '591', '592', '593', '594', '595', '596', '597', '598', '601', '603', '604', '607', '608', '612', '615', '618', '619', '620', '622', '623', '625', '629', '632', '634', '635', '636', '638', '639', '646', '648', '650', '652', '654', '656', '658', '659', '663', '664', '666', '669', '671', '674', '676', '677', '678', '679', '681', '689', '690', '691', '692', '694', '701', '702', '703', '705', '706', '708', '712', '713', '717', '718', '719', '720', '723', '724', '726', '729', '732', '734', '737', '744', '745', '746', '747', '749', '754', '757', '758', '760', '762', '763', '764', '765', '768', '770', '771', '774', '775', '776', '777', '779', '780', '781', '785', '787', '788', '789', '790', '793', '797', '799', '801', '802', '804', '807', '810', '812', '814', '815', '817', '818', '820', '821', '825', '826', '827', '831', '832', '833', '834', '838', '839', '841', '843', '846', '847', '848', '850', '851', '853', '854', '855', '857', '859', '860', '863', '866', '869', '870', '871', '873', '875', '877', '878', '879', '880', '883', '884', '887', '888', '892', '894', '899', '903', '905', '906', '907', '909', '910', '917', '920', '921', '923', '925', '927', '928', '933', '935', '937', '938', '939', '940', '944', '945', '946', '947', '950', '951', '952', '956', '957', '959', '960', '961', '964', '966', '968', '969', '970', '972', '975', '977', '979', '980', '982', '983', '985', '992', '994', '995', '996', '997', '998', '1001', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '1010', '1013', '1016', '1018', '1025', '1026', '1027', '1028', '1030', '1031', '1033', '1034', '1035', '1036', '1041', '1042', '1045', '1046', '1048', '1049', '1051', '1053', '1054', '1055', '1057', '1058', '1059', '1061', '1065', '1067', '1068', '1070', '1075', '1076', '1077', '1079', '1083', '1084', '1085', '1086', '1087', '1089', '1091', '1092', '1095', '1096', '1097', '1099', '1101', '1102', '1104', '1106', '1109', '1110', '1112', '1113', '1115', '1116', '1118', '1120', '1121', '1124', '1126', '1127', '1129', '1130', '1131', '1137', '1140', '1141', '1143', '1148', '1149', '1150', '1151', '1154', '1157', '1159', '1160', '1162', '1164', '1165', '1166', '1167', '1169', '1171', '1172', '1173', '1175', '1176', '1178', '1181', '1183', '1184', '1185', '1188', '1191', '1192', '1194', '1195', '1196', '1197', '1198', '1199', '1200', '1204', '1205', '1208', '1210', '1215', '1219', '1220', '1222', '1223', '1224', '1227', '1229', '1230', '1231', '1232', '1233', '1234', '1236', '1237', '1239', '1240', '1244', '1245', '1251', '1252', '1253', '1255', '1256', '1257', '1258', '1259', '1261', '1268', '1269', '1270', '1273', '1274', '1276', '1278', '1279', '1280', '1281', '1282', '1283', '1285', '1286', '1288', '1289', '1290', '1293', '1301', '1305', '1306', '1307', '1314', '1316', '1317', '1321', '1322', '1323', '1325', '1326', '1327', '1328', '1330', '1332', '1333', '1335', '1336', '1338', '1340', '1346', '1349', '1350', '1355', '1356', '1357', '1359', '1361', '1364', '1365', '1367', '1368', '1369', '1370', '1374', '1377', '1379', '1380', '1383', '1384', '1385', '1386', '1388', '1389', '1393', '1394', '1396', '1398', '1400', '1401', '1403', '1404', '1405', '1406', '1412', '1413', '1414', '1415', '1416', '1418', '1419', '1421', '1422', '1423', '1425', '1426', '1430', '1431', '1434', '1437', '1439', '1440', '1441', '1447', '1448', '1449', '1450', '1451', '1452', '1454', '1455', '1457', '1458', '1461', '1465', '1468', '1470', '1471']

Hangzhou_5=['0', '16', '52', '59', '71', '75', '99', '100', '129', '132', '137', '144', '145', '173', '176', '188', '252', '285', '304', '319', '338', '352', '394', '426', '433', '475', '477', '481', '492', '497', '518', '534', '539', '550', '553', '595', '608', '641', '651', '656', '679', '695', '702', '718', '724', '757', '766', '773', '778', '781', '829', '844', '851', '863', '901', '935', '953', '989', '1003', '1008', '1016', '1027', '1051', '1060', '1066', '1080', '1088', '1138', '1174', '1211', '1263', '1280', '1309', '1310', '1311', '1390', '1391', '1411', '1447', '1450', '1472', '1487', '1494', '1560', '1566', '1570', '1578', '1633']
Hangzhou_10=['1', '11', '36', '44', '47', '70', '71', '86', '95', '97', '102', '108', '112', '136', '145', '154', '177', '183', '199', '200', '211', '219', '239', '244', '264', '269', '284', '288', '308', '335', '342', '343', '371', '400', '401', '405', '418', '468', '482', '488', '491', '495', '503', '512', '520', '538', '550', '557', '562', '566', '579', '584', '592', '599', '601', '602', '610', '619', '624', '625', '634', '643', '644', '661', '663', '664', '681', '708', '710', '721', '722', '729', '735', '743', '745', '748', '757', '763', '772', '784', '801', '805', '807', '818', '821', '854', '855', '856', '858', '864', '868', '919', '931', '939', '946', '958', '963', '966', '984', '992', '1009', '1041', '1049', '1050', '1052', '1054', '1070', '1076', '1087', '1105', '1110', '1127', '1137', '1141', '1160', '1167', '1168', '1175', '1178', '1194', '1205', '1220', '1228', '1230', '1243', '1245', '1250', '1261', '1265', '1281', '1293', '1307', '1318', '1337', '1350', '1353', '1362', '1393', '1396', '1405', '1406', '1407', '1413', '1420', '1421', '1445', '1468', '1473', '1499', '1504', '1517', '1520', '1522', '1531', '1538', '1540', '1566', '1570', '1581', '1583', '1612', '1623', '1626', '1630', '1634', '1643']
Hangzhou_20=['8', '14', '21', '22', '25', '28', '33', '34', '45', '47', '48', '49', '51', '56', '76', '78', '80', '85', '88', '97', '99', '105', '112', '131', '143', '147', '148', '154', '159', '160', '163', '166', '168', '174', '175', '182', '192', '195', '196', '197', '201', '207', '212', '216', '217', '219', '221', '226', '233', '235', '236', '259', '260', '261', '264', '266', '267', '282', '289', '299', '301', '310', '314', '317', '321', '325', '328', '329', '331', '336', '340', '342', '351', '352', '354', '357', '364', '365', '372', '377', '382', '388', '393', '394', '397', '406', '409', '410', '420', '422', '424', '430', '431', '434', '436', '439', '440', '441', '449', '450', '452', '456', '461', '463', '467', '468', '469', '470', '481', '487', '488', '494', '499', '500', '508', '517', '519', '521', '523', '525', '528', '529', '538', '542', '544', '549', '559', '561', '564', '570', '575', '583', '585', '594', '596', '618', '623', '628', '644', '646', '650', '658', '678', '694', '697', '698', '700', '701', '704', '711', '714', '717', '723', '724', '731', '732', '734', '736', '741', '748', '754', '758', '759', '761', '775', '778', '780', '783', '793', '794', '803', '806', '809', '815', '818', '820', '821', '828', '836', '839', '842', '843', '848', '850', '854', '856', '858', '860', '864', '888', '889', '891', '894', '902', '906', '908', '913', '916', '920', '924', '930', '941', '944', '953', '956', '961', '969', '983', '986', '989', '990', '991', '997', '998', '1007', '1017', '1025', '1029', '1032', '1034', '1035', '1036', '1040', '1042', '1051', '1053', '1054', '1058', '1060', '1061', '1062', '1065', '1076', '1078', '1082', '1085', '1091', '1117', '1120', '1135', '1140', '1145', '1147', '1149', '1150', '1154', '1158', '1159', '1163', '1166', '1170', '1173', '1202', '1203', '1209', '1215', '1218', '1221', '1227', '1234', '1235', '1250', '1259', '1260', '1261', '1262', '1265', '1267', '1291', '1296', '1302', '1303', '1307', '1318', '1322', '1325', '1333', '1342', '1343', '1346', '1366', '1374', '1375', '1378', '1380', '1386', '1413', '1416', '1422', '1423', '1425', '1431', '1445', '1448', '1451', '1486', '1487', '1488', '1491', '1492', '1496', '1499', '1503', '1512', '1519', '1523', '1531', '1533', '1539', '1542', '1544', '1548', '1549', '1554', '1568', '1578', '1583', '1584', '1587', '1599', '1601', '1603', '1606', '1609', '1615', '1618', '1626', '1629', '1638', '1641', '1649', '1650']
Hangzhou_30=['2', '4', '5', '7', '8', '12', '18', '20', '21', '22', '25', '28', '31', '32', '34', '39', '40', '47', '50', '65', '66', '68', '70', '72', '79', '84', '91', '93', '96', '97', '98', '106', '107', '110', '111', '114', '116', '117', '129', '132', '135', '140', '142', '147', '148', '151', '153', '161', '162', '164', '168', '172', '174', '175', '179', '180', '181', '190', '199', '203', '205', '206', '210', '211', '222', '223', '224', '226', '230', '231', '232', '240', '243', '244', '248', '249', '251', '252', '255', '259', '261', '262', '263', '265', '267', '268', '269', '270', '273', '274', '275', '279', '282', '297', '301', '308', '309', '310', '313', '317', '321', '324', '327', '328', '333', '336', '337', '339', '342', '345', '353', '354', '358', '367', '368', '370', '371', '374', '378', '379', '380', '381', '383', '385', '386', '390', '392', '394', '402', '404', '405', '407', '418', '420', '424', '426', '427', '430', '436', '437', '439', '440', '443', '450', '452', '458', '460', '461', '464', '465', '474', '475', '487', '489', '491', '494', '498', '499', '502', '505', '506', '512', '517', '519', '520', '521', '532', '535', '536', '538', '540', '541', '547', '548', '552', '553', '556', '558', '559', '560', '561', '578', '581', '590', '591', '593', '595', '596', '598', '603', '606', '612', '613', '615', '621', '623', '624', '625', '627', '631', '637', '638', '641', '642', '643', '647', '650', '651', '663', '664', '665', '669', '673', '674', '675', '689', '694', '700', '704', '705', '706', '709', '711', '713', '714', '718', '722', '724', '727', '728', '730', '735', '737', '739', '740', '745', '748', '749', '752', '761', '764', '765', '768', '769', '774', '780', '781', '783', '788', '789', '793', '796', '799', '803', '809', '810', '811', '812', '818', '822', '824', '826', '829', '830', '834', '839', '840', '843', '846', '848', '849', '854', '856', '862', '863', '865', '873', '874', '875', '881', '884', '886', '887', '889', '891', '894', '895', '908', '909', '911', '912', '914', '918', '919', '923', '928', '934', '935', '936', '939', '940', '942', '943', '952', '956', '957', '962', '963', '975', '978', '988', '1000', '1012', '1013', '1023', '1026', '1028', '1029', '1031', '1036', '1042', '1043', '1046', '1053', '1055', '1056', '1057', '1059', '1062', '1063', '1065', '1068', '1069', '1073', '1076', '1077', '1084', '1086', '1088', '1090', '1096', '1098', '1099', '1100', '1107', '1124', '1126', '1129', '1135', '1140', '1143', '1144', '1146', '1147', '1152', '1156', '1158', '1162', '1163', '1167', '1170', '1174', '1176', '1179', '1184', '1186', '1187', '1191', '1192', '1194', '1195', '1201', '1208', '1218', '1222', '1231', '1232', '1234', '1236', '1237', '1239', '1248', '1249', '1264', '1265', '1271', '1273', '1275', '1280', '1281', '1283', '1284', '1286', '1291', '1292', '1293', '1297', '1303', '1308', '1310', '1314', '1315', '1317', '1319', '1322', '1325', '1327', '1331', '1333', '1338', '1339', '1343', '1346', '1348', '1349', '1350', '1353', '1361', '1362', '1364', '1367', '1373', '1376', '1395', '1396', '1397', '1401', '1403', '1411', '1416', '1423', '1424', '1431', '1435', '1438', '1439', '1441', '1443', '1446', '1449', '1462', '1463', '1465', '1473', '1475', '1476', '1479', '1483', '1486', '1488', '1491', '1496', '1497', '1503', '1504', '1505', '1507', '1508', '1509', '1511', '1513', '1521', '1526', '1527', '1529', '1539', '1542', '1555', '1557', '1558', '1570', '1577', '1578', '1585', '1588', '1604', '1605', '1606', '1608', '1609', '1611', '1612', '1613', '1614', '1617', '1619', '1620', '1624', '1630', '1631', '1633', '1639', '1643', '1644', '1645', '1647', '1652', '1653']
Hangzhou_40=['0', '1', '3', '5', '8', '10', '12', '16', '17', '19', '20', '24', '26', '30', '31', '32', '40', '41', '47', '48', '49', '50', '51', '57', '60', '62', '64', '67', '68', '69', '70', '72', '73', '74', '77', '83', '86', '87', '89', '91', '96', '98', '99', '101', '104', '108', '110', '115', '117', '122', '127', '130', '131', '140', '142', '144', '147', '148', '149', '152', '153', '154', '156', '159', '160', '162', '163', '167', '171', '175', '177', '179', '182', '183', '186', '187', '188', '189', '197', '198', '201', '202', '203', '204', '205', '211', '212', '218', '219', '221', '224', '226', '229', '231', '239', '243', '245', '246', '247', '248', '249', '250', '256', '257', '259', '261', '264', '267', '273', '275', '282', '284', '285', '287', '289', '290', '292', '294', '295', '296', '297', '303', '306', '308', '311', '312', '315', '316', '317', '318', '321', '322', '323', '324', '325', '329', '331', '333', '335', '337', '338', '340', '346', '347', '348', '349', '352', '353', '359', '363', '364', '371', '374', '379', '389', '390', '391', '392', '393', '396', '399', '400', '403', '409', '410', '411', '412', '413', '414', '416', '419', '421', '422', '423', '426', '427', '430', '435', '438', '439', '445', '451', '454', '455', '457', '459', '465', '466', '471', '472', '475', '477', '480', '484', '488', '489', '491', '494', '495', '496', '497', '498', '499', '501', '502', '504', '506', '512', '513', '515', '516', '520', '521', '523', '527', '529', '532', '533', '535', '537', '538', '544', '546', '547', '548', '550', '551', '552', '553', '557', '558', '562', '564', '565', '568', '570', '574', '576', '577', '580', '582', '583', '590', '593', '595', '599', '600', '601', '604', '605', '607', '608', '610', '612', '616', '618', '620', '622', '624', '626', '627', '628', '633', '640', '645', '646', '647', '648', '650', '652', '654', '655', '660', '663', '666', '667', '673', '674', '675', '676', '678', '680', '682', '683', '686', '691', '695', '697', '699', '700', '702', '704', '706', '707', '709', '710', '713', '718', '719', '720', '727', '728', '734', '735', '737', '744', '745', '747', '748', '751', '753', '757', '760', '763', '768', '771', '772', '773', '774', '775', '780', '781', '785', '786', '787', '789', '790', '795', '797', '798', '803', '805', '812', '813', '814', '819', '821', '822', '824', '826', '827', '833', '838', '840', '841', '842', '843', '845', '853', '854', '855', '857', '858', '859', '862', '863', '866', '867', '870', '871', '873', '876', '880', '885', '891', '893', '911', '913', '916', '917', '918', '920', '921', '923', '926', '927', '932', '933', '934', '936', '937', '938', '939', '943', '944', '947', '948', '950', '951', '953', '957', '960', '962', '963', '964', '965', '966', '968', '973', '974', '977', '980', '982', '985', '988', '990', '993', '996', '997', '999', '1000', '1005', '1006', '1011', '1013', '1015', '1023', '1025', '1027', '1028', '1029', '1030', '1031', '1032', '1033', '1037', '1040', '1041', '1042', '1045', '1046', '1049', '1050', '1051', '1052', '1056', '1068', '1070', '1075', '1077', '1078', '1079', '1083', '1091', '1097', '1100', '1101', '1102', '1103', '1108', '1109', '1110', '1115', '1117', '1123', '1124', '1127', '1128', '1131', '1134', '1135', '1138', '1142', '1149', '1151', '1153', '1154', '1157', '1158', '1162', '1164', '1166', '1167', '1168', '1169', '1171', '1172', '1173', '1174', '1176', '1177', '1178', '1182', '1183', '1186', '1187', '1191', '1193', '1196', '1198', '1199', '1202', '1211', '1214', '1215', '1216', '1220', '1225', '1226', '1228', '1229', '1231', '1232', '1239', '1243', '1244', '1246', '1249', '1252', '1253', '1255', '1257', '1258', '1259', '1261', '1271', '1272', '1274', '1277', '1278', '1281', '1282', '1283', '1284', '1289', '1290', '1304', '1306', '1307', '1312', '1313', '1314', '1316', '1317', '1322', '1324', '1325', '1326', '1327', '1331', '1336', '1337', '1339', '1340', '1341', '1343', '1345', '1347', '1350', '1353', '1361', '1363', '1365', '1367', '1369', '1370', '1371', '1378', '1380', '1384', '1389', '1390', '1391', '1393', '1396', '1397', '1398', '1400', '1401', '1405', '1411', '1412', '1413', '1417', '1418', '1420', '1421', '1424', '1425', '1427', '1428', '1429', '1430', '1431', '1432', '1435', '1438', '1439', '1447', '1448', '1451', '1457', '1458', '1462', '1467', '1470', '1472', '1473', '1474', '1477', '1478', '1479', '1480', '1482', '1483', '1484', '1485', '1486', '1489', '1495', '1496', '1497', '1498', '1500', '1501', '1502', '1503', '1504', '1508', '1509', '1510', '1520', '1524', '1526', '1532', '1541', '1543', '1544', '1545', '1548', '1550', '1551', '1552', '1554', '1557', '1563', '1571', '1573', '1574', '1575', '1576', '1577', '1578', '1583', '1586', '1588', '1590', '1598', '1601', '1603', '1605', '1613', '1622', '1628', '1631', '1632', '1634', '1635', '1636', '1638', '1642', '1650', '1652', '1657']
Hangzhou_50=['0', '3', '4', '5', '6', '7', '8', '10', '12', '13', '14', '15', '16', '18', '19', '20', '21', '25', '26', '28', '31', '32', '33', '34', '38', '40', '42', '43', '45', '47', '53', '54', '56', '60', '61', '62', '63', '66', '68', '70', '72', '74', '75', '76', '77', '79', '80', '81', '82', '84', '85', '86', '90', '95', '96', '97', '99', '100', '106', '109', '110', '112', '113', '118', '120', '121', '123', '124', '127', '129', '131', '132', '133', '136', '140', '141', '142', '143', '145', '146', '148', '150', '157', '158', '162', '167', '169', '170', '172', '173', '174', '176', '177', '180', '181', '182', '187', '188', '193', '197', '198', '200', '202', '204', '206', '207', '208', '211', '212', '213', '214', '215', '216', '218', '220', '221', '223', '224', '226', '228', '230', '231', '234', '237', '238', '240', '243', '244', '245', '246', '248', '250', '257', '259', '260', '262', '263', '264', '265', '268', '271', '272', '273', '276', '279', '280', '282', '284', '285', '287', '288', '292', '293', '295', '296', '297', '298', '301', '307', '308', '309', '311', '313', '315', '319', '321', '322', '323', '324', '328', '330', '331', '335', '336', '337', '340', '341', '342', '347', '350', '351', '352', '354', '357', '359', '361', '362', '363', '365', '371', '372', '373', '374', '382', '383', '385', '388', '390', '391', '392', '396', '397', '401', '402', '403', '404', '408', '411', '412', '413', '417', '418', '420', '421', '431', '433', '434', '436', '437', '438', '442', '443', '445', '446', '448', '449', '450', '451', '454', '456', '458', '459', '461', '467', '468', '469', '470', '471', '472', '474', '475', '476', '477', '478', '481', '485', '487', '488', '491', '493', '494', '495', '496', '497', '498', '499', '500', '501', '506', '507', '508', '511', '514', '515', '516', '518', '520', '522', '523', '526', '527', '528', '529', '531', '534', '535', '536', '537', '538', '540', '541', '544', '545', '546', '550', '551', '552', '555', '557', '559', '562', '564', '565', '566', '567', '568', '570', '575', '576', '580', '581', '582', '585', '586', '587', '588', '591', '593', '596', '600', '601', '602', '603', '604', '609', '611', '613', '616', '617', '619', '620', '622', '624', '628', '629', '631', '634', '635', '636', '637', '638', '639', '640', '643', '644', '645', '647', '648', '651', '652', '656', '658', '659', '660', '661', '662', '663', '665', '666', '667', '668', '669', '670', '671', '672', '673', '674', '678', '682', '684', '687', '690', '692', '693', '696', '697', '699', '700', '702', '706', '709', '710', '711', '712', '713', '714', '716', '717', '722', '723', '724', '728', '729', '730', '731', '735', '736', '737', '743', '748', '749', '750', '753', '754', '755', '756', '758', '759', '760', '761', '762', '763', '764', '765', '767', '768', '769', '771', '773', '774', '775', '778', '781', '783', '784', '789', '792', '793', '794', '796', '797', '799', '800', '803', '805', '807', '809', '811', '813', '816', '818', '821', '824', '825', '826', '831', '834', '837', '838', '840', '841', '843', '848', '850', '853', '854', '856', '857', '858', '862', '863', '865', '866', '867', '868', '869', '871', '872', '873', '875', '877', '880', '881', '882', '883', '887', '888', '889', '892', '896', '899', '901', '902', '907', '908', '909', '910', '912', '914', '916', '920', '921', '923', '925', '928', '929', '930', '932', '934', '937', '939', '941', '944', '945', '948', '950', '951', '953', '956', '957', '958', '960', '962', '964', '965', '966', '967', '968', '969', '972', '975', '979', '982', '983', '984', '986', '989', '991', '998', '999', '1009', '1010', '1011', '1012', '1014', '1015', '1016', '1018', '1019', '1020', '1022', '1025', '1031', '1032', '1035', '1038', '1039', '1040', '1043', '1051', '1052', '1057', '1061', '1065', '1067', '1070', '1071', '1072', '1074', '1077', '1081', '1082', '1083', '1084', '1088', '1092', '1094', '1100', '1101', '1102', '1104', '1107', '1109', '1112', '1113', '1115', '1116', '1122', '1123', '1125', '1126', '1128', '1130', '1131', '1133', '1134', '1136', '1139', '1140', '1141', '1143', '1145', '1152', '1154', '1155', '1157', '1158', '1159', '1160', '1161', '1164', '1165', '1168', '1170', '1174', '1175', '1176', '1177', '1181', '1183', '1184', '1185', '1186', '1187', '1191', '1194', '1195', '1196', '1198', '1199', '1201', '1202', '1203', '1204', '1207', '1208', '1209', '1211', '1212', '1216', '1219', '1220', '1221', '1222', '1224', '1226', '1227', '1235', '1236', '1238', '1239', '1240', '1244', '1246', '1250', '1251', '1252', '1255', '1257', '1259', '1260', '1262', '1265', '1266', '1267', '1270', '1272', '1273', '1274', '1276', '1277', '1278', '1279', '1280', '1282', '1285', '1286', '1288', '1289', '1292', '1293', '1294', '1295', '1298', '1299', '1301', '1303', '1307', '1308', '1313', '1314', '1317', '1319', '1321', '1324', '1329', '1330', '1332', '1334', '1337', '1340', '1343', '1344', '1346', '1348', '1350', '1351', '1353', '1359', '1365', '1367', '1368', '1370', '1371', '1372', '1373', '1374', '1375', '1378', '1380', '1384', '1386', '1389', '1390', '1391', '1392', '1394', '1398', '1400', '1405', '1406', '1407', '1410', '1411', '1412', '1416', '1417', '1419', '1424', '1425', '1431', '1432', '1433', '1437', '1440', '1441', '1448', '1449', '1450', '1454', '1457', '1458', '1460', '1461', '1465', '1466', '1468', '1469', '1472', '1473', '1476', '1477', '1479', '1480', '1481', '1482', '1485', '1487', '1492', '1493', '1501', '1503', '1505', '1508', '1509', '1510', '1519', '1520', '1522', '1523', '1524', '1526', '1527', '1528', '1529', '1530', '1539', '1541', '1543', '1545', '1548', '1549', '1550', '1551', '1555', '1556', '1557', '1559', '1561', '1562', '1570', '1571', '1573', '1575', '1578', '1579', '1580', '1581', '1583', '1584', '1592', '1593', '1594', '1595', '1596', '1598', '1599', '1603', '1605', '1607', '1608', '1610', '1611', '1614', '1616', '1618', '1620', '1621', '1624', '1625', '1627', '1628', '1634', '1635', '1637', '1639', '1640', '1642', '1643', '1646', '1647', '1650', '1651', '1652', '1653', '1654', '1656', '1657', '1658', '1659']

In [ ]:
start_edges = ['1'',''2'',''3'',''4']
end_edges = ['5'',''6'',''7'',''8']
for i in range(4):
    vehicle_id = f"veh_{i}" 
    start_edge = start_edges[i]
    print(type(start_edge))
    end_edge = end_edges[i]
    print(f"{vehicle_id} {start_edge} {end_edge}")


In [ ]:
import xml.etree.ElementTree as ET
#输出stert_edges
def extract_start_edges_from_xml(file_path):
    # 解析指定路径的XML文件
    tree = ET.parse(file_path)
    root = tree.getroot()

    # 初始化一个列表来存储起点edges
    start_edges = []

    # 遍历每一个vehicle
    for vehicle in root.findall('vehicle'):
        # 获取route标签内的edges属性
        edges = vehicle.find('route').get('edges')
        # 将edges字符串分割成列表,并取第一个元素作为起点
        start_edge = edges.split()[0]
        start_edges.append(start_edge)

    return start_edges

# 指定rou.xml的路径
rou_xml_path = r"D:\intereting\DynamicNavigation\findPath\findPath\grid_50_vechle.xml"  # 请替换为你的rou.xml文件的实际路径

# 调用函数并打印结果
start_edges = extract_start_edges_from_xml(rou_xml_path)
print("start_edges ="','start_edges)

In [ ]:
#输出终端
import xml.etree.ElementTree as ET

def extract_end_edges_from_xml(file_path):
    # 解析指定路径的XML文件
    tree = ET.parse(file_path)
    root = tree.getroot()

    # 初始化一个列表来存储终点edges
    end_edges = []

    # 遍历每一个vehicle
    for vehicle in root.findall('vehicle'):
        # 获取route标签内的edges属性
        edges = vehicle.find('route').get('edges')
        # 将edges字符串分割成列表,并取最后一个元素作为终点
        # 注意：split()会移除末尾的空格,因此不需要担心因空格导致的错误
        end_edge = edges.split()[-1]
        end_edges.append(end_edge)

    return end_edges

# 指定rou.xml的路径
rou_xml_path = r"D:\intereting\DynamicNavigation\findPath\findPath\Hangzhou_50_vechle.xml"  # 请替换为你的rou.xml文件的实际路径

# 调用函数并打印结果
end_edges = extract_end_edges_from_xml(rou_xml_path)
print("end_edges ="','end_edges)

In [ ]:
net_file = r'D:\intereting\DynamicNavigation\findPath\findPath\test_data\孙倩\grid4x4\grid4x4.net.xml'# 路网文件
rou_file = r'D:\intereting\DynamicNavigation\findPath\findPath\test_data\孙倩\grid4x4\grid4x4.rou.xml'# 路网文件
cfg_file = r'D:\intereting\DynamicNavigation\findPath\findPath\test_data\孙倩\grid4x4\grid4x4.sumocfg'
###grid_5
car_num = 73
start_edges = ['left3A3'',''bottom0A0'',''right0D0'',''top1B3'',''right1D1'',''right2D2'',''top3D3'',''bottom3D0'',''top3D3'',''right1D1'',''top0A3'',''right2D2'',''right2D2'',''bottom2C0'',''right3D3'',''top0A3'',''right0D0'',''top0A3'',''bottom0A0'',''right2D2'',''left1A1'',''right0D0'',''top2C3'',''left3A3'',''right1D1'',''right2D2'',''left0A0'',''right3D3'',''bottom0A0'',''left1A1'',''right3D3'',''top1B3'',''bottom1B0'',''right3D3'',''left2A2'',''bottom3D0'',''bottom2C0'',''right2D2'',''right1D1'',''top1B3'',''bottom1B0'',''bottom3D0'',''left3A3'',''right3D3'',''left3A3'',''right1D1'',''top3D3'',''left3A3'',''bottom1B0'',''right2D2'',''bottom0A0'',''top2C3'',''left1A1'',''right3D3'',''bottom1B0'',''left3A3'',''right0D0'',''bottom3D0'',''left0A0'',''bottom0A0'',''bottom2C0'',''top0A3'',''top3D3'',''left0A0'',''top0A3'',''left2A2'',''right2D2'',''top1B3'',''bottom0A0'',''bottom2C0'',''left0A0'',''right3D3'',''top1B3']
end_edges = ['A2left2'',''A1left1'',''D3top3'',''A1left1'',''A0left0'',''D1right1'',''D3top3'',''D3top3'',''D3top3'',''A0left0'',''D0right0'',''D0right0'',''D1right1'',''D3top3'',''D2right2'',''D0bottom3'',''A2left2'',''B0bottom1'',''D2right2'',''D0bottom3'',''C0bottom2'',''A3left3'',''B0bottom1'',''C0bottom2'',''B0bottom1'',''A2left2'',''A3top0'',''D1right1'',''D3right3'',''D3right3'',''A0bottom0'',''D0bottom3'',''D1right1'',''D0right0'',''B0bottom1'',''D1right1'',''D3top3'',''C3top2'',''D1right1'',''D1right1'',''A3left3'',''D1right1'',''D3top3'',''C0bottom2'',''A2left2'',''D0bottom3'',''D3top3'',''A0bottom0'',''B3top1'',''B3top1'',''D0right0'',''A3top0'',''C3top2'',''B0bottom1'',''A3top0'',''D2right2'',''B3top1'',''C3top2'',''A0bottom0'',''B0bottom1'',''D0right0'',''A0bottom0'',''D1right1'',''A2left2'',''D1right1'',''D3right3'',''D1right1'',''B0bottom1'',''D1right1'',''C0bottom2'',''A0left0'',''D2right2'',''C0bottom2']
print(len(start_edges))
print(len(end_edges))
# ###grid_10
# car_num = 147
# start_edges = ['right1D1'',''left0A0'',''left1A1'',''left3A3'',''left0A0'',''top3D3'',''left0A0'',''left2A2'',''bottom0A0'',''bottom2C0'',''top3D3'',''right0D0'',''right1D1'',''bottom0A0'',''right1D1'',''left0A0'',''right3D3'',''bottom1B0'',''bottom3D0'',''right0D0'',''top0A3'',''top3D3'',''left0A0'',''top1B3'',''right0D0'',''left1A1'',''top0A3'',''top0A3'',''top2C3'',''top3D3'',''left1A1'',''left0A0'',''left3A3'',''right0D0'',''right2D2'',''left3A3'',''top2C3'',''right3D3'',''bottom1B0'',''left2A2'',''bottom3D0'',''bottom1B0'',''left3A3'',''bottom2C0'',''right2D2'',''right2D2'',''right1D1'',''top3D3'',''bottom2C0'',''bottom2C0'',''top1B3'',''top3D3'',''top0A3'',''top2C3'',''right2D2'',''left2A2'',''bottom3D0'',''left0A0'',''top3D3'',''bottom1B0'',''bottom1B0'',''left3A3'',''left1A1'',''bottom2C0'',''left2A2'',''left0A0'',''top1B3'',''right3D3'',''right1D1'',''right3D3'',''top3D3'',''top1B3'',''top1B3'',''left1A1'',''left1A1'',''left1A1'',''left0A0'',''bottom2C0'',''right0D0'',''bottom1B0'',''top3D3'',''top3D3'',''top2C3'',''bottom0A0'',''left0A0'',''left1A1'',''left2A2'',''left1A1'',''bottom3D0'',''top3D3'',''bottom3D0'',''right2D2'',''right0D0'',''left1A1'',''top2C3'',''top0A3'',''bottom0A0'',''top3D3'',''left3A3'',''bottom1B0'',''top0A3'',''top0A3'',''right0D0'',''left0A0'',''top2C3'',''top2C3'',''right1D1'',''left1A1'',''left1A1'',''top3D3'',''top0A3'',''right3D3'',''left3A3'',''top1B3'',''top3D3'',''left2A2'',''right1D1'',''top3D3'',''bottom1B0'',''top1B3'',''bottom1B0'',''bottom2C0'',''right0D0'',''left2A2'',''left1A1'',''left3A3'',''left3A3'',''bottom2C0'',''right2D2'',''left0A0'',''right1D1'',''left3A3'',''left2A2'',''bottom1B0'',''left1A1'',''right3D3'',''left3A3'',''bottom1B0'',''right3D3'',''bottom1B0'',''left0A0'',''top2C3'',''right3D3'',''left0A0'',''top1B3'',''bottom0A0'',''left2A2']
# end_edges = ['B3top1'',''B3top1'',''A2left2'',''D1right1'',''A0bottom0'',''D2right2'',''D3top3'',''A2left2'',''B0bottom1'',''D2right2'',''A3left3'',''B0bottom1'',''A0left0'',''B0bottom1'',''C3top2'',''A1left1'',''D3top3'',''C3top2'',''D2right2'',''D1right1'',''A3top0'',''A0left0'',''B3top1'',''D3right3'',''A3top0'',''B0bottom1'',''B0bottom1'',''D3right3'',''D3right3'',''A3left3'',''C0bottom2'',''A0left0'',''A3top0'',''D0bottom3'',''D0bottom3'',''A3left3'',''A0left0'',''D0bottom3'',''D3top3'',''D1right1'',''A1left1'',''B0bottom1'',''D2right2'',''D0bottom3'',''D0bottom3'',''D3right3'',''A3top0'',''A3top0'',''D0bottom3'',''C3top2'',''B3top1'',''B3top1'',''B0bottom1'',''D3top3'',''D3top3'',''B0bottom1'',''D2right2'',''C3top2'',''D0right0'',''A0left0'',''D0bottom3'',''A3left3'',''D0bottom3'',''A0left0'',''A3top0'',''A1left1'',''B0bottom1'',''A0left0'',''D1right1'',''C3top2'',''D0bottom3'',''B3top1'',''C0bottom2'',''D1right1'',''A1left1'',''C0bottom2'',''D3right3'',''C3top2'',''A3left3'',''A2left2'',''D2right2'',''D0bottom3'',''A0bottom0'',''A3left3'',''B3top1'',''D3top3'',''D3right3'',''A0bottom0'',''D2right2'',''A0left0'',''D2right2'',''A0bottom0'',''B0bottom1'',''A2left2'',''D1right1'',''D1right1'',''A1left1'',''D3top3'',''D1right1'',''D3right3'',''D2right2'',''A1left1'',''A1left1'',''D2right2'',''D1right1'',''A3top0'',''D1right1'',''A2left2'',''C3top2'',''D0right0'',''B3top1'',''A3left3'',''D1right1'',''D3right3'',''B3top1'',''C3top2'',''D3top3'',''C0bottom2'',''A3left3'',''B3top1'',''A0left0'',''D0right0'',''D3top3'',''C0bottom2'',''A0bottom0'',''D3right3'',''B3top1'',''A3top0'',''A1left1'',''A3left3'',''C0bottom2'',''A3top0'',''D3top3'',''D3top3'',''D3right3'',''A0left0'',''C0bottom2'',''D0right0'',''B0bottom1'',''A2left2'',''D2right2'',''B3top1'',''B0bottom1'',''D3right3'',''A3top0'',''B3top1'',''A0left0']
# ##grid_20
# car_num = 295
# start_edges = ['top2C3'',''bottom0A0'',''right3D3'',''left2A2'',''left3A3'',''bottom1B0'',''left1A1'',''right0D0'',''top2C3'',''bottom0A0'',''left1A1'',''bottom3D0'',''right1D1'',''right1D1'',''bottom1B0'',''right2D2'',''bottom1B0'',''right0D0'',''bottom2C0'',''right0D0'',''left0A0'',''left2A2'',''bottom3D0'',''left0A0'',''right3D3'',''left0A0'',''bottom2C0'',''right2D2'',''top3D3'',''right0D0'',''right1D1'',''top1B3'',''top0A3'',''right3D3'',''right0D0'',''bottom0A0'',''right2D2'',''bottom1B0'',''bottom0A0'',''top3D3'',''left3A3'',''left2A2'',''bottom0A0'',''left0A0'',''bottom0A0'',''right2D2'',''left1A1'',''right3D3'',''right2D2'',''top2C3'',''left2A2'',''left2A2'',''bottom1B0'',''left1A1'',''top2C3'',''top1B3'',''right3D3'',''right2D2'',''left1A1'',''bottom0A0'',''top2C3'',''left1A1'',''bottom3D0'',''left2A2'',''right0D0'',''left3A3'',''bottom2C0'',''left3A3'',''right1D1'',''top0A3'',''top0A3'',''right3D3'',''bottom1B0'',''top0A3'',''left0A0'',''bottom3D0'',''bottom2C0'',''left3A3'',''top1B3'',''top0A3'',''top0A3'',''left2A2'',''bottom2C0'',''bottom1B0'',''bottom1B0'',''right3D3'',''left3A3'',''right3D3'',''bottom1B0'',''top0A3'',''bottom3D0'',''top3D3'',''bottom1B0'',''right2D2'',''right3D3'',''bottom0A0'',''bottom1B0'',''left2A2'',''right1D1'',''right1D1'',''top0A3'',''top0A3'',''bottom3D0'',''right3D3'',''bottom2C0'',''top3D3'',''right1D1'',''left2A2'',''left3A3'',''right3D3'',''left1A1'',''bottom2C0'',''top2C3'',''left1A1'',''bottom3D0'',''left0A0'',''top3D3'',''bottom1B0'',''bottom1B0'',''right3D3'',''left2A2'',''left3A3'',''bottom0A0'',''right3D3'',''top3D3'',''bottom2C0'',''bottom2C0'',''left2A2'',''left0A0'',''left3A3'',''right3D3'',''right1D1'',''bottom0A0'',''right1D1'',''left3A3'',''right0D0'',''right1D1'',''top0A3'',''bottom3D0'',''top3D3'',''left0A0'',''right1D1'',''bottom1B0'',''right2D2'',''top3D3'',''left1A1'',''left0A0'',''right3D3'',''right2D2'',''top1B3'',''top2C3'',''bottom3D0'',''left2A2'',''left2A2'',''top2C3'',''right1D1'',''top3D3'',''right3D3'',''right1D1'',''left2A2'',''left1A1'',''left0A0'',''bottom2C0'',''left0A0'',''left2A2'',''left2A2'',''bottom3D0'',''left1A1'',''right2D2'',''right1D1'',''top0A3'',''left2A2'',''left3A3'',''top3D3'',''right2D2'',''top0A3'',''right0D0'',''right0D0'',''top3D3'',''bottom0A0'',''right2D2'',''left3A3'',''top2C3'',''top2C3'',''top3D3'',''right3D3'',''bottom3D0'',''left1A1'',''bottom3D0'',''left3A3'',''right2D2'',''right2D2'',''right1D1'',''left0A0'',''right2D2'',''right1D1'',''left1A1'',''top2C3'',''top1B3'',''left3A3'',''top1B3'',''left2A2'',''left1A1'',''right1D1'',''top2C3'',''bottom3D0'',''right3D3'',''bottom2C0'',''bottom2C0'',''right3D3'',''left1A1'',''bottom2C0'',''top3D3'',''right0D0'',''top0A3'',''left1A1'',''top0A3'',''right3D3'',''top2C3'',''left1A1'',''bottom2C0'',''right1D1'',''right2D2'',''top3D3'',''top2C3'',''left3A3'',''top0A3'',''bottom1B0'',''top0A3'',''left2A2'',''bottom3D0'',''top2C3'',''top1B3'',''right0D0'',''left0A0'',''bottom1B0'',''left2A2'',''top0A3'',''left1A1'',''left0A0'',''right1D1'',''right3D3'',''right0D0'',''right3D3'',''left0A0'',''right0D0'',''bottom2C0'',''right0D0'',''right3D3'',''top2C3'',''right2D2'',''top0A3'',''left2A2'',''right1D1'',''bottom2C0'',''bottom3D0'',''right1D1'',''top3D3'',''left2A2'',''top2C3'',''right1D1'',''right0D0'',''left2A2'',''top3D3'',''left1A1'',''right3D3'',''bottom0A0'',''bottom1B0'',''bottom1B0'',''right3D3'',''bottom1B0'',''left2A2'',''left3A3'',''right2D2'',''right3D3'',''left2A2'',''bottom3D0'',''left0A0'',''right1D1'',''top1B3'',''top0A3'',''top0A3'',''bottom1B0'',''top3D3'',''right1D1'',''right3D3'',''left0A0'',''top1B3'',''right0D0'',''right1D1'',''top1B3'',''top2C3'',''bottom2C0'',''left2A2'',''left0A0']
# end_edges = ['D3top3'',''A0left0'',''D0bottom3'',''B3top1'',''B0bottom1'',''D3right3'',''D1right1'',''D2right2'',''A3left3'',''D2right2'',''A2left2'',''D0bottom3'',''D1right1'',''D0right0'',''C0bottom2'',''D3right3'',''B3top1'',''A0left0'',''A2left2'',''D0right0'',''D0right0'',''A2left2'',''C3top2'',''D0right0'',''C0bottom2'',''A2left2'',''A1left1'',''D3top3'',''D3top3'',''D1right1'',''A0left0'',''B0bottom1'',''D0right0'',''B3top1'',''D0bottom3'',''A0left0'',''C0bottom2'',''A0left0'',''D0bottom3'',''A0left0'',''D0right0'',''B3top1'',''B0bottom1'',''A1left1'',''D1right1'',''D1right1'',''D3top3'',''D3top3'',''D3right3'',''D3right3'',''B0bottom1'',''D0right0'',''A0bottom0'',''D0right0'',''C0bottom2'',''D3right3'',''D2right2'',''D3right3'',''B3top1'',''D0right0'',''C3top2'',''A3top0'',''C0bottom2'',''D0right0'',''A2left2'',''D3right3'',''A0left0'',''D3right3'',''A0bottom0'',''B0bottom1'',''D0right0'',''D3right3'',''D3right3'',''B0bottom1'',''A3top0'',''A2left2'',''D0bottom3'',''A3top0'',''A3left3'',''A0left0'',''A0left0'',''D1right1'',''A2left2'',''A1left1'',''A3top0'',''B3top1'',''A1left1'',''D0bottom3'',''D0bottom3'',''A2left2'',''A1left1'',''D0right0'',''B0bottom1'',''D3right3'',''D1right1'',''D0bottom3'',''A2left2'',''D0bottom3'',''A3left3'',''C0bottom2'',''C0bottom2'',''C0bottom2'',''A3left3'',''D0right0'',''A0left0'',''A1left1'',''B0bottom1'',''C3top2'',''C3top2'',''A3top0'',''D3top3'',''C3top2'',''D3top3'',''D3top3'',''D2right2'',''C3top2'',''D0right0'',''A0left0'',''D0bottom3'',''D0bottom3'',''D0bottom3'',''A3left3'',''A0left0'',''A0bottom0'',''D0right0'',''A0left0'',''C3top2'',''A3top0'',''A1left1'',''A1left1'',''D0bottom3'',''D0bottom3'',''A2left2'',''D1right1'',''A1left1'',''A2left2'',''D3top3'',''D2right2'',''C0bottom2'',''D0right0'',''B0bottom1'',''A1left1'',''D0right0'',''D0right0'',''A3top0'',''D1right1'',''D3right3'',''A3left3'',''D1right1'',''A0bottom0'',''B0bottom1'',''C3top2'',''D1right1'',''A0bottom0'',''A0left0'',''C0bottom2'',''D0bottom3'',''A2left2'',''D1right1'',''B0bottom1'',''D1right1'',''D3top3'',''A2left2'',''A3left3'',''A1left1'',''C0bottom2'',''D1right1'',''A3top0'',''B0bottom1'',''B0bottom1'',''A1left1'',''A2left2'',''D0right0'',''A0left0'',''A0bottom0'',''A0bottom0'',''D1right1'',''A0bottom0'',''A0bottom0'',''D2right2'',''A2left2'',''D1right1'',''D0right0'',''A1left1'',''D3top3'',''D1right1'',''D3top3'',''A3top0'',''B3top1'',''A2left2'',''A0bottom0'',''A1left1'',''D3top3'',''D2right2'',''D0bottom3'',''D1right1'',''A2left2'',''A0bottom0'',''A1left1'',''B0bottom1'',''D2right2'',''B0bottom1'',''C3top2'',''B0bottom1'',''C3top2'',''B0bottom1'',''A3left3'',''D0bottom3'',''D0bottom3'',''B0bottom1'',''D3right3'',''C3top2'',''A0left0'',''A3top0'',''B0bottom1'',''A2left2'',''C3top2'',''A1left1'',''A1left1'',''C0bottom2'',''A0left0'',''D0bottom3'',''D3right3'',''C0bottom2'',''D3top3'',''B0bottom1'',''A3top0'',''D3top3'',''A0left0'',''A2left2'',''C3top2'',''A2left2'',''A0left0'',''A2left2'',''A0left0'',''D1right1'',''A0left0'',''D0bottom3'',''A0bottom0'',''A2left2'',''B0bottom1'',''C3top2'',''D0right0'',''D3top3'',''A0bottom0'',''A3top0'',''D3right3'',''A3left3'',''A0bottom0'',''D0right0'',''A1left1'',''D3top3'',''D3right3'',''C0bottom2'',''C0bottom2'',''D0bottom3'',''D0right0'',''C0bottom2'',''D3top3'',''D0bottom3'',''B0bottom1'',''D3top3'',''A3top0'',''D0bottom3'',''D0right0'',''A0left0'',''A0left0'',''A2left2'',''D0right0'',''B0bottom1'',''A2left2', 'A0left0', 'B0bottom1', 'C0bottom2', 'D1right1', 'C0bottom2', 'C3top2', 'D2right2', 'D3top3', 'A0bottom0', 'D1right1', 'B0bottom1', 'A0bottom0', 'D0right0', 'A0left0', 'C0bottom2', 'D0bottom3', 'D1right1', 'D3top3', 'A1left1', 'C0bottom2', 'A3left3', 'A0left0', 'A0left0', 'D0bottom3']
# ##grid_30
# car_num = 442
# start_edges = ['bottom0A0', 'right3D3', 'top2C3', 'top2C3', 'left2A2', 'right1D1', 'right1D1', 'top2C3', 'left3A3', 'top3D3', 'top3D3', 'bottom1B0', 'bottom3D0', 'top1B3', 'bottom0A0', 'left1A1', 'right2D2', 'right1D1', 'right0D0', 'right1D1', 'right3D3', 'top3D3', 'left3A3', 'top3D3', 'top0A3', 'right3D3', 'right0D0', 'bottom3D0', 'bottom3D0', 'top1B3', 'bottom3D0', 'left0A0', 'right0D0', 'bottom1B0', 'left0A0', 'bottom2C0', 'top2C3', 'right2D2', 'left2A2', 'top3D3', 'right0D0', 'bottom3D0', 'right0D0', 'top1B3', 'bottom1B0', 'bottom1B0', 'bottom0A0', 'bottom1B0', 'left3A3', 'bottom2C0', 'right0D0', 'left2A2', 'bottom0A0', 'left0A0', 'bottom0A0', 'right2D2', 'top0A3', 'right3D3', 'left2A2', 'left1A1', 'right1D1', 'top3D3', 'left3A3', 'left2A2', 'left0A0', 'top3D3', 'top3D3', 'right2D2', 'bottom2C0', 'top0A3', 'right1D1', 'right3D3', 'right0D0', 'left3A3', 'bottom2C0', 'top0A3', 'right1D1', 'left0A0', 'left1A1', 'bottom2C0', 'left1A1', 'bottom2C0', 'right0D0', 'top3D3', 'bottom2C0', 'bottom3D0', 'left1A1', 'top0A3', 'top0A3', 'top2C3', 'bottom3D0', 'top2C3', 'top3D3', 'left0A0', 'top3D3', 'left2A2', 'bottom2C0', 'right1D1', 'left3A3', 'top1B3', 'top3D3', 'top0A3', 'left3A3', 'left3A3', 'bottom1B0', 'top0A3', 'right1D1', 'top0A3', 'left2A2', 'right1D1', 'top2C3', 'bottom2C0', 'bottom0A0', 'left3A3', 'bottom1B0', 'right0D0', 'left2A2', 'top0A3', 'bottom1B0', 'bottom3D0', 'bottom2C0', 'right3D3', 'bottom0A0', 'bottom3D0', 'top0A3', 'right1D1', 'right3D3', 'right1D1', 'left2A2', 'left2A2', 'top1B3', 'top1B3', 'top0A3', 'top3D3', 'top0A3', 'top3D3', 'left3A3', 'top0A3', 'left1A1', 'bottom1B0', 'left2A2', 'right0D0', 'left3A3', 'top1B3', 'bottom0A0', 'bottom3D0', 'top1B3', 'right0D0', 'right1D1', 'top0A3', 'top2C3', 'top3D3', 'bottom2C0', 'right3D3', 'bottom2C0', 'top2C3', 'bottom3D0', 'left3A3', 'right3D3', 'left2A2', 'top3D3', 'top0A3', 'bottom2C0', 'top2C3', 'left1A1', 'right3D3', 'right1D1', 'right1D1', 'left1A1', 'left0A0', 'left0A0', 'bottom2C0', 'bottom2C0', 'left0A0', 'top1B3', 'right2D2', 'right0D0', 'bottom1B0', 'right1D1', 'bottom1B0', 'bottom0A0', 'right0D0', 'bottom0A0', 'right1D1', 'left0A0', 'left3A3', 'left0A0', 'bottom1B0', 'bottom1B0', 'top1B3', 'left3A3', 'left1A1', 'right1D1', 'left2A2', 'bottom0A0', 'right3D3', 'right1D1', 'right2D2', 'right0D0', 'right1D1', 'left3A3', 'right0D0', 'top1B3', 'left0A0', 'right3D3', 'right3D3', 'right1D1', 'left3A3', 'left3A3', 'bottom3D0', 'left2A2', 'left1A1', 'left2A2', 'top3D3', 'right3D3', 'top0A3', 'left1A1', 'left0A0', 'bottom2C0', 'right3D3', 'left0A0', 'top2C3', 'left2A2', 'left2A2', 'left2A2', 'right2D2', 'bottom0A0', 'left1A1', 'left3A3', 'top3D3', 'right2D2', 'right0D0', 'left2A2', 'left1A1', 'left2A2', 'left1A1', 'top0A3', 'top1B3', 'bottom3D0', 'top3D3', 'right0D0', 'left1A1', 'right0D0', 'top0A3', 'right1D1', 'left1A1', 'right2D2', 'top0A3', 'left3A3', 'right1D1', 'top2C3', 'right0D0', 'right3D3', 'right2D2', 'bottom0A0', 'left3A3', 'top2C3', 'bottom0A0', 'bottom1B0', 'left2A2', 'top3D3', 'right3D3', 'left3A3', 'bottom3D0', 'top3D3', 'left3A3', 'left2A2', 'left0A0', 'top2C3', 'bottom3D0', 'left3A3', 'right3D3', 'bottom0A0', 'top2C3', 'bottom2C0', 'bottom2C0', 'right2D2', 'bottom2C0', 'top2C3', 'right1D1', 'right2D2', 'right2D2', 'bottom0A0', 'left0A0', 'left1A1', 'left0A0', 'top2C3', 'top2C3', 'top2C3', 'bottom3D0', 'top3D3', 'left3A3', 'top0A3', 'left2A2', 'bottom2C0', 'left1A1', 'right0D0', 'top0A3', 'top3D3', 'top1B3', 'right3D3', 'top2C3', 'right0D0', 'right3D3', 'left0A0', 'left3A3', 'left3A3', 'right0D0', 'left1A1', 'right2D2', 'top2C3', 'left0A0', 'left1A1', 'right0D0', 'left3A3', 'left3A3', 'bottom3D0', 'right0D0', 'bottom1B0', 'bottom3D0', 'right3D3', 'top3D3', 'left0A0', 'top3D3', 'bottom3D0', 'top2C3', 'top1B3', 'top0A3', 'right2D2', 'top1B3', 'left0A0', 'left2A2', 'bottom0A0', 'top2C3', 'left3A3', 'left2A2', 'right2D2', 'right0D0', 'top3D3', 'top2C3', 'left0A0', 'bottom2C0', 'right1D1', 'right0D0', 'top0A3', 'left1A1', 'left0A0', 'left1A1', 'left0A0', 'left2A2', 'right2D2', 'top3D3', 'top2C3', 'top1B3', 'right1D1', 'top3D3', 'top0A3', 'right1D1', 'right3D3', 'top2C3', 'bottom0A0', 'right0D0', 'bottom3D0', 'right2D2', 'top1B3', 'top2C3', 'top3D3', 'left1A1', 'bottom1B0', 'right1D1', 'bottom0A0', 'bottom2C0', 'right1D1', 'left3A3', 'top2C3', 'bottom3D0', 'top3D3', 'top3D3', 'right2D2', 'bottom2C0', 'left3A3', 'bottom3D0', 'top0A3', 'right0D0', 'bottom0A0', 'left2A2', 'bottom1B0', 'right3D3', 'bottom3D0', 'left1A1', 'bottom2C0', 'left1A1', 'bottom2C0', 'bottom1B0', 'right3D3', 'bottom0A0', 'top2C3', 'bottom1B0', 'right3D3', 'right1D1', 'right3D3', 'bottom3D0', 'top2C3', 'right2D2', 'right0D0', 'bottom0A0', 'top1B3', 'left2A2', 'top2C3', 'top2C3', 'bottom2C0', 'left2A2', 'right2D2', 'right3D3', 'left0A0', 'bottom3D0', 'bottom1B0', 'left0A0', 'bottom1B0', 'top3D3', 'top1B3', 'bottom1B0', 'bottom2C0', 'left0A0', 'left0A0', 'right0D0', 'bottom0A0', 'top1B3', 'top0A3', 'top1B3', 'bottom3D0', 'top0A3', 'left2A2', 'top2C3', 'right3D3', 'right3D3', 'right3D3', 'left2A2', 'left3A3', 'right2D2', 'left2A2', 'left0A0']
# end_edges = ['A0left0', 'A3top0', 'A2left2', 'A0left0', 'B3top1', 'B3top1', 'B0bottom1', 'C3top2', 'B0bottom1', 'D2right2', 'A0left0', 'D3right3', 'D1right1', 'A1left1', 'D2right2', 'A2left2', 'A0bottom0', 'D1right1', 'A3top0', 'D0right0', 'D3right3', 'D3top3', 'A3left3', 'D2right2', 'A3top0', 'D1right1', 'D0right0', 'A2left2', 'D3top3', 'A1left1', 'C3top2', 'D3top3', 'D0right0', 'C0bottom2', 'A2left2', 'A1left1', 'A0left0', 'D3top3', 'B0bottom1', 'D3top3', 'D1right1', 'D3top3', 'B0bottom1', 'B0bottom1', 'A2left2', 'D0bottom3', 'D0bottom3', 'A1left1', 'D0right0', 'C3top2', 'A0bottom0', 'B3top1', 'B0bottom1', 'A1left1', 'D1right1', 'D0right0', 'D3right3', 'D3top3', 'A2left2', 'A3top0', 'C0bottom2', 'A0left0', 'D3right3', 'B0bottom1', 'C3top2', 'A0left0', 'D0right0', 'A1left1', 'B3top1', 'B3top1', 'D0bottom3', 'D2right2', 'C3top2', 'D3right3', 'A1left1', 'D0bottom3', 'A3left3', 'D3top3', 'D3top3', 'A0left0', 'A3top0', 'A3left3', 'C0bottom2', 'B0bottom1', 'D1right1', 'A0left0', 'B3top1', 'B0bottom1', 'D0right0', 'D3right3', 'B3top1', 'A0bottom0', 'A3top0', 'A3top0', 'A3left3', 'D0right0', 'D0bottom3', 'B0bottom1', 'A3top0', 'A3left3', 'B3top1', 'A0left0', 'A3left3', 'A0bottom0', 'D0bottom3', 'A0left0', 'B0bottom1', 'A2left2', 'D1right1', 'B0bottom1', 'A0left0', 'A2left2', 'D2right2', 'A1left1', 'D0bottom3', 'A0left0', 'D1right1', 'C3top2', 'C0bottom2', 'D0right0', 'D0bottom3', 'C3top2', 'D3right3', 'D0bottom3', 'C0bottom2', 'B0bottom1', 'D0right0', 'C0bottom2', 'D0bottom3', 'A1left1', 'D0bottom3', 'D3top3', 'D3right3', 'A3top0', 'C0bottom2', 'A3top0', 'A3left3', 'D1right1', 'D0right0', 'D1right1', 'C0bottom2', 'A0bottom0', 'A0left0', 'B3top1', 'D1right1', 'D0right0', 'D3top3', 'D1right1', 'B0bottom1', 'B0bottom1', 'A1left1', 'D3top3', 'A3top0', 'A3top0', 'C3top2', 'D3top3', 'D1right1', 'A2left2', 'C0bottom2', 'B0bottom1', 'D0right0', 'A3top0', 'D3top3', 'D0right0', 'C3top2', 'D0bottom3', 'A0left0', 'D2right2', 'A0left0', 'A2left2', 'A1left1', 'A0left0', 'C3top2', 'D2right2', 'D1right1', 'B3top1', 'D3top3', 'D1right1', 'B3top1', 'D1right1', 'A0bottom0', 'C3top2', 'A2left2', 'D3top3', 'C0bottom2', 'A1left1', 'B0bottom1', 'B3top1', 'D0right0', 'C0bottom2', 'D2right2', 'D1right1', 'D1right1', 'A3top0', 'A3top0', 'A3left3', 'A0left0', 'D1right1', 'D2right2', 'A0bottom0', 'A1left1', 'A3left3', 'A0left0', 'D3top3', 'A0bottom0', 'C3top2', 'D3top3', 'A2left2', 'C3top2', 'C3top2', 'D1right1', 'D1right1', 'A0bottom0', 'D0bottom3', 'A2left2', 'D3top3', 'D1right1', 'C0bottom2', 'A2left2', 'C0bottom2', 'B3top1', 'A0bottom0', 'C0bottom2', 'D0bottom3', 'D0bottom3', 'C3top2', 'A0left0', 'A3top0', 'A0bottom0', 'D0bottom3', 'C3top2', 'B3top1', 'D3right3', 'A1left1', 'D3right3', 'A0bottom0', 'A1left1', 'A0left0', 'D2right2', 'A0left0', 'D0bottom3', 'D1right1', 'B0bottom1', 'A0bottom0', 'B0bottom1', 'A2left2', 'A1left1', 'B3top1', 'D3top3', 'D2right2', 'D1right1', 'A0bottom0', 'C0bottom2', 'A2left2', 'A3top0', 'D1right1', 'D0right0', 'A1left1', 'A1left1', 'D3top3', 'D3top3', 'D1right1', 'A0bottom0', 'D3top3', 'A3left3', 'D1right1', 'D3top3', 'D2right2', 'C0bottom2', 'A3left3', 'A0bottom0', 'D3right3', 'A2left2', 'D1right1', 'D2right2', 'D3right3', 'B3top1', 'B0bottom1', 'D0bottom3', 'A3left3', 'D0bottom3', 'A1left1', 'D2right2', 'B0bottom1', 'B0bottom1', 'C0bottom2', 'A3top0', 'D1right1', 'A0bottom0', 'A3left3', 'B0bottom1', 'B0bottom1', 'A0left0', 'B0bottom1', 'D3top3', 'C3top2', 'A0bottom0', 'A0left0', 'B3top1', 'D1right1', 'A3top0', 'C3top2', 'A1left1', 'A3left3', 'D2right2', 'D1right1', 'B0bottom1', 'A3left3', 'D2right2', 'A1left1', 'C0bottom2', 'D2right2', 'D3right3', 'A3top0', 'D3top3', 'D3right3', 'A0bottom0', 'B3top1', 'A3top0', 'C3top2', 'D0bottom3', 'A3left3', 'A0bottom0', 'D3top3', 'A0left0', 'D3top3', 'D0right0', 'A3top0', 'A0bottom0', 'B3top1', 'D3top3', 'C3top2', 'C3top2', 'C3top2', 'A3top0', 'A2left2', 'D3top3', 'A2left2', 'A2left2', 'A3top0', 'A0left0', 'D3right3', 'D1right1', 'D3top3', 'D0bottom3', 'A0bottom0', 'A2left2', 'D2right2', 'A0bottom0', 'B3top1', 'D2right2', 'A0bottom0', 'B0bottom1', 'A3top0', 'A2left2', 'A0left0', 'D1right1', 'D3top3', 'D3top3', 'A3top0', 'D1right1', 'A3top0', 'D3top3', 'D3right3', 'A0bottom0', 'A0bottom0', 'A1left1', 'B0bottom1', 'A1left1', 'A3top0', 'C0bottom2', 'D3right3', 'D3top3', 'A1left1', 'D2right2', 'B3top1', 'A3left3', 'D2right2', 'A1left1', 'C0bottom2', 'C3top2', 'D0bottom3', 'D3right3', 'B0bottom1', 'C0bottom2', 'D3right3', 'A1left1', 'C3top2', 'A1left1', 'D3right3', 'A3top0', 'A0bottom0', 'B3top1', 'D0right0', 'B0bottom1', 'A2left2', 'D3right3', 'A2left2', 'A3top0', 'A2left2', 'D1right1', 'B0bottom1', 'D3top3', 'D2right2', 'D0bottom3', 'D1right1', 'D1right1', 'C0bottom2', 'B3top1', 'A1left1', 'B0bottom1', 'D1right1', 'C0bottom2', 'B0bottom1', 'D2right2', 'A3top0', 'A1left1', 'D3right3', 'A0bottom0', 'D0right0', 'A0bottom0', 'C3top2', 'C0bottom2', 'D0bottom3', 'A0bottom0', 'A2left2', 'A0left0', 'C3top2', 'B0bottom1', 'A3top0', 'C0bottom2', 'D3top3', 'A0bottom0', 'D0right0', 'D2right2', 'A1left1', 'C0bottom2', 'A0left0', 'D0right0', 'A0bottom0', 'A0left0', 'B0bottom1']
# ##grid_40
# car_num = 589
# start_edges = ['right1D1', 'left3A3', 'top2C3', 'right3D3', 'left2A2', 'right1D1', 'right1D1', 'left1A1', 'left3A3', 'right0D0', 'left0A0', 'left3A3', 'bottom0A0', 'bottom0A0', 'top3D3', 'bottom1B0', 'left1A1', 'right2D2', 'right0D0', 'bottom0A0', 'top0A3', 'bottom3D0', 'top1B3', 'bottom0A0', 'left1A1', 'right2D2', 'bottom3D0', 'left3A3', 'right1D1', 'top0A3', 'right0D0', 'left0A0', 'right1D1', 'top3D3', 'bottom1B0', 'top3D3', 'right3D3', 'right1D1', 'bottom1B0', 'bottom2C0', 'bottom2C0', 'left3A3', 'top2C3', 'top0A3', 'left2A2', 'bottom0A0', 'right1D1', 'bottom3D0', 'bottom3D0', 'left0A0', 'bottom0A0', 'bottom3D0', 'right3D3', 'left0A0', 'right2D2', 'top0A3', 'right2D2', 'bottom3D0', 'left2A2', 'right3D3', 'top1B3', 'top0A3', 'right3D3', 'left0A0', 'right2D2', 'right0D0', 'right3D3', 'right2D2', 'bottom1B0', 'top2C3', 'bottom0A0', 'bottom1B0', 'left0A0', 'bottom0A0', 'right0D0', 'bottom0A0', 'left0A0', 'bottom0A0', 'left3A3', 'bottom2C0', 'right2D2', 'bottom3D0', 'left1A1', 'left1A1', 'right1D1', 'top3D3', 'bottom3D0', 'right2D2', 'top2C3', 'left2A2', 'left1A1', 'right0D0', 'left2A2', 'top1B3', 'top0A3', 'left2A2', 'right0D0', 'right2D2', 'bottom2C0', 'right3D3', 'right1D1', 'left3A3', 'left2A2', 'right3D3', 'bottom2C0', 'right0D0', 'top0A3', 'right1D1', 'right2D2', 'top2C3', 'bottom2C0', 'right3D3', 'left1A1', 'left2A2', 'right0D0', 'left3A3', 'left3A3', 'right0D0', 'right2D2', 'bottom0A0', 'bottom0A0', 'top0A3', 'top0A3', 'top2C3', 'bottom1B0', 'left2A2', 'right3D3', 'top1B3', 'bottom3D0', 'bottom2C0', 'left2A2', 'bottom0A0', 'bottom0A0', 'right0D0', 'right1D1', 'top3D3', 'right0D0', 'bottom1B0', 'right2D2', 'bottom0A0', 'top0A3', 'top3D3', 'bottom3D0', 'right1D1', 'right2D2', 'bottom2C0', 'left3A3', 'top2C3', 'bottom1B0', 'left3A3', 'bottom0A0', 'bottom2C0', 'top2C3', 'left0A0', 'top1B3', 'right3D3', 'left3A3', 'right3D3', 'bottom1B0', 'top0A3', 'right3D3', 'top2C3', 'left2A2', 'bottom1B0', 'bottom3D0', 'bottom3D0', 'bottom2C0', 'bottom0A0', 'left2A2', 'bottom3D0', 'bottom0A0', 'left1A1', 'right3D3', 'bottom1B0', 'left0A0', 'bottom3D0', 'left3A3', 'bottom2C0', 'top1B3', 'top1B3', 'left2A2', 'left0A0', 'right2D2', 'bottom3D0', 'right1D1', 'top1B3', 'bottom3D0', 'top0A3', 'bottom3D0', 'top1B3', 'right3D3', 'top2C3', 'top3D3', 'left0A0', 'top0A3', 'bottom3D0', 'bottom3D0', 'left2A2', 'right0D0', 'right3D3', 'bottom3D0', 'bottom2C0', 'right3D3', 'top3D3', 'bottom3D0', 'left2A2', 'top0A3', 'top0A3', 'top1B3', 'right1D1', 'top0A3', 'top2C3', 'right0D0', 'left3A3', 'left2A2', 'top3D3', 'bottom2C0', 'right3D3', 'top2C3', 'left3A3', 'right2D2', 'left1A1', 'right2D2', 'bottom3D0', 'top2C3', 'bottom1B0', 'bottom1B0', 'right2D2', 'top1B3', 'right1D1', 'top2C3', 'bottom1B0', 'left2A2', 'right1D1', 'left1A1', 'right0D0', 'bottom0A0', 'top2C3', 'right2D2', 'left0A0', 'top3D3', 'left1A1', 'right0D0', 'bottom2C0', 'bottom3D0', 'bottom2C0', 'left0A0', 'right1D1', 'right0D0', 'top1B3', 'right3D3', 'top3D3', 'right1D1', 'bottom1B0', 'bottom0A0', 'right0D0', 'right1D1', 'left0A0', 'right1D1', 'right0D0', 'left3A3', 'right2D2', 'right1D1', 'right1D1', 'bottom3D0', 'right3D3', 'left1A1', 'bottom0A0', 'top3D3', 'right0D0', 'right1D1', 'right2D2', 'right2D2', 'bottom0A0', 'left2A2', 'top1B3', 'bottom0A0', 'right3D3', 'top2C3', 'right1D1', 'right2D2', 'bottom1B0', 'right1D1', 'left3A3', 'right0D0', 'top1B3', 'left0A0', 'right3D3', 'left1A1', 'top3D3', 'top1B3', 'left3A3', 'left3A3', 'top2C3', 'right1D1', 'left2A2', 'left1A1', 'right0D0', 'right1D1', 'top3D3', 'right3D3', 'left3A3', 'top0A3', 'top0A3', 'bottom2C0', 'bottom1B0', 'right1D1', 'left1A1', 'left0A0', 'bottom2C0', 'left0A0', 'top1B3', 'bottom0A0', 'bottom3D0', 'left3A3', 'bottom1B0', 'bottom1B0', 'left1A1', 'left3A3', 'right2D2', 'top0A3', 'top1B3', 'left1A1', 'right1D1', 'top2C3', 'left2A2', 'right0D0', 'top1B3', 'bottom3D0', 'top3D3', 'bottom3D0', 'top3D3', 'right3D3', 'left3A3', 'right0D0', 'left2A2', 'top3D3', 'left0A0', 'top3D3', 'bottom0A0', 'top2C3', 'left3A3', 'right0D0', 'bottom1B0', 'right1D1', 'right1D1', 'left0A0', 'right2D2', 'right2D2', 'left3A3', 'right1D1', 'right0D0', 'right0D0', 'top3D3', 'right3D3', 'bottom1B0', 'bottom3D0', 'left3A3', 'bottom0A0', 'left3A3', 'right1D1', 'bottom3D0', 'left0A0', 'top2C3', 'bottom1B0', 'right2D2', 'bottom3D0', 'left2A2', 'right0D0', 'right2D2', 'left2A2', 'left0A0', 'left3A3', 'right3D3', 'left1A1', 'bottom2C0', 'bottom0A0', 'left2A2', 'right2D2', 'bottom2C0', 'top3D3', 'right2D2', 'right1D1', 'right0D0', 'bottom3D0', 'right2D2', 'bottom3D0', 'left0A0', 'right2D2', 'right1D1', 'right1D1', 'left0A0', 'bottom0A0', 'top2C3', 'bottom2C0', 'bottom0A0', 'right1D1', 'top3D3', 'left1A1', 'top2C3', 'top2C3', 'right2D2', 'bottom3D0', 'top1B3', 'left3A3', 'top1B3', 'top0A3', 'left1A1', 'right0D0', 'right2D2', 'right0D0', 'top3D3', 'right3D3', 'top2C3', 'right0D0', 'bottom0A0', 'top2C3', 'bottom2C0', 'left2A2', 'left1A1', 'left3A3', 'left1A1', 'left3A3', 'right3D3', 'bottom1B0', 'top2C3', 'right1D1', 'top2C3', 'top2C3', 'right2D2', 'left1A1', 'bottom2C0', 'top1B3', 'left3A3', 'left3A3', 'top0A3', 'left1A1', 'right0D0', 'top3D3', 'left2A2', 'bottom0A0', 'bottom2C0', 'right3D3', 'bottom1B0', 'bottom1B0', 'top2C3', 'right1D1', 'right1D1', 'left0A0', 'left1A1', 'left3A3', 'top3D3', 'left3A3', 'right2D2', 'top1B3', 'right2D2', 'left1A1', 'top0A3', 'bottom2C0', 'left3A3', 'right3D3', 'top1B3', 'left2A2', 'bottom3D0', 'left3A3', 'right2D2', 'top1B3', 'right0D0', 'top3D3', 'left0A0', 'right1D1', 'bottom3D0', 'left1A1', 'top0A3', 'left0A0', 'left1A1', 'left0A0', 'left0A0', 'left2A2', 'top0A3', 'left3A3', 'top2C3', 'bottom1B0', 'right3D3', 'bottom2C0', 'right0D0', 'right3D3', 'left3A3', 'top1B3', 'top2C3', 'right3D3', 'right3D3', 'top0A3', 'left0A0', 'top0A3', 'right1D1', 'top3D3', 'right2D2', 'top2C3', 'bottom1B0', 'left1A1', 'right1D1', 'left3A3', 'bottom2C0', 'bottom2C0', 'bottom2C0', 'top1B3', 'bottom0A0', 'bottom2C0', 'left3A3', 'left0A0', 'right1D1', 'top3D3', 'right2D2', 'top2C3', 'top0A3', 'bottom2C0', 'bottom3D0', 'right1D1', 'right1D1', 'right0D0', 'bottom1B0', 'right3D3', 'top3D3', 'right3D3', 'right3D3', 'bottom1B0', 'left1A1', 'left2A2', 'right1D1', 'top2C3', 'bottom3D0', 'left1A1', 'left3A3', 'right1D1', 'bottom2C0', 'left0A0', 'bottom1B0', 'right3D3', 'top2C3', 'bottom0A0', 'left0A0', 'bottom2C0', 'top2C3', 'left3A3', 'top1B3', 'right2D2', 'bottom1B0', 'left0A0', 'right3D3', 'right1D1', 'bottom0A0', 'left0A0', 'bottom3D0', 'left1A1', 'right1D1', 'left2A2', 'bottom1B0', 'top2C3', 'top2C3', 'right3D3', 'left0A0', 'right2D2', 'left0A0', 'bottom1B0', 'left3A3', 'bottom2C0', 'top2C3', 'top1B3', 'right0D0', 'right0D0', 'top0A3', 'top1B3', 'bottom1B0', 'left1A1', 'right3D3', 'top0A3', 'top3D3', 'left0A0', 'right3D3', 'right2D2', 'left2A2', 'right3D3', 'right0D0', 'right1D1', 'left2A2', 'bottom1B0', 'right0D0', 'top2C3']
# end_edges = ['A3left3', 'A2left2', 'A0left0', 'D0bottom3', 'B3top1', 'B3top1', 'B0bottom1', 'D0bottom3', 'B0bottom1', 'D0right0', 'B3top1', 'A0bottom0', 'A3left3', 'D0bottom3', 'D2right2', 'D3right3', 'D1right1', 'D3top3', 'D2right2', 'D2right2', 'B3top1', 'D1right1', 'A1left1', 'D2right2', 'A2left2', 'A0bottom0', 'D0bottom3', 'D1right1', 'D1right1', 'A0left0', 'A3top0', 'A0bottom0', 'D0right0', 'D1right1', 'C0bottom2', 'A0left0', 'D3right3', 'B3top1', 'B3top1', 'D1right1', 'A2left2', 'A3left3', 'D3top3', 'A3top0', 'D3top3', 'D3right3', 'B3top1', 'A2left2', 'D3top3', 'D0right0', 'B0bottom1', 'C3top2', 'C0bottom2', 'A2left2', 'D3top3', 'B0bottom1', 'A2left2', 'D3top3', 'D3top3', 'B3top1', 'B0bottom1', 'D0right0', 'B3top1', 'B3top1', 'D0right0', 'D0bottom3', 'A3left3', 'C0bottom2', 'A1left1', 'A0left0', 'A0left0', 'D1right1', 'A1left1', 'A3left3', 'A0bottom0', 'B0bottom1', 'A1left1', 'D1right1', 'A3top0', 'A0left0', 'D0bottom3', 'B0bottom1', 'A2left2', 'A3top0', 'C0bottom2', 'A0left0', 'A1left1', 'D3right3', 'D3right3', 'B0bottom1', 'D3top3', 'D1right1', 'D0right0', 'A0left0', 'A3top0', 'B0bottom1', 'D3right3', 'A1left1', 'B3top1', 'A3left3', 'A3top0', 'A3left3', 'C0bottom2', 'D2right2', 'A1left1', 'B3top1', 'D0bottom3', 'A3left3', 'D3right3', 'C3top2', 'A3left3', 'A1left1', 'B0bottom1', 'D0right0', 'A2left2', 'D3right3', 'D3right3', 'C0bottom2', 'C0bottom2', 'D0right0', 'B0bottom1', 'D0right0', 'D3right3', 'D3right3', 'D3right3', 'A0left0', 'D3top3', 'A3left3', 'A2left2', 'D0bottom3', 'B3top1', 'A1left1', 'D2right2', 'A3left3', 'A0left0', 'A0bottom0', 'D0bottom3', 'D0bottom3', 'D0bottom3', 'A3left3', 'A2left2', 'D1right1', 'C3top2', 'B0bottom1', 'A2left2', 'A2left2', 'A3left3', 'B3top1', 'A1left1', 'A0bottom0', 'D2right2', 'D0bottom3', 'A0left0', 'A2left2', 'D1right1', 'B3top1', 'A1left1', 'D0bottom3', 'D3top3', 'A3top0', 'D1right1', 'A3top0', 'D1right1', 'C0bottom2', 'A1left1', 'A1left1', 'D0bottom3', 'D3right3', 'A0bottom0', 'D0bottom3', 'D1right1', 'D3right3', 'D0right0', 'A3top0', 'C0bottom2', 'A0bottom0', 'D0bottom3', 'A3left3', 'D0bottom3', 'D3top3', 'A1left1', 'A2left2', 'D3right3', 'A2left2', 'C0bottom2', 'D0bottom3', 'B0bottom1', 'A1left1', 'D0bottom3', 'D3top3', 'D0bottom3', 'D2right2', 'A3top0', 'B0bottom1', 'D1right1', 'C0bottom2', 'A3left3', 'C0bottom2', 'A0bottom0', 'D0right0', 'D1right1', 'D0bottom3', 'B3top1', 'B3top1', 'D0right0', 'A0bottom0', 'B3top1', 'B0bottom1', 'D0bottom3', 'B0bottom1', 'B0bottom1', 'A1left1', 'D3right3', 'A3top0', 'C3top2', 'D3top3', 'A3top0', 'A3top0', 'D3top3', 'A2left2', 'D3top3', 'D3top3', 'B0bottom1', 'D2right2', 'D0bottom3', 'A0left0', 'A3left3', 'C3top2', 'D3top3', 'A0left0', 'D2right2', 'C0bottom2', 'D0bottom3', 'D2right2', 'A0left0', 'A1left1', 'A3top0', 'A1left1', 'A2left2', 'A1left1', 'D0right0', 'A2left2', 'A2left2', 'A1left1', 'B0bottom1', 'C3top2', 'A1left1', 'D0right0', 'D0right0', 'B0bottom1', 'C3top2', 'A0left0', 'B3top1', 'D1right1', 'A0bottom0', 'C3top2', 'D3top3', 'C0bottom2', 'D1right1', 'A3top0', 'A1left1', 'A1left1', 'D3top3', 'B3top1', 'C0bottom2', 'C3top2', 'D2right2', 'A0bottom0', 'D0right0', 'A3left3', 'D1right1', 'D3top3', 'C0bottom2', 'A0left0', 'A3top0', 'D1right1', 'A3top0', 'A3left3', 'A1left1', 'A0left0', 'D1right1', 'D0bottom3', 'A0bottom0', 'A1left1', 'A3left3', 'A0left0', 'D3top3', 'A0bottom0', 'A3top0', 'D2right2', 'A1left1', 'A2left2', 'C3top2', 'B0bottom1', 'D3right3', 'D1right1', 'D1right1', 'D1right1', 'C0bottom2', 'D0bottom3', 'A2left2', 'A0left0', 'D3top3', 'B0bottom1', 'D0bottom3', 'C3top2', 'D1right1', 'D1right1', 'D3top3', 'A2left2', 'B3top1', 'D0bottom3', 'A0bottom0', 'B0bottom1', 'D1right1', 'A3left3', 'B3top1', 'A3top0', 'A0bottom0', 'C3top2', 'D0bottom3', 'D0right0', 'A0bottom0', 'B0bottom1', 'D2right2', 'A2left2', 'A0left0', 'A0left0', 'D2right2', 'A0left0', 'D2right2', 'D3top3', 'D0right0', 'A3left3', 'D0bottom3', 'D2right2', 'A0bottom0', 'D3top3', 'D1right1', 'A1left1', 'B0bottom1', 'B0bottom1', 'B0bottom1', 'A1left1', 'A3top0', 'B0bottom1', 'B0bottom1', 'A1left1', 'A2left2', 'D3top3', 'D2right2', 'D1right1', 'A0bottom0', 'A0left0', 'A1left1', 'D3top3', 'D1right1', 'A2left2', 'A3top0', 'D0right0', 'D0bottom3', 'B3top1', 'C3top2', 'D0right0', 'A1left1', 'D3top3', 'D3top3', 'D2right2', 'B3top1', 'D1right1', 'D3top3', 'D2right2', 'D0right0', 'C0bottom2', 'A1left1', 'D3right3', 'A2left2', 'D2right2', 'A1left1', 'D2right2', 'A0bottom0', 'A3top0', 'D3top3', 'D0bottom3', 'A3left3', 'B3top1', 'A0bottom0', 'D2right2', 'B3top1', 'A0bottom0', 'D0bottom3', 'A3left3', 'D0right0', 'D0bottom3', 'A3left3', 'D2right2', 'A3top0', 'D3top3', 'B0bottom1', 'A3top0', 'D1right1', 'A2left2', 'A3left3', 'A1left1', 'B0bottom1', 'D2right2', 'A0left0', 'C3top2', 'A0bottom0', 'A0bottom0', 'D1right1', 'B3top1', 'A3top0', 'C3top2', 'A1left1', 'B3top1', 'A3left3', 'D0bottom3', 'D3right3', 'B0bottom1', 'D3right3', 'D2right2', 'D1right1', 'B0bottom1', 'A3top0', 'A0bottom0', 'D0bottom3', 'A3left3', 'C0bottom2', 'C3top2', 'D3right3', 'C3top2', 'C0bottom2', 'D3top3', 'D2right2', 'B0bottom1', 'A2left2', 'B3top1', 'B3top1', 'C3top2', 'D2right2', 'B3top1', 'A1left1', 'A3top0', 'D3top3', 'A1left1', 'D3top3', 'D0bottom3', 'A0bottom0', 'B0bottom1', 'C3top2', 'D3top3', 'B0bottom1', 'A0bottom0', 'B3top1', 'D3right3', 'C0bottom2', 'A0left0', 'D2right2', 'A3top0', 'D0right0', 'A3top0', 'A2left2', 'C3top2', 'D0bottom3', 'D3top3', 'A0left0', 'A2left2', 'A2left2', 'A0left0', 'D1right1', 'D2right2', 'C3top2', 'D0bottom3', 'A1left1', 'A0bottom0', 'A2left2', 'A0bottom0', 'B3top1', 'D1right1', 'D3right3', 'D3right3', 'B3top1', 'B0bottom1', 'A2left2', 'D0right0', 'A0bottom0', 'B3top1', 'A3top0', 'B0bottom1', 'D3top3', 'D3right3', 'B3top1', 'A0bottom0', 'D0right0', 'A1left1', 'C3top2', 'A3top0', 'A0bottom0', 'A1left1', 'A1left1', 'A3top0', 'A0bottom0', 'D3right3', 'D0right0', 'A3top0', 'A0left0', 'C3top2', 'D3right3', 'B0bottom1', 'C3top2', 'D0right0', 'A3left3', 'D3top3', 'D0right0', 'D3top3', 'A2left2', 'D0bottom3', 'D0right0', 'B0bottom1', 'B0bottom1', 'A1left1', 'B3top1', 'D0bottom3', 'C3top2', 'A3left3', 'D3top3', 'D3right3', 'D3top3', 'A3left3', 'D3right3', 'C0bottom2', 'A0bottom0', 'B3top1', 'B0bottom1', 'B3top1', 'D0right0', 'A2left2', 'B0bottom1', 'D3right3', 'D1right1', 'D0bottom3', 'A3top0', 'D0bottom3', 'B0bottom1', 'B0bottom1', 'C0bottom2', 'D1right1', 'D2right2', 'D1right1', 'A3top0', 'D1right1', 'D1right1', 'D0right0', 'A3left3', 'D3right3', 'C0bottom2', 'A2left2', 'A1left1', 'D3top3', 'B0bottom1', 'A1left1', 'D2right2', 'D3right3', 'A0bottom0', 'A1left1', 'A2left2', 'D1right1', 'A0bottom0', 'B0bottom1', 'A2left2', 'D1right1', 'C3top2', 'A0bottom0', 'D3top3', 'D0right0', 'D3top3', 'D0right0', 'D0bottom3', 'A1left1', 'D0right0', 'A0left0', 'D0bottom3', 'D3top3', 'A1left1', 'A0left0', 'A0left0', 'D2right2', 'D0bottom3']
# ###grid_50
# car_num = 736
# start_edges = ['top2C3', 'right1D1', 'left3A3', 'bottom0A0', 'top1B3', 'bottom0A0', 'left1A1', 'top0A3', 'top0A3', 'top0A3', 'right3D3', 'top3D3', 'right1D1', 'left1A1', 'top0A3', 'bottom0A0', 'right0D0', 'left0A0', 'left3A3', 'bottom0A0', 'bottom0A0', 'right0D0', 'bottom1B0', 'left1A1', 'right0D0', 'bottom0A0', 'bottom3D0', 'top1B3', 'left2A2', 'bottom0A0', 'right2D2', 'bottom3D0', 'left3A3', 'right1D1', 'bottom0A0', 'top3D3', 'right2D2', 'right0D0', 'left0A0', 'right3D3', 'right1D1', 'left2A2', 'top2C3', 'bottom1B0', 'right0D0', 'bottom2C0', 'left3A3', 'bottom1B0', 'left0A0', 'top0A3', 'left2A2', 'right3D3', 'bottom3D0', 'left1A1', 'left0A0', 'bottom3D0', 'left0A0', 'right0D0', 'left3A3', 'left0A0', 'bottom3D0', 'left3A3', 'right2D2', 'left2A2', 'top2C3', 'top3D3', 'top3D3', 'right2D2', 'right0D0', 'right1D1', 'right3D3', 'bottom2C0', 'right3D3', 'right0D0', 'bottom1B0', 'top3D3', 'bottom1B0', 'left0A0', 'bottom0A0', 'right0D0', 'left2A2', 'right1D1', 'left0A0', 'bottom0A0', 'bottom1B0', 'bottom0A0', 'right2D2', 'bottom2C0', 'right2D2', 'right2D2', 'top0A3', 'right3D3', 'right2D2', 'left2A2', 'left1A1', 'bottom3D0', 'left1A1', 'left1A1', 'left3A3', 'left3A3', 'bottom3D0', 'right2D2', 'top0A3', 'bottom1B0', 'top1B3', 'top0A3', 'top3D3', 'right2D2', 'top3D3', 'right2D2', 'bottom2C0', 'top0A3', 'bottom3D0', 'top0A3', 'top0A3', 'left3A3', 'top0A3', 'left2A2', 'left0A0', 'right3D3', 'right0D0', 'bottom2C0', 'right0D0', 'top0A3', 'right1D1', 'left1A1', 'top1B3', 'left1A1', 'bottom2C0', 'left0A0', 'left1A1', 'bottom2C0', 'bottom3D0', 'left2A2', 'right0D0', 'bottom1B0', 'bottom1B0', 'bottom1B0', 'bottom3D0', 'bottom1B0', 'right0D0', 'right0D0', 'right2D2', 'bottom2C0', 'top3D3', 'top0A3', 'bottom0A0', 'top0A3', 'bottom0A0', 'top0A3', 'right3D3', 'top0A3', 'left3A3', 'top3D3', 'left2A2', 'right3D3', 'top0A3', 'bottom1B0', 'right0D0', 'bottom3D0', 'left0A0', 'top3D3', 'left2A2', 'bottom2C0', 'left0A0', 'left3A3', 'top1B3', 'bottom0A0', 'top0A3', 'right0D0', 'top3D3', 'bottom2C0', 'bottom1B0', 'top2C3', 'left3A3', 'bottom0A0', 'left3A3', 'bottom0A0', 'top0A3', 'top1B3', 'right1D1', 'right2D2', 'bottom2C0', 'right2D2', 'left3A3', 'bottom1B0', 'bottom0A0', 'bottom2C0', 'left0A0', 'top0A3', 'left2A2', 'left3A3', 'right3D3', 'right3D3', 'right1D1', 'right3D3', 'top2C3', 'right0D0', 'left2A2', 'left0A0', 'bottom3D0', 'bottom1B0', 'bottom3D0', 'right2D2', 'bottom3D0', 'left3A3', 'bottom3D0', 'right0D0', 'right3D3', 'right3D3', 'bottom0A0', 'left2A2', 'bottom3D0', 'bottom0A0', 'right2D2', 'left0A0', 'bottom3D0', 'top1B3', 'left2A2', 'top1B3', 'left2A2', 'right2D2', 'bottom3D0', 'top1B3', 'top1B3', 'top0A3', 'top3D3', 'top0A3', 'right0D0', 'left1A1', 'bottom1B0', 'right0D0', 'right2D2', 'right3D3', 'left3A3', 'left3A3', 'bottom2C0', 'left0A0', 'bottom2C0', 'top0A3', 'top1B3', 'right0D0', 'bottom2C0', 'right1D1', 'top2C3', 'top0A3', 'right0D0', 'left2A2', 'top3D3', 'right3D3', 'left1A1', 'bottom2C0', 'top2C3', 'left3A3', 'right2D2', 'right2D2', 'right2D2', 'left2A2', 'left1A1', 'top3D3', 'left3A3', 'right0D0', 'right2D2', 'top2C3', 'bottom1B0', 'right1D1', 'left2A2', 'right1D1', 'right0D0', 'bottom0A0', 'left2A2', 'left1A1', 'bottom2C0', 'left2A2', 'left0A0', 'right2D2', 'bottom0A0', 'right3D3', 'bottom2C0', 'left3A3', 'bottom2C0', 'bottom2C0', 'bottom3D0', 'left0A0', 'bottom2C0', 'left0A0', 'left1A1', 'bottom0A0', 'right1D1', 'bottom0A0', 'right0D0', 'left2A2', 'right3D3', 'right3D3', 'right3D3', 'left2A2', 'right1D1', 'bottom1B0', 'top2C3', 'left0A0', 'bottom0A0', 'left0A0', 'left3A3', 'left3A3', 'top1B3', 'right0D0', 'right1D1', 'top0A3', 'top3D3', 'top1B3', 'left0A0', 'right1D1', 'bottom1B0', 'right2D2', 'top3D3', 'bottom0A0', 'top1B3', 'left3A3', 'right2D2', 'left2A2', 'top3D3', 'top1B3', 'left2A2', 'bottom0A0', 'left0A0', 'right3D3', 'bottom2C0', 'top1B3', 'bottom1B0', 'left3A3', 'right0D0', 'left0A0', 'top3D3', 'top2C3', 'top1B3', 'left3A3', 'left3A3', 'bottom3D0', 'left2A2', 'left1A1', 'right1D1', 'bottom3D0', 'top3D3', 'right3D3', 'top1B3', 'top3D3', 'bottom2C0', 'left1A1', 'right2D2', 'left1A1', 'bottom2C0', 'right1D1', 'left2A2', 'right2D2', 'left2A2', 'left0A0', 'left1A1', 'left3A3', 'bottom1B0', 'bottom0A0', 'left0A0', 'left3A3', 'left2A2', 'top0A3', 'right3D3', 'top2C3', 'left1A1', 'left3A3', 'right2D2', 'top0A3', 'bottom1B0', 'top1B3', 'right2D2', 'right0D0', 'left1A1', 'left1A1', 'left2A2', 'bottom3D0', 'right1D1', 'left2A2', 'top1B3', 'left3A3', 'top0A3', 'bottom3D0', 'right3D3', 'bottom3D0', 'top0A3', 'right0D0', 'top3D3', 'left0A0', 'top3D3', 'right1D1', 'right0D0', 'top2C3', 'bottom1B0', 'bottom1B0', 'right1D1', 'left1A1', 'top2C3', 'left0A0', 'right2D2', 'left3A3', 'right1D1', 'top0A3', 'right0D0', 'right3D3', 'right2D2', 'bottom1B0', 'left1A1', 'top3D3', 'right3D3', 'bottom0A0', 'top0A3', 'left3A3', 'bottom2C0', 'left3A3', 'right1D1', 'left0A0', 'bottom1B0', 'top2C3', 'left2A2', 'right3D3', 'left3A3', 'right2D2', 'bottom3D0', 'bottom3D0', 'left2A2', 'right3D3', 'top3D3', 'left2A2', 'left0A0', 'right2D2', 'bottom3D0', 'right1D1', 'left3A3', 'top0A3', 'top0A3', 'top0A3', 'bottom2C0', 'right3D3', 'top3D3', 'right0D0', 'bottom2C0', 'right2D2', 'bottom2C0', 'right0D0', 'left1A1', 'left0A0', 'right2D2', 'right1D1', 'bottom1B0', 'right1D1', 'left0A0', 'top2C3', 'bottom2C0', 'right1D1', 'bottom0A0', 'bottom2C0', 'right0D0', 'bottom0A0', 'right1D1', 'left0A0', 'left0A0', 'right1D1', 'top2C3', 'bottom3D0', 'top1B3', 'left1A1', 'top2C3', 'top1B3', 'top1B3', 'right0D0', 'left3A3', 'top0A3', 'left2A2', 'left1A1', 'bottom1B0', 'right2D2', 'top3D3', 'right3D3', 'top2C3', 'top3D3', 'right0D0', 'top3D3', 'left1A1', 'bottom0A0', 'bottom2C0', 'right3D3', 'right3D3', 'left0A0', 'right1D1', 'left3A3', 'left3A3', 'right1D1', 'right0D0', 'left2A2', 'left3A3', 'left2A2', 'right2D2', 'right1D1', 'right3D3', 'bottom1B0', 'right1D1', 'top2C3', 'right0D0', 'top2C3', 'right2D2', 'bottom2C0', 'top1B3', 'left2A2', 'bottom2C0', 'right3D3', 'right0D0', 'left3A3', 'left3A3', 'top0A3', 'left1A1', 'left2A2', 'top0A3', 'right0D0', 'bottom2C0', 'right3D3', 'bottom1B0', 'bottom1B0', 'bottom2C0', 'right3D3', 'right1D1', 'right3D3', 'top3D3', 'top3D3', 'left1A1', 'bottom1B0', 'left3A3', 'top2C3', 'left0A0', 'left3A3', 'bottom0A0', 'left3A3', 'top1B3', 'left0A0', 'right2D2', 'top1B3', 'right2D2', 'left1A1', 'bottom1B0', 'bottom1B0', 'bottom1B0', 'top2C3', 'top0A3', 'bottom2C0', 'left3A3', 'top2C3', 'top1B3', 'left2A2', 'top2C3', 'left2A2', 'right2D2', 'right2D2', 'top3D3', 'top2C3', 'bottom2C0', 'top1B3', 'right1D1', 'bottom3D0', 'left2A2', 'top2C3', 'left1A1', 'top0A3', 'left0A0', 'left0A0', 'right1D1', 'bottom2C0', 'left3A3', 'top0A3', 'left3A3', 'top2C3', 'right3D3', 'right3D3', 'top3D3', 'left1A1', 'top2C3', 'bottom2C0', 'right3D3', 'top1B3', 'top3D3', 'top2C3', 'left3A3', 'right3D3', 'top0A3', 'left0A0', 'top2C3', 'top1B3', 'bottom0A0', 'top0A3', 'top3D3', 'right1D1', 'top3D3', 'left1A1', 'right1D1', 'top3D3', 'left3A3', 'left1A1', 'left3A3', 'bottom2C0', 'bottom2C0', 'bottom0A0', 'bottom2C0', 'right3D3', 'right1D1', 'bottom0A0', 'left3A3', 'right1D1', 'left1A1', 'bottom2C0', 'left0A0', 'right1D1', 'bottom1B0', 'bottom3D0', 'right0D0', 'right2D2', 'top2C3', 'right2D2', 'top0A3', 'left0A0', 'left2A2', 'right1D1', 'right1D1', 'bottom2C0', 'left3A3', 'bottom2C0', 'right3D3', 'right1D1', 'right2D2', 'top2C3', 'right0D0', 'left2A2', 'right0D0', 'bottom1B0', 'right0D0', 'right3D3', 'right2D2', 'top2C3', 'top3D3', 'bottom1B0', 'right3D3', 'top1B3', 'bottom1B0', 'top2C3', 'left1A1', 'bottom0A0', 'right3D3', 'right1D1', 'bottom2C0', 'left0A0', 'right3D3', 'bottom0A0', 'top2C3', 'left0A0', 'bottom2C0', 'top2C3', 'bottom1B0', 'left1A1', 'left3A3', 'bottom0A0', 'left0A0', 'top1B3', 'right2D2', 'left0A0', 'right2D2', 'right0D0', 'bottom0A0', 'left3A3', 'left1A1', 'bottom2C0', 'right1D1', 'left1A1', 'top2C3', 'bottom2C0', 'right2D2', 'bottom1B0', 'top2C3', 'top1B3', 'right0D0', 'top2C3', 'left0A0', 'bottom3D0', 'right0D0', 'bottom1B0', 'left0A0', 'left1A1', 'top2C3', 'left3A3', 'right3D3', 'bottom2C0', 'top3D3', 'top1B3', 'bottom2C0', 'right0D0', 'left0A0', 'right0D0', 'bottom0A0', 'top0A3', 'bottom1B0', 'right1D1', 'bottom3D0', 'bottom1B0', 'top1B3', 'bottom3D0', 'top3D3', 'top3D3', 'right3D3', 'bottom0A0', 'left1A1', 'right3D3', 'top1B3', 'right0D0', 'bottom2C0', 'right0D0', 'left2A2', 'top1B3', 'right0D0', 'right2D2', 'bottom0A0', 'left2A2', 'bottom3D0', 'left2A2', 'top3D3', 'left3A3', 'bottom3D0', 'bottom1B0', 'right3D3', 'right1D1', 'bottom2C0', 'left0A0', 'top2C3', 'left0A0']
# end_edges = ['D3top3', 'A3left3', 'A2left2', 'A0left0', 'A1left1', 'A0left0', 'B3top1', 'C0bottom2', 'A3left3', 'A1left1', 'D0bottom3', 'D3right3', 'B0bottom1', 'D0bottom3', 'D0bottom3', 'A1left1', 'D0right0', 'B3top1', 'A0bottom0', 'A3left3', 'D0bottom3', 'C3top2', 'D3right3', 'D1right1', 'D2right2', 'D2right2', 'D1right1', 'C3top2', 'B0bottom1', 'D2right2', 'A0bottom0', 'D0bottom3', 'D1right1', 'D0right0', 'A3top0', 'A0left0', 'A1left1', 'A3top0', 'A0bottom0', 'D2right2', 'B3top1', 'D0bottom3', 'D3top3', 'A2left2', 'A0left0', 'A2left2', 'A3left3', 'B0bottom1', 'D3top3', 'A3top0', 'D3top3', 'D1right1', 'A2left2', 'D2right2', 'D0right0', 'C3top2', 'D0right0', 'D0right0', 'C3top2', 'A2left2', 'D3right3', 'A2left2', 'D3top3', 'B0bottom1', 'D3top3', 'A3left3', 'D0right0', 'A2left2', 'B0bottom1', 'C0bottom2', 'B3top1', 'D0right0', 'B3top1', 'D0bottom3', 'D0bottom3', 'A0left0', 'D1right1', 'A1left1', 'A3left3', 'A0bottom0', 'B3top1', 'A0left0', 'A1left1', 'A0left0', 'A0left0', 'D1right1', 'A0bottom0', 'A0left0', 'D0right0', 'D1right1', 'D3right3', 'D3top3', 'D0bottom3', 'A2left2', 'A2left2', 'B0bottom1', 'A2left2', 'A3top0', 'D1right1', 'D3right3', 'A1left1', 'D3right3', 'A2left2', 'A0bottom0', 'A0left0', 'A3top0', 'A0left0', 'D0bottom3', 'D0right0', 'A1left1', 'B3top1', 'A3top0', 'A1left1', 'D1right1', 'B3top1', 'A3left3', 'C0bottom2', 'C0bottom2', 'A3top0', 'D2right2', 'C3top2', 'A1left1', 'B3top1', 'D0bottom3', 'A3left3', 'D3top3', 'D3right3', 'B3top1', 'A0left0', 'D2right2', 'A3top0', 'A3left3', 'C0bottom2', 'D0right0', 'A2left2', 'A2left2', 'A2left2', 'A0bottom0', 'A0left0', 'D0right0', 'C0bottom2', 'C0bottom2', 'C0bottom2', 'A0left0', 'A2left2', 'B0bottom1', 'B0bottom1', 'D0right0', 'A3left3', 'D3right3', 'D3right3', 'A1left1', 'D3right3', 'D2right2', 'A0left0', 'D3top3', 'B0bottom1', 'B0bottom1', 'A3left3', 'C3top2', 'A3top0', 'A3left3', 'D0right0', 'D0bottom3', 'A0left0', 'A3top0', 'A3left3', 'A1left1', 'A0left0', 'A3left3', 'D0bottom3', 'A3top0', 'D0bottom3', 'B0bottom1', 'C0bottom2', 'C3top2', 'D2right2', 'A3left3', 'A2left2', 'A0bottom0', 'B0bottom1', 'A2left2', 'A2left2', 'D3top3', 'A0bottom0', 'A3top0', 'D2right2', 'D0bottom3', 'A2left2', 'D3right3', 'A3top0', 'A1left1', 'D0bottom3', 'D0bottom3', 'D0bottom3', 'D1right1', 'A3top0', 'A0left0', 'D1right1', 'A3top0', 'A1left1', 'B0bottom1', 'D0right0', 'D3right3', 'A3left3', 'B3top1', 'A1left1', 'D1right1', 'C3top2', 'D2right2', 'D3right3', 'A0bottom0', 'D0bottom3', 'D1right1', 'D0bottom3', 'C0bottom2', 'A0bottom0', 'C0bottom2', 'D0bottom3', 'D0bottom3', 'A1left1', 'D3right3', 'A2left2', 'D0bottom3', 'D3top3', 'C0bottom2', 'A3top0', 'C0bottom2', 'D2right2', 'D0right0', 'D1right1', 'A0bottom0', 'D0bottom3', 'D0right0', 'A2left2', 'A0left0', 'D0bottom3', 'A3left3', 'A3top0', 'B3top1', 'D3top3', 'D1right1', 'A1left1', 'D3right3', 'A1left1', 'A3left3', 'D3right3', 'C3top2', 'D3top3', 'A3top0', 'D3top3', 'C3top2', 'D3top3', 'A2left2', 'D3top3', 'A0left0', 'B0bottom1', 'B0bottom1', 'D3top3', 'D0right0', 'D2right2', 'B3top1', 'C3top2', 'A0bottom0', 'C0bottom2', 'A0left0', 'D0bottom3', 'D2right2', 'A1left1', 'A3top0', 'D0bottom3', 'D0bottom3', 'D2right2', 'D3right3', 'A2left2', 'A2left2', 'A0left0', 'A0bottom0', 'A3top0', 'D0bottom3', 'A1left1', 'A0left0', 'B0bottom1', 'D1right1', 'C3top2', 'D2right2', 'D0right0', 'D1right1', 'D0right0', 'B0bottom1', 'D0right0', 'B3top1', 'D0bottom3', 'C3top2', 'A0left0', 'D0bottom3', 'B3top1', 'D1right1', 'A1left1', 'B0bottom1', 'A0bottom0', 'C0bottom2', 'A1left1', 'A1left1', 'D2right2', 'A2left2', 'B3top1', 'D2right2', 'D0right0', 'B3top1', 'B0bottom1', 'A1left1', 'B3top1', 'C0bottom2', 'A3top0', 'A0left0', 'D0bottom3', 'D2right2', 'D0right0', 'A3top0', 'D2right2', 'D1right1', 'A3top0', 'A3top0', 'D3right3', 'A3left3', 'C3top2', 'D1right1', 'A0bottom0', 'A1left1', 'A3left3', 'D3top3', 'D2right2', 'D1right1', 'A1left1', 'A2left2', 'C3top2', 'C3top2', 'D1right1', 'D1right1', 'C0bottom2', 'D1right1', 'D0bottom3', 'A2left2', 'A3top0', 'D3top3', 'D0bottom3', 'D2right2', 'B0bottom1', 'D1right1', 'A2left2', 'D0right0', 'A1left1', 'D3top3', 'C0bottom2', 'B3top1', 'C0bottom2', 'D1right1', 'A3left3', 'A3left3', 'B3top1', 'A2left2', 'D0bottom3', 'A1left1', 'A3left3', 'A0left0', 'A3top0', 'D0right0', 'C3top2', 'D0bottom3', 'D3top3', 'D0right0', 'B0bottom1', 'B3top1', 'A1left1', 'A0bottom0', 'A2left2', 'A3left3', 'B0bottom1', 'A2left2', 'A0left0', 'D0right0', 'A2left2', 'D2right2', 'D0right0', 'D0right0', 'A0bottom0', 'D3right3', 'A0bottom0', 'D3top3', 'D1right1', 'C3top2', 'A0bottom0', 'B0bottom1', 'A1left1', 'D3top3', 'A3top0', 'A2left2', 'A1left1', 'B0bottom1', 'A1left1', 'D3top3', 'D2right2', 'D1right1', 'A0left0', 'A1left1', 'B3top1', 'D3top3', 'D3right3', 'A0bottom0', 'C0bottom2', 'D2right2', 'D1right1', 'A2left2', 'D1right1', 'D0right0', 'D0bottom3', 'C3top2', 'A1left1', 'A1left1', 'D3top3', 'D1right1', 'A0bottom0', 'D3top3', 'D3top3', 'B3top1', 'D2right2', 'B0bottom1', 'D3top3', 'D3top3', 'D2right2', 'A2left2', 'A3left3', 'D1right1', 'A2left2', 'D2right2', 'A1left1', 'B0bottom1', 'D3right3', 'D3right3', 'A0bottom0', 'A0bottom0', 'A0bottom0', 'A3top0', 'D2right2', 'D0bottom3', 'A2left2', 'D2right2', 'B3top1', 'A0bottom0', 'D1right1', 'D0bottom3', 'A3left3', 'D0bottom3', 'A3left3', 'A3left3', 'D2right2', 'B3top1', 'A1left1', 'D2right2', 'A3left3', 'B0bottom1', 'C0bottom2', 'A0bottom0', 'A3top0', 'A3left3', 'A3left3', 'A2left2', 'A0bottom0', 'A3left3', 'A1left1', 'A0left0', 'B0bottom1', 'A0left0', 'B0bottom1', 'C3top2', 'B0bottom1', 'A0bottom0', 'B3top1', 'A3top0', 'C3top2', 'C0bottom2', 'A1left1', 'D0right0', 'C3top2', 'B3top1', 'D1right1', 'A3left3', 'A3left3', 'D2right2', 'C3top2', 'D1right1', 'B0bottom1', 'D3right3', 'A3left3', 'D3right3', 'C3top2', 'D2right2', 'A1left1', 'A2left2', 'B0bottom1', 'A3top0', 'D0bottom3', 'A3left3', 'A3left3', 'C0bottom2', 'A3left3', 'C3top2', 'C0bottom2', 'A0bottom0', 'A0left0', 'D3top3', 'A3top0', 'D3right3', 'D2right2', 'B0bottom1', 'A2left2', 'D2right2', 'A1left1', 'B3top1', 'B3top1', 'A1left1', 'A3top0', 'D3top3', 'A0left0', 'D2right2', 'D0bottom3', 'D0bottom3', 'C0bottom2', 'A0left0', 'B0bottom1', 'A3left3', 'C3top2', 'A0left0', 'C3top2', 'B0bottom1', 'B0bottom1', 'B0bottom1', 'D0right0', 'D3top3', 'A0bottom0', 'B3top1', 'D3right3', 'C0bottom2', 'D3top3', 'A1left1', 'A0left0', 'D3right3', 'A0left0', 'D2right2', 'A3top0', 'A0left0', 'A3top0', 'A2left2', 'A2left2', 'C0bottom2', 'D3top3', 'D3top3', 'D1right1', 'A3top0', 'D3right3', 'B3top1', 'D1right1', 'D2right2', 'C0bottom2', 'C3top2', 'C3top2', 'D0bottom3', 'A2left2', 'A0bottom0', 'B0bottom1', 'D1right1', 'D0right0', 'D1right1', 'D3right3', 'D3right3', 'B0bottom1', 'B0bottom1', 'A0bottom0', 'C0bottom2', 'D3right3', 'A2left2', 'A0bottom0', 'A3top0', 'A0left0', 'B0bottom1', 'A1left1', 'D3top3', 'B3top1', 'A0bottom0', 'A3top0', 'D0right0', 'D1right1', 'D0right0', 'C3top2', 'A1left1', 'A3left3', 'A2left2', 'D3top3', 'C3top2', 'A3left3', 'B0bottom1', 'A0bottom0', 'D3right3', 'D0right0', 'C0bottom2', 'A3top0', 'A0bottom0', 'D3top3', 'C3top2', 'A1left1', 'A0bottom0', 'D1right1', 'D3right3', 'C3top2', 'D0right0', 'A3top0', 'B3top1', 'D1right1', 'D3top3', 'C0bottom2', 'A1left1', 'D2right2', 'A3left3', 'D3right3', 'C0bottom2', 'A2left2', 'C0bottom2', 'A3top0', 'C0bottom2', 'A0bottom0', 'A3top0', 'A3top0', 'D0bottom3', 'B0bottom1', 'D3right3', 'D3top3', 'A1left1', 'B3top1', 'B3top1', 'D1right1', 'B0bottom1', 'D0bottom3', 'B0bottom1', 'C3top2', 'A1left1', 'D3top3', 'D3right3', 'D0right0', 'D2right2', 'A0left0', 'B0bottom1', 'B3top1', 'D0right0', 'B0bottom1', 'A2left2', 'D3right3', 'D0bottom3', 'A3top0', 'D0bottom3', 'A2left2', 'D0bottom3', 'B0bottom1', 'A2left2', 'A1left1', 'B0bottom1', 'C0bottom2', 'D2right2', 'D2right2', 'D0bottom3', 'D1right1', 'D0bottom3', 'A3left3', 'D2right2', 'D3right3', 'C3top2', 'A1left1', 'B0bottom1', 'A0bottom0', 'A0left0', 'D3top3', 'D3top3', 'D0bottom3', 'C3top2', 'D2right2', 'A3top0', 'D3right3', 'A1left1', 'D3right3', 'D3top3', 'A2left2', 'A1left1', 'B3top1', 'A2left2', 'D0right0', 'A0bottom0', 'C0bottom2', 'B0bottom1', 'A0bottom0', 'A2left2', 'A0left0', 'D1right1', 'D0right0', 'D0right0', 'D3top3', 'A0bottom0', 'A3top0', 'C0bottom2', 'D0right0', 'D3right3', 'C0bottom2', 'B3top1', 'D3top3', 'D0right0', 'A3left3', 'D3right3', 'A2left2', 'D3right3', 'D0right0', 'D1right1', 'C3top2', 'D0right0', 'B3top1', 'C3top2', 'A0left0', 'A0left0', 'C0bottom2', 'D0right0', 'D2right2', 'B0bottom1', 'D0bottom3', 'A1left1', 'A0left0', 'B0bottom1', 'D0bottom3', 'D0bottom3']
# ####hangzhou
# net_file = r'D:\intereting\DynamicNavigation\findPath\findPath\test_data\孙倩\Hangzhou-4x4\hangzhou.net.xml'# 路网文件
# rou_file = r'D:\intereting\DynamicNavigation\findPath\findPath\test_data\孙倩\Hangzhou-4x4\hangzhou.rou.xml'# 路网文件
# cfg_file = r'D:\intereting\DynamicNavigation\findPath\findPath\test_data\孙倩\Hangzhou-4x4\hangzhou.sumocfg'
# ####hangzhou_5
# car_num = 88
# start_edges = ['road_4_0_1', 'road_5_4_2', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_0_1_0', 'road_0_1_0', 'road_1_0_1', 'road_4_0_1', 'road_0_4_0', 'road_4_0_1', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_5_2_2', 'road_5_4_2', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_1_0', 'road_0_1_0', 'road_0_1_0', 'road_1_5_3', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_5_2_2', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_5_2_2', 'road_5_3_2', 'road_1_0_1', 'road_5_4_2', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_5_2_2', 'road_4_0_1', 'road_1_0_1', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_5_3_2', 'road_0_4_0', 'road_5_2_2', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_1_5_3', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_0_4_0', 'road_5_3_2', 'road_5_3_2', 'road_5_2_2', 'road_4_0_1', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2']
# end_edges = ['road_4_2_0', 'road_1_3_2', 'road_1_4_1', 'road_3_4_1', 'road_1_3_2', 'road_4_4_1', 'road_2_4_1', 'road_4_1_3', 'road_1_1_3', 'road_2_1_3', 'road_4_1_3', 'road_4_4_0', 'road_4_1_0', 'road_1_1_3', 'road_4_4_1', 'road_4_4_1', 'road_3_1_3', 'road_4_3_0', 'road_3_4_1', 'road_1_2_2', 'road_1_4_1', 'road_1_3_2', 'road_4_1_0', 'road_3_1_3', 'road_4_1_0', 'road_1_3_2', 'road_1_3_2', 'road_4_1_3', 'road_1_1_3', 'road_4_4_1', 'road_4_3_0', 'road_4_4_0', 'road_1_3_2', 'road_4_4_0', 'road_1_4_2', 'road_1_1_2', 'road_1_4_2', 'road_3_4_1', 'road_1_1_2', 'road_4_1_3', 'road_1_4_1', 'road_1_1_2', 'road_4_1_3', 'road_2_4_1', 'road_3_4_1', 'road_3_4_1', 'road_2_1_3', 'road_3_4_1', 'road_4_4_1', 'road_4_3_3', 'road_4_1_0', 'road_1_2_2', 'road_1_4_2', 'road_1_3_2', 'road_3_1_3', 'road_1_2_2', 'road_4_4_1', 'road_1_1_2', 'road_1_3_2', 'road_4_1_0', 'road_1_1_2', 'road_1_3_2', 'road_4_4_1', 'road_4_3_0', 'road_4_1_3', 'road_1_2_2', 'road_1_4_1', 'road_1_1_3', 'road_4_1_3', 'road_1_2_2', 'road_2_1_3', 'road_4_1_0', 'road_4_1_3', 'road_1_4_1', 'road_4_2_0', 'road_4_3_0', 'road_4_4_1', 'road_4_4_0', 'road_3_4_1', 'road_4_2_0', 'road_4_4_0', 'road_4_4_0', 'road_1_1_3', 'road_1_1_3', 'road_4_4_0', 'road_1_3_2', 'road_1_4_1', 'road_3_4_1']
# ###hangzhou_10
# car_num = 166
# start_edges = ['road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_0_1_0', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_1_5_3', 'road_4_0_1', 'road_0_4_0', 'road_0_4_0', 'road_1_5_3', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_4_0_1', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_0_1_0', 'road_4_0_1', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_0_1_0', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_1_5_3', 'road_1_5_3', 'road_5_2_2', 'road_1_0_1', 'road_5_4_2', 'road_0_2_0', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_5_2_2', 'road_5_4_2', 'road_4_0_1', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_0_4_0', 'road_1_0_1', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_5_2_2', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_0_2_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_2_0', 'road_0_1_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_1_0_1', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_1_0_1', 'road_5_4_2', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_0_2_0', 'road_0_4_0', 'road_0_4_0', 'road_1_5_3', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_4_0_1', 'road_0_1_0', 'road_0_1_0', 'road_4_0_1', 'road_0_1_0', 'road_0_2_0', 'road_0_1_0', 'road_5_2_2', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_5_3_2', 'road_0_4_0', 'road_5_2_2', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0']
# end_edges = ['road_3_1_3', 'road_1_4_1', 'road_2_1_3', 'road_4_2_0', 'road_4_4_0', 'road_3_4_1', 'road_1_3_2', 'road_4_4_3', 'road_1_3_2', 'road_3_1_3', 'road_2_1_3', 'road_4_3_0', 'road_1_4_2', 'road_2_4_1', 'road_4_1_0', 'road_4_4_1', 'road_1_2_2', 'road_2_1_3', 'road_3_4_1', 'road_4_1_0', 'road_4_4_0', 'road_4_4_1', 'road_4_1_3', 'road_2_4_1', 'road_4_4_1', 'road_4_4_1', 'road_4_3_0', 'road_1_1_3', 'road_4_2_0', 'road_4_4_1', 'road_1_1_3', 'road_1_3_2', 'road_1_4_1', 'road_1_3_2', 'road_3_4_1', 'road_2_1_3', 'road_4_1_0', 'road_1_3_2', 'road_2_4_1', 'road_1_3_2', 'road_4_4_0', 'road_1_4_2', 'road_3_4_1', 'road_1_2_2', 'road_1_4_2', 'road_4_2_0', 'road_4_4_0', 'road_1_2_2', 'road_2_4_1', 'road_3_1_3', 'road_4_4_1', 'road_3_1_3', 'road_2_1_3', 'road_1_2_2', 'road_1_4_2', 'road_4_1_0', 'road_4_4_0', 'road_2_1_3', 'road_4_1_0', 'road_4_1_0', 'road_1_4_1', 'road_2_4_1', 'road_4_1_0', 'road_4_4_1', 'road_3_1_3', 'road_3_3_3', 'road_4_4_1', 'road_4_3_0', 'road_1_4_1', 'road_3_1_3', 'road_3_4_1', 'road_2_4_1', 'road_4_1_0', 'road_4_1_0', 'road_1_1_1', 'road_3_4_1', 'road_3_4_1', 'road_1_1_3', 'road_2_4_1', 'road_4_1_3', 'road_2_4_1', 'road_3_4_1', 'road_2_1_3', 'road_1_2_2', 'road_3_1_3', 'road_4_4_1', 'road_1_1_3', 'road_1_2_2', 'road_2_4_1', 'road_2_1_3', 'road_1_1_2', 'road_4_1_3', 'road_1_2_2', 'road_3_4_1', 'road_1_1_3', 'road_4_4_1', 'road_3_4_1', 'road_4_1_3', 'road_1_3_2', 'road_4_1_0', 'road_1_4_1', 'road_4_4_1', 'road_4_1_0', 'road_1_4_1', 'road_2_1_2', 'road_4_2_0', 'road_3_4_1', 'road_1_4_2', 'road_1_1_3', 'road_3_4_1', 'road_1_2_2', 'road_2_1_3', 'road_4_4_1', 'road_1_4_2', 'road_4_2_0', 'road_1_3_2', 'road_3_1_3', 'road_3_4_1', 'road_1_4_2', 'road_1_4_2', 'road_4_3_0', 'road_3_4_1', 'road_1_4_2', 'road_1_1_3', 'road_2_4_1', 'road_1_1_3', 'road_3_1_3', 'road_4_4_1', 'road_4_1_0', 'road_4_1_3', 'road_3_1_3', 'road_4_4_1', 'road_4_1_0', 'road_1_1_3', 'road_2_1_3', 'road_4_3_0', 'road_3_1_3', 'road_1_4_1', 'road_3_4_1', 'road_2_4_1', 'road_1_1_2', 'road_3_4_1', 'road_1_1_3', 'road_1_3_2', 'road_3_4_1', 'road_1_1_3', 'road_4_4_1', 'road_1_3_2', 'road_1_1_2', 'road_4_4_0', 'road_3_4_1', 'road_4_4_1', 'road_4_1_0', 'road_3_4_1', 'road_1_4_1', 'road_1_4_2', 'road_4_4_0', 'road_1_3_2', 'road_1_1_2', 'road_1_2_2', 'road_1_3_2', 'road_4_4_1', 'road_4_2_0', 'road_1_3_2', 'road_1_1_2', 'road_4_4_1']
# ####hangzhou_20
# car_num = 332
# start_edges = ['road_5_3_2', 'road_4_0_1', 'road_1_0_1', 'road_1_5_3', 'road_5_3_2', 'road_0_1_0', 'road_5_2_2', 'road_4_0_1', 'road_4_0_1', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_5_3_2', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_4_0_1', 'road_1_5_3', 'road_5_4_2', 'road_0_1_0', 'road_5_3_2', 'road_1_0_1', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_5_4_2', 'road_0_2_0', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_5_4_2', 'road_4_0_1', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_5_2_2', 'road_5_3_2', 'road_0_4_0', 'road_4_0_1', 'road_0_1_0', 'road_0_1_0', 'road_5_3_2', 'road_0_1_0', 'road_1_0_1', 'road_1_5_3', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_0_4_0', 'road_1_5_3', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_2_2', 'road_5_4_2', 'road_5_2_2', 'road_4_0_1', 'road_0_4_0', 'road_5_2_2', 'road_5_4_2', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_5_2_2', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_0_2_0', 'road_1_0_1', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_0_4_0', 'road_1_0_1', 'road_0_2_0', 'road_0_4_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_4_0_1', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_0_1_0', 'road_5_2_2', 'road_5_3_2', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_2_0', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_0_1_0', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_5_2_2', 'road_0_1_0', 'road_1_5_3', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_4_0_1', 'road_5_2_2', 'road_0_1_0', 'road_0_2_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_0_1_0', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_5_4_2', 'road_5_2_2', 'road_5_3_2', 'road_0_4_0', 'road_1_0_1', 'road_5_4_2', 'road_1_5_3', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_1_5_3', 'road_5_4_2', 'road_5_3_2', 'road_0_1_0', 'road_1_5_3', 'road_0_4_0', 'road_4_0_1', 'road_4_0_1', 'road_0_1_0', 'road_0_2_0', 'road_5_4_2', 'road_0_1_0', 'road_5_3_2', 'road_1_0_1', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_0_1_0', 'road_5_4_2', 'road_4_0_1', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_1_0_1', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_0_1_0', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_0_4_0', 'road_0_2_0', 'road_4_0_1', 'road_0_4_0', 'road_0_2_0', 'road_1_5_3', 'road_0_4_0', 'road_5_3_2', 'road_0_1_0', 'road_5_2_2', 'road_1_0_1', 'road_1_5_3', 'road_4_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_5_2_2', 'road_1_5_3', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_5_4_2', 'road_5_4_2', 'road_4_0_1', 'road_0_2_0', 'road_5_4_2', 'road_0_4_0']
# end_edges = ['road_4_4_0', 'road_1_1_3', 'road_1_4_1', 'road_2_4_1', 'road_4_4_1', 'road_1_1_3', 'road_2_4_1', 'road_4_4_0', 'road_1_1_2', 'road_4_4_0', 'road_4_4_1', 'road_1_2_2', 'road_4_4_0', 'road_1_2_2', 'road_3_4_1', 'road_1_3_2', 'road_1_1_2', 'road_4_4_0', 'road_1_4_1', 'road_3_1_3', 'road_2_4_1', 'road_4_2_0', 'road_1_4_2', 'road_2_4_1', 'road_4_2_0', 'road_4_4_1', 'road_4_1_3', 'road_4_4_1', 'road_3_4_1', 'road_2_4_1', 'road_2_4_1', 'road_3_4_1', 'road_4_4_0', 'road_1_4_2', 'road_4_1_0', 'road_1_3_2', 'road_4_4_1', 'road_1_4_1', 'road_4_4_1', 'road_1_2_2', 'road_4_4_1', 'road_4_2_0', 'road_3_4_1', 'road_2_1_3', 'road_1_1_3', 'road_4_4_1', 'road_2_1_3', 'road_1_1_2', 'road_3_1_3', 'road_2_1_3', 'road_4_4_0', 'road_4_4_0', 'road_4_1_3', 'road_1_4_2', 'road_4_4_1', 'road_4_1_0', 'road_1_1_3', 'road_3_1_3', 'road_4_4_1', 'road_4_1_0', 'road_1_3_1', 'road_3_1_3', 'road_1_1_3', 'road_1_3_2', 'road_4_3_0', 'road_4_1_0', 'road_3_4_1', 'road_4_3_0', 'road_4_1_0', 'road_4_1_0', 'road_4_4_0', 'road_1_1_3', 'road_4_1_3', 'road_1_3_2', 'road_3_1_3', 'road_4_4_0', 'road_2_4_1', 'road_1_4_1', 'road_1_1_3', 'road_2_4_1', 'road_1_4_2', 'road_2_1_3', 'road_1_1_2', 'road_4_1_0', 'road_3_3_3', 'road_4_4_1', 'road_1_2_2', 'road_4_1_0', 'road_4_4_0', 'road_4_4_1', 'road_3_4_1', 'road_1_3_2', 'road_1_4_1', 'road_2_4_1', 'road_3_3_1', 'road_4_1_3', 'road_1_4_2', 'road_1_3_2', 'road_4_3_0', 'road_4_1_0', 'road_4_1_0', 'road_4_4_1', 'road_4_4_1', 'road_2_4_1', 'road_1_2_2', 'road_1_3_2', 'road_1_1_3', 'road_3_1_3', 'road_4_1_3', 'road_1_4_2', 'road_1_3_2', 'road_2_1_3', 'road_2_4_1', 'road_3_4_1', 'road_3_1_3', 'road_1_1_3', 'road_2_1_3', 'road_4_4_0', 'road_1_1_3', 'road_3_1_3', 'road_3_4_1', 'road_4_1_0', 'road_4_2_0', 'road_3_4_1', 'road_4_4_1', 'road_1_1_2', 'road_1_4_1', 'road_2_1_3', 'road_4_4_0', 'road_3_1_3', 'road_4_1_0', 'road_1_1_3', 'road_2_4_1', 'road_1_4_2', 'road_1_4_2', 'road_4_4_0', 'road_1_3_2', 'road_1_1_3', 'road_4_1_0', 'road_1_4_2', 'road_4_2_0', 'road_1_2_2', 'road_4_4_1', 'road_4_1_0', 'road_1_1_2', 'road_2_4_1', 'road_4_4_1', 'road_3_1_3', 'road_4_1_3', 'road_2_4_1', 'road_3_4_1', 'road_1_3_2', 'road_1_3_2', 'road_3_4_1', 'road_1_3_2', 'road_1_3_2', 'road_1_4_1', 'road_3_4_1', 'road_1_4_1', 'road_3_4_1', 'road_1_2_2', 'road_1_4_1', 'road_1_4_1', 'road_3_1_3', 'road_1_2_2', 'road_4_4_1', 'road_3_4_1', 'road_4_4_1', 'road_4_1_3', 'road_4_3_0', 'road_4_4_1', 'road_4_4_1', 'road_1_1_2', 'road_2_4_1', 'road_1_2_2', 'road_4_1_0', 'road_3_1_3', 'road_4_4_1', 'road_1_2_2', 'road_1_2_2', 'road_1_4_1', 'road_2_1_3', 'road_4_1_3', 'road_1_1_2', 'road_4_4_1', 'road_1_2_2', 'road_2_4_1', 'road_4_1_3', 'road_2_1_3', 'road_3_4_1', 'road_1_1_2', 'road_4_4_0', 'road_4_4_1', 'road_4_1_3', 'road_1_1_3', 'road_3_1_3', 'road_3_4_1', 'road_3_4_1', 'road_2_1_3', 'road_1_3_1', 'road_3_1_3', 'road_3_1_3', 'road_1_1_3', 'road_4_4_1', 'road_4_3_0', 'road_1_3_2', 'road_2_1_3', 'road_4_3_0', 'road_3_4_1', 'road_1_1_2', 'road_1_3_2', 'road_3_1_3', 'road_4_4_1', 'road_4_2_0', 'road_4_1_0', 'road_4_3_0', 'road_4_4_0', 'road_2_4_1', 'road_4_4_1', 'road_1_2_2', 'road_2_1_3', 'road_4_1_3', 'road_4_1_0', 'road_1_4_2', 'road_4_4_1', 'road_2_4_1', 'road_4_2_0', 'road_3_4_1', 'road_4_3_0', 'road_4_1_3', 'road_1_3_1', 'road_4_1_0', 'road_1_4_2', 'road_1_4_2', 'road_3_4_1', 'road_3_4_1', 'road_4_4_0', 'road_1_2_2', 'road_2_4_1', 'road_4_2_0', 'road_4_1_0', 'road_1_1_3', 'road_4_4_1', 'road_3_1_3', 'road_1_4_1', 'road_1_3_2', 'road_4_1_3', 'road_3_4_1', 'road_3_1_3', 'road_1_4_2', 'road_1_1_3', 'road_4_3_0', 'road_4_1_0', 'road_2_1_3', 'road_4_1_3', 'road_4_4_1', 'road_1_1_3', 'road_1_3_2', 'road_2_1_3', 'road_4_4_1', 'road_1_4_1', 'road_3_1_3', 'road_1_1_2', 'road_1_1_3', 'road_4_4_1', 'road_4_1_3', 'road_4_1_0', 'road_4_4_1', 'road_4_4_0', 'road_1_2_2', 'road_1_1_3', 'road_1_4_2', 'road_4_4_1', 'road_4_1_0', 'road_1_2_2', 'road_4_3_0', 'road_1_1_2', 'road_1_4_1', 'road_4_4_1', 'road_4_4_1', 'road_1_4_2', 'road_4_4_1', 'road_1_1_2', 'road_1_4_1', 'road_4_1_0', 'road_1_2_2', 'road_1_1_3', 'road_3_4_1', 'road_1_4_1', 'road_2_1_3', 'road_1_1_3', 'road_2_4_1', 'road_1_1_3', 'road_4_4_1', 'road_4_4_1', 'road_1_1_2', 'road_4_4_0', 'road_1_4_2', 'road_1_4_2', 'road_1_2_2', 'road_1_4_1', 'road_1_1_2', 'road_2_1_3', 'road_4_1_0', 'road_2_4_1', 'road_1_1_1', 'road_3_4_1', 'road_2_4_1', 'road_2_4_1', 'road_4_1_0', 'road_4_1_3', 'road_1_2_2', 'road_4_3_0', 'road_4_4_1', 'road_1_1_3', 'road_1_4_1', 'road_1_2_2', 'road_3_4_1', 'road_4_4_1', 'road_2_1_3', 'road_4_3_0', 'road_1_4_2', 'road_2_4_1', 'road_4_4_1', 'road_1_3_2', 'road_2_1_3', 'road_4_2_0', 'road_4_4_1', 'road_4_1_0', 'road_1_3_2', 'road_1_4_2', 'road_2_4_1']
# ##hangzhou_30
# car_num = 498
# start_edges = ['road_5_4_2', 'road_5_2_2', 'road_1_0_1', 'road_0_4_0', 'road_5_3_2', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_0_2_0', 'road_4_0_1', 'road_0_4_0', 'road_1_0_1', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_4_0_1', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_4_0_1', 'road_5_3_2', 'road_0_1_0', 'road_5_3_2', 'road_0_4_0', 'road_1_0_1', 'road_5_4_2', 'road_4_0_1', 'road_5_3_2', 'road_0_1_0', 'road_1_0_1', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_3_2', 'road_1_0_1', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_5_3_2', 'road_0_4_0', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_0_1_0', 'road_0_4_0', 'road_1_5_3', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_0_1_0', 'road_0_1_0', 'road_0_2_0', 'road_0_4_0', 'road_4_0_1', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_1_0', 'road_0_2_0', 'road_5_3_2', 'road_0_1_0', 'road_0_1_0', 'road_1_5_3', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_5_3_2', 'road_5_2_2', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_0_1_0', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_5_3_2', 'road_5_2_2', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_1_0', 'road_4_0_1', 'road_5_2_2', 'road_1_5_3', 'road_5_3_2', 'road_0_4_0', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_5_4_2', 'road_1_0_1', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_0_1_0', 'road_1_5_3', 'road_5_3_2', 'road_0_4_0', 'road_0_2_0', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_2_0', 'road_5_4_2', 'road_5_3_2', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_5_3_2', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_1_0_1', 'road_1_5_3', 'road_0_2_0', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_5_2_2', 'road_5_4_2', 'road_5_3_2', 'road_4_0_1', 'road_0_4_0', 'road_5_4_2', 'road_5_2_2', 'road_4_0_1', 'road_0_1_0', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_4_0_1', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_0_4_0', 'road_5_4_2', 'road_5_2_2', 'road_1_0_1', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_2_0', 'road_5_4_2', 'road_0_4_0', 'road_1_5_3', 'road_0_2_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_4_0_1', 'road_5_4_2', 'road_5_4_2', 'road_1_0_1', 'road_0_2_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_5_2_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_2_0', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_4_0_1', 'road_0_2_0', 'road_1_5_3', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_1_5_3', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_4_0_1', 'road_0_4_0', 'road_0_4_0', 'road_5_2_2', 'road_0_1_0', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_5_2_2', 'road_0_4_0', 'road_1_5_3', 'road_0_4_0', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_1_0_1', 'road_0_1_0', 'road_0_4_0', 'road_0_2_0', 'road_5_3_2', 'road_0_4_0', 'road_0_1_0', 'road_1_5_3', 'road_5_2_2', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_0_1_0', 'road_0_1_0', 'road_1_0_1', 'road_1_5_3', 'road_0_2_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_1_0_1', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_0_1_0', 'road_1_0_1', 'road_0_4_0', 'road_5_2_2', 'road_0_1_0', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_0_1_0', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_1_5_3', 'road_5_3_2', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_1_5_3', 'road_5_4_2', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_1_0', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_5_2_2', 'road_1_0_1', 'road_4_0_1', 'road_0_1_0', 'road_0_4_0', 'road_5_3_2', 'road_1_5_3', 'road_0_4_0', 'road_4_0_1', 'road_0_1_0', 'road_1_5_3', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_4_0', 'road_1_0_1', 'road_5_2_2', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_5_2_2', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_0_2_0', 'road_0_1_0', 'road_5_3_2', 'road_1_0_1', 'road_1_5_3', 'road_4_0_1', 'road_5_2_2', 'road_4_0_1', 'road_5_3_2', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_5_3_2', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_1_0_1', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_2_2', 'road_5_4_2', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_1_0_1', 'road_0_4_0', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_1_0_1']
# end_edges = ['road_1_4_1', 'road_2_1_3', 'road_1_4_1', 'road_2_3_1', 'road_4_4_0', 'road_3_1_3', 'road_4_1_3', 'road_3_4_1', 'road_1_4_1', 'road_2_4_1', 'road_4_4_1', 'road_1_1_3', 'road_1_3_2', 'road_1_4_1', 'road_4_4_0', 'road_2_1_3', 'road_4_1_0', 'road_4_4_0', 'road_4_2_0', 'road_2_4_1', 'road_4_1_0', 'road_2_1_3', 'road_3_4_1', 'road_4_4_0', 'road_1_1_3', 'road_4_1_0', 'road_1_4_2', 'road_1_4_1', 'road_3_4_1', 'road_3_1_3', 'road_4_4_0', 'road_1_1_3', 'road_1_4_1', 'road_1_3_1', 'road_1_4_1', 'road_3_4_1', 'road_2_4_1', 'road_3_4_1', 'road_1_1_3', 'road_2_1_3', 'road_3_4_1', 'road_1_2_2', 'road_1_1_2', 'road_4_4_1', 'road_4_1_3', 'road_2_1_3', 'road_1_3_2', 'road_4_1_0', 'road_4_1_3', 'road_1_4_1', 'road_4_4_0', 'road_1_1_3', 'road_1_4_2', 'road_4_1_0', 'road_1_1_3', 'road_3_2_0', 'road_2_1_3', 'road_3_4_1', 'road_3_4_1', 'road_1_4_1', 'road_2_1_3', 'road_1_3_2', 'road_4_1_3', 'road_4_4_0', 'road_1_1_2', 'road_3_1_3', 'road_4_1_0', 'road_1_1_2', 'road_1_4_1', 'road_3_4_1', 'road_1_1_3', 'road_1_4_2', 'road_2_1_3', 'road_2_4_1', 'road_1_4_2', 'road_1_2_2', 'road_4_3_0', 'road_3_1_3', 'road_2_2_1', 'road_4_4_0', 'road_1_4_2', 'road_4_4_1', 'road_3_4_1', 'road_4_1_3', 'road_1_1_3', 'road_3_4_1', 'road_4_4_1', 'road_4_2_0', 'road_4_4_1', 'road_1_2_2', 'road_1_2_2', 'road_3_1_3', 'road_3_1_3', 'road_1_1_2', 'road_1_3_1', 'road_4_2_0', 'road_1_1_3', 'road_3_1_3', 'road_1_4_2', 'road_1_3_2', 'road_4_3_0', 'road_3_4_1', 'road_2_4_1', 'road_3_4_1', 'road_1_1_2', 'road_4_1_0', 'road_2_4_1', 'road_4_3_0', 'road_1_1_3', 'road_3_4_1', 'road_4_4_1', 'road_3_1_3', 'road_3_1_3', 'road_1_1_3', 'road_4_4_0', 'road_2_1_3', 'road_1_4_1', 'road_2_1_3', 'road_1_1_3', 'road_4_4_1', 'road_4_3_0', 'road_1_1_2', 'road_4_4_1', 'road_4_4_0', 'road_1_4_1', 'road_4_4_1', 'road_4_4_1', 'road_4_1_0', 'road_3_1_3', 'road_1_1_3', 'road_2_1_3', 'road_2_1_3', 'road_4_1_0', 'road_4_4_0', 'road_3_4_1', 'road_3_1_3', 'road_1_4_2', 'road_1_3_2', 'road_3_3_1', 'road_4_4_0', 'road_4_1_3', 'road_1_4_2', 'road_1_2_2', 'road_4_1_0', 'road_4_1_0', 'road_1_1_2', 'road_2_4_1', 'road_4_4_1', 'road_4_1_0', 'road_1_3_2', 'road_4_1_0', 'road_1_3_2', 'road_1_4_2', 'road_2_1_3', 'road_4_4_0', 'road_2_1_3', 'road_4_4_0', 'road_2_4_1', 'road_2_4_1', 'road_1_4_2', 'road_4_4_1', 'road_1_2_2', 'road_1_1_3', 'road_2_1_3', 'road_1_4_2', 'road_4_4_0', 'road_4_2_0', 'road_4_4_1', 'road_4_1_3', 'road_4_2_0', 'road_4_1_3', 'road_2_4_1', 'road_2_4_1', 'road_2_1_3', 'road_1_4_1', 'road_1_4_2', 'road_3_4_1', 'road_1_2_2', 'road_1_4_1', 'road_4_4_0', 'road_2_1_3', 'road_4_4_0', 'road_4_1_0', 'road_1_3_2', 'road_3_1_3', 'road_2_4_1', 'road_1_1_2', 'road_1_4_2', 'road_4_1_3', 'road_4_4_1', 'road_4_4_0', 'road_1_3_2', 'road_1_1_3', 'road_4_3_0', 'road_1_4_1', 'road_1_3_2', 'road_4_1_0', 'road_4_1_0', 'road_1_4_2', 'road_3_1_3', 'road_1_1_2', 'road_1_1_3', 'road_3_4_1', 'road_4_3_0', 'road_2_4_1', 'road_1_1_2', 'road_4_2_0', 'road_1_1_2', 'road_3_1_3', 'road_3_3_3', 'road_1_1_2', 'road_3_4_1', 'road_1_4_1', 'road_4_2_0', 'road_1_1_2', 'road_3_1_3', 'road_4_1_0', 'road_4_4_1', 'road_4_1_3', 'road_4_4_0', 'road_4_4_0', 'road_1_4_1', 'road_2_4_1', 'road_4_4_0', 'road_3_4_1', 'road_2_4_1', 'road_3_4_1', 'road_3_4_1', 'road_1_1_3', 'road_2_4_1', 'road_4_4_1', 'road_4_1_0', 'road_2_4_1', 'road_4_4_0', 'road_1_4_1', 'road_1_1_1', 'road_3_4_1', 'road_4_4_0', 'road_2_1_3', 'road_3_1_3', 'road_2_4_1', 'road_3_4_1', 'road_1_1_2', 'road_4_4_1', 'road_1_3_2', 'road_3_4_1', 'road_4_3_3', 'road_4_4_1', 'road_4_4_1', 'road_2_1_3', 'road_4_1_3', 'road_4_4_1', 'road_3_1_3', 'road_4_4_1', 'road_1_1_2', 'road_2_1_3', 'road_1_1_3', 'road_1_2_2', 'road_1_2_2', 'road_1_1_2', 'road_1_1_3', 'road_4_3_0', 'road_4_1_0', 'road_1_3_2', 'road_1_2_2', 'road_1_2_2', 'road_4_4_1', 'road_2_1_3', 'road_2_4_1', 'road_4_1_3', 'road_1_3_2', 'road_4_4_1', 'road_1_2_2', 'road_1_4_1', 'road_1_3_2', 'road_1_3_2', 'road_4_4_1', 'road_2_1_3', 'road_4_4_1', 'road_4_4_1', 'road_1_1_3', 'road_4_4_0', 'road_2_1_3', 'road_1_1_2', 'road_4_4_0', 'road_4_4_1', 'road_1_3_2', 'road_3_1_3', 'road_3_4_1', 'road_2_1_3', 'road_1_3_2', 'road_1_4_1', 'road_4_2_0', 'road_4_1_3', 'road_1_2_2', 'road_1_1_3', 'road_1_4_2', 'road_1_2_2', 'road_2_4_1', 'road_3_4_1', 'road_4_2_0', 'road_2_4_1', 'road_1_2_2', 'road_1_1_3', 'road_4_3_0', 'road_3_1_3', 'road_2_4_1', 'road_3_4_1', 'road_1_1_2', 'road_1_4_2', 'road_1_1_2', 'road_1_3_0', 'road_1_3_2', 'road_4_4_0', 'road_2_4_1', 'road_4_3_0', 'road_1_4_1', 'road_2_4_1', 'road_3_4_1', 'road_4_1_3', 'road_1_4_2', 'road_4_2_0', 'road_2_1_3', 'road_2_4_1', 'road_1_3_2', 'road_1_2_2', 'road_1_4_1', 'road_1_1_2', 'road_1_3_1', 'road_2_1_3', 'road_4_1_0', 'road_3_4_1', 'road_4_1_0', 'road_4_4_0', 'road_1_4_2', 'road_3_1_3', 'road_1_1_3', 'road_1_1_3', 'road_1_4_1', 'road_4_4_0', 'road_1_1_3', 'road_4_2_0', 'road_1_1_3', 'road_2_3_1', 'road_2_4_1', 'road_2_1_3', 'road_1_3_2', 'road_2_4_1', 'road_4_2_0', 'road_4_1_0', 'road_1_4_2', 'road_1_3_2', 'road_1_1_3', 'road_4_4_1', 'road_1_4_2', 'road_1_2_2', 'road_4_1_3', 'road_1_3_2', 'road_3_1_3', 'road_1_3_2', 'road_1_1_3', 'road_4_1_3', 'road_1_1_2', 'road_4_4_0', 'road_1_1_3', 'road_1_4_2', 'road_1_1_3', 'road_4_4_0', 'road_4_1_0', 'road_1_4_2', 'road_4_4_1', 'road_1_3_2', 'road_1_2_2', 'road_1_1_3', 'road_4_1_0', 'road_1_2_2', 'road_4_4_0', 'road_4_4_1', 'road_2_1_3', 'road_1_4_2', 'road_4_2_0', 'road_4_3_0', 'road_1_1_3', 'road_4_4_1', 'road_4_1_0', 'road_1_4_1', 'road_4_3_0', 'road_2_4_1', 'road_4_1_0', 'road_4_1_3', 'road_3_4_1', 'road_1_4_2', 'road_1_3_2', 'road_4_4_0', 'road_4_2_0', 'road_3_1_3', 'road_1_1_3', 'road_1_4_2', 'road_4_1_0', 'road_1_4_1', 'road_4_1_3', 'road_3_4_1', 'road_4_4_1', 'road_1_4_1', 'road_1_2_2', 'road_4_3_0', 'road_2_4_1', 'road_4_1_3', 'road_1_1_2', 'road_4_3_0', 'road_1_2_2', 'road_4_4_1', 'road_4_4_1', 'road_1_1_3', 'road_1_1_3', 'road_2_1_3', 'road_4_3_0', 'road_3_1_3', 'road_3_1_3', 'road_1_3_2', 'road_1_3_2', 'road_1_1_3', 'road_3_1_3', 'road_1_4_1', 'road_3_4_1', 'road_4_4_0', 'road_4_4_0', 'road_4_2_0', 'road_4_4_0', 'road_3_4_1', 'road_2_1_3', 'road_4_2_0', 'road_2_4_1', 'road_1_3_2', 'road_1_2_2', 'road_1_3_2', 'road_4_1_3', 'road_1_4_1', 'road_4_2_0', 'road_4_4_0', 'road_3_4_1', 'road_2_4_1', 'road_4_2_0', 'road_1_3_2', 'road_3_4_1', 'road_2_4_1', 'road_2_1_3', 'road_2_4_1', 'road_1_1_2', 'road_1_4_2', 'road_1_4_2', 'road_1_4_1', 'road_1_4_2', 'road_2_1_3', 'road_4_4_0', 'road_1_1_2', 'road_1_1_2', 'road_4_4_1', 'road_4_4_1', 'road_1_3_2', 'road_1_4_1', 'road_4_4_0', 'road_2_4_1', 'road_4_4_0', 'road_4_4_0', 'road_2_4_1', 'road_4_1_0', 'road_1_4_1', 'road_3_4_1', 'road_4_3_0', 'road_1_3_2', 'road_4_4_1', 'road_1_4_1', 'road_4_4_1', 'road_1_1_3', 'road_2_4_1', 'road_2_1_3', 'road_2_4_1', 'road_4_4_0', 'road_4_4_1', 'road_2_4_1', 'road_1_3_2', 'road_4_4_1', 'road_4_4_1', 'road_3_1_3', 'road_4_4_0', 'road_2_1_3', 'road_4_4_1', 'road_1_3_2', 'road_4_4_1', 'road_3_4_1', 'road_1_1_3', 'road_4_4_1', 'road_4_4_1', 'road_4_4_1', 'road_1_2_2', 'road_1_4_1', 'road_1_1_2']
# ####hangzhou_40
# car_num = 664
# start_edges = ['road_4_0_1', 'road_0_1_0', 'road_0_2_0', 'road_1_0_1', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_1_0_1', 'road_1_5_3', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_1_0_1', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_1_0_1', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_5_2_2', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_5_3_2', 'road_1_0_1', 'road_1_5_3', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_5_4_2', 'road_0_1_0', 'road_4_0_1', 'road_0_2_0', 'road_0_4_0', 'road_0_4_0', 'road_1_5_3', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_4_0_1', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_5_3_2', 'road_4_0_1', 'road_5_4_2', 'road_1_0_1', 'road_0_1_0', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_1_0_1', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_1_5_3', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_0_1_0', 'road_1_5_3', 'road_4_0_1', 'road_0_4_0', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_2_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_5_3_2', 'road_4_0_1', 'road_0_1_0', 'road_0_2_0', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_5_3_2', 'road_5_2_2', 'road_0_1_0', 'road_5_4_2', 'road_4_0_1', 'road_1_5_3', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_5_2_2', 'road_4_0_1', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_5_3_2', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_5_3_2', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_1_0_1', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_1_5_3', 'road_5_3_2', 'road_0_4_0', 'road_0_2_0', 'road_0_4_0', 'road_1_0_1', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_1_0_1', 'road_5_3_2', 'road_0_4_0', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_4_0_1', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_5_3_2', 'road_1_0_1', 'road_1_5_3', 'road_5_4_2', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_1_0_1', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_4_0_1', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_4_0_1', 'road_0_2_0', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_5_3_2', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_5_3_2', 'road_0_4_0', 'road_0_4_0', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_3_2', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_0_4_0', 'road_4_0_1', 'road_0_1_0', 'road_4_0_1', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_1_0_1', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_5_3_2', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_5_2_2', 'road_0_1_0', 'road_5_3_2', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_1_5_3', 'road_5_3_2', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_5_4_2', 'road_5_3_2', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_0_1_0', 'road_5_3_2', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_5_2_2', 'road_4_0_1', 'road_1_0_1', 'road_5_3_2', 'road_0_1_0', 'road_0_1_0', 'road_5_2_2', 'road_1_5_3', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_3_2', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_5_3_2', 'road_0_2_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_1_0_1', 'road_1_5_3', 'road_5_3_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_1_0_1', 'road_5_4_2', 'road_0_1_0', 'road_1_0_1', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_4_0_1', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_1_0_1', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_2_2', 'road_0_4_0', 'road_1_5_3', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_5_3_2', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_0_2_0', 'road_5_2_2', 'road_0_2_0', 'road_0_4_0', 'road_4_0_1', 'road_1_5_3', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_4_0_1', 'road_0_4_0', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_5_3_2', 'road_0_1_0', 'road_1_0_1', 'road_0_1_0', 'road_5_2_2', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_0_2_0', 'road_0_4_0', 'road_4_0_1', 'road_1_0_1', 'road_5_4_2', 'road_4_0_1', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_1_5_3', 'road_4_0_1', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_1_5_3', 'road_0_1_0', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_5_2_2', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_5_3_2', 'road_5_2_2', 'road_4_0_1', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_5_3_2', 'road_1_0_1', 'road_1_5_3', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_4_0_1', 'road_5_4_2', 'road_5_2_2', 'road_5_3_2', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_5_2_2', 'road_1_5_3', 'road_0_4_0', 'road_5_3_2', 'road_0_2_0', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_4_0_1', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0']
# end_edges = ['road_4_2_0', 'road_3_1_3', 'road_1_1_3', 'road_1_4_1', 'road_4_4_0', 'road_2_4_1', 'road_3_1_3', 'road_1_3_2', 'road_1_2_2', 'road_4_3_0', 'road_3_4_1', 'road_4_4_1', 'road_2_4_1', 'road_2_1_3', 'road_1_3_2', 'road_1_4_1', 'road_4_1_0', 'road_3_1_3', 'road_4_4_0', 'road_4_4_1', 'road_1_2_2', 'road_4_2_0', 'road_4_4_0', 'road_1_4_1', 'road_4_4_1', 'road_4_2_0', 'road_4_4_0', 'road_3_4_3', 'road_2_1_3', 'road_1_4_1', 'road_3_4_1', 'road_4_4_0', 'road_2_1_3', 'road_4_3_0', 'road_4_2_0', 'road_3_4_1', 'road_4_4_3', 'road_2_4_1', 'road_2_1_3', 'road_1_4_2', 'road_3_4_1', 'road_4_4_0', 'road_2_4_1', 'road_2_1_3', 'road_4_4_0', 'road_4_3_0', 'road_1_3_1', 'road_1_4_1', 'road_3_4_1', 'road_2_4_1', 'road_4_3_0', 'road_1_4_1', 'road_2_4_1', 'road_1_2_2', 'road_1_1_2', 'road_4_4_0', 'road_4_4_1', 'road_4_1_3', 'road_1_3_2', 'road_3_3_3', 'road_1_3_2', 'road_4_4_1', 'road_1_3_2', 'road_3_4_1', 'road_2_4_1', 'road_4_1_3', 'road_2_4_1', 'road_1_4_2', 'road_1_4_2', 'road_4_1_0', 'road_1_2_2', 'road_1_1_3', 'road_1_3_2', 'road_2_1_3', 'road_1_3_2', 'road_2_4_1', 'road_4_4_1', 'road_4_4_1', 'road_1_2_2', 'road_1_4_1', 'road_4_4_1', 'road_4_4_0', 'road_1_4_1', 'road_1_3_2', 'road_2_1_3', 'road_4_4_0', 'road_3_4_1', 'road_4_2_0', 'road_4_4_1', 'road_2_1_3', 'road_4_1_0', 'road_1_1_2', 'road_1_3_2', 'road_3_4_1', 'road_4_1_3', 'road_2_1_3', 'road_3_1_3', 'road_2_4_1', 'road_1_1_2', 'road_1_4_2', 'road_1_2_2', 'road_3_1_3', 'road_1_4_2', 'road_1_4_1', 'road_4_4_0', 'road_1_4_2', 'road_4_4_1', 'road_1_1_3', 'road_4_4_1', 'road_1_2_2', 'road_3_1_3', 'road_4_3_0', 'road_4_3_0', 'road_4_1_0', 'road_4_4_1', 'road_1_1_2', 'road_1_1_3', 'road_2_1_3', 'road_4_4_1', 'road_3_4_1', 'road_1_1_2', 'road_2_1_3', 'road_1_2_2', 'road_4_2_0', 'road_1_3_2', 'road_4_2_0', 'road_2_1_3', 'road_4_1_3', 'road_1_3_2', 'road_2_1_3', 'road_4_3_0', 'road_1_2_2', 'road_1_1_3', 'road_3_4_1', 'road_4_1_0', 'road_4_3_0', 'road_4_1_0', 'road_1_1_2', 'road_4_4_1', 'road_2_4_1', 'road_1_4_1', 'road_4_4_0', 'road_1_2_2', 'road_2_1_3', 'road_3_4_1', 'road_4_2_0', 'road_1_3_2', 'road_4_4_1', 'road_2_4_1', 'road_4_1_0', 'road_2_4_1', 'road_1_4_1', 'road_2_1_3', 'road_4_4_1', 'road_4_4_0', 'road_4_4_1', 'road_2_1_3', 'road_4_4_1', 'road_1_1_2', 'road_1_1_3', 'road_1_4_1', 'road_1_3_2', 'road_3_4_1', 'road_1_2_2', 'road_4_1_0', 'road_1_3_2', 'road_4_1_0', 'road_1_4_2', 'road_2_1_3', 'road_3_4_1', 'road_1_4_1', 'road_2_1_3', 'road_4_4_1', 'road_4_3_0', 'road_3_1_3', 'road_1_4_2', 'road_1_3_2', 'road_1_1_2', 'road_1_3_2', 'road_4_1_3', 'road_1_1_3', 'road_4_4_1', 'road_2_1_3', 'road_1_4_1', 'road_1_1_1', 'road_2_1_3', 'road_1_3_2', 'road_1_1_3', 'road_1_1_3', 'road_3_4_1', 'road_1_3_2', 'road_1_3_2', 'road_2_1_3', 'road_4_1_3', 'road_1_3_2', 'road_2_1_3', 'road_4_4_0', 'road_2_1_3', 'road_1_4_2', 'road_1_1_2', 'road_4_4_1', 'road_4_4_0', 'road_2_4_1', 'road_4_4_0', 'road_2_4_1', 'road_1_4_2', 'road_4_4_1', 'road_1_2_2', 'road_4_2_0', 'road_2_1_3', 'road_3_4_1', 'road_1_4_2', 'road_4_4_0', 'road_1_1_3', 'road_1_3_2', 'road_4_1_0', 'road_4_2_0', 'road_4_4_0', 'road_4_4_1', 'road_4_3_0', 'road_4_2_0', 'road_4_4_1', 'road_4_3_0', 'road_2_4_1', 'road_2_1_3', 'road_4_4_0', 'road_1_4_1', 'road_1_4_1', 'road_1_4_2', 'road_1_2_2', 'road_1_2_2', 'road_2_4_1', 'road_4_4_0', 'road_1_4_1', 'road_1_4_1', 'road_3_1_3', 'road_4_4_1', 'road_1_4_2', 'road_1_3_2', 'road_1_4_1', 'road_3_4_1', 'road_1_1_3', 'road_1_3_2', 'road_2_4_1', 'road_1_1_2', 'road_1_2_2', 'road_3_1_3', 'road_1_4_2', 'road_1_1_3', 'road_3_1_3', 'road_4_2_0', 'road_1_4_2', 'road_4_4_0', 'road_1_3_2', 'road_2_4_1', 'road_4_4_0', 'road_1_3_2', 'road_1_3_1', 'road_4_1_0', 'road_2_1_3', 'road_1_4_2', 'road_1_1_3', 'road_3_4_1', 'road_3_4_1', 'road_4_1_0', 'road_1_4_2', 'road_1_1_2', 'road_4_4_0', 'road_4_2_0', 'road_3_1_3', 'road_1_1_3', 'road_1_1_3', 'road_4_3_0', 'road_3_1_3', 'road_1_4_1', 'road_2_4_1', 'road_1_4_1', 'road_4_2_0', 'road_1_1_2', 'road_4_4_0', 'road_4_4_1', 'road_4_1_3', 'road_4_4_0', 'road_4_4_1', 'road_3_4_1', 'road_2_4_1', 'road_1_1_2', 'road_1_1_2', 'road_1_3_2', 'road_4_4_1', 'road_4_1_3', 'road_4_1_3', 'road_4_4_0', 'road_1_4_2', 'road_1_4_1', 'road_1_4_1', 'road_4_4_0', 'road_2_4_1', 'road_1_1_3', 'road_4_4_0', 'road_1_1_3', 'road_2_4_1', 'road_1_4_1', 'road_4_1_0', 'road_2_4_1', 'road_2_1_3', 'road_1_1_1', 'road_4_3_0', 'road_3_4_1', 'road_4_3_0', 'road_1_3_2', 'road_3_4_1', 'road_4_2_0', 'road_1_1_3', 'road_1_1_2', 'road_1_4_2', 'road_2_4_1', 'road_3_4_1', 'road_1_3_2', 'road_1_2_2', 'road_3_4_1', 'road_4_3_3', 'road_3_4_1', 'road_3_1_3', 'road_3_1_3', 'road_2_1_3', 'road_1_2_2', 'road_1_3_1', 'road_3_4_1', 'road_1_3_2', 'road_4_4_1', 'road_3_4_1', 'road_1_2_2', 'road_1_3_2', 'road_2_4_1', 'road_1_3_2', 'road_3_1_3', 'road_1_1_2', 'road_1_1_3', 'road_4_3_0', 'road_4_1_0', 'road_4_2_0', 'road_4_2_0', 'road_4_4_1', 'road_4_1_0', 'road_1_4_1', 'road_2_1_3', 'road_1_2_2', 'road_4_1_3', 'road_4_4_1', 'road_1_1_3', 'road_2_1_3', 'road_2_4_1', 'road_4_3_0', 'road_1_4_1', 'road_1_3_2', 'road_2_1_3', 'road_1_3_2', 'road_1_1_3', 'road_4_1_3', 'road_4_4_1', 'road_1_1_3', 'road_4_4_1', 'road_1_1_3', 'road_4_4_0', 'road_2_4_1', 'road_2_1_3', 'road_3_4_1', 'road_3_4_1', 'road_1_1_2', 'road_4_2_0', 'road_2_1_3', 'road_1_4_2', 'road_1_2_2', 'road_4_4_1', 'road_2_4_1', 'road_2_1_3', 'road_4_1_0', 'road_1_4_2', 'road_2_4_1', 'road_1_1_3', 'road_4_4_1', 'road_3_4_1', 'road_1_2_2', 'road_1_1_3', 'road_3_4_1', 'road_1_2_2', 'road_1_4_2', 'road_1_1_2', 'road_4_4_1', 'road_3_1_3', 'road_3_1_3', 'road_2_4_1', 'road_3_4_1', 'road_4_3_0', 'road_1_1_2', 'road_4_1_3', 'road_3_4_1', 'road_1_3_2', 'road_4_2_0', 'road_4_1_3', 'road_4_4_0', 'road_2_1_3', 'road_2_1_3', 'road_1_1_2', 'road_1_3_2', 'road_1_3_2', 'road_4_1_3', 'road_4_4_1', 'road_1_4_2', 'road_1_3_0', 'road_4_4_1', 'road_1_4_1', 'road_2_4_1', 'road_4_4_0', 'road_1_4_2', 'road_2_4_1', 'road_4_4_0', 'road_1_3_2', 'road_1_4_1', 'road_2_4_1', 'road_1_4_2', 'road_3_4_1', 'road_4_4_1', 'road_4_2_0', 'road_4_1_3', 'road_4_1_0', 'road_4_4_1', 'road_1_4_2', 'road_4_4_1', 'road_2_1_3', 'road_4_1_0', 'road_1_4_1', 'road_4_4_1', 'road_2_1_2', 'road_1_2_2', 'road_3_4_1', 'road_3_4_1', 'road_1_1_3', 'road_3_1_3', 'road_1_4_2', 'road_4_4_1', 'road_4_4_0', 'road_4_4_0', 'road_4_4_1', 'road_2_3_1', 'road_1_1_2', 'road_3_4_1', 'road_2_1_3', 'road_4_4_1', 'road_2_4_1', 'road_1_2_2', 'road_1_4_2', 'road_1_2_2', 'road_3_4_1', 'road_2_1_3', 'road_2_1_3', 'road_1_1_3', 'road_1_3_2', 'road_3_1_3', 'road_4_2_0', 'road_1_1_3', 'road_4_2_0', 'road_3_1_3', 'road_2_1_3', 'road_4_1_0', 'road_1_3_2', 'road_1_1_3', 'road_4_1_3', 'road_1_3_2', 'road_2_4_1', 'road_1_4_2', 'road_1_3_2', 'road_3_1_3', 'road_1_4_2', 'road_3_1_3', 'road_1_2_2', 'road_4_3_0', 'road_4_1_3', 'road_1_1_2', 'road_2_4_1', 'road_1_4_2', 'road_1_1_3', 'road_1_4_2', 'road_1_4_2', 'road_1_1_3', 'road_4_4_0', 'road_1_4_1', 'road_2_4_1', 'road_1_4_2', 'road_3_4_1', 'road_4_1_0', 'road_1_2_2', 'road_2_1_3', 'road_4_4_1', 'road_1_2_2', 'road_3_4_1', 'road_4_1_3', 'road_1_1_3', 'road_1_4_2', 'road_4_2_0', 'road_1_2_2', 'road_4_4_0', 'road_4_2_0', 'road_2_4_1', 'road_4_4_1', 'road_2_1_3', 'road_1_1_3', 'road_2_4_1', 'road_1_3_2', 'road_3_4_1', 'road_1_1_3', 'road_1_2_2', 'road_1_1_2', 'road_4_4_1', 'road_1_4_1', 'road_1_4_2', 'road_1_3_2', 'road_2_1_3', 'road_4_1_3', 'road_4_1_3', 'road_1_3_2', 'road_3_4_1', 'road_1_4_2', 'road_1_3_2', 'road_3_1_3', 'road_1_1_2', 'road_4_1_0', 'road_4_4_1', 'road_4_4_0', 'road_4_4_1', 'road_4_1_3', 'road_1_2_2', 'road_4_4_1', 'road_1_2_2', 'road_1_4_2', 'road_4_3_0', 'road_4_4_1', 'road_2_4_1', 'road_4_1_3', 'road_4_4_1', 'road_1_1_3', 'road_1_2_2', 'road_1_1_3', 'road_2_4_1', 'road_4_4_1', 'road_1_1_3', 'road_1_4_1', 'road_2_1_3', 'road_4_3_0', 'road_3_1_3', 'road_1_4_2', 'road_4_1_0', 'road_1_3_2', 'road_4_4_1', 'road_4_1_0', 'road_2_1_3', 'road_1_4_1', 'road_4_1_0', 'road_4_4_1', 'road_4_4_1', 'road_4_3_0', 'road_4_4_1', 'road_1_4_1', 'road_3_4_1', 'road_4_4_0', 'road_3_1_3', 'road_2_1_3', 'road_4_4_0', 'road_2_4_1', 'road_4_4_0', 'road_1_4_2', 'road_1_1_3', 'road_1_4_1', 'road_4_1_3', 'road_1_3_2', 'road_3_4_1', 'road_4_2_0', 'road_1_1_3', 'road_3_1_3', 'road_1_4_2', 'road_1_3_2', 'road_4_2_0', 'road_2_4_1', 'road_1_1_3', 'road_1_3_2', 'road_1_2_2', 'road_1_3_2', 'road_3_4_1', 'road_4_4_1', 'road_4_4_1', 'road_1_4_1', 'road_4_1_3', 'road_3_4_1', 'road_3_1_1', 'road_1_3_2', 'road_4_4_0', 'road_1_3_2', 'road_4_4_1', 'road_1_1_2', 'road_1_3_2', 'road_2_1_3', 'road_4_4_1', 'road_3_4_1', 'road_2_4_1', 'road_3_4_1', 'road_2_1_3', 'road_1_1_2', 'road_1_3_2', 'road_1_3_2', 'road_1_4_1', 'road_1_4_2', 'road_1_2_2', 'road_3_4_1', 'road_1_4_1', 'road_4_1_0', 'road_2_1_3', 'road_4_4_0', 'road_4_4_1', 'road_4_4_1', 'road_1_4_0', 'road_4_4_1', 'road_4_3_0', 'road_2_4_1', 'road_4_4_1', 'road_1_2_2', 'road_4_2_0', 'road_4_1_3', 'road_1_2_2', 'road_1_2_2', 'road_1_2_2', 'road_4_4_0', 'road_4_4_1', 'road_4_4_1', 'road_3_4_1', 'road_3_4_1', 'road_1_2_2', 'road_3_4_1', 'road_4_1_0', 'road_3_1_3', 'road_3_4_1', 'road_4_4_1', 'road_1_4_1', 'road_1_2_2', 'road_2_1_3', 'road_1_1_3', 'road_4_4_1', 'road_1_4_1', 'road_4_3_0', 'road_1_4_2', 'road_2_1_3', 'road_4_4_1', 'road_2_1_3', 'road_2_4_1', 'road_4_4_1', 'road_1_3_2', 'road_1_1_2', 'road_4_4_0', 'road_1_3_2', 'road_4_1_0', 'road_4_3_0', 'road_2_4_1', 'road_1_4_1', 'road_4_2_0']
# ###hangzhou_50
# car_num = 830
# end_edges = ['road_4_2_0', 'road_1_1_3', 'road_2_1_3', 'road_1_4_1', 'road_1_2_2', 'road_2_3_1', 'road_4_4_0', 'road_2_4_1', 'road_3_1_3', 'road_4_3_0', 'road_1_1_3', 'road_4_4_1', 'road_1_3_2', 'road_4_1_3', 'road_4_3_0', 'road_3_4_1', 'road_1_4_1', 'road_4_4_1', 'road_2_4_1', 'road_1_1_3', 'road_1_3_2', 'road_1_4_1', 'road_2_4_1', 'road_4_4_0', 'road_4_4_1', 'road_4_1_0', 'road_2_4_1', 'road_2_4_1', 'road_1_1_2', 'road_4_4_0', 'road_4_4_0', 'road_4_3_0', 'road_1_2_2', 'road_4_4_1', 'road_2_1_3', 'road_4_2_0', 'road_1_2_2', 'road_4_1_0', 'road_2_1_3', 'road_3_4_1', 'road_4_4_0', 'road_4_3_0', 'road_4_4_1', 'road_3_4_1', 'road_4_2_0', 'road_1_1_3', 'road_1_1_2', 'road_2_1_3', 'road_4_2_0', 'road_4_1_0', 'road_4_4_0', 'road_4_4_3', 'road_4_4_0', 'road_1_3_2', 'road_3_4_1', 'road_3_1_3', 'road_2_4_1', 'road_4_1_3', 'road_1_1_3', 'road_1_4_1', 'road_1_3_1', 'road_1_4_2', 'road_2_1_3', 'road_1_4_1', 'road_2_4_1', 'road_3_4_1', 'road_2_2_1', 'road_2_4_1', 'road_4_3_0', 'road_1_1_3', 'road_2_4_1', 'road_2_1_3', 'road_1_1_3', 'road_2_4_1', 'road_1_2_2', 'road_4_4_1', 'road_1_1_2', 'road_4_2_0', 'road_4_1_0', 'road_3_1_3', 'road_4_1_3', 'road_1_3_0', 'road_4_3_0', 'road_3_1_3', 'road_4_1_3', 'road_1_4_2', 'road_4_4_0', 'road_1_1_3', 'road_1_1_3', 'road_1_1_3', 'road_1_4_2', 'road_4_4_1', 'road_1_2_2', 'road_3_2_0', 'road_2_1_3', 'road_1_3_2', 'road_2_4_1', 'road_4_4_1', 'road_1_1_3', 'road_1_2_2', 'road_1_4_1', 'road_4_1_0', 'road_4_4_0', 'road_1_3_2', 'road_1_3_2', 'road_4_2_0', 'road_4_4_1', 'road_4_4_0', 'road_3_4_1', 'road_1_4_1', 'road_4_4_1', 'road_1_2_2', 'road_2_1_3', 'road_4_2_0', 'road_4_4_0', 'road_2_1_3', 'road_3_1_3', 'road_4_1_0', 'road_1_1_2', 'road_4_3_0', 'road_1_4_1', 'road_3_4_1', 'road_3_4_1', 'road_2_2_0', 'road_4_3_0', 'road_1_4_2', 'road_2_1_3', 'road_2_4_1', 'road_3_1_3', 'road_2_4_1', 'road_1_4_2', 'road_3_1_3', 'road_1_4_1', 'road_4_4_0', 'road_4_1_3', 'road_4_4_1', 'road_3_4_1', 'road_4_4_1', 'road_4_1_3', 'road_3_4_1', 'road_1_4_2', 'road_1_1_3', 'road_4_4_1', 'road_1_3_2', 'road_3_1_3', 'road_1_4_2', 'road_3_1_3', 'road_4_3_0', 'road_4_3_0', 'road_4_1_0', 'road_1_1_3', 'road_1_1_3', 'road_1_4_1', 'road_4_4_1', 'road_3_4_1', 'road_1_1_2', 'road_4_3_0', 'road_1_3_1', 'road_4_4_1', 'road_4_2_0', 'road_1_1_3', 'road_1_3_2', 'road_1_4_2', 'road_2_1_3', 'road_1_2_2', 'road_4_3_0', 'road_1_2_2', 'road_1_1_3', 'road_3_4_1', 'road_3_4_1', 'road_1_3_2', 'road_4_1_0', 'road_4_4_1', 'road_4_1_0', 'road_2_4_1', 'road_4_4_0', 'road_1_2_2', 'road_1_1_3', 'road_2_1_3', 'road_2_4_1', 'road_4_1_3', 'road_1_3_2', 'road_3_1_3', 'road_4_4_0', 'road_2_4_1', 'road_4_4_1', 'road_4_4_0', 'road_4_1_0', 'road_1_4_1', 'road_1_4_1', 'road_1_1_3', 'road_4_1_3', 'road_2_1_3', 'road_1_4_2', 'road_4_4_1', 'road_4_4_0', 'road_2_1_3', 'road_4_4_1', 'road_2_1_3', 'road_4_4_1', 'road_1_1_3', 'road_3_3_3', 'road_3_4_1', 'road_3_1_3', 'road_3_4_1', 'road_1_1_3', 'road_2_1_3', 'road_1_3_2', 'road_4_1_0', 'road_1_4_2', 'road_4_4_0', 'road_4_1_0', 'road_4_4_0', 'road_2_1_3', 'road_1_4_1', 'road_4_1_0', 'road_2_4_1', 'road_3_3_1', 'road_4_4_0', 'road_1_3_2', 'road_1_1_2', 'road_1_2_2', 'road_1_1_3', 'road_4_4_1', 'road_3_4_1', 'road_4_3_0', 'road_4_1_0', 'road_4_4_1', 'road_2_1_3', 'road_4_4_1', 'road_1_1_2', 'road_2_1_3', 'road_4_4_1', 'road_1_2_2', 'road_1_3_2', 'road_1_1_3', 'road_3_1_3', 'road_1_1_3', 'road_3_4_1', 'road_4_1_0', 'road_1_3_2', 'road_1_4_1', 'road_1_3_2', 'road_4_1_0', 'road_4_1_3', 'road_4_3_3', 'road_1_4_2', 'road_1_3_2', 'road_4_4_0', 'road_4_4_0', 'road_2_1_3', 'road_1_4_2', 'road_1_1_2', 'road_4_4_1', 'road_4_4_0', 'road_2_4_1', 'road_3_4_1', 'road_4_4_0', 'road_4_4_1', 'road_1_4_1', 'road_3_1_3', 'road_1_3_2', 'road_4_3_3', 'road_2_1_3', 'road_3_4_1', 'road_4_3_0', 'road_1_4_2', 'road_4_2_0', 'road_1_1_3', 'road_1_4_2', 'road_1_3_2', 'road_3_4_1', 'road_4_1_0', 'road_1_4_1', 'road_4_4_0', 'road_4_4_1', 'road_4_1_3', 'road_4_3_0', 'road_4_2_0', 'road_4_1_3', 'road_2_4_1', 'road_4_4_1', 'road_4_2_0', 'road_4_3_0', 'road_4_4_0', 'road_1_4_1', 'road_1_4_1', 'road_1_4_1', 'road_1_2_2', 'road_1_4_1', 'road_2_4_1', 'road_4_4_0', 'road_1_4_1', 'road_3_1_3', 'road_4_3_0', 'road_1_4_1', 'road_3_1_3', 'road_4_1_0', 'road_1_4_2', 'road_1_4_1', 'road_4_1_0', 'road_3_4_1', 'road_2_4_1', 'road_2_4_1', 'road_4_4_1', 'road_3_4_1', 'road_3_1_3', 'road_2_4_1', 'road_1_4_2', 'road_3_1_3', 'road_1_4_2', 'road_4_1_0', 'road_4_4_1', 'road_1_1_3', 'road_2_4_1', 'road_2_1_3', 'road_1_1_3', 'road_2_4_1', 'road_1_1_2', 'road_2_1_3', 'road_1_3_2', 'road_1_3_1', 'road_4_1_0', 'road_1_1_3', 'road_1_1_3', 'road_3_1_3', 'road_1_4_1', 'road_1_1_2', 'road_1_2_2', 'road_1_1_2', 'road_1_1_3', 'road_4_1_0', 'road_3_4_1', 'road_2_4_1', 'road_4_1_0', 'road_4_1_0', 'road_1_1_2', 'road_4_4_0', 'road_1_1_2', 'road_3_1_3', 'road_4_1_3', 'road_1_2_2', 'road_2_1_3', 'road_4_3_0', 'road_4_4_1', 'road_1_2_2', 'road_3_1_3', 'road_1_1_2', 'road_1_4_1', 'road_2_4_1', 'road_3_1_3', 'road_3_4_1', 'road_4_4_0', 'road_4_1_0', 'road_2_4_1', 'road_1_4_1', 'road_4_2_0', 'road_4_4_1', 'road_4_4_0', 'road_1_3_2', 'road_1_3_2', 'road_4_4_1', 'road_4_2_0', 'road_2_1_3', 'road_3_1_3', 'road_1_1_2', 'road_1_3_2', 'road_4_4_1', 'road_4_1_3', 'road_4_4_0', 'road_1_4_1', 'road_1_4_1', 'road_2_4_1', 'road_1_1_2', 'road_4_4_0', 'road_3_4_1', 'road_1_3_2', 'road_1_3_2', 'road_3_4_1', 'road_1_3_2', 'road_3_4_1', 'road_2_4_1', 'road_2_4_1', 'road_4_4_1', 'road_1_3_2', 'road_4_1_0', 'road_3_4_1', 'road_2_4_1', 'road_4_1_0', 'road_3_4_1', 'road_4_4_0', 'road_4_4_1', 'road_1_3_2', 'road_1_2_2', 'road_2_4_1', 'road_1_1_3', 'road_1_4_1', 'road_1_4_1', 'road_4_2_0', 'road_3_1_3', 'road_4_4_1', 'road_1_1_3', 'road_2_4_1', 'road_3_4_1', 'road_4_1_0', 'road_1_1_2', 'road_4_4_1', 'road_1_4_2', 'road_3_4_1', 'road_1_3_2', 'road_1_2_2', 'road_4_4_1', 'road_4_3_3', 'road_4_4_1', 'road_4_1_3', 'road_2_1_3', 'road_3_1_3', 'road_4_1_3', 'road_4_3_0', 'road_4_4_1', 'road_3_4_1', 'road_3_1_3', 'road_4_4_1', 'road_4_4_1', 'road_3_4_1', 'road_2_1_3', 'road_1_1_2', 'road_1_1_3', 'road_1_3_2', 'road_4_1_0', 'road_1_2_2', 'road_3_1_3', 'road_1_1_3', 'road_1_4_1', 'road_4_3_0', 'road_3_4_1', 'road_1_2_2', 'road_4_4_1', 'road_4_2_0', 'road_4_4_1', 'road_4_1_0', 'road_2_1_3', 'road_4_1_3', 'road_1_1_2', 'road_4_1_3', 'road_4_4_1', 'road_1_2_2', 'road_2_1_3', 'road_2_4_1', 'road_1_4_1', 'road_1_3_2', 'road_1_3_2', 'road_2_1_3', 'road_1_3_2', 'road_1_1_2', 'road_4_4_0', 'road_4_1_3', 'road_1_1_2', 'road_4_4_1', 'road_4_4_1', 'road_1_4_1', 'road_4_4_1', 'road_4_4_1', 'road_1_3_2', 'road_3_4_1', 'road_2_1_3', 'road_3_4_1', 'road_1_1_2', 'road_1_4_1', 'road_4_2_0', 'road_1_4_1', 'road_3_1_3', 'road_4_1_3', 'road_4_2_0', 'road_3_1_3', 'road_3_4_1', 'road_4_4_1', 'road_1_3_2', 'road_1_4_1', 'road_3_4_1', 'road_2_1_3', 'road_1_4_2', 'road_1_2_2', 'road_1_1_3', 'road_1_1_3', 'road_4_4_1', 'road_3_1_3', 'road_2_1_3', 'road_1_4_2', 'road_1_1_3', 'road_3_4_1', 'road_3_1_3', 'road_1_1_3', 'road_1_1_3', 'road_1_2_2', 'road_1_4_2', 'road_1_1_2', 'road_4_4_1', 'road_4_3_0', 'road_3_1_3', 'road_4_4_1', 'road_3_1_3', 'road_2_4_1', 'road_4_3_0', 'road_1_1_2', 'road_4_1_3', 'road_4_4_0', 'road_3_4_1', 'road_2_1_3', 'road_4_2_0', 'road_1_1_2', 'road_2_4_1', 'road_2_1_3', 'road_4_3_0', 'road_1_3_2', 'road_3_4_1', 'road_1_1_2', 'road_3_1_3', 'road_4_2_0', 'road_1_4_2', 'road_1_4_1', 'road_3_1_3', 'road_2_4_1', 'road_1_3_2', 'road_2_1_3', 'road_1_4_2', 'road_1_1_2', 'road_1_3_2', 'road_2_1_3', 'road_4_4_1', 'road_1_1_3', 'road_4_4_0', 'road_3_4_1', 'road_4_4_1', 'road_2_1_3', 'road_1_3_2', 'road_4_1_0', 'road_4_1_0', 'road_4_2_0', 'road_4_4_1', 'road_2_1_2', 'road_1_4_1', 'road_4_1_3', 'road_4_1_0', 'road_4_1_3', 'road_3_4_1', 'road_4_4_0', 'road_2_1_3', 'road_4_3_0', 'road_3_1_3', 'road_1_1_3', 'road_3_4_1', 'road_4_4_0', 'road_1_1_3', 'road_1_4_1', 'road_4_1_0', 'road_1_2_2', 'road_2_3_1', 'road_1_1_2', 'road_3_4_1', 'road_4_4_1', 'road_2_4_1', 'road_2_4_1', 'road_1_3_2', 'road_1_2_2', 'road_1_4_2', 'road_4_4_0', 'road_1_4_2', 'road_3_4_1', 'road_4_4_0', 'road_1_3_2', 'road_1_1_3', 'road_4_3_0', 'road_1_3_2', 'road_1_4_2', 'road_3_1_3', 'road_2_1_3', 'road_4_2_0', 'road_4_1_0', 'road_1_4_2', 'road_1_4_2', 'road_1_1_3', 'road_1_4_2', 'road_1_3_2', 'road_1_2_2', 'road_1_1_3', 'road_4_1_3', 'road_3_4_1', 'road_4_2_0', 'road_3_4_1', 'road_2_4_1', 'road_1_1_2', 'road_3_1_3', 'road_1_1_3', 'road_4_1_3', 'road_3_4_1', 'road_1_1_2', 'road_2_4_1', 'road_3_4_1', 'road_1_4_2', 'road_1_1_3', 'road_2_1_3', 'road_1_4_2', 'road_1_1_3', 'road_4_4_0', 'road_1_4_2', 'road_4_4_1', 'road_2_4_1', 'road_1_4_2', 'road_3_4_1', 'road_1_3_2', 'road_4_1_0', 'road_2_1_3', 'road_3_1_3', 'road_3_4_1', 'road_1_2_2', 'road_4_1_3', 'road_1_2_2', 'road_4_1_3', 'road_1_2_2', 'road_4_4_1', 'road_3_4_1', 'road_1_3_2', 'road_4_1_0', 'road_4_1_0', 'road_1_1_3', 'road_2_1_3', 'road_1_4_1', 'road_2_1_3', 'road_2_4_1', 'road_4_2_0', 'road_3_4_1', 'road_4_4_1', 'road_2_1_3', 'road_3_1_3', 'road_1_4_2', 'road_2_4_1', 'road_3_4_1', 'road_1_1_3', 'road_1_1_2', 'road_1_1_3', 'road_4_1_3', 'road_4_1_0', 'road_1_1_3', 'road_4_4_1', 'road_1_1_3', 'road_1_4_2', 'road_4_3_0', 'road_1_3_2', 'road_1_4_2', 'road_2_1_3', 'road_4_1_3', 'road_2_1_3', 'road_4_1_0', 'road_1_3_2', 'road_2_1_3', 'road_1_3_2', 'road_3_4_0', 'road_1_3_2', 'road_4_2_0', 'road_3_1_3', 'road_4_1_3', 'road_1_2_2', 'road_4_4_1', 'road_3_1_3', 'road_2_1_3', 'road_1_4_2', 'road_4_4_1', 'road_4_1_0', 'road_4_4_1', 'road_4_1_3', 'road_4_4_1', 'road_1_4_1', 'road_1_3_2', 'road_1_4_2', 'road_1_4_1', 'road_1_4_2', 'road_4_2_0', 'road_1_4_2', 'road_1_1_3', 'road_1_1_3', 'road_4_4_1', 'road_4_4_1', 'road_4_4_1', 'road_1_1_3', 'road_2_1_3', 'road_3_4_1', 'road_4_3_0', 'road_1_4_1', 'road_4_1_0', 'road_1_3_2', 'road_1_4_2', 'road_4_1_0', 'road_2_1_3', 'road_1_3_2', 'road_1_1_3', 'road_4_4_1', 'road_1_1_2', 'road_1_4_1', 'road_4_1_0', 'road_4_4_1', 'road_1_2_2', 'road_4_4_1', 'road_4_3_0', 'road_4_4_1', 'road_1_1_3', 'road_2_4_1', 'road_3_1_3', 'road_2_1_3', 'road_2_4_1', 'road_1_1_2', 'road_3_4_1', 'road_3_4_1', 'road_4_4_0', 'road_1_4_2', 'road_3_4_1', 'road_1_4_1', 'road_3_4_1', 'road_4_2_0', 'road_1_1_3', 'road_2_4_1', 'road_1_1_3', 'road_4_2_0', 'road_4_3_0', 'road_1_3_2', 'road_4_1_3', 'road_4_4_1', 'road_4_4_0', 'road_4_2_0', 'road_3_4_1', 'road_1_4_1', 'road_4_1_3', 'road_4_4_1', 'road_2_1_3', 'road_4_2_0', 'road_1_1_3', 'road_4_4_1', 'road_4_4_0', 'road_4_4_0', 'road_1_3_2', 'road_2_4_1', 'road_1_1_2', 'road_2_1_3', 'road_4_4_1', 'road_1_1_2', 'road_3_4_1', 'road_2_1_3', 'road_4_4_0', 'road_1_2_2', 'road_4_4_1', 'road_1_4_1', 'road_2_1_3', 'road_1_1_2', 'road_4_4_1', 'road_4_4_1', 'road_1_4_0', 'road_2_4_1', 'road_4_4_1', 'road_4_1_0', 'road_1_1_1', 'road_4_3_0', 'road_2_4_1', 'road_4_4_0', 'road_1_4_2', 'road_4_4_0', 'road_2_1_3', 'road_2_4_1', 'road_1_2_2', 'road_4_2_0', 'road_1_2_2', 'road_1_2_2', 'road_4_3_0', 'road_1_2_2', 'road_4_4_0', 'road_1_4_1', 'road_1_4_1', 'road_3_4_1', 'road_4_4_0', 'road_1_4_2', 'road_2_1_3', 'road_1_3_2', 'road_1_2_2', 'road_3_4_1', 'road_3_1_3', 'road_1_4_1', 'road_2_1_3', 'road_1_1_2', 'road_1_1_2', 'road_1_2_2', 'road_3_4_1', 'road_2_1_3', 'road_1_4_1', 'road_1_4_2', 'road_1_4_1', 'road_4_1_0', 'road_1_4_1', 'road_2_1_3', 'road_1_4_2', 'road_2_1_3', 'road_3_4_1', 'road_4_4_0', 'road_1_2_2', 'road_2_4_1', 'road_4_4_1', 'road_4_4_1', 'road_2_1_3', 'road_2_1_3', 'road_3_4_1', 'road_4_4_1', 'road_1_1_3', 'road_4_2_0', 'road_2_4_1', 'road_1_1_2', 'road_4_4_0', 'road_4_1_3', 'road_1_1_3', 'road_4_4_1', 'road_4_3_0', 'road_4_4_1', 'road_2_4_1', 'road_1_2_2', 'road_2_4_1', 'road_1_3_2', 'road_1_4_1', 'road_1_1_2', 'road_1_4_2', 'road_1_4_1', 'road_4_2_0', 'road_4_1_0', 'road_4_4_1']
# start_edges = ['road_4_0_1', 'road_0_2_0', 'road_5_2_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_4_0_1', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_4_0_1', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_5_3_2', 'road_0_4_0', 'road_4_0_1', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_5_3_2', 'road_1_0_1', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_4_0_1', 'road_0_4_0', 'road_5_3_2', 'road_0_2_0', 'road_1_5_3', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_1_5_3', 'road_0_1_0', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_0_1_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_4_0_1', 'road_5_3_2', 'road_0_4_0', 'road_4_0_1', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_1_5_3', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_5_3_2', 'road_1_0_1', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_1_5_3', 'road_5_3_2', 'road_0_2_0', 'road_0_4_0', 'road_1_0_1', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_5_2_2', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_4_0_1', 'road_0_1_0', 'road_1_5_3', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_5_3_2', 'road_4_0_1', 'road_0_2_0', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_5_2_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_5_3_2', 'road_0_4_0', 'road_0_1_0', 'road_0_1_0', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_1_5_3', 'road_5_3_2', 'road_5_2_2', 'road_5_4_2', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_5_2_2', 'road_1_0_1', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_5_3_2', 'road_0_1_0', 'road_0_1_0', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_0_1_0', 'road_5_2_2', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_5_2_2', 'road_1_5_3', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_5_3_2', 'road_1_0_1', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_5_3_2', 'road_5_4_2', 'road_0_1_0', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_4_0_1', 'road_0_1_0', 'road_1_0_1', 'road_1_5_3', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_4_0_1', 'road_0_4_0', 'road_5_4_2', 'road_5_2_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_4_0_1', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_4_0_1', 'road_0_2_0', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_4_0_1', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_1_0_1', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_4_0_1', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_5_3_2', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_0_4_0', 'road_5_4_2', 'road_1_5_3', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_2_2', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_4_0_1', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_2_2', 'road_1_5_3', 'road_5_4_2', 'road_4_0_1', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_4_0_1', 'road_0_1_0', 'road_0_2_0', 'road_5_3_2', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_0_2_0', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_4_0_1', 'road_5_4_2', 'road_5_3_2', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_0_4_0', 'road_1_0_1', 'road_0_2_0', 'road_5_2_2', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_1_0', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_1_5_3', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_1_5_3', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_0_1_0', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_1_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_4_0_1', 'road_0_1_0', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_0_1_0', 'road_5_3_2', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_4_0_1', 'road_1_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_5_3_2', 'road_0_4_0', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_0_1_0', 'road_1_0_1', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_0_2_0', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_5_3_2', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_5_3_2', 'road_0_2_0', 'road_5_2_2', 'road_5_4_2', 'road_1_0_1', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_1_5_3', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_0_1_0', 'road_0_2_0', 'road_0_4_0', 'road_1_0_1', 'road_0_4_0', 'road_1_0_1', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_4_0_1', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_0_4_0', 'road_0_1_0', 'road_1_0_1', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_5_4_2', 'road_0_2_0', 'road_4_0_1', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_0_1_0', 'road_5_3_2', 'road_0_1_0', 'road_4_0_1', 'road_5_4_2', 'road_5_2_2', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_4_0_1', 'road_1_5_3', 'road_0_2_0', 'road_0_1_0', 'road_5_3_2', 'road_4_0_1', 'road_5_4_2', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_5_2_2', 'road_1_0_1', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_4_0_1', 'road_1_0_1', 'road_1_5_3', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_4_0_1', 'road_5_4_2', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_5_4_2', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_1_5_3', 'road_0_1_0', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_1_0_1', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_3_2', 'road_0_2_0', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_5_4_2', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_5_3_2', 'road_0_4_0', 'road_0_2_0', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_5_4_2', 'road_5_3_2', 'road_5_4_2', 'road_0_2_0', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_5_3_2', 'road_5_4_2', 'road_0_4_0', 'road_4_0_1', 'road_0_4_0', 'road_0_1_0', 'road_1_5_3', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_4_0_1', 'road_1_0_1', 'road_0_4_0', 'road_0_1_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_3_2', 'road_0_4_0', 'road_1_0_1', 'road_5_3_2', 'road_0_1_0', 'road_5_4_2', 'road_5_2_2', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_1_5_3', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_5_2_2']

In [ ]:
['A0A1'',''A0B0'',''A0bottom0'',''A0left0'',''A1A0'',''A1A2'',''A1B1'',''A1left1'',''A2A1',
  'A2A3'',''A2B2'',''A2left2'',''A3A2'',''A3B3'',''A3left3'',''A3top0'',''B0A0'',''B0B1'',''B0C0'',''B0bottom1'',''B1A1'','
 
 'B1B0'',''B1B2'',''B1C1'',''B2A2'',''B2B1'',''B2B3'',''B2C2'',''B3A3'',''B3B2'','
 'B3C3'',''B3top1'',''C0B0'',''C0C1'',''C0D0'',''C0bottom2'',''C1B1'',''C1C0'',''C1C2'','
 'C1D1'',''C2B2'',''C2C1'',''C2C3'',''C2D2'',''C3B3'',''C3C2'',''C3D3'',''C3top2'',''D0C0'',''D0D1'','
 'D0bottom3'',''D0right0'',''D1C1'',''D1D0'',''D1D2'',''D1right1'',''D2C2'',''D2D1'',''D2D3'','
 'D2right2'',''D3C3'',''D3D2'',''D3right3'',''D3top3'',''bottom0A0'',''bottom1B0'',''bottom2C0'','
 'bottom3D0'',''left0A0'',''left1A1'',''left2A2'',''left3A3'',''right0D0'',''right1D1'','
 'right2D2'',''right3D3'',''top0A3'',''top1B3'',''top2C3'',''top3D3']

In [ ]:
start_edges = ['left0A0_0','bottom2C0_0','left3A3_0','bottom3D0_0','bottom0A0'',''right0D0'',''right1D1'',''right2D2'',''top3D3','top3D3'',''right1D1'',''right2D2'',''right2D2'',''bottom2C0','right2D2'',''left1A1'',''right0D0'',''top2C3'',''left3A3'',''right1D1'',''bottom0A0'',''left1A1'',''bottom1B0'',''left2A2'',''bottom2C0'',''right2D2'',''right1D1'',''bottom1B0'',''left3A3'',''left3A3','left3A3'',''bottom1B0'',''right2D2'',''top2C3','bottom1B0'',''left3A3'',''right0D0'',''bottom0A0'',''bottom2C0'',''top3D3','right2D2','left0A0_0','bottom2C0_0','left3A3_0','bottom3D0_0','bottom0A0'',''right0D0',]
end_edges = [ 'A0bottom0_0','D0right0_0','A2left2_0','D1right1_0','A1left1'',''D3top3'',''A0left0'',''D1right1'',''D3top3'',''D3top3'',''A0left0'',''D0right0'',''D1right1'',''D3top3'',''D0bottom3'',''C0bottom2'',''A3left3'',''B0bottom1'',''C0bottom2'',''B0bottom1','D3right3'',''D3right3'',''D1right1','B0bottom1'',''D3top3'',''C3top2'',''D1right1','A3left3'',''D3top3'',''C0bottom2','D0bottom3'',''D3top3'',''A0bottom0','D0right0'',''A3top0'',''C3top2'',''A3top0','C3top2','D0right0'',''A0bottom0'',''D1right1','A0bottom0_0','D0right0_0','A2left2_0','D1right1_0','A1left1'',''D3top3']
   
print(len(start_edges))
print(len(end_edges))

In [ ]:
import xml.etree.ElementTree as ET

# 加载并解析XML文件
tree = ET.parse(r'D:\intereting\DynamicNavigation\findPath\findPath\test_data\孙倩\grid4x4\tripinfo_Grid_10.xml')
  # 请替换为你的XML文件路径
root = tree.getroot()

# 初始化变量
total_delay = 0
total_travel_time = 0
total_waiting_time = 0
trip_count = 0

# 遍历所有<tripinfo>标签
for tripinfo in root.findall('tripinfo'):
    # 计算总延误、总旅行时间和总等待时间
    total_delay += float(tripinfo.get('timeLoss'))
    total_travel_time += float(tripinfo.get('duration'))
    total_waiting_time += float(tripinfo.get('waitingTime'))
    # 增加计数器
    trip_count += 1

# 计算平均旅行时间和平均等待时间
if trip_count > 0:
    avg_travel_time = total_travel_time / trip_count
    avg_waiting_time = total_waiting_time / trip_count
else:
    avg_travel_time = 0
    avg_waiting_time = 0

# 打印结果
print(f"Total Delay (s): {total_delay:.2f}")
print(f"Total Travel Time (s): {total_travel_time:.2f}")
print(f"Average Travel Time (s): {avg_travel_time:.2f}")
print(f"Total Waiting Time (s): {total_waiting_time:.2f}")
print(f"Average Waiting Time (s): {avg_waiting_time:.2f}")


RP=0%
_____________________________________________________________________________________________________________________________ 
|Data        Method         | Total Delay(s) | Travel Time(s) | AV Travel Time(s) |  Waiting Time(s)  |AV Waiting Time(s)
-----------------------------------------------------------------------------------------------------------------------------
|Grid 4*4    XRouting       |
|            DQN navigation |
|           LLM_navigation  |
|                           |
|hangzhou    XRouting       |
|            DQN navigation |
|            LLM_navigation |
|_____________________________________________________________________________________________________________________________  

RP=10%           
_____________________________________________________________________________________________________________________________            
Data        Method         | Total Delay(s) | Travel Time(s) | AV Travel Time(s) |  Waiting Time(s)  |AV Waiting Time(s)|
-----------------------------------------------------------------------------------------------------------------------------
Grid 4*4    XRouting       |
            DQN navigation |
            LLM_navigation |
                           |
hangzhou    XRouting       |
            DQN navigation |
            LLM_navigation |
_____________________________________________________________________________________________________________________________ 

RP=20%
_____________________________________________________________________________________________________________________________            
Data        Method         | Total Delay(s) | Travel Time(s) | AV Travel Time(s) |  Waiting Time(s)  |AV Waiting Time(s)|
-----------------------------------------------------------------------------------------------------------------------------
Grid 4*4    XRouting       |
            DQN navigation |
            LLM_navigation |
                           |
hangzhou    XRouting       |
            DQN navigation |
            LLM_navigation |
_____________________________________________________________________________________________________________________________ 

RP=30%
_____________________________________________________________________________________________________________________________            
Data        Method         | Total Delay(s) | Travel Time(s) | AV Travel Time(s) |  Waiting Time(s)  |AV Waiting Time(s)|
-----------------------------------------------------------------------------------------------------------------------------
Grid 4*4    XRouting       |
            DQN navigation |
            LLM_navigation |
                           |
hangzhou    XRouting       |
            DQN navigation |
            LLM_navigation |
_____________________________________________________________________________________________________________________________ 

RP=40%
_____________________________________________________________________________________________________________________________            
Data        Method         | Total Delay(s) | Travel Time(s) | AV Travel Time(s) |  Waiting Time(s)  |AV Waiting Time(s)|
-----------------------------------------------------------------------------------------------------------------------------
Grid 4*4    XRouting       |
            DQN navigation |
            LLM_navigation |
                           |
hangzhou    XRouting       |
            DQN navigation |
            LLM_navigation |
_____________________________________________________________________________________________________________________________ 

RP=50%
_____________________________________________________________________________________________________________________________            
Data        Method         | Total Delay(s) | Travel Time(s) | AV Travel Time(s) |  Waiting Time(s)  |AV Waiting Time(s)|
-----------------------------------------------------------------------------------------------------------------------------
Grid 4*4    XRouting       |
            DQN navigation |
            LLM_navigation |
                           |
hangzhou    XRouting       |
            DQN navigation |
            LLM_navigation |
_____________________________________________________________________________________________________________________________ 

In [ ]:
start_edges = ['left3A3'',''bottom0A0'',''right0D0'',''top1B3'',''right1D1'',''right2D2'',''top3D3'',''top3D3'',''right1D1'',''top0A3'',''right2D2'',''right2D2'',''bottom2C0'',''right3D3'',''top0A3'',''right0D0'',''top0A3'',''bottom0A0'',''right2D2'',''left1A1'',''right0D0'',''top2C3'',''left3A3'',''right1D1'',''right2D2'',''left0A0'',''right3D3'',''bottom0A0'',''left1A1'',''right3D3'',''top1B3'',''bottom1B0'',''right3D3'',''left2A2'',''bottom3D0'',''bottom2C0'',''right2D2'',''right1D1'',''top1B3'',''bottom1B0'',''bottom3D0'',''left3A3'',''right3D3'',''left3A3'',''right1D1'',''top3D3'',''left3A3'',''bottom1B0'',''right2D2'',''bottom0A0'',''top2C3'',''left1A1'',''right3D3'',''bottom1B0'',''left3A3'',''right0D0'',''bottom3D0'',''left0A0'',''bottom0A0'',''bottom2C0'',''top0A3'',''top3D3'',''left0A0'',''top0A3'',''left2A2'',''right2D2'',''top1B3'',''bottom0A0'',''bottom2C0'',''left0A0'',''right3D3'',''top1B3']
end_edges = ['A2left2'',''A1left1'',''D3top3'',''A1left1'',''A0left0'',''D1right1'',''D3top3'',''D3top3'',''A0left0'',''D0right0'',''D0right0'',''D1right1'',''D3top3'',''D2right2'',''D0bottom3'',''A2left2'',''B0bottom1'',''D2right2'',''D0bottom3'',''C0bottom2'',''A3left3'',''B0bottom1'',''C0bottom2'',''B0bottom1'',''A2left2'',''A3top0'',''D1right1'',''D3right3'',''D3right3'',''A0bottom0'',''D0bottom3'',''D1right1'',''D0right0'',''B0bottom1'',''D1right1'',''D3top3'',''C3top2'',''D1right1'',''D1right1'',''A3left3'',''D1right1'',''D3top3'',''C0bottom2'',''A2left2'',''D0bottom3'',''D3top3'',''A0bottom0'',''B3top1'',''B3top1'',''D0right0'',''A3top0'',''C3top2'',''B0bottom1'',''A3top0'',''D2right2'',''B3top1'',''C3top2'',''A0bottom0'',''B0bottom1'',''D0right0'',''A0bottom0'',''D1right1'',''A2left2'',''D1right1'',''D3right3'',''D1right1'',''B0bottom1'',''D1right1'',''C0bottom2'',''A0left0'',''D2right2'',''C0bottom2']
print(len(start_edges))
print(len(end_edges))

In [ ]:
现在我确定了一个思路
但是思路又断了,我直接问孙倩要一下Xrouting的代码？？？
现在我有一点点烦,什么代码都复现不出来


我需要求助于GPT4

In [ ]:
import json
import requests
api_url= "https://gpt-api.hkust-gz.edu.cn/v1/chat/completions"
api_key = "160ecb30bc154b9dac0d0209bfae62955a5577f813cc45b9a21cb97aa1cb01d6"

def create_chat_completion(api_key','messages','temperature=0.5):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        # "model": "gpt-3.5-turbo"','# 或者其他你想要使用的模型
        "model": "gpt-4",
        "messages": messages,
        "temperature": temperature
    }
    
    response = requests.post(
        api_url,
        headers=headers,
        data=json.dumps(data)
    )
    
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        response.raise_for_status()

prompt=f'''
Step #0.00 (0ms ?*RT. ?UPS','TraCI: 50ms','vehicles TOT 0 ACT 0 BUF 0)
Connection ID DQN-grid4x4-0-veh_only-queue
Traceback (most recent call last):
  File "main.py"','line 363','in <module>
    main()
  File "main.py"','line 65','in main
    run(args)
  File "main.py"','line 156','in run
    env.add_cav()
  File "NavTL\multi_signal.py"','line 117','in add_cav
    existing_routes = self.sumo.route.getIDList()
  File "traci\domain.py"','line 174','in getIDList
    return self._getUniversal(tc.TRACI_ID_LIST','"")
  File "traci\domain.py"','line 150','in _getUniversal
    return _parse(self._retValFunc','varID','self._getCmd(varID','objectID','format','*values))
  File "traci\domain.py"','line 155','in _getCmd
    r = self._connection._sendCmd(self._cmdGetID','varID','objID','format','*values)
  File "traci\connection.py"','line 234','in _sendCmd
    return self._sendExact()
  File "traci\connection.py"','line 129','in _sendExact
    raise FatalTraCIError("Connection already closed.")
traci.exceptions.FatalTraCIError: Connection already closed.
'''
messages = [{"role": "user"','"content": prompt}]
response_text = create_chat_completion(api_key','messages)
print(response_text)

In [ ]:
start_edges = ['right1D1'',''left0A0'',''left1A1'',''left3A3'',''left0A0'',''top3D3'',''left0A0'',''left2A2'',''bottom0A0'',''bottom2C0'',''top3D3'',''right0D0'',''right1D1'',''bottom0A0'',''right1D1'',''left0A0'',''right3D3'',''bottom1B0'',''bottom3D0'',''right0D0'',''top0A3'',''top3D3'',''left0A0'',''top1B3'',''right0D0'',''left1A1'',''top0A3'',''top0A3'',''top2C3'',''top3D3'',''left1A1'',''left0A0'',''left3A3'',''right0D0'',''right2D2'',''left3A3'',''top2C3'',''right3D3'',''bottom1B0'',''left2A2'',''bottom3D0'',''bottom1B0'',''left3A3'',''bottom2C0'',''right2D2'',''right2D2'',''right1D1'',''top3D3'',''bottom2C0'',''bottom2C0'',''top1B3'',''top3D3'',''top0A3'',''top2C3'',''right2D2'',''left2A2'',''bottom3D0'',''left0A0'',''top3D3'',''bottom1B0'',''bottom1B0'',''left3A3'',''left1A1'',''bottom2C0'',''left2A2'',''left0A0'',''top1B3'',''right3D3'',''right1D1'',''right3D3'',''top3D3'',''top1B3'',''top1B3'',''left1A1'',''left1A1'',''left1A1'',''left0A0'',''bottom2C0'',''right0D0'',''bottom1B0'',''top3D3'',''top3D3'',''top2C3'',''bottom0A0'',''left0A0'',''left1A1'',''left2A2'',''left1A1'',''bottom3D0'',''top3D3'',''bottom3D0'',''right2D2'',''right0D0'',''left1A1'',''top2C3'',''top0A3'',''bottom0A0'',''top3D3'',''left3A3'',''bottom1B0'',''top0A3'',''top0A3'',''right0D0'',''left0A0'',''top2C3'',''top2C3'',''right1D1'',''left1A1'',''left1A1'',''top3D3'',''top0A3'',''right3D3'',''left3A3'',''top1B3'',''top3D3'',''left2A2'',''right1D1'',''top3D3'',''bottom1B0'',''top1B3'',''bottom1B0'',''bottom2C0'',''right0D0'',''left2A2'',''left1A1'',''left3A3'',''left3A3'',''bottom2C0'',''right2D2'',''left0A0'',''right1D1'',''left3A3'',''left2A2'',''bottom1B0'',''left1A1'',''right3D3'',''left3A3'',''bottom1B0'',''right3D3'',''bottom1B0'',''left0A0'',''top2C3'',''right3D3'',''left0A0'',''top1B3'',''bottom0A0'',''left2A2']
end_edges = ['B3top1'',''B3top1'',''A2left2'',''D1right1'',''A0bottom0'',''D2right2'',''D3top3'',''A2left2'',''B0bottom1'',''D2right2'',''A3left3'',''B0bottom1'',''A0left0'',''B0bottom1'',''C3top2'',''A1left1'',''D3top3'',''C3top2'',''D2right2'',''D1right1'',''A3top0'',''A0left0'',''B3top1'',''D3right3'',''A3top0'',''B0bottom1'',''B0bottom1'',''D3right3'',''D3right3'',''A3left3'',''C0bottom2'',''A0left0'',''A3top0'',''D0bottom3'',''D0bottom3'',''A3left3'',''A0left0'',''D0bottom3'',''D3top3'',''D1right1'',''A1left1'',''B0bottom1'',''D2right2'',''D0bottom3'',''D0bottom3'',''D3right3'',''A3top0'',''A3top0'',''D0bottom3'',''C3top2'',''B3top1'',''B3top1'',''B0bottom1'',''D3top3'',''D3top3'',''B0bottom1'',''D2right2'',''C3top2'',''D0right0'',''A0left0'',''D0bottom3'',''A3left3'',''D0bottom3'',''A0left0'',''A3top0'',''A1left1'',''B0bottom1'',''A0left0'',''D1right1'',''C3top2'',''D0bottom3'',''B3top1'',''C0bottom2'',''D1right1'',''A1left1'',''C0bottom2'',''D3right3'',''C3top2'',''A3left3'',''A2left2'',''D2right2'',''D0bottom3'',''A0bottom0'',''A3left3'',''B3top1'',''D3top3'',''D3right3'',''A0bottom0'',''D2right2'',''A0left0'',''D2right2'',''A0bottom0'',''B0bottom1'',''A2left2'',''D1right1'',''D1right1'',''A1left1'',''D3top3'',''D1right1'',''D3right3'',''D2right2'',''A1left1'',''A1left1'',''D2right2'',''D1right1'',''A3top0'',''D1right1'',''A2left2'',''C3top2'',''D0right0'',''B3top1'',''A3left3'',''D1right1'',''D3right3'',''B3top1'',''C3top2'',''D3top3'',''C0bottom2'',''A3left3'',''B3top1'',''A0left0'',''D0right0'',''D3top3'',''C0bottom2'',''A0bottom0'',''D3right3'',''B3top1'',''A3top0'',''A1left1'',''A3left3'',''C0bottom2'',''A3top0'',''D3top3'',''D3top3'',''D3right3'',''A0left0'',''C0bottom2'',''D0right0'',''B0bottom1'',''A2left2'',''D2right2'',''B3top1'',''B0bottom1'',''D3right3'',''A3top0'',''B3top1'',''A0left0']


# 使用 zip 合并列表并转换为元组列表
merged_tuple_list = list(zip(start_edges','end_edges))

print(merged_tuple_list)

In [ ]:
start_edges = ['right1D1','top2C3'',''right2D2'',''left0A0'',''right2D2']
end_edges = ['B3top1','D3right3','B3top1'',''A3top0'',''D1right1']

#现在确定一下接下来要做的工作
1.编写函数,将孙倩数据里的speed=10的车辆全部提取出来 √
2.将提取出来的车辆的起点,终点,以及depart都提取出来,放在列表里  √
3.修改代码,使得添加代码按照车辆进入的时间添加
4.编写统计函数,统计文件中的原有车辆数量,添加的车辆数量,Travel Time,AV Travel Time 统计出结果,这样就可以比对效果了

In [ ]:
departLane = ['left0A0_0','right0D0_0','left2A2_0','bottom3D0_0','left3A3_0','left3A3_0','top0A3_0','top0A3_0','bottom3D0_0','right3D3_0','left0A0_0','right3D3_0','top1B3_0','left2A2_0','left1A1_0','right3D3_0','left3A3_0','top1B3_0','right2D2_0','bottom3D0_0','top0A3_0','left1A1_0','right1D1_0','right0D0_0','top1B3_0','left0A0_0','right0D0_0','top3D3_0','bottom2C0_0','left0A0_0','right2D2_0','top0A3_0','top1B3_0','top2C3_0','bottom0A0_0','left3A3_0','left2A2_0','top1B3_0','right2D2_0','right0D0_0','bottom0A0_0','left3A3_0','right2D2_0','right3D3_0','right2D2_0','right2D2_0','right2D2_0','bottom1B0_0','top2C3_0','top0A3_0','bottom3D0_0','left3A3_0','left0A0_0','left0A0_0','left1A1_0','left3A3_0','bottom2C0_0','top1B3_0','left1A1_0','top0A3_0','bottom3D0_0','left0A0_0','right3D3_0','right2D2_0','bottom3D0_0','bottom3D0_0','top3D3_0','bottom3D0_0','top1B3_0','right3D3_0','left2A2_0','top3D3_0','right1D1_0','top0A3_0','right0D0_0','bottom0A0_0','right0D0_0','left3A3_0','left0A0_0','right2D2_0','top2C3_0','bottom0A0_0','bottom3D0_0','right3D3_0','bottom3D0_0','right0D0_0','right1D1_0','top3D3_0','left3A3_0','left3A3_0','left0A0_0','left3A3_0','top2C3_0','bottom3D0_0','bottom0A0_0','bottom3D0_0','left0A0_0','left1A1_0','top1B3_0','bottom2C0_0','right2D2_0','bottom3D0_0','right1D1_0','top2C3_0','top1B3_0','bottom1B0_0','left1A1_0','top0A3_0','right0D0_0','right2D2_0','right0D0_0','top3D3_0','right3D3_0','top2C3_0','top1B3_0','bottom1B0_0','left3A3_0','bottom3D0_0','top3D3_0','left0A0_0','left0A0_0','left0A0_0','top0A3_0','top1B3_0','left2A2_0','right2D2_0','left1A1_0','left3A3_0','bottom3D0_0','left0A0_0','bottom3D0_0','left2A2_0','top2C3_0','bottom0A0_0','left0A0_0','right1D1_0','bottom1B0_0','top3D3_0','left0A0_0','right1D1_0','bottom1B0_0','top2C3_0','top3D3_0','right0D0_0','bottom2C0_0','bottom1B0_0','bottom0A0_0','top1B3_0','right2D2_0','top0A3_0','top1B3_0','left3A3_0','left1A1_0','right3D3_0','bottom0A0_0','top2C3_0','right0D0_0','bottom2C0_0','right0D0_0','bottom0A0_0','right0D0_0','right0D0_0','right0D0_0','left3A3_0','left1A1_0','left0A0_0','right1D1_0','right2D2_0','right3D3_0','bottom0A0_0','top3D3_0','top0A3_0','bottom3D0_0','top0A3_0','right3D3_0','left0A0_0','bottom2C0_0','bottom1B0_0','top1B3_0','bottom1B0_0','left0A0_0','left0A0_0','right2D2_0','left2A2_0','right2D2_0','left0A0_0','left0A0_0','top3D3_0','top0A3_0','top0A3_0','right3D3_0','bottom2C0_0','bottom3D0_0','top3D3_0','right2D2_0','top1B3_0','bottom1B0_0','bottom0A0_0','top2C3_0','left3A3_0','left0A0_0','top3D3_0','bottom3D0_0','bottom0A0_0','bottom1B0_0','bottom0A0_0','right2D2_0','left1A1_0','bottom2C0_0','left2A2_0','top1B3_0','left3A3_0','right1D1_0','bottom0A0_0','top1B3_0','top0A3_0','left3A3_0','bottom2C0_0','bottom1B0_0','bottom2C0_0','right0D0_0','right0D0_0','bottom3D0_0','left1A1_0','left0A0_0','top1B3_0','left2A2_0','left0A0_0','top0A3_0','left0A0_0','top3D3_0','top0A3_0','bottom3D0_0','top0A3_0','bottom1B0_0','left2A2_0','right0D0_0','left2A2_0','bottom3D0_0','bottom0A0_0','left0A0_0','right0D0_0','left3A3_0','right2D2_0','left3A3_0','top3D3_0','top0A3_0','top3D3_0','top3D3_0','right0D0_0','right3D3_0','left0A0_0','bottom0A0_0','right0D0_0','right3D3_0','left3A3_0','top1B3_0','left3A3_0','bottom2C0_0','top2C3_0','left1A1_0','right1D1_0','right0D0_0','top0A3_0','bottom0A0_0','top3D3_0','top3D3_0','left2A2_0','top1B3_0','left2A2_0','left1A1_0','left0A0_0','left2A2_0','bottom0A0_0','right2D2_0','top2C3_0','top1B3_0','left2A2_0','bottom3D0_0','left1A1_0','bottom1B0_0','bottom0A0_0','bottom2C0_0','left1A1_0','top3D3_0','right0D0_0','top2C3_0','top1B3_0','left3A3_0','left3A3_0','left0A0_0','left0A0_0','top2C3_0','right3D3_0','right3D3_0','left0A0_0','bottom2C0_0','top2C3_0','left2A2_0','top3D3_0']
arrivalLane = ['left0A0_1','right0D0_1','A2A3_0','bottom3D0_2','left3A3_1','left3A3_0','top0A3_2','top0A3_1','bottom3D0_1','right3D3_1','left0A0_0','right3D3_2','B3C3_0','A2A3_0','A1A2_0','right3D3_2','left3A3_1','B3C3_1','C2C3_1','bottom3D0_2','top0A3_2','A1A2_1','D1D2_2','D0D1_2','B3A3_2','left0A0_1','right0D0_0','top3D3_0','C0B0_0','left0A0_1','B2A2_0','top0A3_1','B3A3_2','B2B3_2','bottom0A0_1','left3A3_1','A2A1_2','B3C3_1','D2D3_2','right0D0_1','bottom0A0_1','left3A3_2','B1A1_2','right3D3_0','C1D1_0','D2D3_2','D2D1_0','B0C0_2','C3D3_0','top0A3_1','bottom3D0_2','left3A3_1','left0A0_1','left0A0_2','A1A0_2','left3A3_0','C0B0_0','B3A3_2','A1A2_0','top0A3_1','bottom3D0_2','left0A0_0','right3D3_1','D2D3_2','bottom3D0_1','bottom3D0_2','top3D3_0','bottom3D0_1','C3top2_0','right3D3_2','A2A1_1','top3D3_0','D1D0_1','top0A3_2','right0D0_1','bottom0A0_1','right0D0_0','left3A3_1','left0A0_0','D2D3_2','C1D1_0','bottom0A0_0','bottom3D0_2','right3D3_2','bottom3D0_2','right0D0_1','D1D0_0','top3D3_0','left3A3_0','left3A3_1','left0A0_2','left3A3_1','C3B3_2','bottom3D0_2','bottom0A0_0','bottom3D0_0','A0bottom0_0','A1A0_2','B3C3_0','C0B0_0','C1D1_1','bottom3D0_2','B1A1_2','C3D3_0','B3C3_0','B0A0_0','A1A0_2','top0A3_1','right0D0_1','D2D1_0','right0D0_0','D3D2_0','right3D3_1','C3D3_1','B3C3_1','B0A0_0','left3A3_0','bottom3D0_2','top3D3_1','left0A0_0','left0A0_1','left0A0_1','top0A3_2','B2A2_0','A2A3_0','D2D1_0','A1A2_0','left3A3_0','bottom3D0_2','left0A0_1','bottom3D0_1','A2A1_1','C3B3_2','bottom0A0_0','left0A0_2','D1D0_0','B2B3_2','top3D3_1','left0A0_1','C2C3_2','B0A0_0','C3D3_0','D3right3_0','right0D0_0','C0B0_0','B0A0_0','A0left0_0','B3C3_0','D2D3_1','top0A3_0','B3C3_0','left3A3_0','A1A0_2','right3D3_2','bottom0A0_1','C3B3_2','right0D0_0','C0B0_0','right0D0_0','bottom0A0_0','right0D0_0','right0D0_0','right0D0_2','left3A3_2','A1A0_1','left0A0_0','D1D2_2','D2D1_0','right3D3_1','bottom0A0_0','top3D3_1','top0A3_1','bottom3D0_2','A3B3_0','right3D3_1','left0A0_2','C0D0_2','B0C0_2','B3A3_2','B0C0_2','left0A0_2','left0A0_1','B1B0_0','A2A1_2','D2D1_1','left0A0_2','left0A0_1','top3D3_0','top0A3_2','top0A3_0','right3D3_2','C0D0_2','bottom3D0_2','top3D3_1','B2B3_0','B3C3_0','C2C3_0','bottom0A0_0','C3D3_1','left3A3_0','left0A0_2','top3D3_0','bottom3D0_2','bottom0A0_0','B0A0_0','bottom0A0_0','D2D3_2','A1A0_2','B1A1_0','A2A3_0','C2C3_1','left3A3_0','D1D0_0','bottom0A0_1','B3C3_0','top0A3_1','A3left3_1','C0B0_0','B0A0_0','C0D0_2','right0D0_0','D0bottom3_0','bottom3D0_2','A1A2_0','left0A0_1','B3A3_2','A2A1_1','left0A0_1','top0A3_1','left0A0_0','top3D3_1','top0A3_2','bottom3D0_0','top0A3_2','B0A0_1','C1C0_0','D0C0_0','A2A1_2','bottom3D0_2','bottom0A0_1','left0A0_1','right0D0_0','left3A3_0','D2D3_2','left3A3_0','top3D3_1','top0A3_1','top3D3_1','top3D3_0','right0D0_0','right3D3_2','left0A0_2','bottom0A0_1','right0D0_0','right3D3_1','left3A3_0','B3A3_2','left3A3_0','C1D1_2','B2B3_1','B1B0_2','C2C3_0','right0D0_0','top0A3_1','bottom0A0_1','top3D3_2','top3D3_1','A2A3_0','B2A2_2','A2A1_1','A1A0_2','left0A0_1','B1A1_2','bottom0A0_1','D2D1_0','B1A1_0','B3A3_2','A2A1_2','bottom3D0_1','A1A0_2','B0C0_2','bottom0A0_1','C0D0_2','B1B0_0','top3D3_0','right0D0_1','C3B3_0','B3A3_2','left3A3_1','left3A3_1','left0A0_1','left0A0_1','C3B3_2','right3D3_0','right3D3_1','left0A0_1','C0D0_2','C3D3_0','A2A1_2','top3D3_0']
depart_times= [ 1145,1199','1205','1306','1329','1385','1395','1417','1495','1501','1605','1608','1659','1684','1763','1773','1809','1854','1858','1863','1992','1993','2000','2049','2051','2169','2192','2254','2377','2479','2824','2848]
  
print(len(depart_times))



In [ ]:
depart_times= [112,876,877,1199,1205,1204,1277,1145,1197,1329,1053,1417,1306,1385,1395,1495,1501,1132,1605,1608,1659,1684,1763,1773,1809,1854,1858,1863,2000,1992,1993,2049,2169,2192,2254,2051,2377,2479,2824,2848,2988]
start_edges = ['top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3','top1B3']
end_edges = ['B3A3','B3A3','B3A3','B3A3','B3A3','A3top0','B3C3','B3top1','D2right2','B3A3','D0bottom3','B3A3','A0bottom0','D3right3','D3right3','B3A3','B3A3','D0right0','B3A3','B3A3','B3A3','B3A3','B3A3','B3A3','B3A3','B3A3','B3A3','B3A3','B3A3','B3A3','B3A3','B3A3','B3A3','B3A3','D0right0','B3A3','B3A3','B3A3','B3A3','B3A3']

zipped_lists = list(zip(depart_times','start_edges','end_edges))
sorted_zipped_lists = sorted(zipped_lists','key=lambda x: x[0])

In [ ]:

print(sorted_zipped_lists)
# 假设你已经有了排序后的元组列表 sorted_zipped_lists
# 使用列表推导式拆分元组列表为原来的三个列表
depart_times,start_edges,end_edges = [
    [item[i] for item in sorted_zipped_lists] for i in range(3)
]
print(len(depart_times))
print("depart_times="','depart_times)
print("start_edges=",start_edges)
print("end_edges=",end_edges)

In [ ]:

start_edges= ['top1B3_0','bottom2C0_0','left0A0_0','right0D0_0','right3D3_0','left0A0_0','bottom3D0_0','left0A0_0','left0A0_0','left0A0_0','top0A3_0','right3D3_0','bottom2C0_0','right1D1_0','top2C3_0','bottom1B0_0','top1B3_0','top1B3_0','top3D3_0','bottom1B0_0','right3D3_0','right3D3_0','right0D0_0','bottom3D0_0','bottom2C0_0','right2D2_0','bottom0A0_0','top2C3_0','bottom1B0_0','bottom3D0_0','bottom3D0_0','bottom0A0_0','left3A3_0','top2C3_0','top3D3_0','right2D2_0','left3A3_0','top1B3_0','bottom2C0_0','right2D2_0','top1B3_0','right1D1_0','bottom3D0_0','top1B3_0','right0D0_0','right2D2_0','right2D2_0','top3D3_0','top3D3_0','top3D3_0','right2D2_0','right0D0_0','right2D2_0','bottom0A0_0','bottom1B0_0','left2A2_0','bottom2C0_0','bottom1B0_0','bottom0A0_0','top1B3_0','left0A0_0','left3A3_0','left0A0_0','bottom3D0_0','bottom2C0_0','top1B3_0','top1B3_0','right1D1_0','left3A3_0','top3D3_0','left1A1_0','left3A3_0','top3D3_0','bottom0A0_0','right0D0_0','right0D0_0','right2D2_0','left1A1_0','top1B3_0','left3A3_0','top2C3_0','right1D1_0','right1D1_0','bottom3D0_0','top1B3_0','left2A2_0','top3D3_0','right0D0_0','bottom0A0_0','left0A0_0','right3D3_0','right2D2_0','right2D2_0','left3A3_0','left0A0_0','bottom2C0_0','top1B3_0','top3D3_0','bottom3D0_0','top2C3_0','left1A1_0','top1B3_0','bottom2C0_0','top2C3_0','left0A0_0','left0A0_0','left3A3_0','bottom3D0_0','top2C3_0','left3A3_0','top2C3_0','left3A3_0','right1D1_0','right3D3_0','bottom3D0_0','top3D3_0','bottom3D0_0','top0A3_0','top1B3_0','right0D0_0','right3D3_0','bottom3D0_0','bottom0A0_0','top2C3_0','left0A0_0','top1B3_0','left3A3_0','right0D0_0','left3A3_0','bottom0A0_0','left0A0_0','left2A2_0','bottom3D0_0','bottom2C0_0','left1A1_0','left0A0_0','left3A3_0','top2C3_0','left2A2_0','bottom3D0_0','right2D2_0','left0A0_0','right0D0_0','right0D0_0','right2D2_0','top1B3_0','right0D0_0','left1A1_0','bottom3D0_0','left3A3_0','left2A2_0','top3D3_0','right1D1_0','bottom1B0_0','right3D3_0','right2D2_0','right3D3_0','top0A3_0','bottom0A0_0','left1A1_0','right3D3_0','left1A1_0','top1B3_0','top3D3_0','bottom3D0_0','right3D3_0','left3A3_0','left2A2_0','left3A3_0','bottom0A0_0','top1B3_0','top1B3_0','bottom3D0_0','top0A3_0','top3D3_0','right1D1_0','top2C3_0','top0A3_0','bottom1B0_0','left1A1_0','top0A3_0','left3A3_0','bottom2C0_0','left3A3_0','top2C3_0','left1A1_0','bottom2C0_0','bottom0A0_0','right3D3_0','right1D1_0','left3A3_0','left1A1_0','bottom1B0_0','left1A1_0','bottom1B0_0','left2A2_0','bottom0A0_0','right1D1_0','right0D0_0','right0D0_0','left3A3_0','right2D2_0','right0D0_0','top0A3_0','top2C3_0','top0A3_0','bottom0A0_0','top1B3_0','top2C3_0','top0A3_0','top0A3_0','bottom1B0_0','left0A0_0','right0D0_0','top1B3_0','top1B3_0','left3A3_0','right1D1_0','left3A3_0','left1A1_0','top0A3_0','bottom1B0_0','bottom3D0_0','bottom2C0_0','right0D0_0','top2C3_0','top2C3_0','bottom1B0_0','bottom3D0_0','right0D0_0','top0A3_0','bottom3D0_0','right1D1_0','right3D3_0','top0A3_0','right2D2_0','top2C3_0','bottom3D0_0','left3A3_0','left0A0_0','right2D2_0','bottom0A0_0','left0A0_0','bottom1B0_0','left2A2_0','top1B3_0','left1A1_0','left2A2_0','left3A3_0','right2D2_0','top1B3_0','left1A1_0','right1D1_0','left2A2_0','left0A0_0','right0D0_0','left0A0_0','bottom2C0_0','bottom1B0_0','top2C3_0','top3D3_0','bottom2C0_0','right1D1_0','top3D3_0','left2A2_0','left1A1_0','top1B3_0','top1B3_0','right2D2_0','bottom2C0_0','right0D0_0','top3D3_0','top3D3_0','top1B3_0','left1A1_0','left3A3_0','bottom3D0_0','left3A3_0','top3D3_0','right3D3_0','right3D3_0','left3A3_0','bottom2C0_0','bottom0A0_0','bottom0A0_0','left0A0_0','left2A2_0','left1A1_0','left1A1_0','right1D1_0','left1A1_0','bottom1B0_0','bottom0A0_0','top1B3_0','right1D1_0','right0D0_0','bottom1B0_0','top1B3_0','top2C3_0','bottom3D0_0','right2D2_0','bottom1B0_0','right3D3_0','top0A3_0','bottom3D0_0','left2A2_0','left3A3_0','right2D2_0','bottom2C0_0']
end_edges = ['B3A3_2','C0D0_2','left0A0_1','right0D0_0','right3D3_1','left0A0_2','bottom3D0_2','left0A0_1','left0A0_1','left0A0_2','B2A2_0','right3D3_0','C0D0_2','D1D2_2','C3B3_2','B0C0_1','B3A3_2','B3A3_2','D3C3_2','B0C0_2','right3D3_1','right3D3_1','right0D0_0','bottom3D0_0','C0D0_1','D2D3_2','bottom0A0_0','C3B3_1','B0C0_1','bottom3D0_2','bottom3D0_2','bottom0A0_1','A3A2_2','C3B3_1','top3D3_0','D2D3_2','left3A3_0','B3A3_1','C0D0_2','D2D3_2','B3A3_2','D1D2_2','D0C0_0','B3C3_0','right0D0_0','D2D3_2','D2D3_2','top3D3_0','top3D3_0','top3D3_0','D2D3_1','right0D0_0','D2D3_2','bottom0A0_0','B0C0_2','A2A1_2','C0D0_2','B0C0_2','bottom0A0_0','B3A3_1','A0bottom0_0','left3A3_0','left0A0_2','bottom3D0_2','D1right1_0','B3A3_2','B3A3_2','D1D2_1','left3A3_0','top3D3_0','A1A0_2','left3A3_1','top3D3_0','bottom0A0_0','right0D0_1','right0D0_0','D2D3_2','A1A0_2','B3A3_2','D0right0_0','C3B3_2','D1D2_2','D1D2_2','bottom3D0_1','B3A3_2','A2A1_2','top3D3_0','right0D0_1','bottom0A0_0','left0A0_2','right3D3_2','D2D3_2','D2D3_0','left3A3_0','left0A0_1','C0D0_2','B3A3_2','top3D3_0','bottom3D0_2','C3B3_1','A1A0_2','B3A3_2','C0D0_2','C3B3_2','left0A0_2','left0A0_1','left3A3_1','bottom3D0_0','C3B3_1','A3A2_1','C3B3_2','left3A3_1','D1D2_2','right3D3_1','D3top3_0','top3D3_0','bottom3D0_0','top0A3_2','B3A3_2','right0D0_0','right3D3_1','bottom3D0_0','bottom0A0_0','C3B3_1','left0A0_0','B3A3_2','left3A3_0','right0D0_0','left3A3_0','bottom0A0_0','left0A0_1','A2A1_1','bottom3D0_2','C0D0_1','A1A0_1','left0A0_1','A3A2_1','C3B3_2','A2A1_1','bottom3D0_0','D2D3_2','left0A0_0','right0D0_0','right0D0_2','D2D3_1','B3A3_0','right0D0_0','A1A0_2','bottom3D0_1','A3A2_1','A2A1_2','D3C3_2','D1D2_2','B0C0_1','right3D3_0','D2D3_1','right3D3_1','top0A3_2','A0B0_0','A1A0_2','right3D3_2','A1A0_2','B3A3_2','top3D3_0','bottom3D0_0','right3D3_2','left3A3_0','A2A1_2','left3A3_2','bottom0A0_0','B3A3_2','B3A3_2','bottom3D0_0','top0A3_2','D3C3_1','D1D2_2','C3B3_1','top0A3_2','B0C0_2','A1A0_1','top0A3_0','left3A3_2','C0D0_2','left3A3_0','C3B3_1','A1A0_2','C0D0_2','bottom0A0_0','right3D3_1','D1D2_2','left3A3_0','A1A0_2','B0C0_1','A1A0_2','B0C0_1','A2A1_2','bottom0A0_0','D1D2_2','D0D1_0','right0D0_0','left3A3_0','D2D3_2','D1right1_0','top0A3_2','C3B3_2','top0A3_1','A0B0_0','B3A3_2','C3B3_2','top0A3_2','top0A3_1','B0C0_1','left0A0_0','D0D1_2','B3A3_2','B3A3_2','left3A3_0','D1D2_2','left3A3_0','D3right3_0','top0A3_0','B0C0_2','bottom3D0_0','C0D0_2','D0D1_0','D0right0_0','C3B3_2','B0C0_2','bottom3D0_2','D0D1_2','top0A3_0','bottom3D0_0','D1D2_2','right3D3_0','top0A3_1','D2D3_1','C3B3_2','bottom3D0_1','left3A3_0','left0A0_1','D2D3_2','bottom0A0_0','left0A0_0','B0C0_1','A2A1_2','B3A3_2','A1A0_1','A2A1_2','left3A3_0','D2D3_2','B3A3_2','A1A0_2','D1D2_1','A2A1_1','left0A0_2','right0D0_0','left0A0_0','C0D0_2','B0C0_2','C3B3_2','D3C3_1','C0D0_2','D1D2_2','top3D3_0','A2A1_2','A1A0_2','D0right0_0','B3A3_2','D2D3_1','C3top2_0','right0D0_1','top3D3_1','top3D3_0','B3A3_2','A1A0_2','left3A3_0','bottom3D0_1','left3A3_0','D3C3_0','right3D3_1','right3D3_2','left3A3_0','C0D0_2','bottom0A0_0','bottom0A0_0','left0A0_1','A2A1_1','A1A0_2','A1A0_2','D1D2_2','A1A0_2','B0C0_2','bottom0A0_0','B3A3_2','D1D0_0','right0D0_0','B0C0_2','B3A3_2','C3B3_2','bottom3D0_2','D2D3_2','B0C0_1','right3D3_0','top0A3_2','bottom3D0_2','A2A1_2','left3A3_0','D2D3_2','C0D0_1']
depart_times= [112.00','140.00','287.00','443.00','474.00','488.00','535.00','547.00','604.00','695.00','638.00','774.00','767.00','819.00','820.00','863.00','876.00','877.00','892.00','926.00','978.00','1001.00','1017.00','1018.00','1009.00','1019.00','1087.00','1055.00','1096.00','1127.00','1134.00','1146.00','1133.00','1128.00','1179.00','1176.00','1219.00','1199.00','1204.00','1220.00','1205.00','1214.00','1273.00','1277.00','1322.00','1293.00','1289.00','1323.00','1351.00','1387.00','1350.00','1386.00','1388.00','1415.00','1396.00','1393.00','1394.00','1395.00','1431.00','1417.00','1447.00','1500.00','1508.00','1513.00','1448.00','1495.00','1501.00','1507.00','1534.00','1541.00','1531.00','1567.00','1572.00','1591.00','1593.00','1613.00','1590.00','1592.00','1605.00','1292.00','1603.00','1607.00','1609.00','1635.00','1608.00','1612.00','1649.00','1653.00','1663.00','1665.00','1669.00','1650.00','1651.00','1683.00','1681.00','1654.00','1659.00','1682.00','1687.00','1662.00','1668.00','1684.00','1686.00','1694.00','1726.00','1730.00','1732.00','1728.00','1712.00','1699.00','1713.00','1755.00','1731.00','1754.00','1604.00','1774.00','1772.00','1786.00','1763.00','1787.00','1788.00','1784.00','1795.00','1770.00','1789.00','1773.00','1804.00','1803.00','1813.00','1808.00','1815.00','1796.00','1825.00','1790.00','1776.00','1824.00','1775.00','1801.00','1805.00','1827.00','1807.00','1834.00','1843.00','1842.00','1808.00','1809.00','1849.00','1820.00','1853.00','1818.00','1814.00','1837.00','1844.00','1819.00','1864.00','1836.00','1870.00','1875.00','1802.00','1838.00','1880.00','1840.00','1854.00','1876.00','1879.00','1882.00','1883.00','1871.00','1888.00','1894.00','1858.00','1863.00','1892.00','1901.00','1850.00','1862.00','1877.00','1904.00','1872.00','1861.00','1898.00','1908.00','1881.00','1909.00','1873.00','1891.00','1887.00','1918.00','1930.00','1900.00','1938.00','1917.00','1874.00','1919.00','1929.00','1927.00','1963.00','1945.00','1931.00','1970.00','1989.00','1968.00','1925.00','1976.00','1977.00','2020.00','1990.00','2000.00','2004.00','2024.00','2025.00','1996.00','2023.00','2005.00','1992.00','1993.00','2037.00','2019.00','2040.00','1807.00','2044.00','2031.00','2052.00','2036.00','2039.00','1800.00','2045.00','2038.00','2084.00','2074.00','2119.00','2120.00','2094.00','2141.00','2129.00','2088.00','2126.00','2158.00','2160.00','2162.00','2142.00','2180.00','2181.00','2159.00','2173.00','2169.00','2161.00','2179.00','2230.00','2236.00','2254.00','2257.00','2258.00','2274.00','2306.00','2311.00','2321.00','2307.00','2304.00','2305.00','2319.00','2320.00','2326.00','2353.00','2333.00','2360.00','2051.00','2377.00','2391.00','2310.00','2471.00','2478.00','2492.00','2479.00','2480.00','2510.00','2516.00','2517.00','2509.00','2550.00','2551.00','2561.00','2564.00','2613.00','2614.00','2629.00','2612.00','2643.00','2649.00','2677.00','2740.00','2746.00','2768.00','2824.00','2855.00','2934.00','2970.00','2988.00','2999.00','3037.00','3118.00','3171.00','3286.00','3316.00','3317.00','3325.00','3428.00','3484.00','3483.00]

# 使用列表推导式和字符串的split方法去掉最后一个下划线和其后的字符
new_start_edges = [s.split('_')[0] for s in start_edges]
new_end_edges = [s.split('_')[0] for s in end_edges]
print(new_start_edges)
print(new_end_edges)

zipped_lists = list(zip(depart_times','start_edges','end_edges))
sorted_zipped_lists = sorted(zipped_lists','key=lambda x: x[0])

print(sorted_zipped_lists)
# 假设你已经有了排序后的元组列表 sorted_zipped_lists
# 使用列表推导式拆分元组列表为原来的三个列表
depart_times,start_edges,end_edges = [
    [item[i] for item in sorted_zipped_lists] for i in range(3)
]
print(len(depart_times))
print("depart_times="','depart_times)
print("start_edges=",start_edges)
print("end_edges=",end_edges)

In [ ]:
sorted_zipped_lists=[(1019.0',''right2D2_0'',''D2D3_2'),(1176.0',''right2D2_0'',''D2D3_2'),(1220.0',''right2D2_0'',''D2D3_2'),(1289.0',''right2D2_0'',''D2D3_2')','(1293.0',''right2D2_0'',''D2D3_2')','(1350.0',''right2D2_0'',''D2D3_1'),(1388.0',''right2D2_0'',''D2D3_2'),(1590.0',''right2D2_0'',''D2D3_2')]

depart_times,start_edges,end_edges = [
    [item[i] for item in sorted_zipped_lists] for i in range(3)
]
print(len(depart_times))
print("depart_times="','depart_times)
print("start_edges=",start_edges)
print("end_edges=",end_edges)


new_start_edges = [s.split('_')[0] for s in start_edges]
new_end_edges = [s.split('_')[0] for s in end_edges]
print("start_edges=",new_start_edges)
print("end_edges=",new_end_edges)

In [ ]:
sorted_zipped_lists=[(820.0',''top2C3_0'',''C3B3_2'),(1055.0',''top2C3_0'',''C3B3_1')','(1128.0',''top2C3_0'',''C3B3_1'),(1603.0,'top2C3_0'',''C3B3_2')
                 ','(1662.0',''top2C3_0'',''C3B3_1')','(1694.0',''top2C3_0'',''C3B3_2')','(1712.0',''top2C3_0'',''C3B3_1'),(1713.0',''top2C3_0'',''C3B3_2')
                 ','(1770.0',''top2C3_0'',''C3B3_1')','(1800.0',''top2C3_0'',''D0right0_0')','(1801.0',''top2C3_0'',''C3B3_2')','(1873.0',''top2C3_0'',''C3B3_1')
                 ','(1877.0',''top2C3_0'',''C3B3_1'),(1977.0',''top2C3_0'',''C3B3_2'),(2004.0',''top2C3_0','C3B3_2')','(2045.0',''top2C3_0'',''C3B3_2')
                 ','(2126.0',''top2C3_0'',''C3B3_2')','(2305.0',''top2C3_0'',''C3B3_2'),(2746.0',''top2C3_0'',''C3B3_2')]
depart_times,start_edges,end_edges = [
    [item[i] for item in sorted_zipped_lists] for i in range(3)
]
print(len(depart_times))
print("depart_times=",depart_times)
print("start_edges=",start_edges)
print("end_edges=",end_edges)


new_start_edges = [s.split('_')[0] for s in start_edges]
new_end_edges = [s.split('_')[0] for s in end_edges]
print("start_edges=",new_start_edges)
print("end_edges=",new_end_edges)

In [ ]:
start_edges = ['right0D0_0','right1D1_0','left3A3_0','right3D3_0','left0A0_0','right1D1_0','top3D3_0','top0A3_0','right0D0_0','right0D0_0','bottom0A0_0','right2D2_0','left1A1_0','bottom0A0_0','left0A0_0','top3D3_0','top1B3_0','left3A3_0','top2C3_0','left2A2_0','bottom1B0_0','bottom2C0_0','right3D3_0','left0A0_0','left0A0_0','top0A3_0','bottom1B0_0','right0D0_0','right2D2_0','left3A3_0','left0A0_0','left3A3_0','top0A3_0','right1D1_0','top2C3_0','bottom3D0_0','bottom0A0_0','top1B3_0','right3D3_0','top2C3_0','bottom2C0_0','bottom2C0_0','right2D2_0','bottom2C0_0','left1A1_0','bottom2C0_0','right1D1_0','bottom0A0_0','bottom0A0_0','right2D2_0','left0A0_0','right2D2_0','left2A2_0','left0A0_0','top0A3_0','left1A1_0','left2A2_0','top0A3_0','left1A1_0','right1D1_0','bottom3D0_0','right0D0_0','bottom3D0_0','top0A3_0','right2D2_0','right2D2_0','left1A1_0','top3D3_0','right3D3_0','right1D1_0','bottom3D0_0','right0D0_0','top0A3_0','left3A3_0','bottom0A0_0','bottom2C0_0','top0A3_0','left0A0_0','top1B3_0','top1B3_0','bottom1B0_0','right1D1_0','bottom3D0_0','left2A2_0','top1B3_0','right1D1_0','left0A0_0','top3D3_0','top2C3_0','left3A3_0','bottom2C0_0','left1A1_0','right0D0_0','right2D2_0','bottom1B0_0','top2C3_0','left1A1_0','bottom0A0_0','bottom3D0_0','bottom0A0_0','left3A3_0','top2C3_0','left1A1_0','bottom0A0_0','bottom2C0_0','right3D3_0','left3A3_0','right1D1_0','left0A0_0','top0A3_0','top2C3_0','right1D1_0','left1A1_0','left2A2_0','top3D3_0','bottom2C0_0','top0A3_0','top0A3_0','bottom3D0_0','left2A2_0','left3A3_0','top0A3_0','right0D0_0','left1A1_0','bottom3D0_0','bottom1B0_0','bottom3D0_0','right3D3_0','top1B3_0','left0A0_0','left0A0_0','right0D0_0','top0A3_0','right0D0_0','right3D3_0','top2C3_0','right3D3_0','right3D3_0','top1B3_0','left3A3_0','top1B3_0','bottom3D0_0','right3D3_0','top3D3_0','right2D2_0','right2D2_0','right2D2_0','left3A3_0','left3A3_0','left3A3_0','left1A1_0','bottom3D0_0','left2A2_0','top2C3_0','left1A1_0','top3D3_0','left3A3_0','top2C3_0','right2D2_0','right1D1_0','top0A3_0','top1B3_0','bottom0A0_0','bottom0A0_0','top1B3_0','right0D0_0','left3A3_0','top3D3_0','bottom3D0_0','top1B3_0','left0A0_0','left0A0_0','right3D3_0','bottom3D0_0','bottom1B0_0','left0A0_0','top1B3_0','right2D2_0','left1A1_0','top2C3_0','top3D3_0','left1A1_0','top0A3_0','bottom2C0_0','right0D0_0','left3A3_0','left1A1_0','left0A0_0','top1B3_0','right2D2_0','top3D3_0','bottom1B0_0','right2D2_0','right2D2_0','right3D3_0','left0A0_0','bottom1B0_0','bottom1B0_0','right0D0_0','left3A3_0','top0A3_0','top0A3_0','left0A0_0','right3D3_0','right2D2_0','top3D3_0','right1D1_0','left3A3_0','top1B3_0','top2C3_0','bottom2C0_0','bottom0A0_0','top0A3_0','right1D1_0','right1D1_0','left0A0_0','right0D0_0','top0A3_0','bottom1B0_0','top1B3_0','bottom1B0_0','top3D3_0','left2A2_0','bottom2C0_0','right1D1_0','top1B3_0','left2A2_0','left0A0_0','bottom1B0_0','right1D1_0','bottom1B0_0','top0A3_0','right0D0_0','bottom1B0_0','top1B3_0','right2D2_0','left0A0_0','left2A2_0','right3D3_0','bottom3D0_0','left0A0_0','left0A0_0','left3A3_0','bottom1B0_0','bottom3D0_0','left3A3_0','top2C3_0','left3A3_0','right0D0_0','right0D0_0','bottom0A0_0','top2C3_0','right1D1_0','right3D3_0','right3D3_0','right0D0_0','right3D3_0','left0A0_0','top0A3_0','bottom2C0_0','bottom0A0_0','left2A2_0','left1A1_0','bottom0A0_0','bottom0A0_0','bottom2C0_0','bottom3D0_0','right1D1_0','left2A2_0','left0A0_0','left0A0_0','bottom0A0_0','top0A3_0','left0A0_0','right3D3_0','right0D0_0','left0A0_0','top0A3_0','left0A0_0','left2A2_0','right3D3_0','right0D0_0','top3D3_0','top2C3_0','right3D3_0','left3A3_0','bottom3D0_0','left1A1_0','left0A0_0','right3D3_0','right1D1_0','bottom3D0_0','left0A0_0','top1B3_0','left0A0_0','left3A3_0','top2C3_0','right2D2_0','top3D3_0','bottom3D0_0','bottom1B0_0','left3A3_0','right3D3_0','top2C3_0','bottom2C0_0','right1D1_0','left1A1_0','right0D0_0','top2C3_0']
end_edges = ['right0D0_0','D1D2_2','left3A3_0','right3D3_1','left0A0_2','D1D2_2','D3C3_1','top0A3_1','right0D0_0','right0D0_0','A0B0_2','D2D3_2','A1A0_2','A0B0_2','left0A0_1','top3D3_0','B3A3_2','left3A3_0','C3B3_2','A2A1_2','B0C0_2','C0D0_2','right3D3_2','left0A0_1','left0A0_1','top0A3_0','B0C0_2','right0D0_1','D2D3_2','left3A3_0','left0A0_1','A3A2_1','A3left3_0','D1D2_2','C3B3_2','bottom3D0_0','A0B0_1','B3A3_2','right3D3_0','C3B3_2','C0D0_2','C0D0_2','D2D3_2','C0D0_2','A1A0_2','C0D0_2','D1D2_1','A0B0_2','A3left3_0','D2D3_2','left0A0_2','D2D3_2','A2A1_2','left0A0_1','top0A3_2','A1A0_2','A2A1_2','top0A3_1','A1A0_2','D1D2_2','D0right0_0','right0D0_0','bottom3D0_1','top0A3_1','D2D3_1','D2D3_2','A1A0_2','top3D3_0','right3D3_1','D1D2_0','bottom3D0_1','D0D1_2','top0A3_1','left3A3_0','A0B0_0','C0D0_2','top0A3_1','left0A0_0','B3A3_1','B3A3_2','B0C0_2','D1D2_2','bottom3D0_2','A2A1_2','B3A3_2','D1D2_2','left0A0_2','top3D3_0','C3B3_1','left3A3_1','C0D0_2','A1A0_2','right0D0_0','D2D3_2','B0C0_0','C3B3_2','A1A0_2','bottom0A0_0','bottom3D0_0','bottom0A0_2','A3A2_1','C3B3_2','A1A0_1','A0B0_1','C0D0_1','right3D3_1','left3A3_1','D1D2_0','left0A0_1','top0A3_1','C3B3_2','D1D2_1','A1A0_1','A2A1_2','C3D3_0','C0D0_0','top0A3_1','top0A3_2','bottom3D0_2','A2A1_2','left3A3_0','A3left3_0','right0D0_1','A1A0_2','bottom3D0_0','B0C0_2','bottom3D0_2','right3D3_1','B3A3_2','left0A0_0','left0A0_2','right0D0_0','top0A3_2','right0D0_0','right3D3_1','C3B3_2','right3D3_1','right3D3_1','B3A3_2','A3A2_0','B3A3_2','bottom3D0_1','right3D3_1','D3C3_2','D2D3_1','D2D3_2','D2D3_0','left3A3_0','left3A3_0','left3A3_2','A1A0_2','bottom3D0_1','A2A1_2','C3B3_2','A1A0_2','D3C3_1','left3A3_0','C3B3_2','D2D3_0','D1D2_2','top0A3_0','B3A3_2','bottom0A0_0','bottom0A0_0','B3A3_2','right0D0_0','left3A3_0','top3D3_0','bottom3D0_0','B3A3_2','left0A0_0','left0A0_1','right3D3_2','bottom3D0_0','B0C0_2','left0A0_0','B3A3_2','D2D3_1','A1A0_1','C3B3_2','D3C3_0','A1A0_2','top0A3_0','C0D0_2','D0D1_1','left3A3_0','A1A0_0','left0A0_0','B3A3_2','D2D3_2','D3C3_1','B0C0_2','D2D3_2','A2left2_0','right3D3_1','left0A0_1','A2left2_0','B0C0_2','right0D0_0','A3A2_2','top0A3_2','top0A3_0','left0A0_1','right3D3_0','D2D3_2','top3D3_0','D3top3_0','A3A2_1','B3A3_2','C3B3_2','C0D0_2','bottom0A0_0','top0A3_0','D1D2_2','D1D2_2','left0A0_0','right0D0_0','top0A3_2','B0C0_2','B3A3_2','B0C0_2','D3C3_1','A2A1_1','C0D0_2','D1D2_2','B3A3_2','A2A1_1','left0A0_1','B0C0_2','D1D2_2','B0C0_2','top0A3_0','right0D0_0','B0C0_2','B3A3_1','D2D3_2','left0A0_1','A2A1_2','B0bottom1_0','bottom3D0_0','left0A0_0','left0A0_1','left3A3_0','B0C0_2','B3top1_0','A3A2_0','C3B3_2','left3A3_0','right0D0_0','right0D0_0','A0B0_2','C3B3_2','D1D2_2','right3D3_2','right3D3_2','right0D0_0','right3D3_0','left0A0_0','top0A3_1','C0D0_2','A0B0_2','A2A1_1','A1A0_2','A0B0_1','bottom0A0_0','C0D0_2','bottom3D0_1','D1D2_1','A2A1_2','left0A0_2','left0A0_2','A0B0_1','top0A3_1','left0A0_1','right3D3_2','D0D1_1','left0A0_0','top0A3_1','left0A0_1','A2A1_0','right3D3_1','right0D0_0','top3D3_0','C3B3_2','right3D3_1','left3A3_0','bottom3D0_1','A1A0_2','left0A0_1','right3D3_2','D1D2_2','bottom3D0_0','left0A0_2','B3A3_2','left0A0_2','left3A3_0','C3B3_1','D2D3_2','D3C3_2','bottom3D0_1','B0C0_2','left3A3_0','right3D3_2','B3top1_0','C0D0_2','D1D2_2','A1A0_2','right0D0_0','C3B3_1']
depart = [140.00, 112.00, 287.00, 443.00, 475.00, 474.00, 488.00, 535.00, 547.00, 604.00, 638.00, 695.00, 767.00, 774.00, 819.00, 820.00, 863.00, 892.00, 877.00, 876.00, 926.00, 978.00, 1009.00, 1017.00, 1019.00, 1018.00, 1001.00, 1055.00, 1087.00, 1127.00, 1128.00, 1096.00, 1134.00, 1133.00, 1146.00, 1179.00, 1161.00, 1199.00, 1219.00, 1204.00, 1205.00, 1214.00, 1220.00, 1272.00, 1277.00, 1289.00, 1290.00, 1291.00, 1178.00, 1322.00, 1351.00, 1329.00, 1350.00, 1394.00, 1417.00, 1393.00, 1396.00, 1431.00, 1387.00, 1447.00, 1448.00, 1495.00, 1507.00, 1508.00, 1486.00, 1501.00, 1500.00, 1541.00, 1566.00, 1534.00, 1567.00, 1557.00, 1590.00, 1591.00, 1571.00, 1572.00, 1601.00, 1612.00, 1592.00, 1603.00, 1602.00, 1607.00, 1635.00, 1613.00, 1609.00, 1608.00, 1655.00, 1656.00, 1634.00, 1668.00, 1651.00, 1654.00, 1681.00, 1653.00, 1649.00, 1665.00, 1658.00, 1682.00, 1685.00, 1687.00, 1663.00, 1669.00, 1662.00, 1683.00, 1684.00, 1730.00, 1731.00, 1694.00, 1727.00, 1728.00, 1713.00, 1699.00, 1712.00, 1732.00, 1650.00, 1744.00, 1772.00, 1774.00, 1773.00, 1754.00, 1776.00, 1756.00, 1790.00, 1763.00, 1787.00, 1770.00, 1789.00, 1795.00, 1775.00, 1788.00, 1799.00, 1805.00, 1805.00, 1808.00, 1813.00, 1786.00, 1815.00, 1818.00, 1784.00, 1796.00, 1806.00, 1830.00, 1834.00, 1807.00, 1798.00, 1809.00, 1808.00, 1842.00, 1837.00, 1843.00, 1825.00, 1854.00, 1826.00, 1828.00, 1827.00, 1836.00, 1861.00, 1820.00, 1838.00, 1840.00, 1862.00, 1841.00, 1872.00, 1875.00, 1844.00, 1873.00, 1878.00, 1874.00, 1881.00, 1853.00, 1891.00, 1894.00, 1895.00, 1898.00, 1870.00, 1901.00, 1864.00, 1858.00, 1863.00, 1883.00, 1871.00, 1888.00, 1919.00, 1908.00, 1900.00, 1930.00, 1916.00, 1938.00, 1918.00, 1917.00, 1876.00, 1926.00, 1937.00, 1818.00, 1967.00, 1970.00, 1804.00, 1945.00, 1990.00, 1931.00, 1991.00, 1992.00, 1997.00, 1993.00, 1976.00, 2004.00, 1893.00, 1977.00, 1989.00, 1996.00, 2000.00, 2019.00, 2024.00, 2005.00, 2020.00, 2040.00, 2045.00, 2051.00, 2025.00, 2037.00, 2031.00, 2039.00, 2010.00, 2038.00, 2052.00, 2047.00, 2054.00, 2084.00, 2074.00, 2088.00, 2094.00, 2129.00, 2141.00, 2119.00, 2120.00, 2126.00, 2158.00, 2142.00, 1834.00, 2169.00, 2180.00, 2181.00, 2192.00, 2173.00, 1969.00, 2159.00, 2172.00, 2230.00, 2254.00, 2257.00, 2236.00, 2258.00, 2274.00, 2304.00, 2308.00, 2309.00, 2311.00, 2320.00, 2332.00, 2305.00, 2307.00, 2306.00, 2319.00, 2326.00, 2353.00, 2336.00, 2377.00, 2360.00, 2391.00, 2471.00, 2479.00, 2480.00, 2510.00, 2517.00, 2516.00, 2492.00, 2551.00, 2561.00, 2550.00, 2564.00, 2612.00, 2614.00, 2629.00, 2613.00, 2649.00, 2643.00, 2673.00, 2677.00, 2740.00, 2768.00, 2746.00, 2824.00, 2848.00, 2855.00, 2934.00, 2970.00, 2988.00, 2999.00, 3037.00, 3118.00, 3171.00, 3317.00, 3325.00, 3286.00, 3316.00, 3429.00, 3428.00, 3484.00, 3483.00]

new_start_edges = [s.split('_')[0] for s in start_edges]
new_end_edges = [s.split('_')[0] for s in end_edges]
print(new_start_edges)
print(new_end_edges)

zipped_lists = list(zip(depart_times,start_edges,end_edges))
sorted_zipped_lists = sorted(zipped_lists,key=lambda x: x[0])

depart_times,start_edges,end_edges = [
    [item[i] for item in sorted_zipped_lists] for i in range(3)
]
print(len(depart_times))
print("depart_times=",depart_times)
print("start_edges=",start_edges)
print("end_edges=",end_edges)

In [ ]:
end_edges = ['road_0_4_0_0','road_1_5_3_0','road_4_0_1_0','road_4_0_1_0','road_1_5_3_0','road_5_4_2_0','road_5_4_2_0','road_1_0_1_0','road_5_2_2_0','road_4_0_1_0','road_0_2_0_0','road_5_4_2_0']
start_edges = ['road_1_2_2_0','road_1_3_2_0','road_4_1_0_0','road_4_2_0_0','road_1_4_2_0','road_2_4_1_0','road_4_4_1_0','road_1_2_2_0','road_4_3_0_1','road_4_2_0_0','road_1_1_2_0','road_4_4_1_0']
depart = [27.00, 79.00, 331.00, 302.00, 433.00, 118.00, 517.00, 302.00, 553.00, 602.00, 565.00, 631.00]
zipped_lists = list(zip(depart,start_edges,end_edges))
sorted_zipped_lists = sorted(zipped_lists,key=lambda x: x[0])
print(zipped_lists )
depart_times,start_edges,end_edges = [
    [item[i] for item in sorted_zipped_lists] for i in range(3)
]
print(len(depart_times))
print("depart_times=",depart_times)
print("start_edges=",start_edges)
print("end_edges=",end_edges)

In [ ]:
depart_times= [27.00, 79.00, 331.00, 302.00, 433.00, 118.00, 517.00, 302.00, 553.00, 602.00, 565.00, 631.00]
start_edges = ['road_4_0_1', 'road_5_4_2', 'road_5_4_2', 'road_0_1_0', 'road_0_4_0', 'road_5_4_2', 'road_5_4_2', 'road_0_1_0', 'road_0_1_0', 'road_1_0_1', 'road_4_0_1', 'road_0_4_0', 'road_4_0_1', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_5_2_2', 'road_5_4_2', 'road_5_4_2', 'road_1_0_1', 'road_1_5_3', 'road_0_1_0', 'road_0_1_0', 'road_0_1_0', 'road_1_5_3', 'road_5_3_2', 'road_0_1_0', 'road_0_4_0', 'road_5_2_2', 'road_5_2_2', 'road_0_4_0', 'road_0_4_0', 'road_5_2_2', 'road_5_3_2', 'road_1_0_1', 'road_5_4_2', 'road_5_3_2', 'road_0_4_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_5_4_2', 'road_5_4_2', 'road_0_1_0', 'road_5_4_2', 'road_5_2_2', 'road_4_0_1', 'road_1_0_1', 'road_5_2_2', 'road_5_4_2', 'road_0_4_0', 'road_0_2_0', 'road_0_4_0', 'road_5_4_2', 'road_1_0_1', 'road_0_4_0', 'road_0_1_0', 'road_0_2_0', 'road_5_3_2', 'road_0_4_0', 'road_5_2_2', 'road_1_5_3', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_1_5_3', 'road_4_0_1', 'road_5_4_2', 'road_0_4_0', 'road_1_0_1', 'road_0_4_0', 'road_5_3_2', 'road_5_3_2', 'road_5_2_2', 'road_4_0_1', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_0_1_0', 'road_0_4_0', 'road_0_4_0', 'road_0_4_0', 'road_5_4_2']
end_edges = ['road_4_2_0', 'road_1_3_2', 'road_1_4_1', 'road_3_4_1', 'road_1_3_2', 'road_4_4_1', 'road_2_4_1', 'road_4_1_3', 'road_1_1_3', 'road_2_1_3', 'road_4_1_3', 'road_4_4_0', 'road_4_1_0', 'road_1_1_3', 'road_4_4_1', 'road_4_4_1', 'road_3_1_3', 'road_4_3_0', 'road_3_4_1', 'road_1_2_2', 'road_1_4_1', 'road_1_3_2', 'road_4_1_0', 'road_3_1_3', 'road_4_1_0', 'road_1_3_2', 'road_1_3_2', 'road_4_1_3', 'road_1_1_3', 'road_4_4_1', 'road_4_3_0', 'road_4_4_0', 'road_1_3_2', 'road_4_4_0', 'road_1_4_2', 'road_1_1_2', 'road_1_4_2', 'road_3_4_1', 'road_1_1_2', 'road_4_1_3', 'road_1_4_1', 'road_1_1_2', 'road_4_1_3', 'road_2_4_1', 'road_3_4_1', 'road_3_4_1', 'road_2_1_3', 'road_3_4_1', 'road_4_4_1', 'road_4_3_3', 'road_4_1_0', 'road_1_2_2', 'road_1_4_2', 'road_1_3_2', 'road_3_1_3', 'road_1_2_2', 'road_4_4_1', 'road_1_1_2', 'road_1_3_2', 'road_4_1_0', 'road_1_1_2', 'road_1_3_2', 'road_4_4_1', 'road_4_3_0', 'road_4_1_3', 'road_1_2_2', 'road_1_4_1', 'road_1_1_3', 'road_4_1_3', 'road_1_2_2', 'road_2_1_3', 'road_4_1_0', 'road_4_1_3', 'road_1_4_1', 'road_4_2_0', 'road_4_3_0', 'road_4_4_1', 'road_4_4_0', 'road_3_4_1', 'road_4_2_0', 'road_4_4_0', 'road_4_4_0', 'road_1_1_3', 'road_1_1_3', 'road_4_4_0', 'road_1_3_2', 'road_1_4_1', 'road_3_4_1']
zipped_lists = list(zip(depart_times,start_edges,end_edges))
sorted_zipped_lists = sorted(zipped_lists,key=lambda x: x[0])

print(sorted_zipped_lists)

In [ ]:
sorted_zipped_lists=[(27.0, 'road_4_0_1', 'road_4_2_0'), (79.0, 'road_5_4_2', 'road_1_3_2'), (118.0, 'road_5_4_2', 'road_4_4_1'), (302.0, 'road_0_1_0', 'road_3_4_1'), (302.0, 'road_0_1_0', 'road_4_1_3'), (331.0, 'road_5_4_2', 'road_1_4_1'), (433.0, 'road_0_4_0', 'road_1_3_2'), (517.0, 'road_5_4_2', 'road_2_4_1'),  (553.0, 'road_0_1_0', 'road_1_1_3'), (565.0, 'road_4_0_1', 'road_4_1_3'), (602.0, 'road_1_0_1', 'road_2_1_3')]


depart_times,start_edges,end_edges = [
    [item[i] for item in sorted_zipped_lists] for i in range(3)
]
print(len(depart_times))
print("depart_times=",depart_times)
print("start_edges=",start_edges)
print("end_edges=",end_edges)

In [ ]:
    depart_times= [302.00, 79.0, 118.0]
    start_edges= ['road_4_0_1', 'road_5_4_2', 'road_5_4_2']
    end_edges= ['road_4_2_0', 'road_1_3_2', 'road_4_4_1']

In [ ]:

<tripinfo id="veh_2" depart="118.00" departLane="road_5_4_2_0" departPos="0.00" departSpeed="10.00" departDelay="0.00" arrival="250.00" arrivalLane="road_4_4_1_0" arrivalPos="586.40" arrivalSpeed="10.66" duration="132.00" routeLength="1381.83" waitingTime="0.00" waitingCount="0" stopTime="0.00" timeLoss="7.91" rerouteNo="0" devices="tripinfo_veh_2 emissions_veh_2" vType="DEFAULT_VEHTYPE" speedFactor="1.01" vaporized="">
 <tripinfo id="cav_32" depart="118.00" departLane="road_5_4_2_0" departPos="0.00" departSpeed="10.00" departDelay="0.00" arrival="591.00" arrivalLane="road_2_4_1_0" arrivalPos="586.40" arrivalSpeed="11.91" duration="473.00" routeLength="4155.87" waitingTime="88.00" waitingCount="8" stopTime="0.00" timeLoss="136.46" rerouteNo="115" devices="tripinfo_cav_32" vType="DEFAULT_VEHTYPE" speedFactor="1.12" vaporized=""/>

<tripinfo id="veh_0" depart="30.00" departLane="road_4_0_1_0" departPos="0.00" departSpeed="10.00" departDelay="3.00" arrival="339.00" arrivalLane="road_4_2_0_0" arrivalPos="786.40" arrivalSpeed="11.46" duration="309.00" routeLength="1982.00" waitingTime="123.00" waitingCount="1" stopTime="0.00" timeLoss="139.90" rerouteNo="0" devices="tripinfo_veh_0 emissions_veh_0" vType="DEFAULT_VEHTYPE" speedFactor="1.06" vaporized="">

<tripinfo id="veh_1" depart="81.00" departLane="road_5_4_2_0" departPos="0.00" departSpeed="10.00" departDelay="2.00" arrival="776.00" arrivalLane="road_1_3_2_0" arrivalPos="786.40" arrivalSpeed="10.26" duration="695.00" routeLength="4584.99" waitingTime="201.00" waitingCount="4" stopTime="0.00" timeLoss="254.42" rerouteNo="0" devices="tripinfo_veh_1 emissions_veh_1" vType="DEFAULT_VEHTYPE" speedFactor="0.94" vaporized="">




In [ ]:
(302.0, 'road_4_2_0_0', 'road_4_0_1_0')
(602.0, 'road_4_2_0_0', 'road_4_0_1_0')
(631.0, 'road_4_4_1_0', 'road_5_4_2_0')

In [ ]:
start_edges = ['road_0_1_0_0','road_0_4_0_0','road_5_4_2_0','road_0_1_0_0','road_5_4_2_0','road_4_0_1_0','road_4_0_1_0','road_5_4_2_0','road_4_0_1_0','road_0_2_0_0','road_5_3_2_0','road_5_4_2_0','road_5_3_2_0','road_0_4_0_0','road_0_1_0_0','road_0_1_0_0','road_5_4_2_0','road_0_4_0_0','road_0_1_0_0','road_1_5_3_0','road_0_1_0_0','road_1_5_3_0','road_1_5_3_0','road_0_4_0_0','road_5_3_2_0','road_5_4_2_0','road_4_0_1_0','road_1_5_3_0','road_0_1_0_0','road_0_1_0_0','road_4_0_1_0','road_0_4_0_0','road_0_4_0_0','road_0_4_0_0','road_5_4_2_0','road_5_3_2_0','road_0_4_0_0','road_5_4_2_0','road_0_1_0_0','road_4_0_1_0','road_0_1_0_0','road_5_3_2_0','road_1_5_3_0','road_4_0_1_0','road_4_0_1_0','road_0_4_0_0','road_1_0_1_0','road_5_4_2_0','road_0_4_0_0','road_0_1_0_0','road_5_4_2_0','road_4_0_1_0','road_5_4_2_0','road_1_0_1_0','road_0_1_0_0','road_1_5_3_0','road_0_1_0_0','road_4_0_1_0','road_0_4_0_0','road_1_5_3_0','road_4_0_1_0','road_0_1_0_0','road_4_0_1_0','road_5_3_2_0','road_4_0_1_0','road_0_4_0_0','road_0_4_0_0','road_0_4_0_0','road_1_0_1_0','road_0_1_0_0','road_0_4_0_0','road_5_3_2_0','road_5_4_2_0','road_0_1_0_0','road_5_4_2_0','road_1_0_1_0','road_0_1_0_0','road_1_5_3_0','road_0_1_0_0','road_0_4_0_0','road_1_5_3_0','road_0_1_0_0','road_5_3_2_0','road_4_0_1_0','road_0_4_0_0','road_4_0_1_0','road_5_4_2_0','road_5_4_2_0','road_0_4_0_0','road_5_3_2_0','road_5_4_2_0','road_0_4_0_0','road_5_3_2_0','road_1_0_1_0','road_5_3_2_0','road_1_0_1_0','road_4_0_1_0','road_0_4_0_0','road_1_5_3_0','road_0_2_0_0','road_5_4_2_0','road_0_1_0_0','road_5_2_2_0','road_5_4_2_0','road_5_4_2_0','road_5_4_2_0','road_4_0_1_0','road_0_4_0_0','road_5_4_2_0','road_5_3_2_0','road_0_1_0_0','road_5_2_2_0','road_0_4_0_0','road_4_0_1_0','road_1_0_1_0','road_1_5_3_0','road_0_4_0_0','road_5_3_2_0','road_1_5_3_0','road_0_4_0_0','road_0_1_0_0','road_4_0_1_0','road_4_0_1_0','road_1_5_3_0','road_0_4_0_0','road_5_2_2_0','road_5_3_2_0','road_5_4_2_0','road_1_0_1_0','road_4_0_1_0','road_4_0_1_0','road_4_0_1_0','road_5_4_2_0','road_1_0_1_0','road_0_2_0_0','road_1_5_3_0','road_0_4_0_0','road_1_0_1_0','road_0_2_0_0','road_5_4_2_0','road_0_1_0_0','road_5_4_2_0','road_0_4_0_0','road_0_1_0_0','road_5_4_2_0','road_5_4_2_0','road_5_4_2_0','road_0_2_0_0','road_0_4_0_0','road_5_4_2_0','road_5_4_2_0','road_0_2_0_0','road_5_2_2_0','road_0_4_0_0','road_5_3_2_0','road_5_3_2_0','road_5_4_2_0','road_1_5_3_0','road_0_1_0_0','road_1_5_3_0','road_5_4_2_0','road_0_4_0_0','road_0_1_0_0','road_1_0_1_0','road_1_0_1_0','road_0_4_0_0','road_5_4_2_0','road_1_0_1_0','road_0_1_0_0','road_0_4_0_0','road_4_0_1_0','road_5_2_2_0','road_5_4_2_0','road_4_0_1_0','road_5_3_2_0','road_0_4_0_0','road_5_4_2_0','road_0_4_0_0','road_5_3_2_0','road_5_4_2_0','road_0_4_0_0','road_4_0_1_0','road_0_1_0_0','road_1_0_1_0','road_0_2_0_0','road_0_4_0_0','road_5_4_2_0','road_5_3_2_0','road_1_5_3_0','road_1_0_1_0','road_0_1_0_0','road_0_2_0_0','road_5_2_2_0','road_5_4_2_0','road_1_0_1_0','road_0_2_0_0','road_0_4_0_0','road_0_1_0_0','road_4_0_1_0','road_5_4_2_0','road_0_1_0_0','road_1_0_1_0','road_0_2_0_0','road_0_2_0_0','road_4_0_1_0','road_0_2_0_0','road_5_3_2_0','road_4_0_1_0','road_5_3_2_0','road_0_4_0_0','road_0_4_0_0','road_0_4_0_0','road_5_4_2_0','road_0_4_0_0','road_5_4_2_0','road_1_0_1_0','road_0_4_0_0','road_5_4_2_0','road_5_4_2_0','road_0_4_0_0','road_1_0_1_0','road_1_5_3_0','road_5_4_2_0','road_4_0_1_0','road_5_4_2_0','road_4_0_1_0','road_4_0_1_0','road_1_0_1_0','road_5_4_2_0','road_0_1_0_0','road_0_2_0_0','road_5_3_2_0','road_0_4_0_0','road_0_4_0_0','road_5_2_2_0','road_1_5_3_0','road_0_1_0_0','road_1_0_1_0','road_5_3_2_0','road_5_4_2_0','road_0_2_0_0','road_5_2_2_0','road_0_4_0_0','road_0_4_0_0','road_0_2_0_0','road_5_4_2_0','road_0_1_0_0','road_0_1_0_0','road_0_4_0_0','road_0_4_0_0','road_0_2_0_0','road_1_5_3_0','road_5_4_2_0','road_5_3_2_0','road_1_0_1_0','road_0_2_0_0','road_5_4_2_0','road_0_1_0_0','road_1_5_3_0','road_5_2_2_0','road_0_1_0_0','road_0_4_0_0','road_0_4_0_0','road_5_4_2_0','road_0_1_0_0','road_0_1_0_0','road_0_1_0_0','road_0_4_0_0','road_0_1_0_0','road_0_1_0_0','road_0_1_0_0','road_0_4_0_0','road_0_1_0_0','road_5_4_2_0','road_1_5_3_0','road_5_3_2_0','road_1_0_1_0','road_5_4_2_0','road_1_0_1_0','road_5_4_2_0','road_0_2_0_0','road_1_0_1_0','road_5_2_2_0','road_0_1_0_0','road_1_0_1_0','road_1_5_3_0','road_5_3_2_0','road_5_3_2_0','road_5_2_2_0','road_0_4_0_0','road_5_2_2_0','road_5_2_2_0','road_5_3_2_0','road_5_3_2_0','road_0_1_0_0','road_0_1_0_0','road_0_2_0_0','road_1_5_3_0','road_1_5_3_0','road_4_0_1_0','road_1_5_3_0','road_4_0_1_0','road_5_2_2_0','road_0_2_0_0','road_1_5_3_0','road_5_3_2_0','road_5_3_2_0','road_5_3_2_0','road_0_2_0_0','road_4_0_1_0','road_5_4_2_0','road_0_4_0_0','road_0_2_0_0','road_5_2_2_0','road_0_2_0_0','road_1_5_3_0','road_1_0_1_0','road_1_0_1_0','road_4_0_1_0','road_4_0_1_0','road_0_4_0_0','road_0_1_0_0','road_0_2_0_0','road_0_4_0_0','road_0_4_0_0','road_0_2_0_0','road_1_5_3_0','road_5_3_2_0','road_5_2_2_0','road_4_0_1_0','road_0_1_0_0','road_0_1_0_0','road_5_4_2_0','road_1_5_3_0','road_1_0_1_0','road_0_4_0_0','road_5_4_2_0','road_0_4_0_0','road_0_4_0_0','road_4_0_1_0','road_0_1_0_0','road_5_2_2_0','road_5_3_2_0','road_0_4_0_0','road_0_4_0_0','road_0_4_0_0','road_0_2_0_0','road_5_3_2_0','road_0_4_0_0','road_0_4_0_0','road_5_4_2_0','road_5_2_2_0','road_5_4_2_0','road_5_4_2_0','road_0_4_0_0','road_1_0_1_0','road_1_0_1_0','road_5_4_2_0','road_5_4_2_0','road_4_0_1_0','road_5_2_2_0','road_0_4_0_0','road_5_3_2_0','road_4_0_1_0','road_0_2_0_0','road_1_0_1_0','road_0_4_0_0','road_0_4_0_0','road_0_1_0_0','road_0_2_0_0','road_0_1_0_0','road_4_0_1_0','road_4_0_1_0','road_5_4_2_0','road_1_5_3_0','road_5_2_2_0','road_0_1_0_0','road_0_2_0_0','road_1_5_3_0','road_0_2_0_0','road_5_4_2_0','road_5_4_2_0','road_1_5_3_0','road_0_4_0_0','road_0_1_0_0','road_1_0_1_0','road_5_3_2_0','road_4_0_1_0','road_5_2_2_0','road_0_4_0_0','road_0_4_0_0','road_0_4_0_0','road_5_4_2_0','road_5_3_2_0','road_0_4_0_0']
end_edges = ['road_1_1_3_0','road_1_4_1_0','road_4_4_1_0','road_1_1_3_0','road_4_4_1_0','road_4_1_0_0','road_4_2_0_0','road_4_4_1_0','road_4_1_0_0','road_1_1_3_0','road_4_4_0_0','road_4_4_1_0','road_4_4_1_0','road_1_4_1_0','road_1_1_3_0','road_1_1_3_0','road_3_4_1_0','road_1_3_2_0','road_2_1_3_0','road_1_4_2_0','road_1_1_3_0','road_1_3_2_0','road_1_2_2_0','road_1_2_2_1','road_4_4_1_0','road_2_4_1_0','road_4_1_0_0','road_1_3_2_0','road_1_1_3_0','road_1_1_3_0','road_3_1_3_0','road_1_4_1_0','road_2_4_1_0','road_1_2_2_0','road_4_4_1_0','road_4_4_0_0','road_2_4_1_0','road_4_4_1_0','road_3_1_3_0','road_4_2_0_0','road_1_1_3_0','road_3_4_1_0','road_1_3_2_0','road_4_2_0_0','road_4_3_0_0','road_1_2_2_0','road_1_4_1_0','road_3_4_1_0','road_1_1_2_0','road_2_1_3_0','road_1_4_1_0','road_4_4_0_0','road_2_4_1_0','road_4_1_0_0','road_2_1_3_0','road_1_3_2_0','road_2_1_3_1','road_4_4_1_0','road_4_4_0_0','road_2_4_1_0','road_4_1_0_0','road_3_1_3_0','road_1_1_3_0','road_3_4_1_0','road_4_3_0_0','road_1_4_1_0','road_1_4_1_0','road_2_3_1_0','road_1_4_1_0','road_1_1_3_0','road_1_3_2_0','road_4_4_0_0','road_4_4_1_0','road_2_1_3_0','road_4_4_1_0','road_3_1_3_0','road_4_1_3_0','road_1_4_2_0','road_1_1_3_0','road_4_4_0_0','road_1_3_2_0','road_1_1_3_0','road_3_4_1_0','road_4_1_0_0','road_1_1_2_0','road_1_1_2_0','road_2_4_1_0','road_2_4_1_1','road_1_2_2_0','road_4_4_0_0','road_2_4_1_0','road_1_4_1_0','road_4_4_0_0','road_1_1_2_0','road_4_4_1_0','road_1_4_1_0','road_4_2_0_0','road_1_2_2_0','road_1_4_2_0','road_1_1_3_0','road_4_4_1_0','road_1_1_3_1','road_2_4_1_0','road_4_4_1_0','road_4_4_1_1','road_1_4_1_0','road_4_3_0_0','road_4_3_0_0','road_4_4_1_0','road_4_4_3_0','road_4_2_0_0','road_1_2_2_0','road_4_3_0_0','road_4_1_0_0','road_2_1_3_0','road_1_4_2_0','road_1_4_1_0','road_3_4_1_0','road_1_3_2_0','road_2_4_1_0','road_1_1_3_0','road_4_1_0_0','road_4_2_0_0','road_1_3_2_0','road_4_4_0_0','road_4_4_1_0','road_3_4_1_0','road_4_4_1_0','road_1_4_1_0','road_4_1_3_0','road_4_2_0_0','road_4_2_0_0','road_4_4_1_0','road_4_1_3_0','road_1_1_2_0','road_1_1_3_0','road_2_1_3_0','road_1_2_2_0','road_2_1_3_0','road_4_4_1_0','road_2_1_3_1','road_4_4_1_0','road_1_1_3_0','road_4_2_0_0','road_4_4_1_1','road_2_4_1_0','road_3_4_1_0','road_1_1_3_0','road_1_4_1_0','road_4_4_1_0','road_4_4_1_0','road_2_4_1_0','road_4_1_3_0','road_4_4_0_0','road_4_4_1_0','road_1_3_2_0','road_4_4_1_0','road_1_4_2_0','road_1_1_3_0','road_1_4_2_0','road_3_4_1_0','road_1_4_1_0','road_2_1_3_0','road_1_2_2_0','road_1_2_2_0','road_1_4_1_0','road_2_4_1_0','road_1_1_2_0','road_4_1_3_0','road_1_1_2_0','road_4_4_1_0','road_4_3_0_0','road_2_4_1_0','road_2_4_1_0','road_3_4_1_0','road_1_2_2_0','road_1_4_2_0','road_1_1_3_1','road_4_4_1_0','road_2_1_3_1','road_2_1_3_0','road_4_3_0_0','road_1_1_3_0','road_4_2_0_1','road_4_2_0_0','road_1_2_2_0','road_4_4_1_0','road_4_4_0_0','road_1_3_2_0','road_1_4_1_0','road_4_2_0_0','road_2_1_3_0','road_3_4_1_0','road_1_3_2_0','road_1_2_2_0','road_1_1_2_0','road_2_1_3_0','road_4_1_0_0','road_4_2_0_0','road_4_4_1_0','road_4_2_0_0','road_1_4_1_0','road_1_1_2_0','road_4_4_0_0','road_4_3_0_0','road_1_4_1_0','road_4_4_1_0','road_4_1_0_0','road_1_1_3_0','road_1_3_2_0','road_1_1_3_0','road_3_4_1_0','road_3_4_1_0','road_1_4_1_0','road_4_4_1_0','road_4_1_3_0','road_1_2_2_0','road_4_4_1_1','road_4_4_1_0','road_1_3_2_0','road_1_1_2_0','road_3_1_3_0','road_3_4_1_0','road_4_1_0_0','road_1_4_1_0','road_4_3_0_0','road_1_2_2_1','road_1_2_2_0','road_3_1_3_0','road_4_1_0_0','road_4_2_0_0','road_4_4_1_0','road_2_1_3_1','road_4_4_0_0','road_1_4_2_0','road_1_1_2_0','road_3_1_3_0','road_2_1_3_0','road_2_4_1_0','road_2_4_1_1','road_1_4_1_0','road_4_1_3_0','road_1_3_2_0','road_1_2_2_0','road_1_1_3_0','road_1_4_2_0','road_4_1_3_0','road_3_1_3_0','road_1_3_2_0','road_1_2_2_0','road_1_1_3_0','road_1_2_2_0','road_4_4_1_0','road_4_1_0_0','road_1_2_2_0','road_1_1_2_0','road_4_4_1_0','road_3_4_1_0','road_1_4_2_0','road_4_4_1_0','road_1_1_3_1','road_4_4_0_0','road_1_1_3_0','road_2_1_3_0','road_2_1_3_1','road_4_1_0_0','road_3_1_3_0','road_1_1_3_0','road_1_1_3_0','road_1_1_3_1','road_2_1_3_0','road_1_3_2_0','road_1_1_3_0','road_4_4_1_0','road_1_4_2_0','road_3_4_1_0','road_4_1_3_0','road_4_4_1_0','road_2_1_3_0','road_4_4_1_0','road_4_2_0_0','road_1_2_2_0','road_4_4_0_0','road_2_1_3_0','road_1_4_1_0','road_1_3_2_0','road_4_4_0_0','road_4_4_1_0','road_2_1_3_0','road_1_3_2_0','road_4_3_0_0','road_4_3_0_1','road_1_4_1_0','road_3_4_1_0','road_2_1_3_0','road_2_1_3_1','road_1_1_3_0','road_1_4_2_0','road_1_1_3_0','road_4_2_0_0','road_1_1_3_0','road_4_1_0_0','road_4_4_0_0','road_2_1_3_0','road_1_2_2_0','road_4_4_0_0','road_4_4_1_0','road_4_4_1_0','road_2_1_3_0','road_4_2_0_0','road_3_4_1_0','road_1_4_1_0','road_1_1_3_0','road_4_3_0_0','road_1_1_3_0','road_1_3_2_0','road_4_1_3_0','road_2_1_3_0','road_4_1_0_1','road_4_1_0_0','road_2_4_1_0','road_4_1_0_0','road_1_1_2_0','road_3_4_1_0','road_2_4_1_0','road_1_1_2_1','road_1_4_2_0','road_1_3_2_0','road_4_3_0_0','road_4_4_0_0','road_2_1_3_0','road_1_1_3_0','road_4_4_1_0','road_3_1_3_0','road_1_1_2_0','road_4_3_0_0','road_2_4_1_0','road_2_4_1_0','road_1_4_1_0','road_4_1_0_0','road_1_1_3_0','road_3_2_0_0','road_3_4_3_0','road_1_3_2_0','road_1_3_2_0','road_1_3_2_1','road_4_3_0_0','road_1_4_1_0','road_4_4_0_0','road_1_3_2_0','road_2_1_3_0','road_2_4_1_0','road_4_4_1_0','road_1_4_2_0','road_1_1_3_0','road_1_4_1_0','road_2_1_3_1','road_4_4_1_1','road_4_1_3_0','road_4_2_0_0','road_3_3_3_0','road_1_2_2_0','road_3_4_1_0','road_4_4_1_0','road_1_1_1_0','road_1_2_2_0','road_1_1_2_0','road_4_1_3_0','road_4_1_0_0','road_1_1_2_0','road_1_1_3_0','road_1_2_2_0','road_4_1_0_0','road_2_4_1_0','road_1_2_2_0','road_4_4_1_0','road_4_1_0_0','road_1_1_3_0','road_1_1_3_0','road_1_1_2_0','road_2_1_3_0','road_1_4_2_0','road_4_3_0_0','road_1_4_1_0','road_3_4_1_0','road_2_1_3_0','road_4_2_0_0','road_1_4_2_0','road_1_2_2_0','road_1_4_1_0','road_4_4_0_0','road_1_4_1_0','road_4_4_1_0','road_4_4_0_1','road_1_4_1_0']
depart=[30.0, 8.0, 42.0, 53.0, 56.0, 78.0, 0.0, 70.0, 104.0, 0.0, 0.0, 91.0, 26.0, 80.0, 93.0, 130.0, 21.0, 36.0, 40.0, 138.0, 160.0, 69.0, 0.0, 26.0, 13.0, 7.0, 182.0, 92.0, 190.0, 170.0, 117.0, 112.0, 52.0, 27.0, 189.0, 117.0, 32.0, 175.0, 10.0, 156.0, 210.0, 91.0, 115.0, 130.0, 65.0, 68.0, 23.0, 84.0, 26.0, 80.0, 0.0, 39.0, 28.0, 46.0, 90.0, 184.0, 110.0, 26.0, 4.0, 23.0, 260.0, 0.0, 13.0, 65.0, 91.0, 144.0, 140.0, 0.0, 0.0, 240.0, 192.0, 208.0, 245.0, 140.0, 238.0, 115.0, 20.0, 299.0, 290.0, 40.0, 253.0, 324.0, 169.0, 323.0, 96.0, 52.0, 49.0, 77.0, 29.0, 52.0, 105.0, 160.0, 273.0, 302.0, 182.0, 69.0, 273.0, 173.0, 317.0, 222.0, 252.0, 330.0, 37.0, 266.0, 273.0, 63.0, 195.0, 12.0, 280.0, 104.0, 50.0, 74.0, 20.0, 346.0, 276.0, 302.0, 204.0, 221.0, 79.0, 55.0, 348.0, 331.0, 302.0, 334.0, 61.0, 259.0, 234.0, 318.0, 138.0, 169.0, 302.0, 369.0, 303.0, 184.0, 356.0, 161.0, 46.0, 334.0, 185.0, 343.0, 270.0, 348.0, 92.0, 60.0, 360.0, 120.0, 140.0, 370.0, 85.0, 355.0, 332.0, 148.0, 314.0, 64.0, 317.0, 156.0, 396.0, 453.0, 408.0, 433.0, 154.0, 244.0, 300.0, 385.0, 351.0, 254.0, 118.0, 453.0, 120.0, 176.0, 234.0, 384.0, 147.0, 143.0, 260.0, 16.0, 112.0, 216.0, 368.0, 35.0, 72.0, 286.0, 450.0, 92.0, 74.0, 268.0, 417.0, 402.0, 419.0, 230.0, 180.0, 296.0, 148.0, 14.0, 368.0, 384.0, 124.0, 230.0, 415.0, 420.0, 100.0, 253.0, 426.0, 111.0, 299.0, 314.0, 419.0, 484.0, 39.0, 84.0, 212.0, 100.0, 168.0, 264.0, 420.0, 299.0, 300.0, 462.0, 450.0, 367.0, 521.0, 46.0, 196.0, 530.0, 133.0, 392.0, 221.0, 436.0, 182.0, 250.0, 259.0, 453.0, 229.0, 76.0, 111.0, 276.0, 336.0, 487.0, 299.0, 161.0, 37.0, 440.0, 380.0, 330.0, 468.0, 126.0, 260.0, 306.0, 395.0, 345.0, 440.0, 368.0, 480.0, 385.0, 207.0, 510.0, 488.0, 70.0, 589.0, 426.0, 522.0, 104.0, 272.0, 98.0, 354.0, 200.0, 342.0, 284.0, 546.0, 552.0, 378.0, 470.0, 570.0, 517.0, 600.0, 487.0, 317.0, 522.0, 538.0, 528.0, 328.0, 302.0, 296.0, 366.0, 402.0, 555.0, 334.0, 538.0, 0.0, 515.0, 342.0, 553.0, 247.0, 143.0, 384.0, 390.0, 524.0, 656.0, 351.0, 553.0, 385.0, 622.0, 454.0, 482.0, 504.0, 572.0, 589.0, 14.0, 496.0, 599.0, 294.0, 505.0, 600.0, 398.0, 552.0, 230.0, 419.0, 600.0, 677.0, 666.0, 168.0, 220.0, 580.0, 116.0, 196.0, 630.0, 672.0, 286.0, 600.0, 439.0, 416.0, 691.0, 616.0, 185.0, 672.0, 132.0, 231.0, 233.0, 590.0, 700.0, 680.0, 222.0, 78.0, 571.0, 545.0, 550.0, 342.0, 351.0, 88.0, 609.0, 224.0, 300.0, 619.0, 210.0, 375.0, 470.0, 504.0, 625.0, 378.0, 602.0, 185.0, 406.0, 472.0, 461.0, 538.0, 636.0, 351.0, 200.0, 282.0, 398.0, 710.0, 247.0, 743.0, 204.0, 487.0, 580.0, 360.0, 660.0, 436.0, 565.0, 201.0, 217.0, 402.0, 636.0, 318.0, 161.0, 600.0, 208.0, 412.0, 633.0, 128.0, 612.0, 631.0, 680.0, 645.0]
start_edge = [item[:-2] for item in start_edges]
end_edge = [item[:-2] for item in end_edges]
zipped_lists = list(zip(depart,start_edge,end_edge))
print(zipped_lists)


In [ ]:
depart = [30.00, 8.00, 42.00, 53.00, 56.00, 78.00, 0.00, 70.00, 104.00, 0.00, 0.00, 91.00, 26.00, 80.00, 93.00, 130.00, 21.00, 36.00, 40.00, 138.00, 160.00, 69.00, 0.00, 26.00, 13.00, 7.00, 182.00, 92.00, 190.00, 170.00, 117.00, 112.00, 52.00, 27.00, 189.00, 117.00, 32.00, 175.00, 10.00, 156.00, 210.00, 91.00, 115.00, 130.00, 65.00, 68.00, 23.00, 84.00, 26.00, 80.00, 0.00, 39.00, 28.00, 46.00, 90.00, 184.00, 110.00, 26.00, 4.00, 23.00, 260.00, 0.00, 13.00, 65.00, 91.00, 144.00, 140.00, 0.00, 0.00, 240.00, 192.00, 208.00, 245.00, 140.00, 238.00, 115.00, 20.00, 299.00, 290.00, 40.00, 253.00, 324.00, 169.00, 323.00, 96.00, 52.00, 49.00, 77.00, 29.00, 52.00, 105.00, 160.00, 273.00, 302.00, 182.00, 69.00, 273.00, 173.00, 317.00, 222.00, 252.00, 330.00, 37.00, 266.00, 273.00, 63.00, 195.00, 12.00, 280.00, 104.00, 50.00, 74.00, 20.00, 346.00, 276.00, 302.00, 204.00, 221.00, 79.00, 55.00, 348.00, 331.00, 302.00, 334.00, 61.00, 259.00, 234.00, 318.00, 138.00, 169.00, 302.00, 369.00, 303.00, 184.00, 356.00, 161.00, 46.00, 334.00, 185.00, 343.00, 270.00, 348.00, 92.00, 60.00, 360.00, 120.00, 140.00, 370.00, 85.00, 355.00, 332.00, 148.00, 314.00, 64.00, 317.00, 156.00, 396.00, 453.00, 408.00, 433.00, 154.00, 244.00, 300.00, 385.00, 351.00, 254.00, 118.00, 453.00, 120.00, 176.00, 234.00, 384.00, 147.00, 143.00, 260.00, 16.00, 112.00, 216.00, 368.00, 35.00, 72.00, 286.00, 450.00, 92.00, 74.00, 268.00, 417.00, 402.00, 419.00, 230.00, 180.00, 296.00, 148.00, 14.00, 368.00, 384.00, 124.00, 230.00, 415.00, 420.00, 100.00, 253.00, 426.00, 111.00, 299.00, 314.00, 419.00, 484.00, 39.00, 84.00, 212.00, 100.00, 168.00, 264.00, 420.00, 299.00, 300.00, 462.00, 450.00, 367.00, 521.00, 46.00, 196.00, 530.00, 133.00, 392.00, 221.00, 436.00, 182.00, 250.00, 259.00, 453.00, 229.00, 76.00, 111.00, 276.00, 336.00, 487.00, 299.00, 161.00, 37.00, 440.00, 380.00, 330.00, 468.00, 126.00, 260.00, 306.00, 395.00, 345.00, 440.00, 368.00, 480.00, 385.00, 207.00, 510.00, 488.00, 70.00, 589.00, 426.00, 522.00, 104.00, 272.00, 98.00, 354.00, 200.00, 342.00, 284.00, 546.00, 552.00, 378.00, 470.00, 570.00, 517.00, 600.00, 487.00, 317.00, 522.00, 538.00, 528.00, 328.00, 302.00, 296.00, 366.00, 402.00, 555.00, 334.00, 538.00, 0.00, 515.00, 342.00, 553.00, 247.00, 143.00, 384.00, 390.00, 524.00, 656.00, 351.00, 553.00, 385.00, 622.00, 454.00, 482.00, 504.00, 572.00, 589.00, 14.00, 496.00, 599.00, 294.00, 505.00, 600.00, 398.00, 552.00, 230.00, 419.00, 600.00, 677.00, 666.00, 168.00, 220.00, 580.00, 116.00, 196.00, 630.00, 672.00, 286.00, 600.00, 439.00, 416.00, 691.00, 616.00, 185.00, 672.00, 132.00, 231.00, 233.00, 590.00, 700.00, 680.00, 222.00, 78.00, 571.00, 545.00, 550.00, 342.00, 351.00, 88.00, 609.00, 224.00, 300.00, 619.00, 210.00, 375.00, 470.00, 504.00, 625.00, 378.00, 602.00, 185.00, 406.00, 472.00, 461.00, 538.00, 636.00, 351.00, 200.00, 282.00, 398.00, 710.00, 247.00, 743.00, 204.00, 487.00, 580.00, 360.00, 660.00, 436.00, 565.00, 201.00, 217.00, 402.00, 636.00, 318.00, 161.00, 600.00, 208.00, 412.00, 633.00, 128.00, 612.00, 631.00, 680.00, 645.00, 1.00, 1.00, 1.00, 2.00, 58.00, 109.00, 120.00, 130.00, 136.00, 148.00, 150.00, 152.00, 156.00, 164.00, 180.00, 184.00, 188.00, 195.00, 15.00, 21.00, 22.00, 38.00, 40.00, 41.00, 47.00, 66.00, 70.00, 71.00, 75.00, 81.00, 92.00, 101.00, 105.00, 106.00, 112.00, 113.00, 116.00, 117.00, 121.00, 131.00, 139.00, 141.00, 144.00, 145.00, 149.00, 157.00, 161.00, 162.00, 169.00, 170.00, 171.00, 181.00, 183.00, 186.00, 196.00, 197.00, 205.00, 207.00, 208.00, 220.00, 226.00, 232.00, 236.00, 240.00, 248.00, 252.00, 258.00, 259.00, 261.00, 278.00, 281.00, 287.00, 288.00, 292.00, 296.00, 300.00, 300.00, 302.00, 305.00, 306.00, 310.00, 312.00, 312.00, 315.00, 320.00, 324.00, 325.00, 328.00, 335.00, 336.00, 340.00, 342.00, 355.00, 356.00, 360.00, 365.00, 366.00, 370.00, 372.00, 372.00, 372.00, 384.00, 385.00, 390.00, 390.00, 396.00, 401.00, 402.00, 402.00, 208.00, 209.00, 211.00, 221.00, 222.00, 223.00, 231.00, 232.00, 235.00, 241.00, 248.00, 249.00, 253.00, 260.00, 261.00, 262.00, 274.00, 278.00, 279.00, 287.00, 288.00, 289.00, 297.00, 302.00, 303.00, 304.00, 307.00, 313.00, 315.00, 316.00, 318.00, 319.00, 325.00, 326.00, 329.00, 335.00, 336.00, 337.00, 349.00, 352.00, 356.00, 357.00, 361.00, 369.00, 370.00, 371.00, 373.00, 379.00, 385.00, 386.00, 386.00, 387.00, 391.00, 397.00, 399.00, 403.00, 403.00, 404.00, 408.00, 410.00, 412.00, 414.00, 415.00, 420.00, 420.00, 425.00, 426.00, 426.00, 430.00, 432.00, 432.00, 435.00, 436.00, 438.00, 438.00, 440.00, 444.00, 444.00, 445.00, 450.00, 454.00, 456.00, 456.00, 456.00, 460.00, 462.00, 465.00, 468.00, 468.00, 468.00, 470.00, 474.00, 474.00, 475.00, 480.00, 480.00, 482.00, 486.00, 486.00, 486.00, 490.00, 492.00, 492.00, 495.00, 496.00, 498.00, 500.00, 500.00, 504.00, 504.00, 504.00, 507.00, 510.00, 510.00, 510.00, 510.00, 516.00, 516.00, 520.00, 521.00, 521.00, 524.00, 525.00, 528.00, 531.00, 534.00, 534.00, 535.00, 538.00, 538.00, 541.00, 541.00, 540.00, 546.00, 552.00, 552.00, 555.00, 555.00, 555.00, 558.00, 558.00]

In [ ]:
    
sorted_zipped_lists = sorted(zipped_lists,key=lambda x: x[0])

depart_times,start_edges,end_edges = [
    [item[i] for item in sorted_zipped_lists] for i in range(3)
]
print(len(depart_times))
print("depart_times=",depart_times)
print("start_edges=",start_edges)
print("end_edges=",end_edges)

In [ ]:
[(0.0, 'road_4_0_1', 'road_4_2_0'),(78.0, 'road_4_0_1', 'road_4_1_0'),(104.0, 'road_4_0_1', 'road_4_1_0'),]

In [ ]:

depart = [30.00, 8.00, 42.00, 53.00, 56.00, 78.00, 0.00, 70.00, 104.00, 0.00, 0.00, 91.00, 26.00, 80.00, 93.00, 130.00, 21.00, 36.00, 40.00, 138.00, 160.00, 69.00, 0.00, 26.00, 13.00, 7.00, 182.00, 92.00, 190.00, 170.00, 117.00, 112.00, 52.00, 27.00, 189.00, 117.00, 32.00, 175.00, 10.00, 156.00, 210.00, 91.00, 115.00, 130.00, 65.00, 68.00, 23.00, 84.00, 26.00, 80.00, 0.00, 39.00, 28.00, 46.00, 90.00, 184.00, 110.00, 26.00, 4.00, 23.00, 260.00, 0.00, 13.00, 65.00, 91.00, 144.00, 140.00, 0.00, 0.00, 240.00, 192.00, 208.00, 245.00, 140.00, 238.00, 115.00, 20.00, 299.00, 290.00, 40.00, 253.00, 324.00, 169.00, 323.00, 96.00, 52.00, 49.00, 77.00, 29.00, 52.00, 105.00, 160.00, 273.00, 302.00, 182.00, 69.00, 273.00, 173.00, 317.00, 222.00, 252.00, 330.00, 37.00, 266.00, 273.00, 63.00, 195.00, 12.00, 280.00, 104.00, 50.00, 74.00, 20.00, 346.00, 276.00, 302.00, 204.00, 221.00, 79.00, 55.00, 348.00, 331.00, 302.00, 334.00, 61.00, 259.00, 234.00, 318.00, 138.00, 169.00, 302.00, 369.00, 303.00, 184.00, 356.00, 161.00, 46.00, 334.00, 185.00, 343.00, 270.00, 348.00, 92.00, 60.00, 360.00, 120.00, 140.00, 370.00, 85.00, 355.00, 332.00, 148.00, 314.00, 64.00, 317.00, 156.00, 396.00, 453.00, 408.00, 433.00, 154.00, 244.00, 300.00, 385.00, 351.00, 254.00, 118.00, 453.00, 120.00, 176.00, 234.00, 384.00, 147.00, 143.00, 260.00, 16.00, 112.00, 216.00, 368.00, 35.00, 72.00, 286.00, 450.00, 92.00, 74.00, 268.00, 417.00, 402.00, 419.00, 230.00, 180.00, 296.00, 148.00, 14.00, 368.00, 384.00, 124.00, 230.00, 415.00, 420.00, 100.00, 253.00, 426.00, 111.00, 299.00, 314.00, 419.00, 484.00, 39.00, 84.00, 212.00, 100.00, 168.00, 264.00, 420.00, 299.00, 300.00, 462.00, 450.00, 367.00, 521.00, 46.00, 196.00, 530.00, 133.00, 392.00, 221.00, 436.00, 182.00, 250.00, 259.00, 453.00, 229.00, 76.00, 111.00, 276.00, 336.00, 487.00, 299.00, 161.00, 37.00, 440.00, 380.00, 330.00, 468.00, 126.00, 260.00, 306.00, 395.00, 345.00, 440.00, 368.00, 480.00, 385.00, 207.00, 510.00, 488.00, 70.00, 589.00, 426.00, 522.00, 104.00, 272.00, 98.00, 354.00, 200.00, 342.00, 284.00, 546.00, 552.00, 378.00, 470.00, 570.00, 517.00, 600.00, 487.00, 317.00, 522.00, 538.00, 528.00, 328.00, 302.00, 296.00, 366.00, 402.00, 555.00, 334.00, 538.00, 0.00, 515.00, 342.00, 553.00, 247.00, 143.00, 384.00, 390.00, 524.00, 656.00, 351.00, 553.00, 385.00, 622.00, 454.00, 482.00, 504.00, 572.00, 589.00, 14.00, 496.00, 599.00, 294.00, 505.00, 600.00, 398.00, 552.00, 230.00, 419.00, 600.00, 677.00, 666.00, 168.00, 220.00, 580.00, 116.00, 196.00, 630.00, 672.00, 286.00, 600.00, 439.00, 416.00, 691.00, 616.00, 185.00, 672.00, 132.00, 231.00, 233.00, 590.00, 700.00, 680.00, 222.00, 78.00, 571.00, 545.00, 550.00, 342.00, 351.00, 88.00, 609.00, 224.00, 300.00, 619.00, 210.00, 375.00, 470.00, 504.00, 625.00, 378.00, 602.00, 185.00, 406.00, 472.00, 461.00, 538.00, 636.00, 351.00, 200.00, 282.00, 398.00, 710.00, 247.00, 743.00, 204.00, 487.00, 580.00, 360.00, 660.00, 436.00, 565.00, 201.00, 217.00, 402.00, 636.00, 318.00, 161.00, 600.00, 208.00, 412.00, 633.00, 128.00, 612.00, 631.00, 680.00, 645.00, 1.00, 1.00, 1.00, 2.00, 58.00, 109.00, 120.00, 130.00, 136.00, 148.00, 150.00, 152.00, 156.00, 164.00, 180.00, 184.00, 188.00, 195.00, 15.00, 21.00, 22.00, 38.00, 40.00, 41.00, 47.00, 66.00, 70.00, 71.00, 75.00, 81.00, 92.00, 101.00, 105.00, 106.00, 112.00, 113.00, 116.00, 117.00, 121.00, 131.00, 139.00, 141.00, 144.00, 145.00, 149.00, 157.00, 161.00, 162.00, 169.00, 170.00, 171.00, 181.00, 183.00, 186.00, 196.00, 197.00, 205.00, 207.00, 208.00, 220.00, 226.00, 232.00, 236.00, 240.00, 248.00, 252.00, 258.00, 259.00, 261.00, 278.00, 281.00, 287.00, 288.00, 292.00, 296.00, 300.00, 300.00, 302.00, 305.00, 306.00, 310.00, 312.00, 312.00, 315.00, 320.00, 324.00, 325.00, 328.00, 335.00, 336.00, 340.00, 342.00, 355.00, 356.00, 360.00, 365.00, 366.00, 370.00, 372.00, 372.00, 372.00, 384.00, 385.00, 390.00, 390.00, 396.00, 401.00, 402.00, 402.00, 208.00, 209.00, 211.00, 221.00, 222.00, 223.00, 231.00, 232.00, 235.00, 241.00, 248.00, 249.00, 253.00, 260.00, 261.00, 262.00, 274.00, 278.00, 279.00, 287.00, 288.00, 289.00, 297.00, 302.00, 303.00, 304.00, 307.00, 313.00, 315.00, 316.00, 318.00, 319.00, 325.00, 326.00, 329.00, 335.00, 336.00, 337.00, 349.00, 352.00, 356.00, 357.00, 361.00, 369.00, 370.00, 371.00, 373.00, 379.00, 385.00, 386.00, 386.00, 387.00, 391.00, 397.00, 399.00, 403.00, 403.00, 404.00, 408.00, 410.00, 412.00, 414.00, 415.00, 420.00, 420.00, 425.00, 426.00, 426.00, 430.00, 432.00, 432.00, 435.00, 436.00, 438.00, 438.00, 440.00, 444.00, 444.00, 445.00, 450.00, 454.00, 456.00, 456.00, 456.00, 460.00, 462.00, 465.00, 468.00, 468.00, 468.00, 470.00, 474.00, 474.00, 475.00, 480.00, 480.00, 482.00, 486.00, 486.00, 486.00, 490.00, 492.00, 492.00, 495.00, 496.00, 498.00, 500.00, 500.00, 504.00, 504.00, 504.00, 507.00, 510.00, 510.00, 510.00, 510.00, 516.00, 516.00, 520.00, 521.00, 521.00, 524.00, 525.00, 528.00, 531.00, 534.00, 534.00, 535.00, 538.00, 538.00, 541.00, 541.00, 540.00, 546.00, 552.00, 552.00, 555.00, 555.00, 555.00, 558.00, 558.00, 561.00, 564.00, 564.00, 565.00, 567.00, 566.00, 570.00, 572.00, 572.00, 575.00, 576.00, 576.00, 576.00, 581.00, 582.00, 582.00, 585.00, 588.00, 588.00, 589.00, 594.00, 594.00, 594.00, 594.00, 595.00, 602.00, 600.00, 600.00, 600.00, 603.00, 604.00, 606.00, 608.00, 409.00, 413.00, 416.00, 421.00, 427.00, 437.00, 439.00, 440.00, 441.00, 445.00, 446.00, 451.00, 454.00, 456.00, 457.00, 463.00, 469.00, 471.00, 475.00, 476.00, 481.00, 483.00, 487.00, 493.00, 497.00, 501.00, 505.00, 505.00, 506.00, 511.00, 511.00, 512.00, 522.00, 523.00, 525.00, 526.00, 529.00, 535.00, 536.00, 539.00, 539.00, 541.00, 542.00, 547.00, 554.00, 556.00, 559.00, 566.00, 567.00, 568.00, 571.00, 572.00, 573.00, 577.00, 581.00, 582.00, 583.00, 589.00, 590.00, 591.00, 595.00, 596.00, 601.00, 603.00, 604.00, 605.00, 610.00, 611.00, 612.00, 615.00, 615.00, 615.00, 616.00, 618.00, 618.00, 618.00, 620.00, 622.00, 621.00, 625.00, 628.00, 631.00, 630.00, 630.00, 631.00, 634.00, 632.00, 633.00, 637.00, 636.00, 639.00, 640.00, 640.00, 642.00, 644.00, 644.00, 645.00, 645.00, 648.00, 648.00, 648.00, 650.00, 652.00, 652.00, 654.00, 654.00, 655.00, 656.00, 658.00, 660.00, 660.00, 660.00, 661.00, 664.00, 664.00, 664.00, 666.00, 668.00, 670.00, 670.00, 672.00, 679.00, 675.00, 675.00, 682.00, 676.00, 683.00, 680.00, 686.00, 684.00, 689.00, 692.00, 688.00, 688.00, 690.00, 690.00, 690.00, 690.00, 694.00, 694.00, 696.00, 697.00, 698.00, 696.00, 702.00, 700.00, 700.00, 705.00, 706.00, 705.00, 705.00, 707.00, 709.00, 708.00, 708.00, 710.00, 710.00, 713.00, 714.00, 712.00, 714.00, 717.00, 717.00, 720.00, 720.00, 720.00, 720.00, 720.00, 721.00, 725.00, 724.00, 726.00, 726.00, 728.00, 728.00, 728.00, 731.00, 730.00, 732.00, 732.00, 734.00, 735.00, 735.00, 737.00, 736.00, 740.00, 740.00, 742.00, 743.00, 745.00, 744.00, 744.00, 744.00, 744.00, 747.00, 748.00, 750.00, 750.00, 750.00, 750.00, 752.00, 753.00, 754.00, 756.00, 756.00, 759.00, 760.00, 760.00, 760.00, 762.00, 762.00, 762.00, 764.00, 766.00, 765.00, 765.00, 765.00, 769.00, 768.00, 770.00, 771.00, 772.00, 774.00, 776.00, 776.00, 776.00, 777.00, 780.00, 780.00, 780.00, 780.00, 780.00, 780.00, 780.00, 783.00, 784.00, 786.00, 787.00, 788.00, 789.00, 790.00, 792.00, 792.00, 792.00, 795.00, 795.00, 795.00, 796.00, 798.00, 798.00, 798.00, 798.00, 800.00, 800.00, 802.00, 804.00, 805.00, 808.00, 808.00, 808.00, 809.00, 810.00, 613.00, 616.00, 617.00, 623.00, 626.00, 632.00, 633.00, 634.00, 635.00, 637.00, 638.00, 641.00, 645.00, 646.00, 649.00, 653.00, 655.00, 656.00, 657.00, 661.00, 662.00, 665.00, 667.00, 671.00, 673.00, 676.00, 677.00, 683.00, 686.00, 689.00, 687.00, 689.00, 690.00, 691.00, 691.00, 694.00, 696.00, 697.00, 698.00, 700.00, 702.00, 703.00, 707.00, 710.00, 709.00, 710.00, 713.00, 711.00, 714.00, 715.00, 718.00, 721.00, 721.00, 722.00, 727.00, 729.00, 733.00, 737.00, 738.00, 741.00, 744.00, 745.00, 745.00, 746.00, 751.00, 757.00, 761.00, 763.00, 766.00, 767.00, 777.00, 778.00, 781.00, 781.00, 782.00, 793.00, 796.00, 797.00, 799.00, 801.00, 809.00, 810.00, 810.00, 810.00, 811.00, 812.00, 814.00, 816.00, 816.00, 816.00, 817.00, 820.00, 820.00, 820.00, 820.00, 823.00, 824.00, 824.00, 825.00, 825.00, 826.00, 828.00, 829.00, 830.00, 831.00, 832.00, 832.00, 834.00, 834.00, 835.00, 836.00, 838.00, 840.00, 840.00, 840.00, 840.00, 841.00, 840.00, 842.00, 843.00, 845.00, 846.00, 848.00, 849.00, 850.00, 852.00, 852.00, 852.00, 852.00, 853.00, 855.00, 855.00, 855.00, 856.00, 856.00, 858.00, 860.00, 860.00, 861.00, 864.00, 864.00, 864.00, 867.00, 868.00, 870.00, 870.00, 870.00, 870.00, 870.00, 870.00, 872.00, 872.00, 873.00, 875.00, 876.00, 876.00, 879.00, 880.00, 880.00, 882.00, 884.00, 885.00, 885.00, 886.00, 888.00, 888.00, 888.00, 888.00, 890.00, 892.00, 896.00, 897.00, 811.00, 812.00, 813.00, 817.00, 818.00, 821.00, 826.00, 826.00, 827.00, 833.00, 835.00, 836.00, 837.00, 841.00, 841.00, 842.00, 843.00, 844.00, 853.00, 854.00, 856.00, 857.00, 861.00, 862.00, 865.00, 871.00, 871.00, 872.00, 873.00, 874.00, 877.00, 881.00, 886.00, 889.00]
departLane = [road_0_1_0_0, road_0_4_0_0, road_5_4_2_0, road_0_1_0_0, road_5_4_2_0, road_4_0_1_0, road_4_0_1_0, road_5_4_2_0, road_4_0_1_0, road_0_2_0_0, road_5_3_2_0, road_5_4_2_0, road_5_3_2_0, road_0_4_0_0, road_0_1_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_1_5_3_0, road_0_1_0_0, road_1_5_3_0, road_1_5_3_0, road_0_4_0_0, road_5_3_2_0, road_5_4_2_0, road_4_0_1_0, road_1_5_3_0, road_0_1_0_0, road_0_1_0_0, road_4_0_1_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_0_4_0_0, road_5_4_2_0, road_0_1_0_0, road_4_0_1_0, road_0_1_0_0, road_5_3_2_0, road_1_5_3_0, road_4_0_1_0, road_4_0_1_0, road_0_4_0_0, road_1_0_1_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_4_0_1_0, road_5_4_2_0, road_1_0_1_0, road_0_1_0_0, road_1_5_3_0, road_0_1_0_0, road_4_0_1_0, road_0_4_0_0, road_1_5_3_0, road_4_0_1_0, road_0_1_0_0, road_4_0_1_0, road_5_3_2_0, road_4_0_1_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_1_0_1_0, road_0_1_0_0, road_0_4_0_0, road_5_3_2_0, road_5_4_2_0, road_0_1_0_0, road_5_4_2_0, road_1_0_1_0, road_0_1_0_0, road_1_5_3_0, road_0_1_0_0, road_0_4_0_0, road_1_5_3_0, road_0_1_0_0, road_5_3_2_0, road_4_0_1_0, road_0_4_0_0, road_4_0_1_0, road_5_4_2_0, road_5_4_2_0, road_0_4_0_0, road_5_3_2_0, road_5_4_2_0, road_0_4_0_0, road_5_3_2_0, road_1_0_1_0, road_5_3_2_0, road_1_0_1_0, road_4_0_1_0, road_0_4_0_0, road_1_5_3_0, road_0_2_0_0, road_5_4_2_0, road_0_1_0_0, road_5_2_2_0, road_5_4_2_0, road_5_4_2_0, road_5_4_2_0, road_4_0_1_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_0_1_0_0, road_5_2_2_0, road_0_4_0_0, road_4_0_1_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_5_3_2_0, road_1_5_3_0, road_0_4_0_0, road_0_1_0_0, road_4_0_1_0, road_4_0_1_0, road_1_5_3_0, road_0_4_0_0, road_5_2_2_0, road_5_3_2_0, road_5_4_2_0, road_1_0_1_0, road_4_0_1_0, road_4_0_1_0, road_4_0_1_0, road_5_4_2_0, road_1_0_1_0, road_0_2_0_0, road_1_5_3_0, road_0_4_0_0, road_1_0_1_0, road_0_2_0_0, road_5_4_2_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_5_4_2_0, road_5_4_2_0, road_0_2_0_0, road_0_4_0_0, road_5_4_2_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_3_2_0, road_5_3_2_0, road_5_4_2_0, road_1_5_3_0, road_0_1_0_0, road_1_5_3_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_1_0_1_0, road_1_0_1_0, road_0_4_0_0, road_5_4_2_0, road_1_0_1_0, road_0_1_0_0, road_0_4_0_0, road_4_0_1_0, road_5_2_2_0, road_5_4_2_0, road_4_0_1_0, road_5_3_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_3_2_0, road_5_4_2_0, road_0_4_0_0, road_4_0_1_0, road_0_1_0_0, road_1_0_1_0, road_0_2_0_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_1_5_3_0, road_1_0_1_0, road_0_1_0_0, road_0_2_0_0, road_5_2_2_0, road_5_4_2_0, road_1_0_1_0, road_0_2_0_0, road_0_4_0_0, road_0_1_0_0, road_4_0_1_0, road_5_4_2_0, road_0_1_0_0, road_1_0_1_0, road_0_2_0_0, road_0_2_0_0, road_4_0_1_0, road_0_2_0_0, road_5_3_2_0, road_4_0_1_0, road_5_3_2_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_1_0_1_0, road_0_4_0_0, road_5_4_2_0, road_5_4_2_0, road_0_4_0_0, road_1_0_1_0, road_1_5_3_0, road_5_4_2_0, road_4_0_1_0, road_5_4_2_0, road_4_0_1_0, road_4_0_1_0, road_1_0_1_0, road_5_4_2_0, road_0_1_0_0, road_0_2_0_0, road_5_3_2_0, road_0_4_0_0, road_0_4_0_0, road_5_2_2_0, road_1_5_3_0, road_0_1_0_0, road_1_0_1_0, road_5_3_2_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_0_4_0_0, road_0_2_0_0, road_5_4_2_0, road_0_1_0_0, road_0_1_0_0, road_0_4_0_0, road_0_4_0_0, road_0_2_0_0, road_1_5_3_0, road_5_4_2_0, road_5_3_2_0, road_1_0_1_0, road_0_2_0_0, road_5_4_2_0, road_0_1_0_0, road_1_5_3_0, road_5_2_2_0, road_0_1_0_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_1_0_0, road_0_1_0_0, road_0_1_0_0, road_0_4_0_0, road_0_1_0_0, road_0_1_0_0, road_0_1_0_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_1_5_3_0, road_5_3_2_0, road_1_0_1_0, road_5_4_2_0, road_1_0_1_0, road_5_4_2_0, road_0_2_0_0, road_1_0_1_0, road_5_2_2_0, road_0_1_0_0, road_1_0_1_0, road_1_5_3_0, road_5_3_2_0, road_5_3_2_0, road_5_2_2_0, road_0_4_0_0, road_5_2_2_0, road_5_2_2_0, road_5_3_2_0, road_5_3_2_0, road_0_1_0_0, road_0_1_0_0, road_0_2_0_0, road_1_5_3_0, road_1_5_3_0, road_4_0_1_0, road_1_5_3_0, road_4_0_1_0, road_5_2_2_0, road_0_2_0_0, road_1_5_3_0, road_5_3_2_0, road_5_3_2_0, road_5_3_2_0, road_0_2_0_0, road_4_0_1_0, road_5_4_2_0, road_0_4_0_0, road_0_2_0_0, road_5_2_2_0, road_0_2_0_0, road_1_5_3_0, road_1_0_1_0, road_1_0_1_0, road_4_0_1_0, road_4_0_1_0, road_0_4_0_0, road_0_1_0_0, road_0_2_0_0, road_0_4_0_0, road_0_4_0_0, road_0_2_0_0, road_1_5_3_0, road_5_3_2_0, road_5_2_2_0, road_4_0_1_0, road_0_1_0_0, road_0_1_0_0, road_5_4_2_0, road_1_5_3_0, road_1_0_1_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_4_0_1_0, road_0_1_0_0, road_5_2_2_0, road_5_3_2_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_2_0_0, road_5_3_2_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_5_2_2_0, road_5_4_2_0, road_5_4_2_0, road_0_4_0_0, road_1_0_1_0, road_1_0_1_0, road_5_4_2_0, road_5_4_2_0, road_4_0_1_0, road_5_2_2_0, road_0_4_0_0, road_5_3_2_0, road_4_0_1_0, road_0_2_0_0, road_1_0_1_0, road_0_4_0_0, road_0_4_0_0, road_0_1_0_0, road_0_2_0_0, road_0_1_0_0, road_4_0_1_0, road_4_0_1_0, road_5_4_2_0, road_1_5_3_0, road_5_2_2_0, road_0_1_0_0, road_0_2_0_0, road_1_5_3_0, road_0_2_0_0, road_5_4_2_0, road_5_4_2_0, road_1_5_3_0, road_0_4_0_0, road_0_1_0_0, road_1_0_1_0, road_5_3_2_0, road_4_0_1_0, road_5_2_2_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_0_4_0_0, road_5_4_2_0, road_0_1_0_0, road_5_3_2_0, road_5_3_2_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_5_3_2_0, road_0_4_0_0, road_0_4_0_0, road_0_1_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_5_3_2_0, road_0_1_0_0, road_1_0_1_0, road_0_4_0_0, road_5_4_2_0, road_5_2_2_0, road_0_4_0_0, road_0_4_0_0, road_0_1_0_0, road_0_1_0_0, road_0_2_0_0, road_0_1_0_0, road_5_4_2_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_5_2_2_0, road_0_4_0_0, road_4_0_1_0, road_1_0_1_0, road_4_0_1_0, road_1_5_3_0, road_0_4_0_0, road_0_4_0_0, road_1_0_1_0, road_1_0_1_0, road_5_4_2_0, road_1_5_3_0, road_0_1_0_0, road_0_2_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_0_2_0_0, road_5_4_2_0, road_4_0_1_0, road_5_4_2_0, road_5_2_2_0, road_1_5_3_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_2_0_0, road_5_3_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_2_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_2_2_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_5_2_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_1_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_1_0_1_0, road_0_4_0_0, road_0_1_0_0, road_5_3_2_0, road_0_1_0_0, road_1_0_1_0, road_0_1_0_0, road_5_4_2_0, road_5_3_2_0, road_0_2_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_1_0_0, road_0_4_0_0, road_4_0_1_0, road_1_0_1_0, road_5_4_2_0, road_1_0_1_0, road_0_1_0_0, road_0_2_0_0, road_0_4_0_0, road_1_0_1_0, road_0_4_0_0, road_5_2_2_0, road_0_1_0_0, road_1_0_1_0, road_0_4_0_0, road_5_4_2_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_5_3_2_0, road_0_1_0_0, road_0_4_0_0, road_5_4_2_0, road_1_0_1_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_2_0_0, road_5_3_2_0, road_0_1_0_0, road_0_4_0_0, road_0_2_0_0, road_0_4_0_0, road_5_4_2_0, road_5_4_2_0, road_0_4_0_0, road_0_2_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_0_4_0_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_5_3_2_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_0_2_0_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_5_2_2_0, road_1_5_3_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_0_4_0_0, road_5_2_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_5_2_2_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_5_3_2_0, road_4_0_1_0, road_0_1_0_0, road_5_4_2_0, road_5_2_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_1_5_3_0, road_5_3_2_0, road_5_2_2_0, road_0_4_0_0, road_0_1_0_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_5_2_2_0, road_1_5_3_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_5_4_2_0, road_5_2_2_0, road_1_0_1_0, road_0_4_0_0, road_5_3_2_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_2_0_0, road_5_2_2_0, road_5_4_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_4_0_1_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_1_0_1_0, road_0_1_0_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_4_0_1_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_5_2_2_0, road_0_1_0_0, road_0_4_0_0, road_4_0_1_0, road_5_4_2_0, road_4_0_1_0, road_0_1_0_0, road_5_3_2_0, road_5_3_2_0, road_0_1_0_0, road_1_0_1_0, road_5_3_2_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_1_0_0, road_5_3_2_0, road_5_4_2_0, road_5_2_2_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_0_4_0_0, road_5_4_2_0, road_0_1_0_0, road_0_1_0_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_5_4_2_0, road_5_2_2_0, road_0_1_0_0, road_5_4_2_0, road_5_3_2_0, road_0_2_0_0, road_0_4_0_0, road_0_4_0_0, road_1_5_3_0, road_0_1_0_0, road_5_4_2_0, road_0_1_0_0, road_0_1_0_0, road_5_3_2_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_5_2_2_0, road_5_3_2_0, road_0_1_0_0, road_0_4_0_0, road_1_0_1_0, road_5_4_2_0, road_5_4_2_0, road_0_1_0_0, road_0_4_0_0, road_5_2_2_0, road_0_4_0_0, road_0_4_0_0, road_1_5_3_0, road_0_4_0_0, road_0_1_0_0, road_1_0_1_0, road_0_1_0_0, road_4_0_1_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_3_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_1_0_0, road_5_2_2_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_4_0_1_0, road_5_4_2_0, road_1_5_3_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_4_0_1_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_5_4_2_0, road_0_4_0_0, road_5_3_2_0, road_0_1_0_0, road_0_4_0_0, road_5_4_2_0, road_1_0_1_0, road_0_4_0_0, road_4_0_1_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_5_2_2_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_4_0_1_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_3_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_4_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_4_0_1_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_4_0_1_0, road_0_4_0_0, road_5_4_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_0_4_0_0, road_0_1_0_0, road_4_0_1_0, road_5_4_2_0, road_0_4_0_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_5_3_2_0, road_0_4_0_0, road_5_4_2_0, road_0_1_0_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_4_0_1_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_5_3_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_5_4_2_0, road_4_0_1_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_4_0_1_0, road_5_4_2_0, road_5_3_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_4_0_1_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_5_3_2_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_4_2_0, road_4_0_1_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_4_0_1_0, road_0_1_0_0, road_5_2_2_0, road_5_3_2_0, road_4_0_1_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_3_2_0, road_5_4_2_0, road_5_3_2_0, road_5_2_2_0, road_1_5_3_0, road_0_4_0_0, road_1_5_3_0, road_0_4_0_0, road_5_4_2_0, road_5_4_2_0, road_5_4_2_0, road_0_2_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_5_3_2_0, road_5_4_2_0, road_1_5_3_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_1_0_0, road_0_1_0_0, road_4_0_1_0, road_4_0_1_0, road_5_4_2_0, road_5_4_2_0, road_0_4_0_0, road_1_5_3_0, road_4_0_1_0, road_4_0_1_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_5_4_2_0, road_1_0_1_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_2_2_0, road_1_0_1_0, road_5_3_2_0, road_0_1_0_0, road_0_4_0_0, road_5_4_2_0, road_4_0_1_0, road_0_4_0_0, road_0_4_0_0, road_4_0_1_0, road_5_2_2_0, road_1_0_1_0, road_0_4_0_0, road_5_4_2_0, road_4_0_1_0, road_0_2_0_0, road_5_4_2_0, road_0_4_0_0, road_0_2_0_0, road_5_4_2_0, road_5_3_2_0, road_0_1_0_0, road_0_4_0_0, road_0_4_0_0, road_0_1_0_0, road_0_4_0_0, road_5_3_2_0, road_5_4_2_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_0_4_0_0, road_4_0_1_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_4_0_1_0, road_0_4_0_0, road_5_4_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_3_2_0, road_4_0_1_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_4_0_1_0, road_0_2_0_0, road_5_2_2_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_4_0_1_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_5_4_2_0, road_0_1_0_0, road_0_2_0_0, road_5_2_2_0, road_1_0_1_0, road_1_5_3_0, road_0_4_0_0, road_5_4_2_0, road_5_3_2_0, road_0_4_0_0, road_4_0_1_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_0_1_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_0_2_0_0, road_5_2_2_0, road_4_0_1_0, road_5_4_2_0, road_1_0_1_0, road_1_5_3_0, road_5_3_2_0, road_0_1_0_0, road_5_4_2_0, road_5_4_2_0, road_4_0_1_0, road_0_2_0_0, road_5_3_2_0, road_0_1_0_0, road_5_4_2_0, road_5_2_2_0, road_0_2_0_0, road_0_4_0_0, road_0_4_0_0, road_1_0_1_0, road_0_4_0_0, road_0_4_0_0, road_0_4_0_0, road_0_1_0_0, road_1_0_1_0, road_5_4_2_0, road_5_4_2_0, road_5_4_2_0, road_1_0_1_0, road_0_1_0_0, road_0_1_0_0, road_0_2_0_0, road_5_4_2_0, road_5_4_2_0, road_1_5_3_0, road_0_4_0_0, road_5_4_2_0, road_0_4_0_0, road_5_4_2_0, road_5_4_2_0, road_0_4_0_0, road_0_4_0_0, road_1_0_1_0, road_0_1_0_0, road_0_2_0_0]
arrivalLane = [road_1_1_3_0, road_1_4_1_0, road_4_4_1_0, road_1_1_3_0, road_4_4_1_0, road_4_1_0_0, road_4_2_0_0, road_4_4_1_0, road_4_1_0_0, road_1_1_3_0, road_4_4_0_0, road_4_4_1_0, road_4_4_1_0, road_1_4_1_0, road_1_1_3_0, road_1_1_3_0, road_3_4_1_0, road_1_3_2_0, road_2_1_3_0, road_1_4_2_0, road_1_1_3_0, road_1_3_2_0, road_1_2_2_0, road_1_2_2_1, road_4_4_1_0, road_2_4_1_0, road_4_1_0_0, road_1_3_2_0, road_1_1_3_0, road_1_1_3_0, road_3_1_3_0, road_1_4_1_0, road_2_4_1_0, road_1_2_2_0, road_4_4_1_0, road_4_4_0_0, road_2_4_1_0, road_4_4_1_0, road_3_1_3_0, road_4_2_0_0, road_1_1_3_0, road_3_4_1_0, road_1_3_2_0, road_4_2_0_0, road_4_3_0_0, road_1_2_2_0, road_1_4_1_0, road_3_4_1_0, road_1_1_2_0, road_2_1_3_0, road_1_4_1_0, road_4_4_0_0, road_2_4_1_0, road_4_1_0_0, road_2_1_3_0, road_1_3_2_0, road_2_1_3_1, road_4_4_1_0, road_4_4_0_0, road_2_4_1_0, road_4_1_0_0, road_3_1_3_0, road_1_1_3_0, road_3_4_1_0, road_4_3_0_0, road_1_4_1_0, road_1_4_1_0, road_2_3_1_0, road_1_4_1_0, road_1_1_3_0, road_1_3_2_0, road_4_4_0_0, road_4_4_1_0, road_2_1_3_0, road_4_4_1_0, road_3_1_3_0, road_4_1_3_0, road_1_4_2_0, road_1_1_3_0, road_4_4_0_0, road_1_3_2_0, road_1_1_3_0, road_3_4_1_0, road_4_1_0_0, road_1_1_2_0, road_1_1_2_0, road_2_4_1_0, road_2_4_1_1, road_1_2_2_0, road_4_4_0_0, road_2_4_1_0, road_1_4_1_0, road_4_4_0_0, road_1_1_2_0, road_4_4_1_0, road_1_4_1_0, road_4_2_0_0, road_1_2_2_0, road_1_4_2_0, road_1_1_3_0, road_4_4_1_0, road_1_1_3_1, road_2_4_1_0, road_4_4_1_0, road_4_4_1_1, road_1_4_1_0, road_4_3_0_0, road_4_3_0_0, road_4_4_1_0, road_4_4_3_0, road_4_2_0_0, road_1_2_2_0, road_4_3_0_0, road_4_1_0_0, road_2_1_3_0, road_1_4_2_0, road_1_4_1_0, road_3_4_1_0, road_1_3_2_0, road_2_4_1_0, road_1_1_3_0, road_4_1_0_0, road_4_2_0_0, road_1_3_2_0, road_4_4_0_0, road_4_4_1_0, road_3_4_1_0, road_4_4_1_0, road_1_4_1_0, road_4_1_3_0, road_4_2_0_0, road_4_2_0_0, road_4_4_1_0, road_4_1_3_0, road_1_1_2_0, road_1_1_3_0, road_2_1_3_0, road_1_2_2_0, road_2_1_3_0, road_4_4_1_0, road_2_1_3_1, road_4_4_1_0, road_1_1_3_0, road_4_2_0_0, road_4_4_1_1, road_2_4_1_0, road_3_4_1_0, road_1_1_3_0, road_1_4_1_0, road_4_4_1_0, road_4_4_1_0, road_2_4_1_0, road_4_1_3_0, road_4_4_0_0, road_4_4_1_0, road_1_3_2_0, road_4_4_1_0, road_1_4_2_0, road_1_1_3_0, road_1_4_2_0, road_3_4_1_0, road_1_4_1_0, road_2_1_3_0, road_1_2_2_0, road_1_2_2_0, road_1_4_1_0, road_2_4_1_0, road_1_1_2_0, road_4_1_3_0, road_1_1_2_0, road_4_4_1_0, road_4_3_0_0, road_2_4_1_0, road_2_4_1_0, road_3_4_1_0, road_1_2_2_0, road_1_4_2_0, road_1_1_3_1, road_4_4_1_0, road_2_1_3_1, road_2_1_3_0, road_4_3_0_0, road_1_1_3_0, road_4_2_0_1, road_4_2_0_0, road_1_2_2_0, road_4_4_1_0, road_4_4_0_0, road_1_3_2_0, road_1_4_1_0, road_4_2_0_0, road_2_1_3_0, road_3_4_1_0, road_1_3_2_0, road_1_2_2_0, road_1_1_2_0, road_2_1_3_0, road_4_1_0_0, road_4_2_0_0, road_4_4_1_0, road_4_2_0_0, road_1_4_1_0, road_1_1_2_0, road_4_4_0_0, road_4_3_0_0, road_1_4_1_0, road_4_4_1_0, road_4_1_0_0, road_1_1_3_0, road_1_3_2_0, road_1_1_3_0, road_3_4_1_0, road_3_4_1_0, road_1_4_1_0, road_4_4_1_0, road_4_1_3_0, road_1_2_2_0, road_4_4_1_1, road_4_4_1_0, road_1_3_2_0, road_1_1_2_0, road_3_1_3_0, road_3_4_1_0, road_4_1_0_0, road_1_4_1_0, road_4_3_0_0, road_1_2_2_1, road_1_2_2_0, road_3_1_3_0, road_4_1_0_0, road_4_2_0_0, road_4_4_1_0, road_2_1_3_1, road_4_4_0_0, road_1_4_2_0, road_1_1_2_0, road_3_1_3_0, road_2_1_3_0, road_2_4_1_0, road_2_4_1_1, road_1_4_1_0, road_4_1_3_0, road_1_3_2_0, road_1_2_2_0, road_1_1_3_0, road_1_4_2_0, road_4_1_3_0, road_3_1_3_0, road_1_3_2_0, road_1_2_2_0, road_1_1_3_0, road_1_2_2_0, road_4_4_1_0, road_4_1_0_0, road_1_2_2_0, road_1_1_2_0, road_4_4_1_0, road_3_4_1_0, road_1_4_2_0, road_4_4_1_0, road_1_1_3_1, road_4_4_0_0, road_1_1_3_0, road_2_1_3_0, road_2_1_3_1, road_4_1_0_0, road_3_1_3_0, road_1_1_3_0, road_1_1_3_0, road_1_1_3_1, road_2_1_3_0, road_1_3_2_0, road_1_1_3_0, road_4_4_1_0, road_1_4_2_0, road_3_4_1_0, road_4_1_3_0, road_4_4_1_0, road_2_1_3_0, road_4_4_1_0, road_4_2_0_0, road_1_2_2_0, road_4_4_0_0, road_2_1_3_0, road_1_4_1_0, road_1_3_2_0, road_4_4_0_0, road_4_4_1_0, road_2_1_3_0, road_1_3_2_0, road_4_3_0_0, road_4_3_0_1, road_1_4_1_0, road_3_4_1_0, road_2_1_3_0, road_2_1_3_1, road_1_1_3_0, road_1_4_2_0, road_1_1_3_0, road_4_2_0_0, road_1_1_3_0, road_4_1_0_0, road_4_4_0_0, road_2_1_3_0, road_1_2_2_0, road_4_4_0_0, road_4_4_1_0, road_4_4_1_0, road_2_1_3_0, road_4_2_0_0, road_3_4_1_0, road_1_4_1_0, road_1_1_3_0, road_4_3_0_0, road_1_1_3_0, road_1_3_2_0, road_4_1_3_0, road_2_1_3_0, road_4_1_0_1, road_4_1_0_0, road_2_4_1_0, road_4_1_0_0, road_1_1_2_0, road_3_4_1_0, road_2_4_1_0, road_1_1_2_1, road_1_4_2_0, road_1_3_2_0, road_4_3_0_0, road_4_4_0_0, road_2_1_3_0, road_1_1_3_0, road_4_4_1_0, road_3_1_3_0, road_1_1_2_0, road_4_3_0_0, road_2_4_1_0, road_2_4_1_0, road_1_4_1_0, road_4_1_0_0, road_1_1_3_0, road_3_2_0_0, road_3_4_3_0, road_1_3_2_0, road_1_3_2_0, road_1_3_2_1, road_4_3_0_0, road_1_4_1_0, road_4_4_0_0, road_1_3_2_0, road_2_1_3_0, road_2_4_1_0, road_4_4_1_0, road_1_4_2_0, road_1_1_3_0, road_1_4_1_0, road_2_1_3_1, road_4_4_1_1, road_4_1_3_0, road_4_2_0_0, road_3_3_3_0, road_1_2_2_0, road_3_4_1_0, road_4_4_1_0, road_1_1_1_0, road_1_2_2_0, road_1_1_2_0, road_4_1_3_0, road_4_1_0_0, road_1_1_2_0, road_1_1_3_0, road_1_2_2_0, road_4_1_0_0, road_2_4_1_0, road_1_2_2_0, road_4_4_1_0, road_4_1_0_0, road_1_1_3_0, road_1_1_3_0, road_1_1_2_0, road_2_1_3_0, road_1_4_2_0, road_4_3_0_0, road_1_4_1_0, road_3_4_1_0, road_2_1_3_0, road_4_2_0_0, road_1_4_2_0, road_1_2_2_0, road_1_4_1_0, road_4_4_0_0, road_1_4_1_0, road_4_4_1_0, road_4_4_0_1, road_1_4_1_0, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ]


original_list = [(0.0, 'road_4_0_1', 'road_4_2_0'), (78.0, 'road_4_0_1', 'road_4_1_0'), (104.0, 'road_4_0_1', 'road_4_1_0')]
new_list = [item for item in original_list if item[1] == 'road_4_0_1']
print(new_list)


In [ ]:
import numpy as np
def find_indices(lst,value):
    pos = np.where(np.array(lst)==value)[0]
    return pos
def extract_elements_by_indices(str_list, index_list):
    """
    根据给定的索引列表,从字符串列表中提取元素,保持索引顺序。
    
    参数:
    str_list -- 包含字符串的列表
    index_list -- 包含要提取元素索引的列表
    
    返回:
    一个新列表,包含按照index_list指定顺序的str_list中的元素
    """
    # 确保索引都在有效范围内
    valid_index_list = [idx for idx in index_list if idx < len(str_list)]
    # 根据索引顺序提取元素
    extracted_elements = [str_list[idx] for idx in valid_index_list]
    return extracted_elements


#将三者合并为元组的形式进行选取,先对start_edges和end_edges进行处理
start_edges = [item[:-2] for item in arrivalLane]
end_edges = [item[:-2] for item in end_edge]

find_value = 'road_4_0_1'
indices = find_indices(start_edges, find_value)

start_edges=['road_1_1_3', 'road_1_4_1', 'road_4_4_1', 'road_1_1_3', 'road_4_4_1', 'road_4_1_0', 'road_4_2_0', 'road_4_4_1', 'road_4_1_0', 'road_1_1_3', 'road_4_4_0', 'road_4_4_1', 'road_4_4_1', 'road_1_4_1', 'road_1_1_3', 'road_1_1_3', 'road_3_4_1', 'road_1_3_2', 'road_2_1_3', 'road_1_4_2', 'road_1_1_3', 'road_1_3_2', 'road_1_2_2', 'road_1_2_2', 'road_4_4_1', 'road_2_4_1', 'road_4_1_0', 'road_1_3_2', 'road_1_1_3', 'road_1_1_3', 'road_3_1_3', 'road_1_4_1', 'road_2_4_1', 'road_1_2_2', 'road_4_4_1', 'road_4_4_0', 'road_2_4_1', 'road_4_4_1', 'road_3_1_3', 'road_4_2_0', 'road_1_1_3', 'road_3_4_1', 'road_1_3_2', 'road_4_2_0', 'road_4_3_0', 'road_1_2_2', 'road_1_4_1', 'road_3_4_1', 'road_1_1_2', 'road_2_1_3', 'road_1_4_1', 'road_4_4_0', 'road_2_4_1', 'road_4_1_0', 'road_2_1_3', 'road_1_3_2', 'road_2_1_3', 'road_4_4_1', 'road_4_4_0', 'road_2_4_1', 'road_4_1_0', 'road_3_1_3', 'road_1_1_3', 'road_3_4_1', 'road_4_3_0', 'road_1_4_1', 'road_1_4_1', 'road_2_3_1', 'road_1_4_1', 'road_1_1_3', 'road_1_3_2', 'road_4_4_0', 'road_4_4_1', 'road_2_1_3', 'road_4_4_1', 'road_3_1_3', 'road_4_1_3', 'road_1_4_2', 'road_1_1_3', 'road_4_4_0', 'road_1_3_2', 'road_1_1_3', 'road_3_4_1', 'road_4_1_0', 'road_1_1_2', 'road_1_1_2', 'road_2_4_1', 'road_2_4_1', 'road_1_2_2', 'road_4_4_0', 'road_2_4_1', 'road_1_4_1', 'road_4_4_0', 'road_1_1_2', 'road_4_4_1', 'road_1_4_1', 'road_4_2_0', 'road_1_2_2', 'road_1_4_2', 'road_1_1_3', 'road_4_4_1', 'road_1_1_3', 'road_2_4_1', 'road_4_4_1', 'road_4_4_1', 'road_1_4_1', 'road_4_3_0', 'road_4_3_0', 'road_4_4_1', 'road_4_4_3', 'road_4_2_0', 'road_1_2_2', 'road_4_3_0', 'road_4_1_0', 'road_2_1_3', 'road_1_4_2', 'road_1_4_1', 'road_3_4_1', 'road_1_3_2', 'road_2_4_1', 'road_1_1_3', 'road_4_1_0', 'road_4_2_0', 'road_1_3_2', 'road_4_4_0', 'road_4_4_1', 'road_3_4_1', 'road_4_4_1', 'road_1_4_1', 'road_4_1_3', 'road_4_2_0', 'road_4_2_0', 'road_4_4_1', 'road_4_1_3', 'road_1_1_2', 'road_1_1_3', 'road_2_1_3', 'road_1_2_2', 'road_2_1_3', 'road_4_4_1', 'road_2_1_3', 'road_4_4_1', 'road_1_1_3', 'road_4_2_0', 'road_4_4_1', 'road_2_4_1', 'road_3_4_1', 'road_1_1_3', 'road_1_4_1', 'road_4_4_1', 'road_4_4_1', 'road_2_4_1', 'road_4_1_3', 'road_4_4_0', 'road_4_4_1', 'road_1_3_2', 'road_4_4_1', 'road_1_4_2', 'road_1_1_3', 'road_1_4_2', 'road_3_4_1', 'road_1_4_1', 'road_2_1_3', 'road_1_2_2', 'road_1_2_2', 'road_1_4_1', 'road_2_4_1', 'road_1_1_2', 'road_4_1_3', 'road_1_1_2', 'road_4_4_1', 'road_4_3_0', 'road_2_4_1', 'road_2_4_1', 'road_3_4_1', 'road_1_2_2', 'road_1_4_2', 'road_1_1_3', 'road_4_4_1', 'road_2_1_3', 'road_2_1_3', 'road_4_3_0', 'road_1_1_3', 'road_4_2_0', 'road_4_2_0', 'road_1_2_2', 'road_4_4_1', 'road_4_4_0', 'road_1_3_2', 'road_1_4_1', 'road_4_2_0', 'road_2_1_3', 'road_3_4_1', 'road_1_3_2', 'road_1_2_2', 'road_1_1_2', 'road_2_1_3', 'road_4_1_0', 'road_4_2_0', 'road_4_4_1', 'road_4_2_0', 'road_1_4_1', 'road_1_1_2', 'road_4_4_0', 'road_4_3_0', 'road_1_4_1', 'road_4_4_1', 'road_4_1_0', 'road_1_1_3', 'road_1_3_2', 'road_1_1_3', 'road_3_4_1', 'road_3_4_1', 'road_1_4_1', 'road_4_4_1', 'road_4_1_3', 'road_1_2_2', 'road_4_4_1', 'road_4_4_1', 'road_1_3_2', 'road_1_1_2', 'road_3_1_3', 'road_3_4_1', 'road_4_1_0', 'road_1_4_1', 'road_4_3_0', 'road_1_2_2', 'road_1_2_2', 'road_3_1_3', 'road_4_1_0', 'road_4_2_0', 'road_4_4_1', 'road_2_1_3', 'road_4_4_0', 'road_1_4_2', 'road_1_1_2', 'road_3_1_3', 'road_2_1_3', 'road_2_4_1', 'road_2_4_1', 'road_1_4_1', 'road_4_1_3', 'road_1_3_2', 'road_1_2_2', 'road_1_1_3', 'road_1_4_2', 'road_4_1_3', 'road_3_1_3', 'road_1_3_2', 'road_1_2_2', 'road_1_1_3', 'road_1_2_2', 'road_4_4_1', 'road_4_1_0', 'road_1_2_2', 'road_1_1_2', 'road_4_4_1', 'road_3_4_1', 'road_1_4_2', 'road_4_4_1', 'road_1_1_3', 'road_4_4_0', 'road_1_1_3', 'road_2_1_3', 'road_2_1_3', 'road_4_1_0', 'road_3_1_3', 'road_1_1_3', 'road_1_1_3', 'road_1_1_3', 'road_2_1_3', 'road_1_3_2', 'road_1_1_3', 'road_4_4_1', 'road_1_4_2', 'road_3_4_1', 'road_4_1_3', 'road_4_4_1', 'road_2_1_3', 'road_4_4_1', 'road_4_2_0', 'road_1_2_2', 'road_4_4_0', 'road_2_1_3', 'road_1_4_1', 'road_1_3_2', 'road_4_4_0', 'road_4_4_1', 'road_2_1_3', 'road_1_3_2', 'road_4_3_0', 'road_4_3_0', 'road_1_4_1', 'road_3_4_1', 'road_2_1_3', 'road_2_1_3', 'road_1_1_3', 'road_1_4_2', 'road_1_1_3', 'road_4_2_0', 'road_1_1_3', 'road_4_1_0', 'road_4_4_0', 'road_2_1_3', 'road_1_2_2', 'road_4_4_0', 'road_4_4_1', 'road_4_4_1', 'road_2_1_3', 'road_4_2_0', 'road_3_4_1', 'road_1_4_1', 'road_1_1_3', 'road_4_3_0', 'road_1_1_3', 'road_1_3_2', 'road_4_1_3', 'road_2_1_3', 'road_4_1_0', 'road_4_1_0', 'road_2_4_1', 'road_4_1_0', 'road_1_1_2', 'road_3_4_1', 'road_2_4_1', 'road_1_1_2', 'road_1_4_2', 'road_1_3_2', 'road_4_3_0', 'road_4_4_0', 'road_2_1_3', 'road_1_1_3', 'road_4_4_1', 'road_3_1_3', 'road_1_1_2', 'road_4_3_0', 'road_2_4_1', 'road_2_4_1', 'road_1_4_1', 'road_4_1_0', 'road_1_1_3', 'road_3_2_0', 'road_3_4_3', 'road_1_3_2', 'road_1_3_2', 'road_1_3_2', 'road_4_3_0', 'road_1_4_1', 'road_4_4_0', 'road_1_3_2', 'road_2_1_3', 'road_2_4_1', 'road_4_4_1', 'road_1_4_2', 'road_1_1_3', 'road_1_4_1', 'road_2_1_3', 'road_4_4_1', 'road_4_1_3', 'road_4_2_0', 'road_3_3_3', 'road_1_2_2', 'road_3_4_1', 'road_4_4_1', 'road_1_1_1', 'road_1_2_2', 'road_1_1_2', 'road_4_1_3', 'road_4_1_0', 'road_1_1_2', 'road_1_1_3', 'road_1_2_2', 'road_4_1_0', 'road_2_4_1', 'road_1_2_2', 'road_4_4_1', 'road_4_1_0', 'road_1_1_3', 'road_1_1_3', 'road_1_1_2', 'road_2_1_3', 'road_1_4_2', 'road_4_3_0', 'road_1_4_1', 'road_3_4_1', 'road_2_1_3', 'road_4_2_0', 'road_1_4_2', 'road_1_2_2', 'road_1_4_1', 'road_4_4_0', 'road_1_4_1', 'road_4_4_1', 'road_4_4_0', 'road_1_4_1']
pos = np.where(np.array(start_edges)=='road_4_0_1')[0]





In [ ]:
find_value = 'road_4_0_1'
indices = find_indices(start_edges, find_value)
print(indices)

In [ ]:
#将筛选出来的list拆解为代码可用形式
depart_times,start_edges,end_edges = [
    [item[i] for item in  new_list] for i in range(3)
]
print(len(depart_times))
print("depart_times=",depart_times)
print("start_edges=",start_edges)
print("end_edges=",end_edges)

In [ ]:
my_list = ["apple", "banana", "cherry", "apple", "date"]
find_value = "apple"
indices = find_indices(my_list, find_value)
print(f"'{find_value}'在列表中的索引有：{indices}")

In [ ]:
list=['road_1_1_3', 'road_1_4_1', 'road_4_4_1', 'road_1_1_3', 'road_4_4_1', 'road_4_1_0', 'road_4_2_0', 'road_4_4_1', 'road_4_1_0', 'road_1_1_3', 'road_4_4_0', 'road_4_4_1', 'road_4_4_1', 'road_1_4_1', 'road_1_1_3', 'road_1_1_3', 'road_3_4_1', 'road_1_3_2', 'road_2_1_3', 'road_1_4_2', 'road_1_1_3', 'road_1_3_2', 'road_1_2_2', 'road_1_2_2', 'road_4_4_1', 'road_2_4_1', 'road_4_1_0', 'road_1_3_2', 'road_1_1_3', 'road_1_1_3', 'road_3_1_3', 'road_1_4_1', 'road_2_4_1', 'road_1_2_2', 'road_4_4_1', 'road_4_4_0', 'road_2_4_1', 'road_4_4_1', 'road_3_1_3', 'road_4_2_0', 'road_1_1_3', 'road_3_4_1', 'road_1_3_2', 'road_4_2_0', 'road_4_3_0', 'road_1_2_2', 'road_1_4_1', 'road_3_4_1', 'road_1_1_2', 'road_2_1_3', 'road_1_4_1', 'road_4_4_0', 'road_2_4_1', 'road_4_1_0', 'road_2_1_3', 'road_1_3_2', 'road_2_1_3', 'road_4_4_1', 'road_4_4_0', 'road_2_4_1', 'road_4_1_0', 'road_3_1_3', 'road_1_1_3', 'road_3_4_1', 'road_4_3_0', 'road_1_4_1', 'road_1_4_1', 'road_2_3_1', 'road_1_4_1', 'road_1_1_3', 'road_1_3_2', 'road_4_4_0', 'road_4_4_1', 'road_2_1_3', 'road_4_4_1', 'road_3_1_3', 'road_4_1_3', 'road_1_4_2', 'road_1_1_3', 'road_4_4_0', 'road_1_3_2', 'road_1_1_3', 'road_3_4_1', 'road_4_1_0', 'road_1_1_2', 'road_1_1_2', 'road_2_4_1', 'road_2_4_1', 'road_1_2_2', 'road_4_4_0', 'road_2_4_1', 'road_1_4_1', 'road_4_4_0', 'road_1_1_2', 'road_4_4_1', 'road_1_4_1', 'road_4_2_0', 'road_1_2_2', 'road_1_4_2', 'road_1_1_3', 'road_4_4_1', 'road_1_1_3', 'road_2_4_1', 'road_4_4_1', 'road_4_4_1', 'road_1_4_1', 'road_4_3_0', 'road_4_3_0', 'road_4_4_1', 'road_4_4_3', 'road_4_2_0', 'road_1_2_2', 'road_4_3_0', 'road_4_1_0', 'road_2_1_3', 'road_1_4_2', 'road_1_4_1', 'road_3_4_1', 'road_1_3_2', 'road_2_4_1', 'road_1_1_3', 'road_4_1_0', 'road_4_2_0', 'road_1_3_2', 'road_4_4_0', 'road_4_4_1', 'road_3_4_1', 'road_4_4_1', 'road_1_4_1', 'road_4_1_3', 'road_4_2_0', 'road_4_2_0', 'road_4_4_1', 'road_4_1_3', 'road_1_1_2', 'road_1_1_3', 'road_2_1_3', 'road_1_2_2', 'road_2_1_3', 'road_4_4_1', 'road_2_1_3', 'road_4_4_1', 'road_1_1_3', 'road_4_2_0', 'road_4_4_1', 'road_2_4_1', 'road_3_4_1', 'road_1_1_3', 'road_1_4_1', 'road_4_4_1', 'road_4_4_1', 'road_2_4_1', 'road_4_1_3', 'road_4_4_0', 'road_4_4_1', 'road_1_3_2', 'road_4_4_1', 'road_1_4_2', 'road_1_1_3', 'road_1_4_2', 'road_3_4_1', 'road_1_4_1', 'road_2_1_3', 'road_1_2_2', 'road_1_2_2', 'road_1_4_1', 'road_2_4_1', 'road_1_1_2', 'road_4_1_3', 'road_1_1_2', 'road_4_4_1', 'road_4_3_0', 'road_2_4_1', 'road_2_4_1', 'road_3_4_1', 'road_1_2_2', 'road_1_4_2', 'road_1_1_3', 'road_4_4_1', 'road_2_1_3', 'road_2_1_3', 'road_4_3_0', 'road_1_1_3', 'road_4_2_0', 'road_4_2_0', 'road_1_2_2', 'road_4_4_1', 'road_4_4_0', 'road_1_3_2', 'road_1_4_1', 'road_4_2_0', 'road_2_1_3', 'road_3_4_1', 'road_1_3_2', 'road_1_2_2', 'road_1_1_2', 'road_2_1_3', 'road_4_1_0', 'road_4_2_0', 'road_4_4_1', 'road_4_2_0', 'road_1_4_1', 'road_1_1_2', 'road_4_4_0', 'road_4_3_0', 'road_1_4_1', 'road_4_4_1', 'road_4_1_0', 'road_1_1_3', 'road_1_3_2', 'road_1_1_3', 'road_3_4_1', 'road_3_4_1', 'road_1_4_1', 'road_4_4_1', 'road_4_1_3', 'road_1_2_2', 'road_4_4_1', 'road_4_4_1', 'road_1_3_2', 'road_1_1_2', 'road_3_1_3', 'road_3_4_1', 'road_4_1_0', 'road_1_4_1', 'road_4_3_0', 'road_1_2_2', 'road_1_2_2', 'road_3_1_3', 'road_4_1_0', 'road_4_2_0', 'road_4_4_1', 'road_2_1_3', 'road_4_4_0', 'road_1_4_2', 'road_1_1_2', 'road_3_1_3', 'road_2_1_3', 'road_2_4_1', 'road_2_4_1', 'road_1_4_1', 'road_4_1_3', 'road_1_3_2', 'road_1_2_2', 'road_1_1_3', 'road_1_4_2', 'road_4_1_3', 'road_3_1_3', 'road_1_3_2', 'road_1_2_2', 'road_1_1_3', 'road_1_2_2', 'road_4_4_1', 'road_4_1_0', 'road_1_2_2', 'road_1_1_2', 'road_4_4_1', 'road_3_4_1', 'road_1_4_2', 'road_4_4_1', 'road_1_1_3', 'road_4_4_0', 'road_1_1_3', 'road_2_1_3', 'road_2_1_3', 'road_4_1_0', 'road_3_1_3', 'road_1_1_3', 'road_1_1_3', 'road_1_1_3', 'road_2_1_3', 'road_1_3_2', 'road_1_1_3', 'road_4_4_1', 'road_1_4_2', 'road_3_4_1', 'road_4_1_3', 'road_4_4_1', 'road_2_1_3', 'road_4_4_1', 'road_4_2_0', 'road_1_2_2', 'road_4_4_0', 'road_2_1_3', 'road_1_4_1', 'road_1_3_2', 'road_4_4_0', 'road_4_4_1', 'road_2_1_3', 'road_1_3_2', 'road_4_3_0', 'road_4_3_0', 'road_1_4_1', 'road_3_4_1', 'road_2_1_3', 'road_2_1_3', 'road_1_1_3', 'road_1_4_2', 'road_1_1_3', 'road_4_2_0', 'road_1_1_3', 'road_4_1_0', 'road_4_4_0', 'road_2_1_3', 'road_1_2_2', 'road_4_4_0', 'road_4_4_1', 'road_4_4_1', 'road_2_1_3', 'road_4_2_0', 'road_3_4_1', 'road_1_4_1', 'road_1_1_3', 'road_4_3_0', 'road_1_1_3', 'road_1_3_2', 'road_4_1_3', 'road_2_1_3', 'road_4_1_0', 'road_4_1_0', 'road_2_4_1', 'road_4_1_0', 'road_1_1_2', 'road_3_4_1', 'road_2_4_1', 'road_1_1_2', 'road_1_4_2', 'road_1_3_2', 'road_4_3_0', 'road_4_4_0', 'road_2_1_3', 'road_1_1_3', 'road_4_4_1', 'road_3_1_3', 'road_1_1_2', 'road_4_3_0', 'road_2_4_1', 'road_2_4_1', 'road_1_4_1', 'road_4_1_0', 'road_1_1_3', 'road_3_2_0', 'road_3_4_3', 'road_1_3_2', 'road_1_3_2', 'road_1_3_2', 'road_4_3_0', 'road_1_4_1', 'road_4_4_0', 'road_1_3_2', 'road_2_1_3', 'road_2_4_1', 'road_4_4_1', 'road_1_4_2', 'road_1_1_3', 'road_1_4_1', 'road_2_1_3', 'road_4_4_1', 'road_4_1_3', 'road_4_2_0', 'road_3_3_3', 'road_1_2_2', 'road_3_4_1', 'road_4_4_1', 'road_1_1_1', 'road_1_2_2', 'road_1_1_2', 'road_4_1_3', 'road_4_1_0', 'road_1_1_2', 'road_1_1_3', 'road_1_2_2', 'road_4_1_0', 'road_2_4_1', 'road_1_2_2', 'road_4_4_1', 'road_4_1_0', 'road_1_1_3', 'road_1_1_3', 'road_1_1_2', 'road_2_1_3', 'road_1_4_2', 'road_4_3_0', 'road_1_4_1', 'road_3_4_1', 'road_2_1_3', 'road_4_2_0', 'road_1_4_2', 'road_1_2_2', 'road_1_4_1', 'road_4_4_0', 'road_1_4_1', 'road_4_4_1', 'road_4_4_0', 'road_1_4_1']
new_list = [i for i, x in enumerate(list) if x == 'road_4_0_1']
print(new_list)


In [128]:

start_edge = [,'road_0_1_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_1_5_3','road_0_1_0','road_1_5_3','road_1_5_3','road_0_4_0','road_5_3_2','road_5_4_2','road_4_0_1','road_1_5_3','road_0_1_0','road_0_1_0','road_4_0_1','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_5_4_2','road_0_1_0','road_4_0_1','road_0_1_0','road_5_3_2','road_1_5_3','road_4_0_1','road_4_0_1','road_0_4_0','road_1_0_1','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_4_0_1','road_5_4_2','road_1_0_1','road_0_1_0','road_1_5_3','road_0_1_0','road_4_0_1','road_0_4_0','road_1_5_3','road_4_0_1','road_0_1_0','road_4_0_1','road_5_3_2','road_4_0_1','road_0_4_0','road_0_4_0','road_0_4_0','road_1_0_1','road_0_1_0','road_0_4_0','road_5_3_2','road_5_4_2','road_0_1_0','road_5_4_2','road_1_0_1','road_0_1_0','road_1_5_3','road_0_1_0','road_0_4_0','road_1_5_3','road_0_1_0','road_5_3_2','road_4_0_1','road_0_4_0','road_4_0_1','road_5_4_2','road_5_4_2','road_0_4_0','road_5_3_2','road_5_4_2','road_0_4_0','road_5_3_2','road_1_0_1','road_5_3_2','road_1_0_1','road_4_0_1','road_0_4_0','road_1_5_3','road_0_2_0','road_5_4_2','road_0_1_0','road_5_2_2','road_5_4_2','road_5_4_2','road_5_4_2','road_4_0_1','road_0_4_0','road_5_4_2','road_5_3_2','road_0_1_0','road_5_2_2','road_0_4_0','road_4_0_1','road_1_0_1','road_1_5_3','road_0_4_0','road_5_3_2','road_1_5_3','road_0_4_0','road_0_1_0','road_4_0_1','road_4_0_1','road_1_5_3','road_0_4_0','road_5_2_2','road_5_3_2','road_5_4_2','road_1_0_1','road_4_0_1','road_4_0_1','road_4_0_1','road_5_4_2','road_1_0_1','road_0_2_0','road_1_5_3','road_0_4_0','road_1_0_1','road_0_2_0','road_5_4_2','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_5_4_2','road_5_4_2','road_0_2_0','road_0_4_0','road_5_4_2','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_3_2','road_5_3_2','road_5_4_2','road_1_5_3','road_0_1_0','road_1_5_3','road_5_4_2','road_0_4_0','road_0_1_0','road_1_0_1','road_1_0_1','road_0_4_0','road_5_4_2','road_1_0_1','road_0_1_0','road_0_4_0','road_4_0_1','road_5_2_2','road_5_4_2','road_4_0_1','road_5_3_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_3_2','road_5_4_2','road_0_4_0','road_4_0_1','road_0_1_0','road_1_0_1','road_0_2_0','road_0_4_0','road_5_4_2','road_5_3_2','road_1_5_3','road_1_0_1','road_0_1_0','road_0_2_0','road_5_2_2','road_5_4_2','road_1_0_1','road_0_2_0','road_0_4_0','road_0_1_0','road_4_0_1','road_5_4_2','road_0_1_0','road_1_0_1','road_0_2_0','road_0_2_0','road_4_0_1','road_0_2_0','road_5_3_2','road_4_0_1','road_5_3_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_1_0_1','road_0_4_0','road_5_4_2','road_5_4_2','road_0_4_0','road_1_0_1','road_1_5_3','road_5_4_2','road_4_0_1','road_5_4_2','road_4_0_1','road_4_0_1','road_1_0_1','road_5_4_2','road_0_1_0','road_0_2_0','road_5_3_2','road_0_4_0','road_0_4_0','road_5_2_2','road_1_5_3','road_0_1_0','road_1_0_1','road_5_3_2','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_0_4_0','road_0_2_0','road_5_4_2','road_0_1_0','road_0_1_0','road_0_4_0','road_0_4_0','road_0_2_0','road_1_5_3','road_5_4_2','road_5_3_2','road_1_0_1','road_0_2_0','road_5_4_2','road_0_1_0','road_1_5_3','road_5_2_2','road_0_1_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_1_0','road_0_1_0','road_0_1_0','road_0_4_0','road_0_1_0','road_0_1_0','road_0_1_0','road_0_4_0','road_0_1_0','road_5_4_2','road_1_5_3','road_5_3_2','road_1_0_1','road_5_4_2','road_1_0_1','road_5_4_2','road_0_2_0','road_1_0_1','road_5_2_2','road_0_1_0','road_1_0_1','road_1_5_3','road_5_3_2','road_5_3_2','road_5_2_2','road_0_4_0','road_5_2_2','road_5_2_2','road_5_3_2','road_5_3_2','road_0_1_0','road_0_1_0','road_0_2_0','road_1_5_3','road_1_5_3','road_4_0_1','road_1_5_3','road_4_0_1','road_5_2_2','road_0_2_0','road_1_5_3','road_5_3_2','road_5_3_2','road_5_3_2','road_0_2_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_2_0','road_5_2_2','road_0_2_0','road_1_5_3','road_1_0_1','road_1_0_1','road_4_0_1','road_4_0_1','road_0_4_0','road_0_1_0','road_0_2_0','road_0_4_0','road_0_4_0','road_0_2_0','road_1_5_3','road_5_3_2','road_5_2_2','road_4_0_1','road_0_1_0','road_0_1_0','road_5_4_2','road_1_5_3','road_1_0_1','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_4_0_1','road_0_1_0','road_5_2_2','road_5_3_2','road_0_4_0','road_0_4_0','road_0_4_0','road_0_2_0','road_5_3_2','road_0_4_0','road_0_4_0','road_5_4_2','road_5_2_2','road_5_4_2','road_5_4_2','road_0_4_0','road_1_0_1','road_1_0_1','road_5_4_2','road_5_4_2','road_4_0_1','road_5_2_2','road_0_4_0','road_5_3_2','road_4_0_1','road_0_2_0','road_1_0_1','road_0_4_0','road_0_4_0','road_0_1_0','road_0_2_0','road_0_1_0','road_4_0_1','road_4_0_1','road_5_4_2','road_1_5_3','road_5_2_2','road_0_1_0','road_0_2_0','road_1_5_3','road_0_2_0','road_5_4_2','road_5_4_2','road_1_5_3','road_0_4_0','road_0_1_0','road_1_0_1','road_5_3_2','road_4_0_1','road_5_2_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_5_4_2','road_0_1_0','road_5_3_2','road_5_3_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_3_2','road_0_4_0','road_0_4_0','road_0_1_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_5_3_2','road_0_1_0','road_1_0_1','road_0_4_0','road_5_4_2','road_5_2_2','road_0_4_0','road_0_4_0','road_0_1_0','road_0_1_0','road_0_2_0','road_0_1_0','road_5_4_2','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_2_2','road_0_4_0','road_4_0_1','road_1_0_1','road_4_0_1','road_1_5_3','road_0_4_0','road_0_4_0','road_1_0_1','road_1_0_1','road_5_4_2','road_1_5_3','road_0_1_0','road_0_2_0','road_5_4_2','road_0_4_0','road_0_4_0','road_0_2_0','road_5_4_2','road_4_0_1','road_5_4_2','road_5_2_2','road_1_5_3','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_2_0','road_5_3_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_2_2','road_0_4_0','road_0_4_0','road_5_4_2','road_5_2_2','road_0_4_0','road_0_1_0','road_5_4_2','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_0_1_0','road_5_4_2','road_0_1_0','road_0_4_0','road_0_4_0','road_0_4_0','road_1_0_1','road_0_4_0','road_0_1_0','road_5_3_2','road_0_1_0','road_1_0_1','road_0_1_0','road_5_4_2','road_5_3_2','road_0_2_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_1_0','road_0_4_0','road_4_0_1','road_1_0_1','road_5_4_2','road_1_0_1','road_0_1_0','road_0_2_0','road_0_4_0','road_1_0_1','road_0_4_0','road_5_2_2','road_0_1_0','road_1_0_1','road_0_4_0','road_5_4_2','road_5_4_2','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_5_3_2','road_5_3_2','road_0_1_0','road_0_4_0','road_5_4_2','road_1_0_1','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_2_0','road_5_3_2','road_0_1_0','road_0_4_0','road_0_2_0','road_0_4_0','road_5_4_2','road_5_4_2','road_0_4_0','road_0_2_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_3_2','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_4_0','road_0_2_0','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_0_1_0','road_5_4_2','road_5_2_2','road_1_5_3','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_5_2_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_2_2','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_5_3_2','road_4_0_1','road_0_1_0','road_5_4_2','road_5_2_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_1_5_3','road_5_3_2','road_5_2_2','road_0_4_0','road_0_1_0','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_2_2','road_1_5_3','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_5_4_2','road_5_2_2','road_1_0_1','road_0_4_0','road_5_3_2','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_2_0','road_5_2_2','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_4_0_1','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_1_0_1','road_0_1_0','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_4_0_1','road_0_1_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_5_2_2','road_0_1_0','road_0_4_0','road_4_0_1','road_5_4_2','road_4_0_1','road_0_1_0','road_5_3_2','road_5_3_2','road_0_1_0','road_1_0_1','road_5_3_2','road_0_4_0','road_0_4_0','road_0_4_0','road_0_1_0','road_5_3_2','road_5_4_2','road_5_2_2','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_5_4_2','road_0_1_0','road_0_1_0','road_0_4_0','road_0_1_0','road_5_4_2','road_5_4_2','road_5_2_2','road_0_1_0','road_5_4_2','road_5_3_2','road_0_2_0','road_0_4_0','road_0_4_0','road_1_5_3','road_0_1_0','road_5_4_2','road_0_1_0','road_0_1_0','road_5_3_2','road_5_4_2','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_5_2_2','road_5_3_2','road_0_1_0','road_0_4_0','road_1_0_1','road_5_4_2','road_5_4_2','road_0_1_0','road_0_4_0','road_5_2_2','road_0_4_0','road_0_4_0','road_1_5_3','road_0_4_0','road_0_1_0','road_1_0_1','road_0_1_0','road_4_0_1','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_3_2','road_1_0_1','road_1_5_3','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_1_0','road_5_2_2','road_0_4_0','road_5_4_2','road_5_3_2','road_4_0_1','road_5_4_2','road_1_5_3','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_4_0_1','road_5_4_2','road_0_2_0','road_5_2_2','road_5_4_2','road_0_4_0','road_5_3_2','road_0_1_0','road_0_4_0','road_5_4_2','road_1_0_1','road_0_4_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_5_2_2','road_0_4_0','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_4_0_1','road_5_4_2','road_0_2_0','road_5_2_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_3_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_4_0_1','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_4_0_1','road_0_4_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_0_1_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_3_2','road_0_4_0','road_5_4_2','road_0_1_0','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_4_0','road_0_1_0','road_5_4_2','road_5_3_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_4_2','road_4_0_1','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_5_4_2','road_0_4_0','road_4_0_1','road_5_4_2','road_5_3_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_2_0','road_5_2_2','road_1_0_1','road_1_5_3','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_3_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_4_0_1','road_1_0_1','road_1_5_3','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_5_3_2','road_4_0_1','road_0_1_0','road_5_2_2','road_5_3_2','road_4_0_1','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_5_3_2','road_5_4_2','road_5_3_2','road_5_2_2','road_1_5_3','road_0_4_0','road_1_5_3','road_0_4_0','road_5_4_2','road_5_4_2','road_5_4_2','road_0_2_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_3_2','road_5_4_2','road_1_5_3','road_0_4_0','road_0_4_0','road_0_4_0','road_0_1_0','road_0_1_0','road_4_0_1','road_4_0_1','road_5_4_2','road_5_4_2','road_0_4_0','road_1_5_3','road_4_0_1','road_4_0_1','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_5_4_2','road_1_0_1','road_0_4_0','road_5_4_2','road_0_4_0','road_5_2_2','road_1_0_1','road_5_3_2','road_0_1_0','road_0_4_0','road_5_4_2','road_4_0_1','road_0_4_0','road_0_4_0','road_4_0_1','road_5_2_2','road_1_0_1','road_0_4_0','road_5_4_2','road_4_0_1','road_0_2_0','road_5_4_2','road_0_4_0','road_0_2_0','road_5_4_2','road_5_3_2','road_0_1_0','road_0_4_0','road_0_4_0','road_0_1_0','road_0_4_0','road_5_3_2','road_5_4_2','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_0_4_0','road_4_0_1','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_5_3_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_4_0_1','road_0_4_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_3_2','road_4_0_1','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_4_0_1','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_0_1_0','road_0_2_0','road_5_2_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_2_0','road_5_2_2','road_4_0_1','road_5_4_2','road_1_0_1','road_1_5_3','road_5_3_2','road_0_1_0','road_5_4_2','road_5_4_2','road_4_0_1','road_0_2_0','road_5_3_2','road_0_1_0','road_5_4_2','road_5_2_2','road_0_2_0','road_0_4_0','road_0_4_0','road_1_0_1','road_0_4_0','road_0_4_0','road_0_4_0','road_0_1_0','road_1_0_1','road_5_4_2','road_5_4_2','road_5_4_2','road_1_0_1','road_0_1_0','road_0_1_0','road_0_2_0','road_5_4_2','road_5_4_2','road_1_5_3','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_5_4_2','road_0_4_0','road_0_4_0','road_1_0_1','road_0_1_0','road_0_2_0_0']
end_edge = [,'road_1_1_3','road_1_1_3','road_3_4_1','road_1_3_2','road_2_1_3','road_1_4_2','road_1_1_3','road_1_3_2','road_1_2_2','road_1_2_2_1, road_4_4_1','road_2_4_1','road_4_1_0','road_1_3_2','road_1_1_3','road_1_1_3','road_3_1_3','road_1_4_1','road_2_4_1','road_1_2_2','road_4_4_1','road_4_4_0','road_2_4_1','road_4_4_1','road_3_1_3','road_4_2_0','road_1_1_3','road_3_4_1','road_1_3_2','road_4_2_0','road_4_3_0','road_1_2_2','road_1_4_1','road_3_4_1','road_1_1_2','road_2_1_3','road_1_4_1','road_4_4_0','road_2_4_1','road_4_1_0','road_2_1_3','road_1_3_2','road_2_1_3_1, road_4_4_1','road_4_4_0','road_2_4_1','road_4_1_0','road_3_1_3','road_1_1_3','road_3_4_1','road_4_3_0','road_1_4_1','road_1_4_1','road_2_3_1','road_1_4_1','road_1_1_3','road_1_3_2','road_4_4_0','road_4_4_1','road_2_1_3','road_4_4_1','road_3_1_3','road_4_1_3','road_1_4_2','road_1_1_3','road_4_4_0','road_1_3_2','road_1_1_3','road_3_4_1','road_4_1_0','road_1_1_2','road_1_1_2','road_2_4_1','road_2_4_1_1, road_1_2_2','road_4_4_0','road_2_4_1','road_1_4_1','road_4_4_0','road_1_1_2','road_4_4_1','road_1_4_1','road_4_2_0','road_1_2_2','road_1_4_2','road_1_1_3','road_4_4_1','road_1_1_3_1, road_2_4_1','road_4_4_1','road_4_4_1_1, road_1_4_1','road_4_3_0','road_4_3_0','road_4_4_1','road_4_4_3','road_4_2_0','road_1_2_2','road_4_3_0','road_4_1_0','road_2_1_3','road_1_4_2','road_1_4_1','road_3_4_1','road_1_3_2','road_2_4_1','road_1_1_3','road_4_1_0','road_4_2_0','road_1_3_2','road_4_4_0','road_4_4_1','road_3_4_1','road_4_4_1','road_1_4_1','road_4_1_3','road_4_2_0','road_4_2_0','road_4_4_1','road_4_1_3','road_1_1_2','road_1_1_3','road_2_1_3','road_1_2_2','road_2_1_3','road_4_4_1','road_2_1_3_1, road_4_4_1','road_1_1_3','road_4_2_0','road_4_4_1_1, road_2_4_1','road_3_4_1','road_1_1_3','road_1_4_1','road_4_4_1','road_4_4_1','road_2_4_1','road_4_1_3','road_4_4_0','road_4_4_1','road_1_3_2','road_4_4_1','road_1_4_2','road_1_1_3','road_1_4_2','road_3_4_1','road_1_4_1','road_2_1_3','road_1_2_2','road_1_2_2','road_1_4_1','road_2_4_1','road_1_1_2','road_4_1_3','road_1_1_2','road_4_4_1','road_4_3_0','road_2_4_1','road_2_4_1','road_3_4_1','road_1_2_2','road_1_4_2','road_1_1_3_1, road_4_4_1','road_2_1_3_1, road_2_1_3','road_4_3_0','road_1_1_3','road_4_2_0_1, road_4_2_0','road_1_2_2','road_4_4_1','road_4_4_0','road_1_3_2','road_1_4_1','road_4_2_0','road_2_1_3','road_3_4_1','road_1_3_2','road_1_2_2','road_1_1_2','road_2_1_3','road_4_1_0','road_4_2_0','road_4_4_1','road_4_2_0','road_1_4_1','road_1_1_2','road_4_4_0','road_4_3_0','road_1_4_1','road_4_4_1','road_4_1_0','road_1_1_3','road_1_3_2','road_1_1_3','road_3_4_1','road_3_4_1','road_1_4_1','road_4_4_1','road_4_1_3','road_1_2_2','road_4_4_1_1, road_4_4_1','road_1_3_2','road_1_1_2','road_3_1_3','road_3_4_1','road_4_1_0','road_1_4_1','road_4_3_0','road_1_2_2_1, road_1_2_2','road_3_1_3','road_4_1_0','road_4_2_0','road_4_4_1','road_2_1_3_1, road_4_4_0','road_1_4_2','road_1_1_2','road_3_1_3','road_2_1_3','road_2_4_1','road_2_4_1_1, road_1_4_1','road_4_1_3','road_1_3_2','road_1_2_2','road_1_1_3','road_1_4_2','road_4_1_3','road_3_1_3','road_1_3_2','road_1_2_2','road_1_1_3','road_1_2_2','road_4_4_1','road_4_1_0','road_1_2_2','road_1_1_2','road_4_4_1','road_3_4_1','road_1_4_2','road_4_4_1','road_1_1_3_1, road_4_4_0','road_1_1_3','road_2_1_3','road_2_1_3_1, road_4_1_0','road_3_1_3','road_1_1_3','road_1_1_3','road_1_1_3_1, road_2_1_3','road_1_3_2','road_1_1_3','road_4_4_1','road_1_4_2','road_3_4_1','road_4_1_3','road_4_4_1','road_2_1_3','road_4_4_1','road_4_2_0','road_1_2_2','road_4_4_0','road_2_1_3','road_1_4_1','road_1_3_2','road_4_4_0','road_4_4_1','road_2_1_3','road_1_3_2','road_4_3_0','road_4_3_0_1, road_1_4_1','road_3_4_1','road_2_1_3','road_2_1_3_1, road_1_1_3','road_1_4_2','road_1_1_3','road_4_2_0','road_1_1_3','road_4_1_0','road_4_4_0','road_2_1_3','road_1_2_2','road_4_4_0','road_4_4_1','road_4_4_1','road_2_1_3','road_4_2_0','road_3_4_1','road_1_4_1','road_1_1_3','road_4_3_0','road_1_1_3','road_1_3_2','road_4_1_3','road_2_1_3','road_4_1_0_1, road_4_1_0','road_2_4_1','road_4_1_0','road_1_1_2','road_3_4_1','road_2_4_1','road_1_1_2_1, road_1_4_2','road_1_3_2','road_4_3_0','road_4_4_0','road_2_1_3','road_1_1_3','road_4_4_1','road_3_1_3','road_1_1_2','road_4_3_0','road_2_4_1','road_2_4_1','road_1_4_1','road_4_1_0','road_1_1_3','road_3_2_0','road_3_4_3','road_1_3_2','road_1_3_2','road_1_3_2_1, road_4_3_0','road_1_4_1','road_4_4_0','road_1_3_2','road_2_1_3','road_2_4_1','road_4_4_1','road_1_4_2','road_1_1_3','road_1_4_1','road_2_1_3_1, road_4_4_1_1, road_4_1_3','road_4_2_0','road_3_3_3','road_1_2_2','road_3_4_1','road_4_4_1','road_1_1_1','road_1_2_2','road_1_1_2','road_4_1_3','road_4_1_0','road_1_1_2','road_1_1_3','road_1_2_2','road_4_1_0','road_2_4_1','road_1_2_2','road_4_4_1','road_4_1_0','road_1_1_3','road_1_1_3','road_1_1_2','road_2_1_3','road_1_4_2','road_4_3_0','road_1_4_1','road_3_4_1','road_2_1_3','road_4_2_0','road_1_4_2','road_1_2_2','road_1_4_1','road_4_4_0','road_1_4_1','road_4_4_1','road_4_4_0_1, road_1_4_1_0']
depart = [  , 130, 21, 36, 40, 138, 160, 69, 0, 26, 13, 7, 182, 92, 190, 170, 117, 112, 52, 27, 189, 117, 32, 175, 10, 156, 210, 91, 115, 130, 65, 68, 23, 84, 26, 80, 0, 39, 28, 46, 90, 184, 110, 26, 4, 23, 260, 0, 13, 65, 91, 144, 140, 0, 0, 240, 192, 208, 245, 140, 238, 115, 20, 299, 290, 40, 253, 324, 169, 323, 96, 52, 49, 77, 29, 52, 105, 160, 273, 302, 182, 69, 273, 173, 317, 222, 252, 330, 37, 266, 273, 63, 195, 12, 280, 104, 50, 74, 20, 346, 276, 302, 204, 221, 79, 55, 348, 331, 302, 334, 61, 259, 234, 318, 138, 169, 302, 369, 303, 184, 356, 161, 46, 334, 185, 343, 270, 348, 92, 60, 360, 120, 140, 370, 85, 355, 332, 148, 314, 64, 317, 156, 396, 453, 408, 433, 154, 244, 300, 385, 351, 254, 118, 453, 120, 176, 234, 384, 147, 143, 260, 16, 112, 216, 368, 35, 72, 286, 450, 92, 74, 268, 417, 402, 419, 230, 180, 296, 148, 14, 368, 384, 124, 230, 415, 420, 100, 253, 426, 111, 299, 314, 419, 484, 39, 84, 212, 100, 168, 264, 420, 299, 300, 462, 450, 367, 521, 46, 196, 530, 133, 392, 221, 436, 182, 250, 259, 453, 229, 76, 111, 276, 336, 487, 299, 161, 37, 440, 380, 330, 468, 126, 260, 306, 395, 345, 440, 368, 480, 385, 207, 510, 488, 70, 589, 426, 522, 104, 272, 98, 354, 200, 342, 284, 546, 552, 378, 470, 570, 517, 600, 487, 317, 522, 538, 528, 328, 302, 296, 366, 402, 555, 334, 538, 0, 515, 342, 553, 247, 143, 384, 390, 524, 656, 351, 553, 385, 622, 454, 482, 504, 572, 589, 14, 496, 599, 294, 505, 600, 398, 552, 230, 419, 600, 677, 666, 168, 220, 580, 116, 196, 630, 672, 286, 600, 439, 416, 691, 616, 185, 672, 132, 231, 233, 590, 700, 680, 222, 78, 571, 545, 550, 342, 351, 88, 609, 224, 300, 619, 210, 375, 470, 504, 625, 378, 602, 185, 406, 472, 461, 538, 636, 351, 200, 282, 398, 710, 247, 743, 204, 487, 580, 360, 660, 436, 565, 201, 217, 402, 636, 318, 161, 600, 208, 412, 633, 128, 612, 631, 680, 645, 1, 1, 1, 2, 58, 109, 120, 130, 136, 148, 150, 152, 156, 164, 180, 184, 188, 195, 15, 21, 22, 38, 40, 41, 47, 66, 70, 71, 75, 81, 92, 101, 105, 106, 112, 113, 116, 117, 121, 131, 139, 141, 144, 145, 149, 157, 161, 162, 169, 170, 171, 181, 183, 186, 196, 197, 205, 207, 208, 220, 226, 232, 236, 240, 248, 252, 258, 259, 261, 278, 281, 287, 288, 292, 296, 300, 300, 302, 305, 306, 310, 312, 312, 315, 320, 324, 325, 328, 335, 336, 340, 342, 355, 356, 360, 365, 366, 370, 372, 372, 372, 384, 385, 390, 390, 396, 401, 402, 402, 208, 209, 211, 221, 222, 223, 231, 232, 235, 241, 248, 249, 253, 260, 261, 262, 274, 278, 279, 287, 288, 289, 297, 302, 303, 304, 307, 313, 315, 316, 318, 319, 325, 326, 329, 335, 336, 337, 349, 352, 356, 357, 361, 369, 370, 371, 373, 379, 385, 386, 386, 387, 391, 397, 399, 403, 403, 404, 408, 410, 412, 414, 415, 420, 420, 425, 426, 426, 430, 432, 432, 435, 436, 438, 438, 440, 444, 444, 445, 450, 454, 456, 456, 456, 460, 462, 465, 468, 468, 468, 470, 474, 474, 475, 480, 480, 482, 486, 486, 486, 490, 492, 492, 495, 496, 498, 500, 500, 504, 504, 504, 507, 510, 510, 510, 510, 516, 516, 520, 521, 521, 524, 525, 528, 531, 534, 534, 535, 538, 538, 541, 541, 540, 546, 552, 552, 555, 555, 555, 558, 558, 561, 564, 564, 565, 567, 566, 570, 572, 572, 575, 576, 576, 576, 581, 582, 582, 585, 588, 588, 589, 594, 594, 594, 594, 595, 602, 600, 600, 600, 603, 604, 606, 608, 409, 413, 416, 421, 427, 437, 439, 440, 441, 445, 446, 451, 454, 456, 457, 463, 469, 471, 475, 476, 481, 483, 487, 493, 497, 501, 505, 505, 506, 511, 511, 512, 522, 523, 525, 526, 529, 535, 536, 539, 539, 541, 542, 547, 554, 556, 559, 566, 567, 568, 571, 572, 573, 577, 581, 582, 583, 589, 590, 591, 595, 596, 601, 603, 604, 605, 610, 611, 612, 615, 615, 615, 616, 618, 618, 618, 620, 622, 621, 625, 628, 631, 630, 630, 631, 634, 632, 633, 637, 636, 639, 640, 640, 642, 644, 644, 645, 645, 648, 648, 648, 650, 652, 652, 654, 654, 655, 656, 658, 660, 660, 660, 661, 664, 664, 664, 666, 668, 670, 670, 672, 679, 675, 675, 682, 676, 683, 680, 686, 684, 689, 692, 688, 688, 690, 690, 690, 690, 694, 694, 696, 697, 698, 696, 702, 700, 700, 705, 706, 705, 705, 707, 709, 708, 708, 710, 710, 713, 714, 712, 714, 717, 717, 720, 720, 720, 720, 720, 721, 725, 724, 726, 726, 728, 728, 728, 731, 730, 732, 732, 734, 735, 735, 737, 736, 740, 740, 742, 743, 745, 744, 744, 744, 744, 747, 748, 750, 750, 750, 750, 752, 753, 754, 756, 756, 759, 760, 760, 760, 762, 762, 762, 764, 766, 765, 765, 765, 769, 768, 770, 771, 772, 774, 776, 776, 776, 777, 780, 780, 780, 780, 780, 780, 780, 783, 784, 786, 787, 788, 789, 790, 792, 792, 792, 795, 795, 795, 796, 798, 798, 798, 798, 800, 800, 802, 804, 805, 808, 808, 808, 809, 810, 613, 616, 617, 623, 626, 632, 633, 634, 635, 637, 638, 641, 645, 646, 649, 653, 655, 656, 657, 661, 662, 665, 667, 671, 673, 676, 677, 683, 686, 689, 687, 689, 690, 691, 691, 694, 696, 697, 698, 700, 702, 703, 707, 710, 709, 710, 713, 711, 714, 715, 718, 721, 721, 722, 727, 729, 733, 737, 738, 741, 744, 745, 745, 746, 751, 757, 761, 763, 766, 767, 777, 778, 781, 781, 782, 793, 796, 797, 799, 801, 809, 810, 810, 810, 811, 812, 814, 816, 816, 816, 817, 820, 820, 820, 820, 823, 824, 824, 825, 825, 826, 828, 829, 830, 831, 832, 832, 834, 834, 835, 836, 838, 840, 840, 840, 840, 841, 840, 842, 843, 845, 846, 848, 849, 850, 852, 852, 852, 852, 853, 855, 855, 855, 856, 856, 858, 860, 860, 861, 864, 864, 864, 867, 868, 870, 870, 870, 870, 870, 870, 872, 872, 873, 875, 876, 876, 879, 880, 880, 882, 884, 885, 885, 886, 888, 888, 888, 888, 890, 892, 896, 897, 811, 812, 813, 817, 818, 821, 826, 826, 827, 833, 835, 836, 837, 841, 841, 842, 843, 844, 853, 854, 856, 857, 861, 862, 865, 871, 871, 872, 873, 874, 877, 881, 886, 889]



In [ ]:
start_edge = ['road_0_1_0','road_5_4_2','road_4_0_1','road_5_4_2','road_1_0_1','road_0_1_0','road_1_5_3','road_0_1_0','road_4_0_1','road_0_4_0','road_1_5_3','road_4_0_1','road_0_1_0','road_4_0_1','road_5_3_2','road_4_0_1','road_0_4_0','road_0_4_0','road_0_4_0','road_1_0_1','road_0_1_0','road_0_4_0','road_5_3_2','road_5_4_2','road_0_1_0','road_5_4_2','road_1_0_1','road_0_1_0','road_1_5_3','road_0_1_0','road_0_4_0','road_1_5_3','road_0_1_0','road_5_3_2','road_4_0_1','road_0_4_0','road_4_0_1','road_5_4_2','road_5_4_2','road_0_4_0','road_5_3_2','road_5_4_2','road_0_4_0','road_5_3_2','road_1_0_1','road_5_3_2','road_1_0_1','road_4_0_1','road_0_4_0','road_1_5_3','road_0_2_0','road_5_4_2','road_0_1_0','road_5_2_2','road_5_4_2','road_5_4_2','road_5_4_2','road_4_0_1','road_0_4_0','road_5_4_2','road_5_3_2','road_0_1_0','road_5_2_2','road_0_4_0','road_4_0_1','road_1_0_1','road_1_5_3','road_0_4_0','road_5_3_2','road_1_5_3','road_0_4_0','road_0_1_0','road_4_0_1','road_4_0_1','road_1_5_3','road_0_4_0','road_5_2_2','road_5_3_2','road_5_4_2','road_1_0_1','road_4_0_1','road_4_0_1','road_4_0_1','road_5_4_2','road_1_0_1','road_0_2_0','road_1_5_3','road_0_4_0','road_1_0_1','road_0_2_0','road_5_4_2','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_5_4_2','road_5_4_2','road_0_2_0','road_0_4_0','road_5_4_2','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_3_2','road_5_3_2','road_5_4_2','road_1_5_3','road_0_1_0','road_1_5_3','road_5_4_2','road_0_4_0','road_0_1_0','road_1_0_1','road_1_0_1','road_0_4_0','road_5_4_2','road_1_0_1','road_0_1_0','road_0_4_0','road_4_0_1','road_5_2_2','road_5_4_2','road_4_0_1','road_5_3_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_3_2','road_5_4_2','road_0_4_0','road_4_0_1','road_0_1_0','road_1_0_1','road_0_2_0','road_0_4_0','road_5_4_2','road_5_3_2','road_1_5_3','road_1_0_1','road_0_1_0','road_0_2_0','road_5_2_2','road_5_4_2','road_1_0_1','road_0_2_0','road_0_4_0','road_0_1_0','road_4_0_1','road_5_4_2','road_0_1_0','road_1_0_1','road_0_2_0','road_0_2_0','road_4_0_1','road_0_2_0','road_5_3_2','road_4_0_1','road_5_3_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_1_0_1','road_0_4_0','road_5_4_2','road_5_4_2','road_0_4_0','road_1_0_1','road_1_5_3','road_5_4_2','road_4_0_1','road_5_4_2','road_4_0_1','road_4_0_1','road_1_0_1','road_5_4_2','road_0_1_0','road_0_2_0','road_5_3_2','road_0_4_0','road_0_4_0','road_5_2_2','road_1_5_3','road_0_1_0','road_1_0_1','road_5_3_2','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_0_4_0','road_0_2_0','road_5_4_2','road_0_1_0','road_0_1_0','road_0_4_0','road_0_4_0','road_0_2_0','road_1_5_3','road_5_4_2','road_5_3_2','road_1_0_1','road_0_2_0','road_5_4_2','road_0_1_0','road_1_5_3','road_5_2_2','road_0_1_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_1_0','road_0_1_0','road_0_1_0','road_0_4_0','road_0_1_0','road_0_1_0','road_0_1_0','road_0_4_0','road_0_1_0','road_5_4_2','road_1_5_3','road_5_3_2','road_1_0_1','road_5_4_2','road_1_0_1','road_5_4_2','road_0_2_0','road_1_0_1','road_5_2_2','road_0_1_0','road_1_0_1','road_1_5_3','road_5_3_2','road_5_3_2','road_5_2_2','road_0_4_0','road_5_2_2','road_5_2_2','road_5_3_2','road_5_3_2','road_0_1_0','road_0_1_0','road_0_2_0','road_1_5_3','road_1_5_3','road_4_0_1','road_1_5_3','road_4_0_1','road_5_2_2','road_0_2_0','road_1_5_3','road_5_3_2','road_5_3_2','road_5_3_2','road_0_2_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_2_0','road_5_2_2','road_0_2_0','road_1_5_3','road_1_0_1','road_1_0_1','road_4_0_1','road_4_0_1','road_0_4_0','road_0_1_0','road_0_2_0','road_0_4_0','road_0_4_0','road_0_2_0','road_1_5_3','road_5_3_2','road_5_2_2','road_4_0_1','road_0_1_0','road_0_1_0','road_5_4_2','road_1_5_3','road_1_0_1','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_4_0_1','road_0_1_0','road_5_2_2','road_5_3_2','road_0_4_0','road_0_4_0','road_0_4_0','road_0_2_0','road_5_3_2','road_0_4_0','road_0_4_0','road_5_4_2','road_5_2_2','road_5_4_2','road_5_4_2','road_0_4_0','road_1_0_1','road_1_0_1','road_5_4_2','road_5_4_2','road_4_0_1','road_5_2_2','road_0_4_0','road_5_3_2','road_4_0_1','road_0_2_0','road_1_0_1','road_0_4_0','road_0_4_0','road_0_1_0','road_0_2_0','road_0_1_0','road_4_0_1','road_4_0_1','road_5_4_2','road_1_5_3','road_5_2_2','road_0_1_0','road_0_2_0','road_1_5_3','road_0_2_0','road_5_4_2','road_5_4_2','road_1_5_3','road_0_4_0','road_0_1_0','road_1_0_1','road_5_3_2','road_4_0_1','road_5_2_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_5_4_2','road_0_1_0','road_5_3_2','road_5_3_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_3_2','road_0_4_0','road_0_4_0','road_0_1_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_5_3_2','road_0_1_0','road_1_0_1','road_0_4_0','road_5_4_2','road_5_2_2','road_0_4_0','road_0_4_0','road_0_1_0','road_0_1_0','road_0_2_0','road_0_1_0','road_5_4_2','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_2_2','road_0_4_0','road_4_0_1','road_1_0_1','road_4_0_1','road_1_5_3','road_0_4_0','road_0_4_0','road_1_0_1','road_1_0_1','road_5_4_2','road_1_5_3','road_0_1_0','road_0_2_0','road_5_4_2','road_0_4_0','road_0_4_0','road_0_2_0','road_5_4_2','road_4_0_1','road_5_4_2','road_5_2_2','road_1_5_3','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_2_0','road_5_3_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_2_2','road_0_4_0','road_0_4_0','road_5_4_2','road_5_2_2','road_0_4_0','road_0_1_0','road_5_4_2','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_0_1_0','road_5_4_2','road_0_1_0','road_0_4_0','road_0_4_0','road_0_4_0','road_1_0_1','road_0_4_0','road_0_1_0','road_5_3_2','road_0_1_0','road_1_0_1','road_0_1_0','road_5_4_2','road_5_3_2','road_0_2_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_0_1_0','road_0_4_0','road_4_0_1','road_1_0_1','road_5_4_2','road_1_0_1','road_0_1_0','road_0_2_0','road_0_4_0','road_1_0_1','road_0_4_0','road_5_2_2','road_0_1_0','road_1_0_1','road_0_4_0','road_5_4_2','road_5_4_2','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_5_3_2','road_5_3_2','road_0_1_0','road_0_4_0','road_5_4_2','road_1_0_1','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_2_0','road_5_3_2','road_0_1_0','road_0_4_0','road_0_2_0','road_0_4_0','road_5_4_2','road_5_4_2','road_0_4_0','road_0_2_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_3_2','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_4_0','road_0_2_0','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_0_1_0','road_5_4_2','road_5_2_2','road_1_5_3','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_5_2_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_2_2','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_5_3_2','road_4_0_1','road_0_1_0','road_5_4_2','road_5_2_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_1_5_3','road_5_3_2','road_5_2_2','road_0_4_0','road_0_1_0','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_2_2','road_1_5_3','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_5_4_2','road_5_2_2','road_1_0_1','road_0_4_0','road_5_3_2','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_2_0','road_5_2_2','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_4_0_1','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_1_0_1','road_0_1_0','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_4_0_1','road_0_1_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_5_2_2','road_0_1_0','road_0_4_0','road_4_0_1','road_5_4_2','road_4_0_1','road_0_1_0','road_5_3_2','road_5_3_2','road_0_1_0','road_1_0_1','road_5_3_2','road_0_4_0','road_0_4_0','road_0_4_0','road_0_1_0','road_5_3_2','road_5_4_2','road_5_2_2','road_0_1_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_5_4_2','road_0_1_0','road_0_1_0','road_0_4_0','road_0_1_0','road_5_4_2','road_5_4_2','road_5_2_2','road_0_1_0','road_5_4_2','road_5_3_2','road_0_2_0','road_0_4_0','road_0_4_0','road_1_5_3','road_0_1_0','road_5_4_2','road_0_1_0','road_0_1_0','road_5_3_2','road_5_4_2','road_0_4_0','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_5_2_2','road_5_3_2','road_0_1_0','road_0_4_0','road_1_0_1','road_5_4_2','road_5_4_2','road_0_1_0','road_0_4_0','road_5_2_2','road_0_4_0','road_0_4_0','road_1_5_3','road_0_4_0','road_0_1_0','road_1_0_1','road_0_1_0','road_4_0_1','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_3_2','road_1_0_1','road_1_5_3','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_4_0','road_0_4_0','road_5_4_2','road_0_1_0','road_5_2_2','road_0_4_0','road_5_4_2','road_5_3_2','road_4_0_1','road_5_4_2','road_1_5_3','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_4_0_1','road_5_4_2','road_0_2_0','road_5_2_2','road_5_4_2','road_0_4_0','road_5_3_2','road_0_1_0','road_0_4_0','road_5_4_2','road_1_0_1','road_0_4_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_5_2_2','road_0_4_0','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_4_0_1','road_5_4_2','road_0_2_0','road_5_2_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_3_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_4_0_1','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_4_0_1','road_0_4_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_0_1_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_3_2','road_0_4_0','road_5_4_2','road_0_1_0','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_4_0','road_0_1_0','road_5_4_2','road_5_3_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_4_2','road_4_0_1','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_0_4_0','road_5_4_2','road_0_4_0','road_4_0_1','road_5_4_2','road_5_3_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_2_0','road_5_2_2','road_1_0_1','road_1_5_3','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_3_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_4_0_1','road_1_0_1','road_1_5_3','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_5_3_2','road_4_0_1','road_0_1_0','road_5_2_2','road_5_3_2','road_4_0_1','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_5_3_2','road_5_4_2','road_5_3_2','road_5_2_2','road_1_5_3','road_0_4_0','road_1_5_3','road_0_4_0','road_5_4_2','road_5_4_2','road_5_4_2','road_0_2_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_3_2','road_5_4_2','road_1_5_3','road_0_4_0','road_0_4_0','road_0_4_0','road_0_1_0','road_0_1_0','road_4_0_1','road_4_0_1','road_5_4_2','road_5_4_2','road_0_4_0','road_1_5_3','road_4_0_1','road_4_0_1','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_5_4_2','road_1_0_1','road_0_4_0','road_5_4_2','road_0_4_0','road_5_2_2','road_1_0_1','road_5_3_2','road_0_1_0','road_0_4_0','road_5_4_2','road_4_0_1','road_0_4_0','road_0_4_0','road_4_0_1','road_5_2_2','road_1_0_1','road_0_4_0','road_5_4_2','road_4_0_1','road_0_2_0','road_5_4_2','road_0_4_0','road_0_2_0','road_5_4_2','road_5_3_2','road_0_1_0','road_0_4_0','road_0_4_0','road_0_1_0','road_0_4_0','road_5_3_2','road_5_4_2','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_0_4_0','road_4_0_1','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_5_3_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_4_0_1','road_0_4_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_2_0','road_5_2_2','road_0_4_0','road_5_3_2','road_4_0_1','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_4_0','road_0_1_0','road_5_4_2','road_1_0_1','road_1_5_3','road_0_4_0','road_4_0_1','road_0_2_0','road_5_2_2','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_4_0','road_5_4_2','road_0_1_0','road_0_2_0','road_5_2_2','road_1_0_1','road_1_5_3','road_0_4_0','road_5_4_2','road_5_3_2','road_0_4_0','road_4_0_1','road_5_4_2','road_0_4_0','road_0_4_0','road_0_1_0','road_5_4_2','road_0_4_0','road_5_4_2','road_0_2_0','road_5_2_2','road_4_0_1','road_5_4_2','road_1_0_1','road_1_5_3','road_5_3_2','road_0_1_0','road_5_4_2','road_5_4_2','road_4_0_1','road_0_2_0','road_5_3_2','road_0_1_0','road_5_4_2','road_5_2_2','road_0_2_0','road_0_4_0','road_0_4_0','road_1_0_1','road_0_4_0','road_0_4_0','road_0_4_0','road_0_1_0','road_1_0_1','road_5_4_2','road_5_4_2','road_5_4_2','road_1_0_1','road_0_1_0','road_0_1_0','road_0_2_0','road_5_4_2','road_5_4_2','road_1_5_3','road_0_4_0','road_5_4_2','road_0_4_0','road_5_4_2','road_5_4_2','road_0_4_0','road_0_4_0','road_1_0_1','road_0_1_0','road_0_2_0_0']
end_edge = ['road_2_1_3','road_1_4_1','road_4_4_0','road_2_4_1','road_4_1_0','road_2_1_3','road_1_3_2','road_2_1_3_1, road_4_4_1','road_4_4_0','road_2_4_1','road_4_1_0','road_3_1_3','road_1_1_3','road_3_4_1','road_4_3_0','road_1_4_1','road_1_4_1','road_2_3_1','road_1_4_1','road_1_1_3','road_1_3_2','road_4_4_0','road_4_4_1','road_2_1_3','road_4_4_1','road_3_1_3','road_4_1_3','road_1_4_2','road_1_1_3','road_4_4_0','road_1_3_2','road_1_1_3','road_3_4_1','road_4_1_0','road_1_1_2','road_1_1_2','road_2_4_1','road_2_4_1_1, road_1_2_2','road_4_4_0','road_2_4_1','road_1_4_1','road_4_4_0','road_1_1_2','road_4_4_1','road_1_4_1','road_4_2_0','road_1_2_2','road_1_4_2','road_1_1_3','road_4_4_1','road_1_1_3_1, road_2_4_1','road_4_4_1','road_4_4_1_1, road_1_4_1','road_4_3_0','road_4_3_0','road_4_4_1','road_4_4_3','road_4_2_0','road_1_2_2','road_4_3_0','road_4_1_0','road_2_1_3','road_1_4_2','road_1_4_1','road_3_4_1','road_1_3_2','road_2_4_1','road_1_1_3','road_4_1_0','road_4_2_0','road_1_3_2','road_4_4_0','road_4_4_1','road_3_4_1','road_4_4_1','road_1_4_1','road_4_1_3','road_4_2_0','road_4_2_0','road_4_4_1','road_4_1_3','road_1_1_2','road_1_1_3','road_2_1_3','road_1_2_2','road_2_1_3','road_4_4_1','road_2_1_3_1, road_4_4_1','road_1_1_3','road_4_2_0','road_4_4_1_1, road_2_4_1','road_3_4_1','road_1_1_3','road_1_4_1','road_4_4_1','road_4_4_1','road_2_4_1','road_4_1_3','road_4_4_0','road_4_4_1','road_1_3_2','road_4_4_1','road_1_4_2','road_1_1_3','road_1_4_2','road_3_4_1','road_1_4_1','road_2_1_3','road_1_2_2','road_1_2_2','road_1_4_1','road_2_4_1','road_1_1_2','road_4_1_3','road_1_1_2','road_4_4_1','road_4_3_0','road_2_4_1','road_2_4_1','road_3_4_1','road_1_2_2','road_1_4_2','road_1_1_3_1, road_4_4_1','road_2_1_3_1, road_2_1_3','road_4_3_0','road_1_1_3','road_4_2_0_1, road_4_2_0','road_1_2_2','road_4_4_1','road_4_4_0','road_1_3_2','road_1_4_1','road_4_2_0','road_2_1_3','road_3_4_1','road_1_3_2','road_1_2_2','road_1_1_2','road_2_1_3','road_4_1_0','road_4_2_0','road_4_4_1','road_4_2_0','road_1_4_1','road_1_1_2','road_4_4_0','road_4_3_0','road_1_4_1','road_4_4_1','road_4_1_0','road_1_1_3','road_1_3_2','road_1_1_3','road_3_4_1','road_3_4_1','road_1_4_1','road_4_4_1','road_4_1_3','road_1_2_2','road_4_4_1_1, road_4_4_1','road_1_3_2','road_1_1_2','road_3_1_3','road_3_4_1','road_4_1_0','road_1_4_1','road_4_3_0','road_1_2_2_1, road_1_2_2','road_3_1_3','road_4_1_0','road_4_2_0','road_4_4_1','road_2_1_3_1, road_4_4_0','road_1_4_2','road_1_1_2','road_3_1_3','road_2_1_3','road_2_4_1','road_2_4_1_1, road_1_4_1','road_4_1_3','road_1_3_2','road_1_2_2','road_1_1_3','road_1_4_2','road_4_1_3','road_3_1_3','road_1_3_2','road_1_2_2','road_1_1_3','road_1_2_2','road_4_4_1','road_4_1_0','road_1_2_2','road_1_1_2','road_4_4_1','road_3_4_1','road_1_4_2','road_4_4_1','road_1_1_3_1, road_4_4_0','road_1_1_3','road_2_1_3','road_2_1_3_1, road_4_1_0','road_3_1_3','road_1_1_3','road_1_1_3','road_1_1_3_1, road_2_1_3','road_1_3_2','road_1_1_3','road_4_4_1','road_1_4_2','road_3_4_1','road_4_1_3','road_4_4_1','road_2_1_3','road_4_4_1','road_4_2_0','road_1_2_2','road_4_4_0','road_2_1_3','road_1_4_1','road_1_3_2','road_4_4_0','road_4_4_1','road_2_1_3','road_1_3_2','road_4_3_0','road_4_3_0_1, road_1_4_1','road_3_4_1','road_2_1_3','road_2_1_3_1, road_1_1_3','road_1_4_2','road_1_1_3','road_4_2_0','road_1_1_3','road_4_1_0','road_4_4_0','road_2_1_3','road_1_2_2','road_4_4_0','road_4_4_1','road_4_4_1','road_2_1_3','road_4_2_0','road_3_4_1','road_1_4_1','road_1_1_3','road_4_3_0','road_1_1_3','road_1_3_2','road_4_1_3','road_2_1_3','road_4_1_0_1, road_4_1_0','road_2_4_1','road_4_1_0','road_1_1_2','road_3_4_1','road_2_4_1','road_1_1_2_1, road_1_4_2','road_1_3_2','road_4_3_0','road_4_4_0','road_2_1_3','road_1_1_3','road_4_4_1','road_3_1_3','road_1_1_2','road_4_3_0','road_2_4_1','road_2_4_1','road_1_4_1','road_4_1_0','road_1_1_3','road_3_2_0','road_3_4_3','road_1_3_2','road_1_3_2','road_1_3_2_1, road_4_3_0','road_1_4_1','road_4_4_0','road_1_3_2','road_2_1_3','road_2_4_1','road_4_4_1','road_1_4_2','road_1_1_3','road_1_4_1','road_2_1_3_1, road_4_4_1_1, road_4_1_3','road_4_2_0','road_3_3_3','road_1_2_2','road_3_4_1','road_4_4_1','road_1_1_1','road_1_2_2','road_1_1_2','road_4_1_3','road_4_1_0','road_1_1_2','road_1_1_3','road_1_2_2','road_4_1_0','road_2_4_1','road_1_2_2','road_4_4_1','road_4_1_0','road_1_1_3','road_1_1_3','road_1_1_2','road_2_1_3','road_1_4_2','road_4_3_0','road_1_4_1','road_3_4_1','road_2_1_3','road_4_2_0','road_1_4_2','road_1_2_2','road_1_4_1','road_4_4_0','road_1_4_1','road_4_4_1','road_4_4_0_1, road_1_4_1_0']
depart = [     39,         28,          46,              90,           184,       110,            26,    4,            23,          260,         0,            13, 65, 91, 144, 140, 0, 0, 240, 192, 208, 245, 140, 238, 115, 20, 299, 290, 40, 253, 324, 169, 323, 96, 52, 49, 77, 29, 52, 105, 160, 273, 302, 182, 69, 273, 173, 317, 222, 252, 330, 37, 266, 273, 63, 195, 12, 280, 104, 50, 74, 20, 346, 276, 302, 204, 221, 79, 55, 348, 331, 302, 334, 61, 259, 234, 318, 138, 169, 302, 369, 303, 184, 356, 161, 46, 334, 185, 343, 270, 348, 92, 60, 360, 120, 140, 370, 85, 355, 332, 148, 314, 64, 317, 156, 396, 453, 408, 433, 154, 244, 300, 385, 351, 254, 118, 453, 120, 176, 234, 384, 147, 143, 260, 16, 112, 216, 368, 35, 72, 286, 450, 92, 74, 268, 417, 402, 419, 230, 180, 296, 148, 14, 368, 384, 124, 230, 415, 420, 100, 253, 426, 111, 299, 314, 419, 484, 39, 84, 212, 100, 168, 264, 420, 299, 300, 462, 450, 367, 521, 46, 196, 530, 133, 392, 221, 436, 182, 250, 259, 453, 229, 76, 111, 276, 336, 487, 299, 161, 37, 440, 380, 330, 468, 126, 260, 306, 395, 345, 440, 368, 480, 385, 207, 510, 488, 70, 589, 426, 522, 104, 272, 98, 354, 200, 342, 284, 546, 552, 378, 470, 570, 517, 600, 487, 317, 522, 538, 528, 328, 302, 296, 366, 402, 555, 334, 538, 0, 515, 342, 553, 247, 143, 384, 390, 524, 656, 351, 553, 385, 622, 454, 482, 504, 572, 589, 14, 496, 599, 294, 505, 600, 398, 552, 230, 419, 600, 677, 666, 168, 220, 580, 116, 196, 630, 672, 286, 600, 439, 416, 691, 616, 185, 672, 132, 231, 233, 590, 700, 680, 222, 78, 571, 545, 550, 342, 351, 88, 609, 224, 300, 619, 210, 375, 470, 504, 625, 378, 602, 185, 406, 472, 461, 538, 636, 351, 200, 282, 398, 710, 247, 743, 204, 487, 580, 360, 660, 436, 565, 201, 217, 402, 636, 318, 161, 600, 208, 412, 633, 128, 612, 631, 680, 645, 1, 1, 1, 2, 58, 109, 120, 130, 136, 148, 150, 152, 156, 164, 180, 184, 188, 195, 15, 21, 22, 38, 40, 41, 47, 66, 70, 71, 75, 81, 92, 101, 105, 106, 112, 113, 116, 117, 121, 131, 139, 141, 144, 145, 149, 157, 161, 162, 169, 170, 171, 181, 183, 186, 196, 197, 205, 207, 208, 220, 226, 232, 236, 240, 248, 252, 258, 259, 261, 278, 281, 287, 288, 292, 296, 300, 300, 302, 305, 306, 310, 312, 312, 315, 320, 324, 325, 328, 335, 336, 340, 342, 355, 356, 360, 365, 366, 370, 372, 372, 372, 384, 385, 390, 390, 396, 401, 402, 402, 208, 209, 211, 221, 222, 223, 231, 232, 235, 241, 248, 249, 253, 260, 261, 262, 274, 278, 279, 287, 288, 289, 297, 302, 303, 304, 307, 313, 315, 316, 318, 319, 325, 326, 329, 335, 336, 337, 349, 352, 356, 357, 361, 369, 370, 371, 373, 379, 385, 386, 386, 387, 391, 397, 399, 403, 403, 404, 408, 410, 412, 414, 415, 420, 420, 425, 426, 426, 430, 432, 432, 435, 436, 438, 438, 440, 444, 444, 445, 450, 454, 456, 456, 456, 460, 462, 465, 468, 468, 468, 470, 474, 474, 475, 480, 480, 482, 486, 486, 486, 490, 492, 492, 495, 496, 498, 500, 500, 504, 504, 504, 507, 510, 510, 510, 510, 516, 516, 520, 521, 521, 524, 525, 528, 531, 534, 534, 535, 538, 538, 541, 541, 540, 546, 552, 552, 555, 555, 555, 558, 558, 561, 564, 564, 565, 567, 566, 570, 572, 572, 575, 576, 576, 576, 581, 582, 582, 585, 588, 588, 589, 594, 594, 594, 594, 595, 602, 600, 600, 600, 603, 604, 606, 608, 409, 413, 416, 421, 427, 437, 439, 440, 441, 445, 446, 451, 454, 456, 457, 463, 469, 471, 475, 476, 481, 483, 487, 493, 497, 501, 505, 505, 506, 511, 511, 512, 522, 523, 525, 526, 529, 535, 536, 539, 539, 541, 542, 547, 554, 556, 559, 566, 567, 568, 571, 572, 573, 577, 581, 582, 583, 589, 590, 591, 595, 596, 601, 603, 604, 605, 610, 611, 612, 615, 615, 615, 616, 618, 618, 618, 620, 622, 621, 625, 628, 631, 630, 630, 631, 634, 632, 633, 637, 636, 639, 640, 640, 642, 644, 644, 645, 645, 648, 648, 648, 650, 652, 652, 654, 654, 655, 656, 658, 660, 660, 660, 661, 664, 664, 664, 666, 668, 670, 670, 672, 679, 675, 675, 682, 676, 683, 680, 686, 684, 689, 692, 688, 688, 690, 690, 690, 690, 694, 694, 696, 697, 698, 696, 702, 700, 700, 705, 706, 705, 705, 707, 709, 708, 708, 710, 710, 713, 714, 712, 714, 717, 717, 720, 720, 720, 720, 720, 721, 725, 724, 726, 726, 728, 728, 728, 731, 730, 732, 732, 734, 735, 735, 737, 736, 740, 740, 742, 743, 745, 744, 744, 744, 744, 747, 748, 750, 750, 750, 750, 752, 753, 754, 756, 756, 759, 760, 760, 760, 762, 762, 762, 764, 766, 765, 765, 765, 769, 768, 770, 771, 772, 774, 776, 776, 776, 777, 780, 780, 780, 780, 780, 780, 780, 783, 784, 786, 787, 788, 789, 790, 792, 792, 792, 795, 795, 795, 796, 798, 798, 798, 798, 800, 800, 802, 804, 805, 808, 808, 808, 809, 810, 613, 616, 617, 623, 626, 632, 633, 634, 635, 637, 638, 641, 645, 646, 649, 653, 655, 656, 657, 661, 662, 665, 667, 671, 673, 676, 677, 683, 686, 689, 687, 689, 690, 691, 691, 694, 696, 697, 698, 700, 702, 703, 707, 710, 709, 710, 713, 711, 714, 715, 718, 721, 721, 722, 727, 729, 733, 737, 738, 741, 744, 745, 745, 746, 751, 757, 761, 763, 766, 767, 777, 778, 781, 781, 782, 793, 796, 797, 799, 801, 809, 810, 810, 810, 811, 812, 814, 816, 816, 816, 817, 820, 820, 820, 820, 823, 824, 824, 825, 825, 826, 828, 829, 830, 831, 832, 832, 834, 834, 835, 836, 838, 840, 840, 840, 840, 841, 840, 842, 843, 845, 846, 848, 849, 850, 852, 852, 852, 852, 853, 855, 855, 855, 856, 856, 858, 860, 860, 861, 864, 864, 864, 867, 868, 870, 870, 870, 870, 870, 870, 872, 872, 873, 875, 876, 876, 879, 880, 880, 882, 884, 885, 885, 886, 888, 888, 888, 888, 890, 892, 896, 897, 811, 812, 813, 817, 818, 821, 826, 826, 827, 833, 835, 836, 837, 841, 841, 842, 843, 844, 853, 854, 856, 857, 861, 862, 865, 871, 871, 872, 873, 874, 877, 881, 886, 889]
